In [1]:
import glob, json, pathlib
from tqdm.notebook import tqdm
sanitychecks = list(glob.glob(r"D:\ig_pipeline\cad\*\*\artifacts\sanitycheck.json"))
jsons = {}
for x in tqdm(sanitychecks):
    with open(x, "r") as f:
        name = "/".join(pathlib.Path(x).parts[-4:-2])
        jsons[name] = json.load(f)

  0%|          | 0/92 [00:00<?, ?it/s]

In [2]:
total = len(jsons)
failed = {name for name, x in jsons.items() if x["errors"]}
total, len(failed)

(92, 92)

In [3]:
from collections import defaultdict
import re
error_templates_and_ignore = {
    "Category (.*) for object .* does not exist on spreadsheet.": False,
    "(.*) has too many vertices: .* > .*": True,
    "(.*) link .* has different .* in instance .* compared to instance .*. .* difference: .*.": False,
    "(.*) is missing in instance .*, so relative transform check cannot be completed.": False,
    "(.*) has bad name.": False,
    "(.*) has negative object offset scale.*": False,
    "(.*) has scale that is not 1. Reset scale.": False,
    ".*No instance ID 0 instance of (.*)": False,
    "Articulated object (.*) instances have different scales for base links. This may have broken things during the match links script.": True,
    "All instances of (.*) do not have contiguous instance IDs. Missing: .*": False,
    "(.*) has disallowed type.*": False,
    "Light object (.*) should not have object offset rotation. Reset pivot.": False,
    "(.*) has different pivot offset position \(by (.*)\). Match pivots on each instance.": False,
    "(.*) has different pivot offset rotation \(by (.*)\). Match pivots on each instance.": False,
    "(.*) has different shear. Match scaling axes on each instance.": False,
    "Model ID (.*) contains 'todo'.": False,
    "(.*) should not have an upper side.": False,
    "(.*) should have an upper side.": False,
    "(.*) has different vertex count than recorded in provider.*": False,
    "(.*) has different face count than recorded in provider.*": False,
    "(.*) has no collision mesh. Create a collision mesh.": False,
    "Inconsistent link sets within model (.*)": False,
    "(.*) is missing meta link: fillable.": False,
    "(.*) is missing meta link: heatsource.": False,
    "(.*) is missing meta link: togglebutton.": False,
    "(.*) is missing meta link: particleapplier.": False,
    "(.*) is missing meta link: fluidsource.": False,
    "(.*) is missing meta link: fluidsink.": False,
    "(.*) is missing meta link: togglebutton.": False,
    "(.*) element .* has too many vertices .*": True,
    "Model ID (.*) requires joints but has no joints.": False,
    "Expected meta links for (.*) are missing: .*": False,
    "Convex mesh fillable meta link (.*) should be of Editable Poly instead of .*": False,
    "Cloth object (.*) should consist of exactly 1 element. Currently it has .* elements.": False,
    "Cannot validate clothness: category (.*) not found in taxonomy.": False,
    "(.*) element .* is not a volume": True,
    "(.*) element .* has elements trimesh still finds splittable": False,
    "Object (.*) has .* fillable meshes. Should have no more than one.": False,
    "Model (.*) has category .* that is neither the from or to element in the rename file.": False,
    "(.*) has meta link not required by its synset: .*": True,
    "(.*) meta type .* ID .* has non-continuous subids .*": False,
    "Cannot validate meta links and joints for model ID (.*): Category .* not found in taxonomy.": False,
    "(.*) is in the deletion queue. Delete the object.": False,
    "(.*) has unapplied rename .*.": True,
    "fillable_seed is an invalid meta link under (.*).": False,
    "(.*) is not unique.": False,
    "Object (.*) has .* meshes. Should have no more than one.": False,
    "(.*) is not under another object but contains part tags {'connectedpart'}.": False,
    "(.*) is not rendering the baked material. Select the baked material for rendering or rebake.": True,
    "(.*) has non-shell material. Run texture baking.": True,
    "(.*) is not rendering the baked material in the viewport. Select the baked material for viewport.": True,
    "(.*) has different material. Match materials on each instance.": True,
    "(.*) has different UV unwrapping than recorded. Reunwrap the object.": True,
    "(.*) material tree contains .* with disallowed type .*. It should be one of .*.": False,
}

SKIP_IGNORE = True

error_templates = list(error_templates_and_ignore.keys())
error_matches = defaultdict(set)
error_matches_by_file = defaultdict(lambda: defaultdict(set))
unknown_errors_by_file = defaultdict(set)
for name, f in jsons.items():
    for error in f["errors"] + f["warnings"]:
        for i, tmpl in enumerate(error_templates):
            match = re.fullmatch(tmpl, error, re.S)
            if match is not None:
                if not (SKIP_IGNORE and error_templates_and_ignore[tmpl]):
                    error_matches[tmpl].add(match)
                    error_matches_by_file[tmpl][name].add(match)

                break
        else:
            print(error)
            unknown_errors_by_file[name].add(error)
        
print("\nCounts")
for tmpl, objs in sorted(error_matches.items(), key=lambda x: len(x[1]), reverse=True):
    print(f"{tmpl}: {len(objs)}")

print("\n\nCounts by file")
for tmpl, matches_by_file in error_matches_by_file.items():
    all_matches = error_matches[tmpl]
    if not all_matches:
        continue
    print(f"\n{tmpl}")
    for name, matches in sorted(matches_by_file.items(), key=lambda x: x[1], reverse=True):
        if not matches:
            break
        print(f"  {name}: {len(matches)}")


Counts
(.*) material tree contains .* with disallowed type .*. It should be one of .*.: 9010
(.*) is in the deletion queue. Delete the object.: 7


Counts by file

(.*) material tree contains .* with disallowed type .*. It should be one of .*.
  objects/batch-00: 35
  objects/batch-01: 73
  objects/batch-02: 31
  objects/batch-03: 18
  objects/batch-04: 9
  objects/batch-05: 41
  objects/batch-06: 47
  objects/batch-07: 33
  objects/batch-08: 49
  objects/batch-09: 75
  objects/batch-10: 50
  objects/batch-11: 12
  objects/batch-12: 7
  objects/batch-13: 5
  objects/legacy_batch-07: 1
  objects/procedural-aa: 28
  objects/school_objects-aa: 15
  objects/task_relavant-ab: 835
  objects/task_relevant-xy: 72
  scenes/Beechwood_0_garden: 28
  scenes/commercial_kitchen_fire_extinguisher: 218
  scenes/commercial_kitchen_pans: 1
  scenes/grocery_store_asian: 341
  scenes/grocery_store_cafe: 25
  scenes/grocery_store_half_stocked: 912
  scenes/hall_arch_wood: 8
  scenes/hall_conference_large:

In [ ]:
import sys
sys.path.append(r"D:\ig_pipeline")
from b1k_pipeline.utils import parse_name
error_objs = {parse_name(x.group(1)).group("model_id") for x in error_matches["(.*) material tree contains .* with disallowed type .*. It should be one of .*."]}
print(len(error_objs))
# with open("bad_materials.json", "w") as f:
#     json.dump(sorted(error_objs), f)

1394


In [6]:
from IPython.display import display, Markdown

# Print the number of kinds of error for each target
output = []
output.append("\n\n# Counts by error")
for name in jsons.keys():
    all_errors = {}
    for tmpl, matches_by_file in error_matches_by_file.items():
      if error_templates_and_ignore[tmpl]:
          continue
      matches = matches_by_file[name]
      if not matches:
          continue
      all_errors[tmpl] = matches

    for error in unknown_errors_by_file[name]:
        all_errors["Unknown"] = all_errors.get("Unknown", set())
        all_errors["Unknown"].add(error)

    if all_errors:
        output.append(f"\n## {name}")
        for k, v in sorted(all_errors.items(), key=lambda x: len(x[1]), reverse=True):
            output.append(f"  ### {k}: {len(v)}")
            for match in v:
                err_str = match if isinstance(match, str) else match.group(0)
                output.append("    " + err_str)
            output.append("")

display(Markdown("\n".join(output)))



# Counts by error

## objects/batch-00
  ### (.*) material tree contains .* with disallowed type .*. It should be one of .*.: 35
    bookend-eyddem-0 material tree contains Bookend06_gold:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bookend-detqsw-0 material tree contains Material #2146935558:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bookend-huxmnl-0 material tree contains Bookend10:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    bookend-eyddem-0 material tree contains Material #2146935517:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bookend-hlpgle-0 material tree contains Material #2146935554:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bookend-huxmnl-0 material tree contains Material #2146935558:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bag_of_brown_rice-tnfazj-0 material tree contains vray_BrownRice_3_Double_mult:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bag_of_brown_rice-tnfazj-0 material tree contains Ground_matte-shadows:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bookend-detqsw-0 material tree contains Bookend10:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-mint-oewhqr-0 material tree contains highball_mint_leaves:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    bag_of_brown_rice-tnfazj-0 material tree contains BrownRice_Light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bookend-eyddem-0 material tree contains Material #2146935554:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bottle_of_aspirin-psvktc-0 material tree contains bottle:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bottle_of_aspirin-psvktc-0 material tree contains label:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bookend-hlpgle-0 material tree contains Material #2146935517:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    icicle_lights-lbwpka-0 material tree contains thread:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bottle_of_aspirin-psvktc-0 material tree contains cup:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bookend-oxfecv-0 material tree contains Bookend08:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-pea-wsitbh-0 material tree contains Sepals_and_petals:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-pea-duigmh-0 material tree contains Sepals_and_petals:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bookend-xqyxrq-0 material tree contains Bookend07:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-pea-unfdjb-0 material tree contains Sepals_and_petals:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bookend-eyddem-0 material tree contains Bookend06_chrome:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-pea-lprbcf-0 material tree contains Sepals_and_petals:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bookend-hlpgle-0 material tree contains Bookend06_chrome:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-pea-hzlhrs-0 material tree contains Sepals_and_petals:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bookend-jmodol-0 material tree contains Bookend09_stand:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-pea-tmsvpt-0 material tree contains Sepals_and_petals:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bookend-rwvcah-0 material tree contains Bookend08:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-pea-absogx-0 material tree contains Sepals_and_petals:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bookend-hlpgle-0 material tree contains Bookend06_gold:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-pea-gikvpd-0 material tree contains Sepals_and_petals:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bookend-ygynyq-0 material tree contains Bookend07:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-pea_pod-qntgxe-0 material tree contains Sepals_and_petals:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bookend-sgiryo-0 material tree contains Bookend09_stand:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.


## objects/batch-01
  ### (.*) material tree contains .* with disallowed type .*. It should be one of .*.: 73
    L-kabob-vderrf-0 material tree contains Meat:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-kabob-icxokf-0 material tree contains Meat:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-green_bean-pnwyiy-1 material tree contains GreenBeans_mat:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-half_green_bean-qqucst-0-Tsubpart material tree contains Glow_mat:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-diamond-xeoqwf-0-Tglass material tree contains Diamond Material:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-sliced_tomato-whfsoo-0 material tree contains tomatoBase:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-sliced_tomato-whfsoo-0 material tree contains tomato:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-parsley-bvekjv-0 material tree contains Parsley:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-green_bean-mcbvwu-3 material tree contains Glow_mat:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-half_green_bean-qqucst-0-Tsubpart material tree contains GreenBeans_mat:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-firewood_grate-jaobtm-0 material tree contains aged bronze:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-sliced_tomato-qjpofs-0 material tree contains tomatoBase:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-sliced_tomato-qjpofs-0 material tree contains tomato:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-green_bean-mcbvwu-3 material tree contains GreenBeans_mat:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-green_bean-mcbvwu-1 material tree contains Glow_mat:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fireplace-gpnsij-0 material tree contains marble_osn:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-parsley-coghab-0 material tree contains Parsley:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-parsley-nasgbx-0 material tree contains Parsley:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-parsley-jirtgx-0 material tree contains Parsley:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-green_bean-iukfzl-2 material tree contains Glow_mat:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-green_bean-mcbvwu-1 material tree contains GreenBeans_mat:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    fireplace-gpnsij-0 material tree contains marble_osn:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-half_parsley-zgpzbo-0-Tsubpart material tree contains Parsley:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-green_bean-iukfzl-2 material tree contains GreenBeans_mat:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-green_bean-mcbvwu-0 material tree contains Glow_mat:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fireplace-gpnsij-0 material tree contains white brick:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-green_bean-iukfzl-3 material tree contains Glow_mat:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-green_bean-pnwyiy-0 material tree contains Glow_mat:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-green_bean-mcbvwu-0 material tree contains GreenBeans_mat:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-scone-sgiols-0 material tree contains Material #40:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-green_bean-pnwyiy-3 material tree contains Glow_mat:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-green_bean-pnwyiy-0 material tree contains GreenBeans_mat:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    blender-dhfqid-0-base_link material tree contains Arch145_034_003:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-half_scone-vykthm-0-Tsubpart material tree contains Material #40:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-green_bean-pnwyiy-3 material tree contains GreenBeans_mat:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-green_bean-mcbvwu-2 material tree contains Glow_mat:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    blender-dhfqid-0-lid-base_link-R-lower-Topenable material tree contains Arch145_034_003:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-instant_pot-lkbvad-0-lid-base_link-R-upper-Topenable material tree contains cgaxis_models_116_04_03:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-half_scone-nruwwp-0-Tsubpart material tree contains Material #40:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-green_bean-mcbvwu-5 material tree contains Glow_mat:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-green_bean-pnwyiy-1 material tree contains Glow_mat:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-green_bean-mcbvwu-2 material tree contains GreenBeans_mat:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    blender-dhfqid-0-lid-base_link-R-upper-Topenable material tree contains Arch145_034_003:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-lid-iuhvrv-0 material tree contains Boxing, Fork_spoon:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-parsley-kavoif-0 material tree contains Parsley:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-green_bean-mcbvwu-5 material tree contains GreenBeans_mat:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-green_bean-iukfzl-1 material tree contains Glow_mat:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    blender-dhfqid-0-glass-base_link-F-lower-Tglass material tree contains Arch145_034_003:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-tablefork-ptqjqv-0 material tree contains Boxing, Fork_spoon:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-green_bean-iukfzl-3 material tree contains GreenBeans_mat:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-green_bean-iukfzl-1 material tree contains GreenBeans_mat:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    chard-linuaq-0 material tree contains Letucce_v1:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    lunch_box-adxzhe-0 material tree contains Boxing, Fork_spoon:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-green_bean-pnwyiy-2 material tree contains Glow_mat:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-green_bean-iukfzl-0 material tree contains Glow_mat:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    half_chard-oajufk-0-Tsubpart material tree contains Letucce_v1:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-half_kabob-oeeitd-0-Tsubpart material tree contains Meat:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-half_parsley-zfzedo-0-Tsubpart material tree contains Parsley:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-green_bean-pnwyiy-2 material tree contains GreenBeans_mat:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-green_bean-iukfzl-0 material tree contains GreenBeans_mat:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    half_chard-shmety-0-Tsubpart material tree contains Letucce_v1:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-half_kabob-eormcp-0-Tsubpart material tree contains Meat:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-parsley-acotgf-0 material tree contains Parsley:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-green_bean-mcbvwu-4 material tree contains Glow_mat:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-half_green_bean-exrkaw-0-Tsubpart material tree contains Glow_mat:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-instant_pot-lkbvad-0-base_link material tree contains cgaxis_models_116_04_03:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-kabob-khdosp-0 material tree contains Meat:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-parsley-paksij-0 material tree contains Parsley:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-parsley-okmcfy-0 material tree contains Parsley:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-green_bean-mcbvwu-4 material tree contains GreenBeans_mat:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-half_green_bean-exrkaw-0-Tsubpart material tree contains GreenBeans_mat:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-instant_pot-lkbvad-0-lid-base_link-R-lower-Topenable material tree contains cgaxis_models_116_04_03:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-parsley-rispvj-0 material tree contains Parsley:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.


## objects/batch-02
  ### (.*) material tree contains .* with disallowed type .*. It should be one of .*.: 31
    L-postage_stamp-biqmtn-0 material tree contains vray_wr_mat_001:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    curtain-trwest-0 material tree contains tulle:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-cupcake-sutaow-0 material tree contains vray Material #1873468054:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-postage_stamp-yszkpy-0 material tree contains vray_wr_mat_005:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-half_cheese_tart-txjdgi-0-Tsubpart material tree contains vray Material #84:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-cupcake-outske-0 material tree contains vray Material #1873468054:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-half_muffin-ypzhst-0-Tsubpart material tree contains vray Material #72:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-cheese_tart-pyynjg-0 material tree contains vray Material #84:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-muffin-upcxno-0 material tree contains vray Material #72:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-half_muffin-ubimyt-0-Tsubpart material tree contains vray Material #72:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-half_doughnut-qhhrmz-0-Tsubpart material tree contains vray Material #77:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-half_doughnut-qcuowh-0-Tsubpart material tree contains vray Material #77:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-half_sourdough-flwdle-0-Tsubpart material tree contains vray Material #1873468057:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-donut-euibpi-0 material tree contains vray Material #77:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-sourdough-ltjltm-0 material tree contains vray Material #1873468057:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-half_sourdough-wgcbuy-0-Tsubpart material tree contains vray Material #1873468057:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-half_cinnamon_roll-mijkzq-0-Tsubpart material tree contains vray Material #1873468061:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-half_cinnamon_roll-tmlned-0-Tsubpart material tree contains vray Material #1873468061:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-cinnamon_roll-xjlptd-0 material tree contains vray Material #1873468061:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-toast-nmxadm-0 material tree contains vray Material #88:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-mint-jezepy-0 material tree contains Material #43:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-mint-jezepy-0 material tree contains vray_mint_stem:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-sourdough-ndcexh-0 material tree contains vray Material #1873468057:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-donut-hfdwgc-0 material tree contains vray Material #1873468040:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-cupcake-mbhweg-0 material tree contains vray Material #116:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-sourdough-lourny-0 material tree contains vray Material #78:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-half_toast-gjpbgv-0-Tsubpart material tree contains vray Material #88:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-half_cupcake-ofuznw-0-Tsubpart material tree contains vray Material #116:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-half_cupcake-opuxgd-0-Tsubpart material tree contains vray Material #116:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-half_toast-xvqnvl-0-Tsubpart material tree contains vray Material #88:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-half_cheese_tart-jldqnm-0-Tsubpart material tree contains vray Material #84:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.


## objects/batch-03
  ### (.*) material tree contains .* with disallowed type .*. It should be one of .*.: 18
    L-water_glass-vcwsbm-0-Tglass material tree contains Material #29:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    half_fried_chicken-axoton-0-Tsubpart material tree contains chicken_03_mat:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-mint-sivqgq-0 material tree contains vray_mint_stem:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-paraffin_wax-nnbvcm-0 material tree contains Sugar_mat:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-weed-nbhoge-0 material tree contains Material #665:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    half_fried_chicken-emcign-0-Tsubpart material tree contains chicken_03_mat:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    fried_chicken-lpqpbn-0 material tree contains chicken_03_mat:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    bandana-wbhliu-0 material tree contains Kerchief:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-mint-sivqgq-0 material tree contains Material #43:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    cd-xkfnrj-0 material tree contains vray_Data:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-paraffin_wax-nnbvcm-0 material tree contains Light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-half_buttermilk_pancake-rfgvmd-0-Tsubpart material tree contains Pancake:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-half_buttermilk_pancake-svndrr-0-Tsubpart material tree contains Pancake:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-paraffin_wax-onopqz-0 material tree contains Light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-paraffin_wax-onopqz-0 material tree contains Sugar_mat:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-hingeless_jar-qdnmwg-0 material tree contains jar_Coat_001:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-cotton_ball-gabqdn-0 material tree contains Jar_01_SSS:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-buttermilk_pancake-ogghhf-0 material tree contains Pancake:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.


## objects/batch-04
  ### (.*) material tree contains .* with disallowed type .*. It should be one of .*.: 9
    L-half_turkey-hbzolq-0-Tsubpart material tree contains Turkey_vray_SSS:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-stapler-wheosp-0 material tree contains Red_paint:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    paper_lantern-hgmreo-0 material tree contains Mat3d66-1514387-48-8577:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-turkey-ppaslw-0 material tree contains Turkey_vray_SSS:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-raspberry-ygrfsn-0 material tree contains berry:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-stapler-wheosp-0 material tree contains Paint:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-raspberry-ygrfsn-0 material tree contains leaf:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-stapler-wheosp-0 material tree contains Plastic:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-half_turkey-xuxpbr-0-Tsubpart material tree contains Turkey_vray_SSS:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.


## objects/batch-05
  ### (.*) material tree contains .* with disallowed type .*. It should be one of .*.: 41
    bottom_cabinet_no_top-gjrero-0-link_0-base_link-R-lower-Topenable material tree contains AE27_554:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-trophy-afownt-0 material tree contains vray_GoldTrophy2:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    bottom_cabinet_no_top-gjrero-0-link_0-base_link-R-upper-Topenable material tree contains AE27_554:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-meringue-rxnxmg-0 material tree contains IceCream:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-okra-mopall-0 material tree contains Okra_mat:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-okra-mopall-0 material tree contains Glow_mat:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-half_rutabaga-afinny-0-Tsubpart material tree contains Rutabaga:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-okra-oezxso-0 material tree contains Okra_mat:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-okra-oezxso-0 material tree contains Glow_mat:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-half_rutabaga-kvelnh-0-Tsubpart material tree contains Rutabaga:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-okra-oibmms-0 material tree contains Okra_mat:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-okra-oibmms-0 material tree contains Glow_mat:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-rutabaga-qpwfps-0 material tree contains Rutabaga:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-sticker-njigvs-0 material tree contains curved:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-whitewall_tire-bjcowe-0 material tree contains Material Set:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-table_lamp-fhsorf-0-arm-base_link-R-upper material tree contains Black_ceiling:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-rice_cooker-qatgeb-0-lid-base_link-R-upper-Topenable material tree contains IH_LCD:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-rice_cooker-qatgeb-0-lid-base_link-R-upper-Topenable material tree contains Material #56:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-table_lamp-fhsorf-0-base_link material tree contains Black_ceiling:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-rice_cooker-qatgeb-0-lid-base_link-R-upper-Topenable material tree contains Material #56:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-rice_cooker-qatgeb-0-lid-base_link-R-upper-Topenable material tree contains vray_IH_button2:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-table_lamp-fhsorf-0-arm-base_link-R-lower material tree contains Black_ceiling:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-rice_cooker-qatgeb-0-lid-base_link-R-lower-Topenable material tree contains Material #56:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-rice_cooker-qatgeb-0-lid-base_link-R-upper-Topenable material tree contains vray_IH_button1:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-rice_cooker-qatgeb-0-lid-base_link-R-lower-Topenable material tree contains vray_IH_button2:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-rice_cooker-qatgeb-0-lid-base_link-R-lower-Topenable material tree contains IH_LCD:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-rice_cooker-qatgeb-0-lid-base_link-R-lower-Topenable material tree contains vray_IH_button1:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-rice_cooker-qatgeb-0-lid-base_link-R-lower-Topenable material tree contains Material #56:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-stockpot-azoiaq-0-Tconnectedpart material tree contains IH_LCD:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-stockpot-azoiaq-0-Tconnectedpart material tree contains Material #56:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-stockpot-azoiaq-0-Tconnectedpart material tree contains vray_IH_button2:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-stockpot-azoiaq-0-Tconnectedpart material tree contains Material #56:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bottom_cabinet_no_top-gjrero-0-link_1-base_link-R-upper-Topenable material tree contains AE27_554:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-rice_cooker-qatgeb-0-base_link material tree contains Material #56:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-stockpot-azoiaq-0-Tconnectedpart material tree contains vray_IH_button1:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-rice_cooker-qatgeb-0-base_link material tree contains vray_IH_button2:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-rice_cooker-qatgeb-0-base_link material tree contains IH_LCD:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-rice_cooker-qatgeb-0-base_link material tree contains vray_IH_button1:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-rice_cooker-qatgeb-0-base_link material tree contains Material #56:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-punching_bag-svkdji-0 material tree contains Leather_brown:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bottom_cabinet_no_top-gjrero-0-link_1-base_link-R-lower-Topenable material tree contains AE27_554:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.


## objects/batch-06
  ### (.*) material tree contains .* with disallowed type .*. It should be one of .*.: 47
    L-half_biscuit_dough-wxqptp-0-Tsubpart material tree contains Dough_02:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-half_sugar_cookie_dough-icwbdb-0-Tsubpart material tree contains Dough_02:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-portafilter-obdthu-0 material tree contains chrome:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-sugar_cookie_dough-qewbbb-0 material tree contains Dough_02:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    curtain_rod-pburmj-0 material tree contains Antique.Bronze:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-half_roll_dough-gwnxss-0-Tsubpart material tree contains Dough_02:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-biscuit_dough-vbhtbo-0 material tree contains Dough_02:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-half_roll_dough-khmxtb-0-Tsubpart material tree contains Dough_02:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-half_edible_cookie_dough-zgxapd-0-Tsubpart material tree contains Dough_02:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-roll_dough-plwpgb-0 material tree contains Dough_02:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-watering_can-pewtns-0 material tree contains Head:Composite with disallowed type compositematerial. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-half_edible_cookie_dough-jkqazq-0-Tsubpart material tree contains Dough_02:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-twine-fngeov-0 material tree contains Leather_Distressed:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-bottle_of_milk-gchrwu-0 material tree contains milk:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-bottle_of_milk-gchrwu-0-glass-base_link-F-lower-Tglass material tree contains milk:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-bottle_of_coke-bmtvvb-0 material tree contains label 2 sides:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-bottle_of_coke-uqusgj-0 material tree contains cap:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-bottle_of_coke-bmtvvb-0-glass-base_link-F-lower-Tglass material tree contains cap:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-bottle_of_coke-bmtvvb-0-glass-base_link-F-lower-Tglass material tree contains label 2 sides:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-bottle_of_coke-bmtvvb-0 material tree contains cap:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-half_pizza_dough-spvbal-0-Tsubpart material tree contains Dough_02:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-bottle_of_coke-uqusgj-0-glass-base_link-F-lower-Tglass material tree contains cap:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-bottle_of_coke-uqusgj-0-glass-base_link-F-lower-Tglass material tree contains label 2 sides:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-half_cookie_dough-gcnxkb-0-Tsubpart material tree contains Dough_02:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-edible_cookie_dough-epwkoi-0 material tree contains Dough_02:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-half_chocolate_cookie_dough-qpfajx-0-Tsubpart material tree contains spicy_cooki_dough:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-half_chocolate_cookie_dough-jpknaw-0-Tsubpart material tree contains spicy_cooki_dough:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-cookie_dough-pewqae-0 material tree contains Dough_02:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    curtain_rod-pburmj-0 material tree contains Bronze:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-half_pastry-ilugji-0-Tsubpart material tree contains Dough_02:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-chocolate_cookie_dough-csnixa-0 material tree contains spicy_cooki_dough:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-half_pizza_dough-xzkcwu-0-Tsubpart material tree contains Dough_02:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-pastry-bqjjie-0 material tree contains Dough_02:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-half_pastry-izfobj-0-Tsubpart material tree contains Dough_02:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-bottle_of_coke-uqusgj-0 material tree contains label 2 sides:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-pizza_dough-jguwbv-0 material tree contains Dough_02:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-half_sugar_cookie_dough-ueipcz-0-Tsubpart material tree contains Dough_02:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-half_bagel_dough-zqdnzp-0-Tsubpart material tree contains Dough_02:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-half_bagel_dough-yhsrmc-0-Tsubpart material tree contains Dough_02:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-half_cookie_dough-gqgssp-0-Tsubpart material tree contains Dough_02:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-paper_coffee_filter-kizndy-0 material tree contains vray 02_chemex_type2_01_paper:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-half_spice_cookie_dough-xzxwml-0-Tsubpart material tree contains spicy_cooki_dough:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-bagel_dough-iuembm-0 material tree contains Dough_02:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    curtain_rod-pburmj-0 material tree contains Plastic.Translucent:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-spice_cookie_dough-iuiioe-0 material tree contains spicy_cooki_dough:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-half_spice_cookie_dough-iajxas-0-Tsubpart material tree contains spicy_cooki_dough:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-half_biscuit_dough-fkbfdp-0-Tsubpart material tree contains Dough_02:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.


## objects/batch-07
  ### (.*) material tree contains .* with disallowed type .*. It should be one of .*.: 33
    L-dahlia_flower-kudrwc-0 material tree contains  Dahlia 1404 2 FLOWERS 1:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-toaster_oven-ctrngh-0-door-base_link-R-lower material tree contains screen:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-dahlia_flower-kudrwc-0 material tree contains PETAL WHITE 1404 1:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-toaster_oven-ctrngh-0-base_link material tree contains screen:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-flower_petal-hhmaur-0 material tree contains PETAL WHITE 1404 1:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-cell_phone-dbhfuh-0 material tree contains vray_phone_mat:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-dahlia_flower-hdkzhl-0 material tree contains  Dahlia 1404 2 FLOWERS 3:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-flower_petal-hhmaur-0 material tree contains PETAL WHITE 1404 3:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-flower_petal-hhmaur-0 material tree contains PETAL WHITE 1404 2:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-dahlia_flower-hdkzhl-0 material tree contains PETAL WHITE 1404 2:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-flower_petal-gzslfo-0 material tree contains PETAL WHITE 1404 1:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-flower_petal-gzslfo-0 material tree contains PETAL WHITE 1404 3:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-lettuce-orsjxq-0 material tree contains nmodel_03_31_salad_mat:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-flower_petal-gzslfo-0 material tree contains PETAL WHITE 1404 2:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-rosemary-qbfkpo-0 material tree contains 15 - Default:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-rosemary-bdsaeg-0 material tree contains 15 - Default:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    digital_camera-zcnxuz-0 material tree contains 13 - Default:Autodesk_Plastic_Vinyl with disallowed type Autodesk_Material. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-power_drill-hdvnxd-0 material tree contains Tip_Blended_Mat:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    digital_camera-zcnxuz-0 material tree contains 18 - Default:Autodesk_Metal with disallowed type Autodesk_Material. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    digital_camera-zcnxuz-0 material tree contains 21 - Default:Autodesk_Generic with disallowed type Autodesk_Material. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    digital_camera-zcnxuz-0 material tree contains Solid Glass:Autodesk_Solid_Glass with disallowed type Autodesk_Material. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    digital_camera-zcnxuz-0 material tree contains 22 - Default:Autodesk_Plastic_Vinyl with disallowed type Autodesk_Material. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    digital_camera-zcnxuz-0 material tree contains 16 - Default:Autodesk_Generic with disallowed type Autodesk_Material. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    digital_camera-zcnxuz-0 material tree contains 15 - Default:Autodesk_Plastic_Vinyl with disallowed type Autodesk_Material. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    digital_camera-zcnxuz-0 material tree contains 12 - Default:Autodesk_Plastic_Vinyl with disallowed type Autodesk_Material. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    digital_camera-zcnxuz-0 material tree contains 19 - Default:Autodesk_Plastic_Vinyl with disallowed type Autodesk_Material. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    digital_camera-zcnxuz-0 material tree contains 23 - Default:Autodesk_Plastic_Vinyl with disallowed type Autodesk_Material. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    digital_camera-zcnxuz-0 material tree contains 14 - Default:Autodesk_Plastic_Vinyl with disallowed type Autodesk_Material. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    baseball_glove-feakpw-0 material tree contains vray_Lether_Yellow_with_text:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    digital_camera-zcnxuz-0 material tree contains 20 - Default:Autodesk_Plastic_Vinyl with disallowed type Autodesk_Material. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-toaster_oven-ctrngh-0-door-base_link-R-upper material tree contains screen:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    baseball_glove-feakpw-0 material tree contains vray_Main_Letaher:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-toaster_oven-ctrngh-0-window-door-F-lower-Tglass material tree contains screen:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.


## objects/batch-08
  ### (.*) material tree contains .* with disallowed type .*. It should be one of .*.: 49
    candied_yam-zzxnmn-0 material tree contains Sweet_Potato:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    candied_yam-axmnvy-0 material tree contains Sweet_Potato:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    lily-yltekp-0 material tree contains TEXTURELilium white 2706 1:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    lily-yltekp-0 material tree contains TEXTURE Lilium white 2706 5:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    candied_yam-ivkukc-0 material tree contains Sweet_Potato:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    candied_yam-mztmgb-0 material tree contains Sweet_Potato:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    lily-jobzwo-0 material tree contains TEXTURE Lilium white 2706 3:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    lily-yltekp-0 material tree contains TEXTURE Lilium white 2706 2:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    half_candied_yam-ckwhwm-0-Tsubpart material tree contains Sweet_Potato:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    half_candied_yam-dqfxuu-0-Tsubpart material tree contains Sweet_Potato:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    lily-jobzwo-0 material tree contains TEXTURE Lilium white 2706 5:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    lily-jobzwo-0 material tree contains TEXTURE Lilium white 2706 7:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-sliced_mushroom-jnfabw-0 material tree contains PortobelloMushroomSlice_mat:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    candied_yam-ayqjzg-0 material tree contains Sweet_Potato:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    lily-jobzwo-0 material tree contains TEXTURE Lilium white 2706 2:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    lily-jobzwo-0 material tree contains TEXTURELilium white 2706 1:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-sliced_mushroom-jnfabw-0 material tree contains Glow_mat:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-frying_pan-hzspwg-0 material tree contains Car_Paint_Standard:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    lily-qqtrts-0 material tree contains TEXTURE Lilium white 2706 7:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    lily-qqtrts-0 material tree contains TEXTURE Lilium white 2706 3:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-frying_pan-hzspwg-0 material tree contains TeflonMtl:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    lily-qqtrts-0 material tree contains TEXTURELilium white 2706 1:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    lily-qqtrts-0 material tree contains TEXTURE Lilium white 2706 5:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    lily-jtouad-0 material tree contains TEXTURELilium white 2706 1:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    lily-qqtrts-0 material tree contains TEXTURE Lilium white 2706 2:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    lily-jfeimu-0 material tree contains TEXTURE Lilium white 2706 7:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    lily-jtouad-0 material tree contains TEXTURE Lilium white 2706 2:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    lily-jfeimu-0 material tree contains TEXTURE Lilium white 2706 2:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    lily-jfeimu-0 material tree contains TEXTURELilium white 2706 1:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    lily-rygjjm-0 material tree contains TEXTURELilium white 2706 1:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    lily-rygjjm-0 material tree contains TEXTURE Lilium white 2706 7:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    lily-mohrqo-0 material tree contains TEXTURE Lilium white 2706 7:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    lily-relerp-0 material tree contains TEXTURE Lilium white 2706 7:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    lily-rygjjm-0 material tree contains TEXTURE Lilium white 2706 2:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    lily-mohrqo-0 material tree contains TEXTURE Lilium white 2706 5:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    lily-relerp-0 material tree contains TEXTURELilium white 2706 1:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    lily-relerp-0 material tree contains TEXTURE Lilium white 2706 2:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    lily-mohrqo-0 material tree contains TEXTURELilium white 2706 1:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    lily-qpqtlz-0 material tree contains TEXTURELilium white 2706 1:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    lily-qpqtlz-0 material tree contains TEXTURE Lilium white 2706 7:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    lily-mohrqo-0 material tree contains TEXTURE Lilium white 2706 2:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    lily-pfodaq-0 material tree contains TEXTURE Lilium white 2706 7:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    lily-qpqtlz-0 material tree contains TEXTURE Lilium white 2706 2:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    lily-yltekp-0 material tree contains TEXTURE Lilium white 2706 3:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    lily-pfodaq-0 material tree contains TEXTURE Lilium white 2706 2:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    lily-pfodaq-0 material tree contains TEXTURELilium white 2706 1:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    lily-yltekp-0 material tree contains TEXTURE Lilium white 2706 7:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    lily-mohrqo-0 material tree contains TEXTURE Lilium white 2706 3:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    lily-jtouad-0 material tree contains TEXTURE Lilium white 2706 7:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.


## objects/batch-09
  ### (.*) material tree contains .* with disallowed type .*. It should be one of .*.: 75
    L-macaron-yhsnod-0 material tree contains Material #3023:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-macaron-yhsnod-0 material tree contains Material #3008:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-macaron-yhsnod-0 material tree contains Material #3021:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-half_macaroon-znmffr-0-Tsubpart material tree contains Material #3021:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    centerpiece-atfpvd-0 material tree contains tulips 1203 26:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-sugar_cookie-feirtr-0 material tree contains Material #2926:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-half_sugar_cookie-hvqdow-0-Tsubpart material tree contains Material #2927:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-cupcake-hvyxpw-0 material tree contains Brioche1:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-macaron-yhsnod-0 material tree contains MacaronChocolat:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-macaron-szcqkk-0 material tree contains MacaronChocolat:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-quiche-fvhqcu-0 material tree contains Quiche:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    centerpiece-atfpvd-0 material tree contains tulips FLOWER 1403 20:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-half_sugar_cookie-hvqdow-0-Tsubpart material tree contains Material #2926:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-cupcake-wdiezm-0 material tree contains Brioche1:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-cupcake-hvyxpw-0 material tree contains Brioche2:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-macaron-szcqkk-0 material tree contains Material #3023:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-half_quiche-murare-0-Tsubpart material tree contains Quiche:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    centerpiece-atfpvd-0 material tree contains tulips FLOWER 2603:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-half_sugar_cookie-jkrbjp-0-Tsubpart material tree contains Material #2927:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-cupcake-wdiezm-0 material tree contains Brioche2:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-chocolate_biscuit-fwnyas-0 material tree contains Material #2926:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-macaron-szcqkk-0 material tree contains Material #3008:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-half_quiche-gdvrux-0-Tsubpart material tree contains Quiche:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    centerpiece-atfpvd-0 material tree contains tulips 1203 20:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-half_sugar_cookie-jkrbjp-0-Tsubpart material tree contains Material #2926:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-half_chocolate_biscuit-owycyc-0-Tsubpart material tree contains Material #2926:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-chocolate_biscuit-fwnyas-0 material tree contains Material #2927:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-macaron-szcqkk-0 material tree contains Material #3021:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-half_macaroon-znmffr-0-Tsubpart material tree contains Material #3008:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-cheese_tart-rnsdha-0 material tree contains TarteCitron:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    centerpiece-atfpvd-0 material tree contains tulips FLOWER 2603 3:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-half_chocolate_biscuit-llipgq-0-Tsubpart material tree contains Material #2926:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-half_chocolate_biscuit-owycyc-0-Tsubpart material tree contains Material #2927:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-macaron-ggffjy-0 material tree contains MacaronChocolat:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-tartlet-zvoeav-0 material tree contains PistachioPie:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    centerpiece-atfpvd-0 material tree contains tulips FLOWER 1403 22:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-chocolate_biscuit-xhmpht-0 material tree contains Material #2926:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-half_chocolate_biscuit-llipgq-0-Tsubpart material tree contains Material #2927:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-macaron-ggffjy-0 material tree contains Material #3023:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-sourdough-znibqw-0 material tree contains BreadArtisan:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    centerpiece-atfpvd-0 material tree contains tulips 1203 33:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-chocolate_biscuit-xhmpht-0 material tree contains Material #2927:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-cupcake-rpadye-0 material tree contains PasteldeNata2:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-macaron-ggffjy-0 material tree contains Material #3008:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-baguette-pjzkeh-0 material tree contains BreadRye:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    centerpiece-atfpvd-0 material tree contains tulips 1203 30:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-cupcake-fabdnw-0 material tree contains PasteldeNata1:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-cupcake-rpadye-0 material tree contains PasteldeNata1:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-macaron-ggffjy-0 material tree contains Material #3021:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-half_baguette-mnhwnp-0-Tsubpart material tree contains BreadRye:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-spice_cookie-fwnyyy-0 material tree contains Material #2926:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-half_croissant-xryfwy-0-Tsubpart material tree contains Croissant:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-half_macaroon-gafeof-0-Tsubpart material tree contains MacaronChocolat:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-half_baguette-mdaqot-0-Tsubpart material tree contains BreadRye:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-spice_cookie-fwnyyy-0 material tree contains Material #2927:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-half_croissant-ytowpu-0-Tsubpart material tree contains Croissant:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-half_macaroon-gafeof-0-Tsubpart material tree contains Material #3023:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-sourdough-jgydml-0 material tree contains BunAlmond:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-half_spice_cookie-mtzrxb-0-Tsubpart material tree contains Material #2926:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-croissant-hnbnap-0 material tree contains Croissant:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-half_macaroon-gafeof-0-Tsubpart material tree contains Material #3008:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    clothesline_pole-smrqrw-0 material tree contains Material #8:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-half_spice_cookie-mtzrxb-0-Tsubpart material tree contains Material #2927:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-tartlet-ewlvxw-0 material tree contains BlueBerryPie:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-half_macaroon-gafeof-0-Tsubpart material tree contains Material #3021:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    clothesline_pole-smrqrw-1 material tree contains Material #8:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-half_spice_cookie-qrwsjv-0-Tsubpart material tree contains Material #2926:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    legging-izafif-0 material tree contains vray_leggings:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-half_macaroon-znmffr-0-Tsubpart material tree contains MacaronChocolat:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    centerpiece-atfpvd-0 material tree contains tulips FLOWER 2603 5:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-half_spice_cookie-qrwsjv-0-Tsubpart material tree contains Material #2927:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-half_macaroon-znmffr-0-Tsubpart material tree contains Material #3023:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    centerpiece-atfpvd-0 material tree contains tulips 2603 3:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-sugar_cookie-feirtr-0 material tree contains Material #2927:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-cupcake-fabdnw-0 material tree contains PasteldeNata2:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.


## objects/batch-10
  ### (.*) material tree contains .* with disallowed type .*. It should be one of .*.: 50
    L-suitcase-njalke-0-lid-base_link-R-upper material tree contains zipper:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-suitcase-oqiwhb-0-lid-base_link-R-upper material tree contains rotating_disc:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-suitcase-njalke-0-lid-base_link-R-upper material tree contains body:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-suitcase-oqiwhb-0-lid-base_link-R-upper material tree contains zipper:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-suitcase-njalke-0-lid-base_link-R-upper material tree contains lock_body:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-suitcase-oqiwhb-0-lid-base_link-R-upper material tree contains body:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-suitcase-njalke-0-lid-base_link-R-lower material tree contains rotating_disc:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-suitcase-oqiwhb-0-lid-base_link-R-lower material tree contains lock_body:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-french_press-zidmyo-0-press-base_link-P-lower material tree contains Material #240:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-suitcase-njalke-0-lid-base_link-R-lower material tree contains zipper:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-suitcase-oqiwhb-0-lid-base_link-R-lower material tree contains rotating_disc:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-french_press-zidmyo-0-base_link material tree contains Material #245:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-suitcase-njalke-0-lid-base_link-R-lower material tree contains body:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-suitcase-oqiwhb-0-lid-base_link-R-lower material tree contains zipper:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-french_press-zidmyo-0-base_link material tree contains matte-material:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-suitcase-njalke-0-lid-base_link-R-lower material tree contains lock_body:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-suitcase-oqiwhb-0-lid-base_link-R-lower material tree contains body:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-suitcase-oqiwhb-0-base_link material tree contains lock_body:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-french_press-zidmyo-0-base_link material tree contains SET-MATTE:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-suitcase-qthouj-0-lid-base_link-R-upper material tree contains zipper:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-french_press-zidmyo-0-glass-base_link-F-lower-Tglass material tree contains SET-MATTE:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-french_press-zidmyo-0-press-base_link-P-upper material tree contains Material #240:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-suitcase-oqiwhb-0-base_link material tree contains rotating_disc:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-suitcase-qthouj-0-lid-base_link-R-upper material tree contains body:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-french_press-zidmyo-0-glass-base_link-F-lower-Tglass material tree contains matte-material:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-seashell-iuoykx-0 material tree contains vray_wr_mat_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-suitcase-oqiwhb-0-base_link material tree contains zipper:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-suitcase-qthouj-0-lid-base_link-R-upper material tree contains lock_body:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-french_press-zidmyo-0-glass-base_link-F-lower-Tglass material tree contains Material #245:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-plug-mkfczr-0 material tree contains vray_PowerPlug6:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-suitcase-oqiwhb-0-base_link material tree contains body:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-suitcase-qthouj-0-lid-base_link-R-upper material tree contains rotating_disc:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-half_pork_rib-xnujmn-0-Tsubpart material tree contains Steak_Bone:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-suitcase-njalke-0-base_link material tree contains lock_body:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-suitcase-qthouj-0-base_link material tree contains zipper:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-suitcase-qthouj-0-lid-base_link-R-lower material tree contains zipper:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-half_pork_rib-pwbysl-0-Tsubpart material tree contains Steak_Bone:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-suitcase-njalke-0-base_link material tree contains body:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-suitcase-qthouj-0-base_link material tree contains body:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-suitcase-qthouj-0-lid-base_link-R-lower material tree contains body:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-pork_rib-tkqvnq-0 material tree contains Steak_Bone:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-suitcase-njalke-0-base_link material tree contains zipper:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-suitcase-qthouj-0-base_link material tree contains lock_body:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-suitcase-qthouj-0-lid-base_link-R-lower material tree contains lock_body:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-saxophone-kladff-0 material tree contains golld_blend_mat:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-suitcase-njalke-0-base_link material tree contains rotating_disc:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-suitcase-qthouj-0-base_link material tree contains rotating_disc:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-suitcase-qthouj-0-lid-base_link-R-lower material tree contains rotating_disc:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-suitcase-oqiwhb-0-lid-base_link-R-upper material tree contains lock_body:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-suitcase-njalke-0-lid-base_link-R-upper material tree contains rotating_disc:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.


## objects/batch-11
  ### (.*) material tree contains .* with disallowed type .*. It should be one of .*.: 12
    chicken_tender-waemzf-0 material tree contains chicken_01_mat:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-lid-zjtjrn-0-Tconnectedpart material tree contains Car_Paint_Metallic:VRayCarPaintMtl with disallowed type VRayCarPaintMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-ice_cream_cone-thwtmp-0 material tree contains IceCream:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    chicken_tender-mjimcu-0 material tree contains chicken_02_mat:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    half_chicken_tender-zzwgnh-0-Tsubpart material tree contains chicken_03_mat:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-roasting_pan-nrecsa-0 material tree contains Car_Paint_Metallic_Lid:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    baguette-xhqnuc-0 material tree contains nmodel_08_06_bread_mat:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    half_chicken_tender-cmfwiz-0-Tsubpart material tree contains chicken_03_mat:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-roasting_pan-nrecsa-0 material tree contains Car_Paint_Metallic:VRayCarPaintMtl with disallowed type VRayCarPaintMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    baguette-ypbyek-0 material tree contains nmodel_08_06_bread_part_mat:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    chicken_tender-eiwmqk-0 material tree contains chicken_03_mat:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-lid-zjtjrn-0-Tconnectedpart material tree contains Car_Paint_Metallic_Lid:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.


## objects/batch-12
  ### (.*) material tree contains .* with disallowed type .*. It should be one of .*.: 7
    L-lid-kbuqww-0-Tglass-Tconnectedpart material tree contains Plastic Clear 1:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-ceremonial_robe-stcrad-0 material tree contains Hoodie:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-hingeless_jar-gejwoi-0-base_link-Tglass material tree contains Plastic Clear 1:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-mannequin-nphsfp-0 material tree contains Mannequin:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-plate-wqgndf-0-Tglass material tree contains Plastic Clear 1:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-mannequin-nphsfp-0 material tree contains FastSSS_Sampler:VRayFastSSS with disallowed type VRayFastSSS. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-bottle_of_paint-volzrj-0 material tree contains GeneralUse_Card:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.


## objects/batch-13
  ### (.*) material tree contains .* with disallowed type .*. It should be one of .*.: 5
    skiff-coanpv-0 material tree contains vray_01 - Defaultвка:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-tank-bsdexp-0-lid-base_link-R-upper material tree contains Aquarium_V1_Light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-tank-bsdexp-0-base_link-Tglass material tree contains Aquarium_V1_Light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    skiff-coanpv-0 material tree contains vray_fishing_boat_01 - Defaultв:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-tank-bsdexp-0-lid-base_link-R-lower material tree contains Aquarium_V1_Light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.


## objects/legacy_batch-07
  ### (.*) material tree contains .* with disallowed type .*. It should be one of .*.: 1
    washer-mrgspe-0-base_link material tree contains Plastic_SSS_01_White #0:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.


## objects/procedural-aa
  ### (.*) material tree contains .* with disallowed type .*. It should be one of .*.: 28
    lime_juice_bottle-bnekjp-0-Tglass material tree contains AE27_653:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    cap-fomiem-2-Tconnectedpart material tree contains AE27_652:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    cap-fomiem-6-Tconnectedpart material tree contains AE27_652:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    cap-fomiem-2-Tconnectedpart material tree contains AE27_653:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    cap-fomiem-6-Tconnectedpart material tree contains AE27_653:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    lemon_juice_bottle-qsdqik-0-Tglass material tree contains AE27_652:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    vinegar_bottle-ykysuc-0-Tglass material tree contains AE27_652:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    lemon_juice_bottle-qsdqik-0-Tglass material tree contains AE27_653:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    vinegar_bottle-ykysuc-0-Tglass material tree contains AE27_653:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    cap-fomiem-1-Tconnectedpart material tree contains AE27_652:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    cap-fomiem-5-Tconnectedpart material tree contains AE27_652:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    cap-fomiem-1-Tconnectedpart material tree contains AE27_653:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    cap-fomiem-5-Tconnectedpart material tree contains AE27_653:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    cooking_oil_bottle-cfdond-0-Tglass material tree contains AE27_652:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    sesame_oil_bottle-bupgpj-0-Tglass material tree contains AE27_652:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    cooking_oil_bottle-cfdond-0-Tglass material tree contains AE27_653:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    sesame_oil_bottle-bupgpj-0-Tglass material tree contains AE27_653:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    cap-fomiem-0-Tconnectedpart material tree contains AE27_652:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    cap-fomiem-4-Tconnectedpart material tree contains AE27_652:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    cap-fomiem-0-Tconnectedpart material tree contains AE27_653:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    cap-fomiem-4-Tconnectedpart material tree contains AE27_653:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    olive_oil_bottle-zfvhus-0-Tglass material tree contains AE27_652:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    oil_bottle-yoxfyu-0-Tglass material tree contains AE27_652:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    oil_bottle-yoxfyu-0-Tglass material tree contains AE27_653:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    olive_oil_bottle-zfvhus-0-Tglass material tree contains AE27_653:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    cap-fomiem-3-Tconnectedpart material tree contains AE27_652:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    cap-fomiem-3-Tconnectedpart material tree contains AE27_653:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    lime_juice_bottle-bnekjp-0-Tglass material tree contains AE27_652:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.


## objects/school_objects-aa
  ### (.*) material tree contains .* with disallowed type .*. It should be one of .*.: 15
    security_camera-tiscsa-0 material tree contains light_on:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    picture-nbwrns-0-glass-base_link-F-lower-Tglass material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    locker-yhthgr-0-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-qebtel-0 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    picture-hxbrop-0-glass-base_link-F-lower-Tglass material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    locker-yhthgr-0-doortop-base_link-R-upper material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-hylkmj-0 material tree contains light_on:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    locker-gemgfz-0-bottomdoor-base_link-R-upper material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    picture-hvjwvj-0-glass-base_link-F-lower-Tglass material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    locker-yhthgr-0-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    locker-yhthgr-0-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    locker-gemgfz-0-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    door-ewqpfv-0-right_glass-right_door-F-lower-Tglass material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    door-xxipyh-0-glass-base_link-F-lower-Tglass material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    locker-yhthgr-0-doorbottom-base_link-R-upper material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.


## objects/task_relavant-ab
  ### (.*) material tree contains .* with disallowed type .*. It should be one of .*.: 835
    piano-qlcpph-0-key53-base_link-R-lower material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key61-base_link-R-lower material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key74-base_link-R-upper material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key82-base_link-R-upper material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key53-base_link-R-lower material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key61-base_link-R-lower material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key74-base_link-R-upper material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key81-base_link-R-upper material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key53-base_link-R-lower material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key61-base_link-R-lower material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key73-base_link-R-upper material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key81-base_link-R-upper material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key53-base_link-R-lower material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key60-base_link-R-lower material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key73-base_link-R-upper material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key81-base_link-R-upper material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key44-base_link-R-lower material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key60-base_link-R-lower material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key73-base_link-R-upper material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key81-base_link-R-upper material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key44-base_link-R-lower material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key60-base_link-R-lower material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key73-base_link-R-upper material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key80-base_link-R-upper material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key44-base_link-R-lower material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key60-base_link-R-lower material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key72-base_link-R-upper material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key80-base_link-R-upper material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key44-base_link-R-lower material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key59-base_link-R-lower material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key72-base_link-R-upper material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key80-base_link-R-upper material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key52-base_link-R-lower material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key59-base_link-R-lower material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key72-base_link-R-upper material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key80-base_link-R-upper material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key52-base_link-R-lower material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key59-base_link-R-lower material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key72-base_link-R-upper material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key79-base_link-R-upper material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key52-base_link-R-lower material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key59-base_link-R-lower material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key71-base_link-R-upper material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key79-base_link-R-upper material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key52-base_link-R-lower material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key58-base_link-R-lower material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key71-base_link-R-upper material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key79-base_link-R-upper material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key51-base_link-R-lower material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key58-base_link-R-lower material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key71-base_link-R-upper material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key79-base_link-R-upper material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key51-base_link-R-lower material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key58-base_link-R-lower material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key71-base_link-R-upper material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key78-base_link-R-upper material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key51-base_link-R-lower material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key58-base_link-R-lower material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key70-base_link-R-upper material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key78-base_link-R-upper material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key51-base_link-R-lower material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key57-base_link-R-lower material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key70-base_link-R-upper material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key78-base_link-R-upper material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key50-base_link-R-lower material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key57-base_link-R-lower material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key70-base_link-R-upper material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key78-base_link-R-upper material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key50-base_link-R-lower material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key57-base_link-R-lower material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key70-base_link-R-upper material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key77-base_link-R-upper material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key50-base_link-R-lower material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key57-base_link-R-lower material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key69-base_link-R-upper material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key77-base_link-R-upper material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key50-base_link-R-lower material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key56-base_link-R-lower material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key69-base_link-R-upper material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key77-base_link-R-upper material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key49-base_link-R-lower material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key56-base_link-R-lower material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key69-base_link-R-upper material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key77-base_link-R-upper material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key49-base_link-R-lower material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key56-base_link-R-lower material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key69-base_link-R-upper material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key76-base_link-R-upper material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key49-base_link-R-lower material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key56-base_link-R-lower material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key68-base_link-R-upper material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key76-base_link-R-upper material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key49-base_link-R-lower material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key55-base_link-R-lower material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key68-base_link-R-upper material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key76-base_link-R-upper material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key48-base_link-R-lower material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key55-base_link-R-lower material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key68-base_link-R-upper material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key76-base_link-R-upper material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key48-base_link-R-lower material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key55-base_link-R-lower material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key68-base_link-R-upper material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key75-base_link-R-upper material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key48-base_link-R-lower material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key55-base_link-R-lower material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key67-base_link-R-upper material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key75-base_link-R-upper material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key48-base_link-R-lower material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key54-base_link-R-lower material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key67-base_link-R-upper material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key75-base_link-R-upper material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key47-base_link-R-lower material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key54-base_link-R-lower material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key67-base_link-R-upper material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key75-base_link-R-upper material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key47-base_link-R-lower material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key54-base_link-R-lower material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key67-base_link-R-upper material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key74-base_link-R-upper material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key47-base_link-R-lower material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key54-base_link-R-lower material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key66-base_link-R-upper material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key74-base_link-R-upper material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key38-base_link-R-lower material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key47-base_link-R-lower material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key58-base_link-R-upper material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key66-base_link-R-upper material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key38-base_link-R-lower material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key46-base_link-R-lower material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key58-base_link-R-upper material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key66-base_link-R-upper material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key37-base_link-R-lower material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key46-base_link-R-lower material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key58-base_link-R-upper material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key66-base_link-R-upper material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key37-base_link-R-lower material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key46-base_link-R-lower material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key58-base_link-R-upper material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key65-base_link-R-upper material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key37-base_link-R-lower material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key46-base_link-R-lower material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key57-base_link-R-upper material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key65-base_link-R-upper material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key37-base_link-R-lower material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key45-base_link-R-lower material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key57-base_link-R-upper material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key65-base_link-R-upper material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key36-base_link-R-lower material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key45-base_link-R-lower material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key57-base_link-R-upper material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key65-base_link-R-upper material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key36-base_link-R-lower material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key45-base_link-R-lower material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key57-base_link-R-upper material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key64-base_link-R-upper material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key36-base_link-R-lower material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key45-base_link-R-lower material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key56-base_link-R-upper material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key64-base_link-R-upper material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key36-base_link-R-lower material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key43-base_link-R-lower material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key56-base_link-R-upper material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key64-base_link-R-upper material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key35-base_link-R-lower material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key43-base_link-R-lower material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key56-base_link-R-upper material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key64-base_link-R-upper material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key35-base_link-R-lower material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key43-base_link-R-lower material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key56-base_link-R-upper material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key63-base_link-R-upper material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key35-base_link-R-lower material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key43-base_link-R-lower material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key55-base_link-R-upper material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key63-base_link-R-upper material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key35-base_link-R-lower material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key42-base_link-R-lower material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key55-base_link-R-upper material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key63-base_link-R-upper material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key34-base_link-R-lower material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key42-base_link-R-lower material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key55-base_link-R-upper material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key63-base_link-R-upper material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key34-base_link-R-lower material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key42-base_link-R-lower material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key55-base_link-R-upper material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key62-base_link-R-upper material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key34-base_link-R-lower material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key42-base_link-R-lower material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key54-base_link-R-upper material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key62-base_link-R-upper material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key34-base_link-R-lower material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key41-base_link-R-lower material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key54-base_link-R-upper material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key62-base_link-R-upper material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key33-base_link-R-lower material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key41-base_link-R-lower material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key54-base_link-R-upper material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key62-base_link-R-upper material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key33-base_link-R-lower material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key41-base_link-R-lower material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key54-base_link-R-upper material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key61-base_link-R-upper material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key33-base_link-R-lower material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key41-base_link-R-lower material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key53-base_link-R-upper material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key61-base_link-R-upper material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key33-base_link-R-lower material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key40-base_link-R-lower material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key53-base_link-R-upper material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key61-base_link-R-upper material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key32-base_link-R-lower material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key40-base_link-R-lower material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key53-base_link-R-upper material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key61-base_link-R-upper material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key32-base_link-R-lower material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key40-base_link-R-lower material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key53-base_link-R-upper material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key60-base_link-R-upper material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key32-base_link-R-lower material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key40-base_link-R-lower material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key44-base_link-R-upper material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key60-base_link-R-upper material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key32-base_link-R-lower material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key39-base_link-R-lower material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key44-base_link-R-upper material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key60-base_link-R-upper material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key31-base_link-R-lower material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key39-base_link-R-lower material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key44-base_link-R-upper material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key60-base_link-R-upper material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key31-base_link-R-lower material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key39-base_link-R-lower material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key44-base_link-R-upper material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key59-base_link-R-upper material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key31-base_link-R-lower material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key39-base_link-R-lower material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key52-base_link-R-upper material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key59-base_link-R-upper material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key31-base_link-R-lower material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key38-base_link-R-lower material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key52-base_link-R-upper material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key59-base_link-R-upper material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key30-base_link-R-lower material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key38-base_link-R-lower material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key52-base_link-R-upper material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key59-base_link-R-upper material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key22-base_link-R-lower material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key30-base_link-R-lower material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key52-base_link-R-upper material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key22-base_link-R-lower material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key30-base_link-R-lower material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key51-base_link-R-upper material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key22-base_link-R-lower material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key30-base_link-R-lower material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key51-base_link-R-upper material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key22-base_link-R-lower material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key29-base_link-R-lower material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key51-base_link-R-upper material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key21-base_link-R-lower material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key29-base_link-R-lower material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key51-base_link-R-upper material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key21-base_link-R-lower material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key29-base_link-R-lower material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key50-base_link-R-upper material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key21-base_link-R-lower material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key29-base_link-R-lower material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key50-base_link-R-upper material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key21-base_link-R-lower material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key28-base_link-R-lower material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key50-base_link-R-upper material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key20-base_link-R-lower material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key28-base_link-R-lower material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key50-base_link-R-upper material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key20-base_link-R-lower material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key28-base_link-R-lower material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key49-base_link-R-upper material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key20-base_link-R-lower material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key28-base_link-R-lower material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key49-base_link-R-upper material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key20-base_link-R-lower material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key27-base_link-R-lower material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key49-base_link-R-upper material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key19-base_link-R-lower material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key27-base_link-R-lower material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key49-base_link-R-upper material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key19-base_link-R-lower material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key27-base_link-R-lower material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key48-base_link-R-upper material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key19-base_link-R-lower material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key27-base_link-R-lower material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key48-base_link-R-upper material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key19-base_link-R-lower material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key26-base_link-R-lower material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key48-base_link-R-upper material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key18-base_link-R-lower material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key26-base_link-R-lower material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key48-base_link-R-upper material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key18-base_link-R-lower material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key26-base_link-R-lower material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key47-base_link-R-upper material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key18-base_link-R-lower material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key26-base_link-R-lower material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key47-base_link-R-upper material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key18-base_link-R-lower material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key25-base_link-R-lower material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key47-base_link-R-upper material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-support-base_link-F-lower material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key17-base_link-R-lower material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key25-base_link-R-lower material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key47-base_link-R-upper material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-support-base_link-F-lower material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key17-base_link-R-lower material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key25-base_link-R-lower material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key46-base_link-R-upper material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-support-base_link-F-lower material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key17-base_link-R-lower material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key25-base_link-R-lower material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key46-base_link-R-upper material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-support-base_link-F-lower material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key17-base_link-R-lower material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key24-base_link-R-lower material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key46-base_link-R-upper material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-cap-base_link-F-lower material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key16-base_link-R-lower material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key24-base_link-R-lower material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key46-base_link-R-upper material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-cap-base_link-F-lower material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key16-base_link-R-lower material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key24-base_link-R-lower material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key45-base_link-R-upper material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-cap-base_link-F-lower material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key16-base_link-R-lower material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key24-base_link-R-lower material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key45-base_link-R-upper material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-cap-base_link-F-lower material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key16-base_link-R-lower material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key23-base_link-R-lower material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key45-base_link-R-upper material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-leg1-base_link-F-lower material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key15-base_link-R-lower material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key23-base_link-R-lower material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key45-base_link-R-upper material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-leg1-base_link-F-lower material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key15-base_link-R-lower material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key23-base_link-R-lower material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key43-base_link-R-upper material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-leg1-base_link-F-lower material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key15-base_link-R-lower material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key23-base_link-R-lower material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key43-base_link-R-upper material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-leg1-base_link-F-lower material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key15-base_link-R-lower material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key43-base_link-R-upper material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-leg2-base_link-F-lower material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key14-base_link-R-lower material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key43-base_link-R-upper material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-leg2-base_link-F-lower material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key14-base_link-R-lower material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key42-base_link-R-upper material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-leg2-base_link-F-lower material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key14-base_link-R-lower material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key42-base_link-R-upper material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-leg2-base_link-F-lower material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key14-base_link-R-lower material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key42-base_link-R-upper material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key88-base_link-R-upper material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key13-base_link-R-lower material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key42-base_link-R-upper material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key88-base_link-R-upper material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key13-base_link-R-lower material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key41-base_link-R-upper material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key88-base_link-R-upper material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key13-base_link-R-lower material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key41-base_link-R-upper material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key88-base_link-R-upper material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key13-base_link-R-lower material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key41-base_link-R-upper material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key87-base_link-R-upper material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key12-base_link-R-lower material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key41-base_link-R-upper material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key87-base_link-R-upper material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key12-base_link-R-lower material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key40-base_link-R-upper material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key87-base_link-R-upper material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key12-base_link-R-lower material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key40-base_link-R-upper material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key87-base_link-R-upper material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key12-base_link-R-lower material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key40-base_link-R-upper material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key86-base_link-R-upper material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key11-base_link-R-lower material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key40-base_link-R-upper material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key86-base_link-R-upper material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key11-base_link-R-lower material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key39-base_link-R-upper material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key86-base_link-R-upper material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key11-base_link-R-lower material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key39-base_link-R-upper material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key86-base_link-R-upper material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key11-base_link-R-lower material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key39-base_link-R-upper material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key85-base_link-R-upper material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key10-base_link-R-lower material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key39-base_link-R-upper material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key85-base_link-R-upper material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key10-base_link-R-lower material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key38-base_link-R-upper material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key85-base_link-R-upper material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key10-base_link-R-lower material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key38-base_link-R-upper material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key85-base_link-R-upper material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key10-base_link-R-lower material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key38-base_link-R-upper material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key84-base_link-R-upper material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key9-base_link-R-lower material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key38-base_link-R-upper material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key84-base_link-R-upper material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key9-base_link-R-lower material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key37-base_link-R-upper material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key84-base_link-R-upper material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key9-base_link-R-lower material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key37-base_link-R-upper material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key84-base_link-R-upper material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key9-base_link-R-lower material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key37-base_link-R-upper material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key83-base_link-R-upper material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key8-base_link-R-lower material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key37-base_link-R-upper material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key83-base_link-R-upper material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key8-base_link-R-lower material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key36-base_link-R-upper material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key83-base_link-R-upper material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key8-base_link-R-lower material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key36-base_link-R-upper material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key83-base_link-R-upper material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key8-base_link-R-lower material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key36-base_link-R-upper material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key82-base_link-R-upper material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key7-base_link-R-lower material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key36-base_link-R-upper material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key82-base_link-R-upper material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key7-base_link-R-lower material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key35-base_link-R-upper material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key82-base_link-R-upper material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    half_enchilada-cgrjfr-0-Tsubpart material tree contains Chicken_Enchilada_m:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    piano-qlcpph-0-key7-base_link-R-lower material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key27-base_link-R-upper material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key35-base_link-R-upper material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    half_enchilada-yyikja-0-Tsubpart material tree contains Chicken_Enchilada_m:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    piano-qlcpph-0-key7-base_link-R-lower material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key27-base_link-R-upper material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key35-base_link-R-upper material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    enchilada-dwvaig-0 material tree contains Chicken_Enchilada_m:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    piano-qlcpph-0-key6-base_link-R-lower material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key27-base_link-R-upper material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key35-base_link-R-upper material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    half_garlic_bread-vipzpb-0-Tsubpart material tree contains cgaxis_models_48_49_01:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    piano-qlcpph-0-key6-base_link-R-lower material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key27-base_link-R-upper material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key34-base_link-R-upper material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    half_garlic_bread-smddfo-0-Tsubpart material tree contains cgaxis_models_48_49_01:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    piano-qlcpph-0-key6-base_link-R-lower material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key26-base_link-R-upper material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key34-base_link-R-upper material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    garlic_bread-kjhxsq-0 material tree contains cgaxis_models_48_49_01:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    piano-qlcpph-0-key6-base_link-R-lower material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key26-base_link-R-upper material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key34-base_link-R-upper material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    cutting_board-olftkr-0 material tree contains cgaxis_models_48_49_03:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    piano-qlcpph-0-key5-base_link-R-lower material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key26-base_link-R-upper material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key34-base_link-R-upper material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    agave-bvlsig-0 material tree contains Aloe 2:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key5-base_link-R-lower material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key26-base_link-R-upper material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key33-base_link-R-upper material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    agave-bvlsig-0 material tree contains Aloe 1:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key5-base_link-R-lower material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key25-base_link-R-upper material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key33-base_link-R-upper material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    half_agave-qylfyr-0-Tsubpart material tree contains Aloe 2:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key5-base_link-R-lower material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key25-base_link-R-upper material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key33-base_link-R-upper material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    half_agave-qylfyr-0-Tsubpart material tree contains Aloe 1:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key4-base_link-R-lower material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key25-base_link-R-upper material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key33-base_link-R-upper material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    half_agave-npxccb-0-Tsubpart material tree contains Aloe 2:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key4-base_link-R-lower material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key25-base_link-R-upper material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key32-base_link-R-upper material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    half_agave-npxccb-0-Tsubpart material tree contains Aloe 1:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key4-base_link-R-lower material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key24-base_link-R-upper material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key32-base_link-R-upper material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    agave-cnlzlq-0 material tree contains Aloe 2:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key4-base_link-R-lower material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key24-base_link-R-upper material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key32-base_link-R-upper material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    agave-cnlzlq-0 material tree contains Aloe 1:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key3-base_link-R-lower material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key24-base_link-R-upper material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key32-base_link-R-upper material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    half_auricularia-ddjzqg-0-Tsubpart material tree contains WoodEarMushroom_mat:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    piano-qlcpph-0-key3-base_link-R-lower material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key24-base_link-R-upper material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key31-base_link-R-upper material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    half_auricularia-putkli-0-Tsubpart material tree contains WoodEarMushroom_mat:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    piano-qlcpph-0-key3-base_link-R-lower material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key23-base_link-R-upper material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key31-base_link-R-upper material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    auricularia-coecnm-0 material tree contains WoodEarMushroom_mat:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    piano-qlcpph-0-key3-base_link-R-lower material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key23-base_link-R-upper material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key31-base_link-R-upper material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    wine_bottle-vjdkci-0-Tglass material tree contains Bottle of Wine glass:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    piano-qlcpph-0-key2-base_link-R-lower material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key23-base_link-R-upper material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key31-base_link-R-upper material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    wine_bottle-hlzfxw-0-Tglass material tree contains Bottle of Wine glass:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    piano-qlcpph-0-key2-base_link-R-lower material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key23-base_link-R-upper material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key30-base_link-R-upper material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    wine_bottle-zuctnl-0-Tglass material tree contains Bottle of Wine glass:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    piano-qlcpph-0-key2-base_link-R-lower material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key22-base_link-R-upper material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key30-base_link-R-upper material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    cutting_board-wagwoy-0 material tree contains bread_wooden_board:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    piano-qlcpph-0-key2-base_link-R-lower material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key22-base_link-R-upper material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key30-base_link-R-upper material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bread_slice-pgzsxe-0 material tree contains bread_4:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    piano-qlcpph-0-key1-base_link-R-lower material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key22-base_link-R-upper material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key30-base_link-R-upper material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bread_slice-yremdf-0 material tree contains bread_3:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    piano-qlcpph-0-key1-base_link-R-lower material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key22-base_link-R-upper material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key29-base_link-R-upper material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    half_banana_bread-myxyfa-0-Tsubpart material tree contains bread_2:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    piano-qlcpph-0-key1-base_link-R-lower material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key21-base_link-R-upper material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key29-base_link-R-upper material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    half_banana_bread-sooaaz-0-Tsubpart material tree contains bread_2:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    piano-qlcpph-0-key1-base_link-R-lower material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key21-base_link-R-upper material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key29-base_link-R-upper material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    banana_bread-outrja-0 material tree contains bread_2:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    music_stool-qwqxew-0 material tree contains chair_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    piano-qlcpph-0-key21-base_link-R-upper material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key29-base_link-R-upper material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    carving_knife-ghhbgo-0 material tree contains bread_knife:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    piano-qlcpph-0-base_link material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key21-base_link-R-upper material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key28-base_link-R-upper material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    half_chanterelle-ftedsa-0-Tsubpart material tree contains Chanterelle:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    piano-qlcpph-0-base_link material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key20-base_link-R-upper material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key28-base_link-R-upper material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    half_chanterelle-bsfuew-0-Tsubpart material tree contains Chanterelle:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    piano-qlcpph-0-base_link material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key20-base_link-R-upper material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key28-base_link-R-upper material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    chanterelle-rwntzn-0 material tree contains Chanterelle:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    piano-qlcpph-0-base_link material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key20-base_link-R-upper material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key28-base_link-R-upper material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bap-enbbud-0 material tree contains Bun:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    half_bean_curd-abzrit-0-Tsubpart material tree contains Tofu_1_mat:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    piano-qlcpph-0-key12-base_link-R-upper material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key20-base_link-R-upper material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    half_ramen-mplogd-0-Tsubpart material tree contains noodles:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    half_bean_curd-mdqpoj-0-Tsubpart material tree contains Tofu_1_mat:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    piano-qlcpph-0-key12-base_link-R-upper material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key19-base_link-R-upper material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    half_ramen-argnww-0-Tsubpart material tree contains noodles:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    bean_curd-hekigc-0 material tree contains Tofu_1_mat:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    piano-qlcpph-0-key11-base_link-R-upper material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key19-base_link-R-upper material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    ramen-ofnvor-0 material tree contains noodles:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    half_venison-ixnagn-0-Tsubpart material tree contains Beef Meat:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    piano-qlcpph-0-key11-base_link-R-upper material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key19-base_link-R-upper material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    cap-xsxeij-0-Tconnectedpart material tree contains tetra_gemina_aseptic_white:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    half_venison-mszwzw-0-Tsubpart material tree contains Beef Meat:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    piano-qlcpph-0-key11-base_link-R-upper material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key19-base_link-R-upper material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    liquid_carton-rsvypp-0 material tree contains tetra_gemina_aseptic_white:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    venison-dhhbvg-0 material tree contains Beef Meat:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    piano-qlcpph-0-key11-base_link-R-upper material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key18-base_link-R-upper material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    squeeze_bottle-snyiat-0 material tree contains yellow:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    steak-jtgckm-0 material tree contains Beef Meat:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    piano-qlcpph-0-key10-base_link-R-upper material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key18-base_link-R-upper material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    squeeze_bottle-ikiwps-0 material tree contains White:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    half_tenderloin-zlimsu-0-Tsubpart material tree contains meatSSShit:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    piano-qlcpph-0-key10-base_link-R-upper material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key18-base_link-R-upper material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    squeeze_bottle-cuvgtq-0 material tree contains RED:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    half_tenderloin-hsxfzx-0-Tsubpart material tree contains meatSSShit:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    piano-qlcpph-0-key10-base_link-R-upper material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key18-base_link-R-upper material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    scoop_of_ice_cream-vdtzvz-0 material tree contains ice_cream_Mat:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    tenderloin-gbyqap-0 material tree contains meatSSShit:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    piano-qlcpph-0-key10-base_link-R-upper material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key17-base_link-R-upper material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    scoop_of_ice_cream-zenrml-0 material tree contains ice_cream_Mat:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    half_pita-fmkiep-0-Tsubpart material tree contains Material #30:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    piano-qlcpph-0-key9-base_link-R-upper material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key17-base_link-R-upper material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    scoop_of_ice_cream-dodndj-0 material tree contains ice_cream_Mat:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    half_pita-mhwlwr-0-Tsubpart material tree contains Material #30:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    piano-qlcpph-0-key9-base_link-R-upper material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key17-base_link-R-upper material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    scoop_of_ice_cream-trpctm-0 material tree contains ice_cream_Mat:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    pita-igejpf-0 material tree contains Material #30:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    piano-qlcpph-0-key9-base_link-R-upper material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key17-base_link-R-upper material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    scoop_of_ice_cream-kmbwth-0 material tree contains ice_cream_Mat:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    half_bleu-gpzeaz-0-Tsubpart material tree contains cheese_2:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    piano-qlcpph-0-key9-base_link-R-upper material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key16-base_link-R-upper material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    cookie_stick-zlhayf-0 material tree contains ice_cream_Mat:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    half_bleu-nelbjr-0-Tsubpart material tree contains cheese_2:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    piano-qlcpph-0-key8-base_link-R-upper material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key16-base_link-R-upper material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bleu-wxahdr-0 material tree contains cheese_2:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    squeeze_bottle-exfkfw-0 material tree contains Bottle_white_03:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key8-base_link-R-upper material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key16-base_link-R-upper material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    squeeze_bottle-exfkfw-0 material tree contains Other_white_03:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    half_mozzarella-yflmjs-0-Tsubpart material tree contains cheese_6:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    piano-qlcpph-0-key8-base_link-R-upper material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key16-base_link-R-upper material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    squeeze_bottle-exfkfw-0 material tree contains Label:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    half_mozzarella-luofri-0-Tsubpart material tree contains cheese_6:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    piano-qlcpph-0-key8-base_link-R-upper material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key15-base_link-R-upper material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    pottable_chili-lpshnm-0 material tree contains Material #108:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    mozzarella-rsiccz-0 material tree contains cheese_6:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    piano-qlcpph-0-key7-base_link-R-upper material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key15-base_link-R-upper material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    pottable_chili-lpshnm-0 material tree contains Material #110:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    necklace-ythgnf-0 material tree contains Yellow_Gold_14k:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key7-base_link-R-upper material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key15-base_link-R-upper material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    pottable_chili-lpshnm-0 material tree contains Material #111:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    necklace-ythgnf-0 material tree contains Yellow_Gold_14k_Mark:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key7-base_link-R-upper material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key15-base_link-R-upper material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    plant_pot-skbcqq-0 material tree contains Material #112:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    lint_screen-wnbcnn-0 material tree contains Mann_Filter_Paper_001:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    piano-qlcpph-0-key7-base_link-R-upper material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key14-base_link-R-upper material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    pottable_chili-pwalol-0 material tree contains Material #83:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    lemon_peel-mrvvpr-0-Tsubpart material tree contains mat_3:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    piano-qlcpph-0-key6-base_link-R-upper material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key14-base_link-R-upper material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    pottable_chili-pwalol-0 material tree contains Material #75:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    lemon_peel-sdvrmi-0-Tsubpart material tree contains mat_3:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    piano-qlcpph-0-key6-base_link-R-upper material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key14-base_link-R-upper material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    pottable_chili-pwalol-0 material tree contains Material #26:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    lemon_peel-qirncy-0 material tree contains mat_3:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    piano-qlcpph-0-key6-base_link-R-upper material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key14-base_link-R-upper material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    necktie-llnxru-0 material tree contains seam:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    half_zucchini-tncuen-0 material tree contains HalfZucchini01_mat:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    piano-qlcpph-0-key6-base_link-R-upper material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key13-base_link-R-upper material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    necktie-llnxru-0 material tree contains tie:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    cheddar-ewvbng-0 material tree contains cheese_Mat:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    piano-qlcpph-0-key5-base_link-R-upper material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key13-base_link-R-upper material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    necktie-llnxru-0 material tree contains label:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    half_cheddar-cortmw-0-Tsubpart material tree contains cheese_Mat:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    piano-qlcpph-0-key5-base_link-R-upper material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key13-base_link-R-upper material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    infant_car_seat-nulroi-0 material tree contains blks:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    half_cheddar-zkwspo-0-Tsubpart material tree contains cheese_Mat:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    piano-qlcpph-0-key5-base_link-R-upper material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key13-base_link-R-upper material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    infant_car_seat-nulroi-0 material tree contains steel:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    cheddar-cheqcd-0 material tree contains cheese_Mat:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    piano-qlcpph-0-key5-base_link-R-upper material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key12-base_link-R-upper material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    infant_car_seat-nulroi-0 material tree contains lthr.004:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bap-pabfde-0 material tree contains Bun:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    piano-qlcpph-0-key4-base_link-R-upper material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key12-base_link-R-upper material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    infant_car_seat-nulroi-0 material tree contains lthr.003:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key84-base_link-R-lower material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key4-base_link-R-upper material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    infant_car_seat-nulroi-0 material tree contains reds:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key84-base_link-R-lower material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key4-base_link-R-upper material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    infant_car_seat-nulroi-0 material tree contains lthr.002:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key84-base_link-R-lower material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key4-base_link-R-upper material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    infant_car_seat-nulroi-0 material tree contains lthr.001:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key84-base_link-R-lower material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key3-base_link-R-upper material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    copper_pot-gqemcq-0 material tree contains coffee_pot_copper:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    piano-qlcpph-0-key83-base_link-R-lower material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key3-base_link-R-upper material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    conch-dovqcj-0 material tree contains VR_Shell_01:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    piano-qlcpph-0-key83-base_link-R-lower material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key3-base_link-R-upper material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    cap-sjsles-0-Tconnectedpart material tree contains Ribbed_Cap_Material:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    piano-qlcpph-0-key83-base_link-R-lower material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key3-base_link-R-upper material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    liquid_carton-ykfkyq-0 material tree contains Body_Material:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    piano-qlcpph-0-key83-base_link-R-lower material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key2-base_link-R-upper material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    pottable_beefsteak_tomato-rrjizb-0 material tree contains Material #36:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key82-base_link-R-lower material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key2-base_link-R-upper material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    pottable_beefsteak_tomato-rrjizb-0 material tree contains Material #36:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key82-base_link-R-lower material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key2-base_link-R-upper material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    pottable_beefsteak_tomato-rrjizb-0 material tree contains Material #98:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key82-base_link-R-lower material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key2-base_link-R-upper material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    pottable_beefsteak_tomato-rrjizb-0 material tree contains Material #97:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key82-base_link-R-lower material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key1-base_link-R-upper material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    pottable_beefsteak_tomato-rrjizb-0 material tree contains Material #36ggg:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key81-base_link-R-lower material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key1-base_link-R-upper material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    pottable_beefsteak_tomato-rrjizb-0 material tree contains Material #36gg:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key81-base_link-R-lower material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key1-base_link-R-upper material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    pottable_beefsteak_tomato-rrjizb-0 material tree contains Material #36g:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key81-base_link-R-lower material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key1-base_link-R-upper material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    pottable_beefsteak_tomato-rrjizb-0 material tree contains Material #99:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key81-base_link-R-lower material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key88-base_link-R-lower material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    plant_pot-ygrtaz-0 material tree contains pot:Standard_Surface with disallowed type ai_standard_surface. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    piano-qlcpph-0-key80-base_link-R-lower material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key88-base_link-R-lower material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key80-base_link-R-lower material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key88-base_link-R-lower material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key80-base_link-R-lower material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key88-base_link-R-lower material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key80-base_link-R-lower material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key87-base_link-R-lower material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key79-base_link-R-lower material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key87-base_link-R-lower material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key79-base_link-R-lower material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key87-base_link-R-lower material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key79-base_link-R-lower material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key87-base_link-R-lower material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key79-base_link-R-lower material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key86-base_link-R-lower material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key78-base_link-R-lower material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key86-base_link-R-lower material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key78-base_link-R-lower material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key86-base_link-R-lower material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key78-base_link-R-lower material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key86-base_link-R-lower material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key78-base_link-R-lower material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key85-base_link-R-lower material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key77-base_link-R-lower material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key85-base_link-R-lower material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key77-base_link-R-lower material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key85-base_link-R-lower material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key77-base_link-R-lower material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key85-base_link-R-lower material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key69-base_link-R-lower material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key77-base_link-R-lower material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key69-base_link-R-lower material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key76-base_link-R-lower material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key68-base_link-R-lower material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key76-base_link-R-lower material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key68-base_link-R-lower material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key76-base_link-R-lower material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key68-base_link-R-lower material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key76-base_link-R-lower material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key68-base_link-R-lower material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key75-base_link-R-lower material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key67-base_link-R-lower material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key75-base_link-R-lower material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key67-base_link-R-lower material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key75-base_link-R-lower material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key67-base_link-R-lower material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key75-base_link-R-lower material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key67-base_link-R-lower material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key74-base_link-R-lower material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key66-base_link-R-lower material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key74-base_link-R-lower material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key66-base_link-R-lower material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key74-base_link-R-lower material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key66-base_link-R-lower material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key74-base_link-R-lower material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key66-base_link-R-lower material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key73-base_link-R-lower material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key65-base_link-R-lower material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key73-base_link-R-lower material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key65-base_link-R-lower material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key73-base_link-R-lower material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key65-base_link-R-lower material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key73-base_link-R-lower material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key65-base_link-R-lower material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key72-base_link-R-lower material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key64-base_link-R-lower material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key72-base_link-R-lower material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key64-base_link-R-lower material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key72-base_link-R-lower material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key64-base_link-R-lower material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key72-base_link-R-lower material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key64-base_link-R-lower material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key71-base_link-R-lower material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key63-base_link-R-lower material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key71-base_link-R-lower material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key63-base_link-R-lower material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key71-base_link-R-lower material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key63-base_link-R-lower material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key71-base_link-R-lower material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key63-base_link-R-lower material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key70-base_link-R-lower material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key62-base_link-R-lower material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key70-base_link-R-lower material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key62-base_link-R-lower material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key70-base_link-R-lower material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key62-base_link-R-lower material tree contains piano_wood_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key70-base_link-R-lower material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key62-base_link-R-lower material tree contains piano_body_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key69-base_link-R-lower material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key61-base_link-R-lower material tree contains piano_velvet_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    piano-qlcpph-0-key69-base_link-R-lower material tree contains piano_metal_mt:PBRMetalRough with disallowed type PBRMetalRough. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.


## objects/task_relevant-xy
  ### (.*) material tree contains .* with disallowed type .*. It should be one of .*.: 72
    broken_light_bulb-cugtye-0-base_link material tree contains ON:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    dog_food_can-vxxcvg-0 material tree contains dog_food_tin_material:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    patty-ejbije-0 material tree contains Glow_mat:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    salad-nffvob-0 material tree contains Celery:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    half_lime-jnbwpy-0 material tree contains LimeHalf_mat:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    belt-gqwddb-0 material tree contains 01 - Default:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    lid-tzbbeo-0-Tconnectedpart material tree contains cat_food_tin_material:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    patty-ejbije-0 material tree contains RawBurgerPatty_mat:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    salad-nffvob-0 material tree contains vray_Candle wax:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    half_lime-jnbwpy-0 material tree contains Glow_mat:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    belt-gqwddb-0 material tree contains 03 - Default:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    cat_food_tin-rclizj-0 material tree contains cat_food_tin_material:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    lampshade-dizlbd-0 material tree contains White frame:Autodesk_Ceramic with disallowed type Autodesk_Material. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    salad-nffvob-0 material tree contains Material #11:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    skateboard_wheel-lzdgcs-0 material tree contains car paint black4:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    can_of_dog_food-rgwfxq-0 material tree contains dog_food_tin_material:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    lampshade-drlveg-0 material tree contains White frame:Autodesk_Ceramic with disallowed type Autodesk_Material. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    salad-nffvob-0 material tree contains Candle wax:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    skateboard_wheel-lzdgcs-0 material tree contains car paint black2:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    can_of_cat_food-omiuox-0 material tree contains cat_food_tin_material:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    lampshade-zadmdc-0 material tree contains White frame:Autodesk_Ceramic with disallowed type Autodesk_Material. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    salad-nffvob-0 material tree contains Candle wax:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    skateboard_wheel-lzdgcs-0 material tree contains car paint black3:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    broken_light_bulb-cugtye-0-glass-base_link-F-lower-Tglass material tree contains basis_head_grip:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    lampshade-crzpcw-0 material tree contains White frame:Autodesk_Ceramic with disallowed type Autodesk_Material. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    pottable_marigold-clbgcs-0 material tree contains Marigolds_Bud_Yellow:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    skateboard_wheel-bbkthh-0 material tree contains car paint black5:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    broken_light_bulb-cugtye-0-glass-base_link-F-lower-Tglass material tree contains coil_light_bulb_OFF:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    hutch-vsqsuy-0-door1-base_link-R-upper material tree contains Metal:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    pottable_marigold-clbgcs-0 material tree contains Marigolds_Bud_Orange_02:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    skateboard_wheel-bbkthh-0 material tree contains Material #1804124019:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    broken_light_bulb-cugtye-0-glass-base_link-F-lower-Tglass material tree contains OFF:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    hutch-vsqsuy-0-door0-base_link-R-upper material tree contains Metal:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    pottable_marigold-clbgcs-0 material tree contains Marigolds_Bud_Orange_01:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    skateboard_wheel-bbkthh-0 material tree contains car paint black3:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    broken_light_bulb-cugtye-0-glass-base_link-F-lower-Tglass material tree contains Glass_used_dirty:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    hutch-vsqsuy-0-door0-base_link-R-lower material tree contains Metal:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    pottable_marigold-clbgcs-0 material tree contains Marigolds_Bud_Orange_03:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    cup_of_ranch-wtlavb-0 material tree contains Tartar:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    broken_light_bulb-cugtye-0-glass-base_link-F-lower-Tglass material tree contains coil_light_bulb_ON:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    hutch-vsqsuy-0-door1-base_link-R-lower material tree contains Metal:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    pottable_marigold-clbgcs-0 material tree contains Marigolds_Leaves:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    salad-nffvob-0 material tree contains vray_Apple:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    hitch-eqwskw-0-base_link material tree contains 2:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    broken_light_bulb-cugtye-0-glass-base_link-F-lower-Tglass material tree contains ON:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    tray-yqtlhy-0-base_link-Tconnectedpart material tree contains Metal:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    vase-toreid-0 material tree contains Marigolds_Bud_Yellow:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    salad-nffvob-0 material tree contains vray_Tomato:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    half_potato-qktjci-0-Tsubpart material tree contains Potato_mat:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    broken_light_bulb-cugtye-0-base_link material tree contains basis_head_grip:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    hutch-vsqsuy-0-base_link material tree contains Metal:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    vase-toreid-0 material tree contains Marigolds_Bud_Orange_02:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    salad-nffvob-0 material tree contains Meat:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    half_potato-qktjci-0-Tsubpart material tree contains Glow_mat:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    broken_light_bulb-cugtye-0-base_link material tree contains coil_light_bulb_OFF:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    dog_collar-dnjpms-0 material tree contains Metal:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    vase-toreid-0 material tree contains Marigolds_Bud_Orange_01:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    salad-nffvob-0 material tree contains vray_Pimiento Verde:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    half_potato-qcnoim-0-Tsubpart material tree contains Potato_mat:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    broken_light_bulb-cugtye-0-base_link material tree contains OFF:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    dog_collar-dnjpms-0 material tree contains Material #269:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    vase-toreid-0 material tree contains Marigolds_Bud_Orange_03:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    salad-nffvob-0 material tree contains vray_Pimiento Verde:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    potato-lgupkq-0 material tree contains Glow_mat:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    half_potato-qcnoim-0-Tsubpart material tree contains Glow_mat:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    broken_light_bulb-cugtye-0-base_link material tree contains Glass_used_dirty:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    lid-swszeb-0-Tconnectedpart material tree contains dog_food_tin_material:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    pottable_cactus-fcqqsz-0 material tree contains nmodel_02_50_kaktus_mat:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    vase-toreid-0 material tree contains Marigolds_Leaves:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    salad-nffvob-0 material tree contains vray_Pimiento Verde:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    potato-lgupkq-0 material tree contains Potato_mat:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    broken_light_bulb-cugtye-0-base_link material tree contains coil_light_bulb_ON:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.


## scenes/Beechwood_0_garden
  ### (.*) material tree contains .* with disallowed type .*. It should be one of .*.: 28
    L-pot_plant-ibvdwh-2 material tree contains AM212_005_pot_flower_05:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-pot_plant-ibvdwh-2 material tree contains AM212_005_flower_petalin:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-glass_lantern-engbrg-0-glass-base_link-F-lower-Tglass material tree contains Candle wax:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-pot_plant-ibvdwh-1 material tree contains AM212_005_flower_petalin:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    pot_plant-mkhbtk-0 material tree contains AM212_005_pot_flower_11:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-glass_lantern-zjuwnf-0-glass-base_link-F-lower-Tglass material tree contains iron:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-pot_plant-ibvdwh-1 material tree contains AM212_005_pot_flower_05:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    pot_plant-mkhbtk-0 material tree contains AM212_005_pot_flower_05:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-glass_lantern-zjuwnf-0-glass-base_link-F-lower-Tglass material tree contains Candle wax:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    pot_plant-mkhbtk-0 material tree contains AM212_005_pot_flower_17:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-glass_lantern-zjuwnf-0 material tree contains Candle wax:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-glass_lantern-zjuwnf-0 material tree contains iron:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    pot_plant-mkhbtk-0 material tree contains AM212_005_flower_petalin:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-coffee_table-mjyitq-0 material tree contains Teak natural:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    pot_plant-mkhbtk-0 material tree contains AM212_005_pot_flower_14:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-coffee_table-rkdwxe-0 material tree contains Teak natural:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-armchair-nibzys-0 material tree contains Teak natural:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-pot_plant-liqyem-0 material tree contains leaf 2:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-pot_plant-ibvdwh-0 material tree contains AM212_005_flower_petalin:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-pot_plant-ibvdwh-0 material tree contains AM212_005_pot_flower_05:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-pot_plant-liqyem-0 material tree contains leaf 1:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-pot_plant-liqyem-0 material tree contains leaf 3:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-pot_plant-liqyem-0 material tree contains iron tray:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-sofa-rqipbs-0 material tree contains Teak natural:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-coffee_table-rkdwxe-1 material tree contains Teak natural:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    C-glass_lantern-engbrg-0 material tree contains iron:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-glass_lantern-engbrg-0 material tree contains Candle wax:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-glass_lantern-engbrg-0-glass-base_link-F-lower-Tglass material tree contains iron:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.


## scenes/commercial_kitchen_fire_extinguisher
  ### (.*) material tree contains .* with disallowed type .*. It should be one of .*.: 218
    F-shelf-dvnwfa-0 material tree contains K_Oven_Flame:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-commercial_kitchen_table-bwlfnu-2 material tree contains K_Table_Button:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-hwozjd-0 material tree contains K_Lamp_2:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-electric_cauldron-qmdgct-0-lid-base_link-P-lower material tree contains K_Oven_Flame:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-electric_cauldron-qmdgct-0-base_link material tree contains K_Hood_Light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-commercial_kitchen_table-bwlfnu-2 material tree contains K_Table_Button_2:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-hwozjd-0 material tree contains K_White_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-electric_cauldron-qmdgct-0-lid-base_link-P-upper material tree contains K_Hood_Light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-electric_cauldron-qmdgct-0-base_link material tree contains K_Panel:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    commercial_kitchen_sink-nmxgbm-0-faucet-base_link-R-lower material tree contains K_Table_Button:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-hwozjd-0 material tree contains K_Blue_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-electric_cauldron-qmdgct-0-lid-base_link-P-upper material tree contains K_Panel:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-electric_cauldron-qmdgct-0-base_link material tree contains K_Oven_Flame:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    commercial_kitchen_sink-nmxgbm-0-faucet-base_link-R-lower material tree contains K_Table_Button_2:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-hwozjd-1 material tree contains K_Yellow_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-electric_cauldron-qmdgct-0-lid-base_link-P-upper material tree contains K_Oven_Flame:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    stove-xspxzg-0-door1-base_link-R-upper material tree contains K_Hood_Light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-commercial_kitchen_table-bwlfnu-3 material tree contains K_Table_Button:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-hwozjd-1 material tree contains K_Lamp_2:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    commercial_kitchen_sink-nmxgbm-0-faucet-base_link-R-upper material tree contains K_Table_Button:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    stove-xspxzg-0-door1-base_link-R-upper material tree contains K_Panel:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-commercial_kitchen_table-bwlfnu-3 material tree contains K_Table_Button_2:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-hwozjd-1 material tree contains K_White_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    commercial_kitchen_sink-nmxgbm-0-faucet-base_link-R-upper material tree contains K_Table_Button_2:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    stove-xspxzg-0-door1-base_link-R-upper material tree contains K_Oven_Flame:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    commercial_kitchen_sink-nmxgbm-0-base_link material tree contains K_Table_Button:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-hwozjd-1 material tree contains K_Blue_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    metal_bottom_cabinet-yfycfx-1-door-base_link-P-lower material tree contains K_Table_Button:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    stove-xspxzg-0-door1-base_link-R-lower material tree contains K_Hood_Light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    commercial_kitchen_sink-nmxgbm-0-base_link material tree contains K_Table_Button_2:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-hwozjd-2 material tree contains K_Yellow_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    metal_bottom_cabinet-yfycfx-1-door-base_link-P-lower material tree contains K_Table_Button_2:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    stove-xspxzg-0-door1-base_link-R-lower material tree contains K_Panel:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    metal_bottom_cabinet-jqyhjy-0-base_link material tree contains K_Table_Button:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-hwozjd-2 material tree contains K_Lamp_2:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    flat_top_grill-rdyuii-1 material tree contains K_Hood_Light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    stove-xspxzg-0-door1-base_link-R-lower material tree contains K_Oven_Flame:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    metal_bottom_cabinet-jqyhjy-0-base_link material tree contains K_Table_Button_2:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-hwozjd-2 material tree contains K_White_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    flat_top_grill-rdyuii-1 material tree contains K_Panel:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    stove-xspxzg-0-door2-base_link-R-lower material tree contains K_Hood_Light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-commercial_kitchen_table-bwlfnu-4 material tree contains K_Table_Button:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-hwozjd-2 material tree contains K_Blue_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    flat_top_grill-rdyuii-1 material tree contains K_Oven_Flame:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    stove-xspxzg-0-door2-base_link-R-lower material tree contains K_Panel:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-commercial_kitchen_table-bwlfnu-4 material tree contains K_Table_Button_2:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-hwozjd-3 material tree contains K_Yellow_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-vljyzl-0 material tree contains K_Environment:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    stove-xspxzg-0-door2-base_link-R-lower material tree contains K_Oven_Flame:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    metal_bottom_cabinet-yfycfx-0-door-base_link-P-upper material tree contains K_Table_Button:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-hwozjd-3 material tree contains K_Lamp_2:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-amolcz-0 material tree contains K_Environment:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    deep_fryer-jcmoji-0-base_link material tree contains K_Hood_Light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    metal_bottom_cabinet-yfycfx-0-door-base_link-P-upper material tree contains K_Table_Button_2:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-hwozjd-3 material tree contains K_White_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-vwttrj-0 material tree contains K_Environment:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    deep_fryer-jcmoji-0-base_link material tree contains K_Panel:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    metal_bottom_cabinet-yfycfx-0-base_link material tree contains K_Table_Button:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-hwozjd-3 material tree contains K_Blue_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-gcjrbz-0 material tree contains K_Environment:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    deep_fryer-jcmoji-0-base_link material tree contains K_Oven_Flame:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    metal_bottom_cabinet-yfycfx-0-base_link material tree contains K_Table_Button_2:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-hwozjd-4 material tree contains K_Yellow_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-plieoa-0 material tree contains K_Environment:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    deep_fryer-jcmoji-0-dish1-base_link-P-lower material tree contains K_Hood_Light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    metal_bottom_cabinet-yfycfx-1-base_link material tree contains K_Table_Button:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-hwozjd-4 material tree contains K_Lamp_2:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-gzymzl-0 material tree contains K_Environment:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    deep_fryer-jcmoji-0-dish1-base_link-P-lower material tree contains K_Panel:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    metal_bottom_cabinet-yfycfx-1-base_link material tree contains K_Table_Button_2:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-hwozjd-4 material tree contains K_White_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-bottom_cabinet-jhymlr-0-link_5-base_link-P-lower material tree contains K_Hood_Light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    deep_fryer-jcmoji-0-dish1-base_link-P-lower material tree contains K_Oven_Flame:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-commercial_kitchen_table-bwlfnu-5 material tree contains K_Table_Button:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-hwozjd-4 material tree contains K_Blue_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-bottom_cabinet-jhymlr-0-link_5-base_link-P-lower material tree contains K_Panel:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    deep_fryer-jcmoji-0-dish1-base_link-P-upper material tree contains K_Hood_Light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-commercial_kitchen_table-bwlfnu-5 material tree contains K_Table_Button_2:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-hwozjd-5 material tree contains K_Yellow_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-bottom_cabinet-jhymlr-0-link_5-base_link-P-lower material tree contains K_Oven_Flame:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    deep_fryer-jcmoji-0-dish1-base_link-P-upper material tree contains K_Panel:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    metal_bottom_cabinet-mvsgdm-0-door-base_link-R-lower material tree contains K_Table_Button:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-hwozjd-5 material tree contains K_Lamp_2:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    deep_fryer-jcmoji-0-dish1-base_link-P-upper material tree contains K_Oven_Flame:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    metal_bottom_cabinet-mvsgdm-0-door-base_link-R-lower material tree contains K_Table_Button_2:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-hwozjd-5 material tree contains K_White_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    deep_fryer-jcmoji-0-dish2-base_link-P-lower material tree contains K_Hood_Light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    metal_bottom_cabinet-mvsgdm-0-base_link material tree contains K_Table_Button:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-hwozjd-5 material tree contains K_Blue_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    deep_fryer-jcmoji-0-dish2-base_link-P-lower material tree contains K_Panel:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    metal_bottom_cabinet-mvsgdm-0-base_link material tree contains K_Table_Button_2:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-hwozjd-6 material tree contains K_Yellow_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    deep_fryer-jcmoji-0-dish2-base_link-P-lower material tree contains K_Oven_Flame:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    metal_bottom_cabinet-mvsgdm-0-door-base_link-R-upper material tree contains K_Table_Button:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-hwozjd-6 material tree contains K_Lamp_2:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    deep_fryer-jcmoji-0-dish2-base_link-P-upper material tree contains K_Hood_Light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    metal_bottom_cabinet-mvsgdm-0-door-base_link-R-upper material tree contains K_Table_Button_2:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-hwozjd-6 material tree contains K_White_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    deep_fryer-jcmoji-0-dish2-base_link-P-upper material tree contains K_Panel:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    commercial_kitchen_sink-ollneo-0 material tree contains K_Table_Button:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-hwozjd-6 material tree contains K_Blue_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    deep_fryer-jcmoji-0-dish2-base_link-P-upper material tree contains K_Oven_Flame:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    commercial_kitchen_sink-ollneo-0 material tree contains K_Table_Button_2:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-hwozjd-7 material tree contains K_Yellow_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    deep_fryer-jcmoji-0-dish3-base_link-P-lower material tree contains K_Hood_Light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    commercial_kitchen_sink-ollneo-0-door-base_link-R-lower material tree contains K_Table_Button:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-hwozjd-7 material tree contains K_Lamp_2:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    deep_fryer-jcmoji-0-dish3-base_link-P-lower material tree contains K_Panel:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    commercial_kitchen_sink-ollneo-0-door-base_link-R-lower material tree contains K_Table_Button_2:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-hwozjd-7 material tree contains K_White_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    deep_fryer-jcmoji-0-dish3-base_link-P-lower material tree contains K_Oven_Flame:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    metal_bottom_cabinet-yfycfx-0-door-base_link-P-lower material tree contains K_Table_Button:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-hwozjd-7 material tree contains K_Blue_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    deep_fryer-jcmoji-0-dish3-base_link-P-upper material tree contains K_Hood_Light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    metal_bottom_cabinet-yfycfx-0-door-base_link-P-lower material tree contains K_Table_Button_2:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fridge-vwylob-0-base_link material tree contains K_Fridge_Lamp_Red:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    deep_fryer-jcmoji-0-dish3-base_link-P-upper material tree contains K_Panel:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    metal_bottom_cabinet-melxmd-0-base_link material tree contains K_Table_Button:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fridge-vwylob-0-base_link material tree contains K_Fridge_Lamp_Green:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    deep_fryer-jcmoji-0-dish3-base_link-P-upper material tree contains K_Oven_Flame:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    metal_bottom_cabinet-melxmd-0-base_link material tree contains K_Table_Button_2:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fridge-vwylob-0-base_link material tree contains K_Fridge_Light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    deep_fryer-jcmoji-0-dish4-base_link-P-lower material tree contains K_Hood_Light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    metal_bottom_cabinet-melxmd-0-door-base_link-P-upper material tree contains K_Table_Button:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fridge-vwylob-0-door-base_link-R-lower material tree contains K_Fridge_Lamp_Red:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    deep_fryer-jcmoji-0-dish4-base_link-P-lower material tree contains K_Panel:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    metal_bottom_cabinet-melxmd-0-door-base_link-P-upper material tree contains K_Table_Button_2:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fridge-vwylob-0-door-base_link-R-lower material tree contains K_Fridge_Lamp_Green:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    deep_fryer-jcmoji-0-dish4-base_link-P-lower material tree contains K_Oven_Flame:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    metal_bottom_cabinet-melxmd-0-door-base_link-P-lower material tree contains K_Table_Button:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fridge-vwylob-0-door-base_link-R-lower material tree contains K_Fridge_Light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    deep_fryer-jcmoji-0-dish4-base_link-P-upper material tree contains K_Hood_Light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    metal_bottom_cabinet-melxmd-0-door-base_link-P-lower material tree contains K_Table_Button_2:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fridge-vwylob-0-door-base_link-R-upper material tree contains K_Fridge_Lamp_Red:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    deep_fryer-jcmoji-0-dish4-base_link-P-upper material tree contains K_Panel:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    flat_top_grill-rdyuii-0 material tree contains K_Hood_Light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fridge-vwylob-0-door-base_link-R-upper material tree contains K_Fridge_Lamp_Green:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    deep_fryer-jcmoji-0-dish4-base_link-P-upper material tree contains K_Oven_Flame:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    flat_top_grill-rdyuii-0 material tree contains K_Panel:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fridge-vwylob-0-door-base_link-R-upper material tree contains K_Fridge_Light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    deep_fryer-jcmoji-0-dish5-base_link-P-lower material tree contains K_Hood_Light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    flat_top_grill-rdyuii-0 material tree contains K_Oven_Flame:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-cauldron-lbcxwi-0-lid-base_link-P-lower material tree contains K_SK_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    deep_fryer-jcmoji-0-dish5-base_link-P-lower material tree contains K_Panel:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    commercial_kitchen_sink-nmxgbm-0-door-base_link-P-lower material tree contains K_Table_Button:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-cauldron-lbcxwi-0-lid-base_link-P-lower material tree contains K_SK_Lamp2:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    deep_fryer-jcmoji-0-dish5-base_link-P-lower material tree contains K_Oven_Flame:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    commercial_kitchen_sink-nmxgbm-0-door-base_link-P-lower material tree contains K_Table_Button_2:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-L-dish_rack_cart-rniujn-0 material tree contains K_EQ_Button:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    deep_fryer-jcmoji-0-dish5-base_link-P-upper material tree contains K_Hood_Light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    commercial_kitchen_sink-nmxgbm-0-door-base_link-P-upper material tree contains K_Table_Button:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-L-dish_rack_cart-rniujn-0 material tree contains K_EQ_Button_2:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    deep_fryer-jcmoji-0-dish5-base_link-P-upper material tree contains K_Panel:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    commercial_kitchen_sink-nmxgbm-0-door-base_link-P-upper material tree contains K_Table_Button_2:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    oven-cgtaer-0-door-base_link-R-lower material tree contains K_Hood_Light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    deep_fryer-jcmoji-0-dish5-base_link-P-upper material tree contains K_Oven_Flame:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    commercial_kitchen_sink-ollneo-0-door-base_link-R-upper material tree contains K_Table_Button:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    oven-cgtaer-0-door-base_link-R-lower material tree contains K_Panel:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    deep_fryer-jcmoji-0-dish6-base_link-P-lower material tree contains K_Hood_Light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    commercial_kitchen_sink-ollneo-0-door-base_link-R-upper material tree contains K_Table_Button_2:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    oven-cgtaer-0-door-base_link-R-lower material tree contains K_Oven_Flame:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    deep_fryer-jcmoji-0-dish6-base_link-P-lower material tree contains K_Panel:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    metal_bottom_cabinet-jqyhjy-0-door-base_link-P-lower material tree contains K_Table_Button:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    oven-cgtaer-0-base_link material tree contains K_Hood_Light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    deep_fryer-jcmoji-0-dish6-base_link-P-lower material tree contains K_Oven_Flame:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    metal_bottom_cabinet-jqyhjy-0-door-base_link-P-lower material tree contains K_Table_Button_2:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    oven-cgtaer-0-base_link material tree contains K_Panel:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    deep_fryer-jcmoji-0-dish6-base_link-P-upper material tree contains K_Hood_Light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    metal_bottom_cabinet-jqyhjy-0-door-base_link-P-upper material tree contains K_Table_Button:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    oven-cgtaer-0-base_link material tree contains K_Oven_Flame:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-cauldron-lbcxwi-0 material tree contains K_SK_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    deep_fryer-jcmoji-0-dish6-base_link-P-upper material tree contains K_Panel:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    metal_bottom_cabinet-jqyhjy-0-door-base_link-P-upper material tree contains K_Table_Button_2:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    oven-cgtaer-0-door-base_link-R-upper material tree contains K_Hood_Light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-cauldron-lbcxwi-0 material tree contains K_SK_Lamp2:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    deep_fryer-jcmoji-0-dish6-base_link-P-upper material tree contains K_Oven_Flame:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    metal_bottom_cabinet-bbjuqu-0-door-base_link-P-lower material tree contains K_Table_Button:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    oven-cgtaer-0-door-base_link-R-upper material tree contains K_Panel:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-serving_cart-revrxw-0 material tree contains K_EQ_Button:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    flat_top_grill-wlzvfj-0 material tree contains K_Hood_Light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    metal_bottom_cabinet-bbjuqu-0-door-base_link-P-lower material tree contains K_Table_Button_2:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    oven-cgtaer-0-door-base_link-R-upper material tree contains K_Oven_Flame:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-serving_cart-revrxw-0 material tree contains K_EQ_Button_2:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    flat_top_grill-wlzvfj-0 material tree contains K_Panel:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    metal_bottom_cabinet-bbjuqu-0-door-base_link-P-upper material tree contains K_Table_Button:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    metal_bottom_cabinet-bbjuqu-0-base_link material tree contains K_Table_Button:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    range_hood-xnokqb-0 material tree contains K_Hood_Light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    flat_top_grill-wlzvfj-0 material tree contains K_Oven_Flame:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    metal_bottom_cabinet-bbjuqu-0-door-base_link-P-upper material tree contains K_Table_Button_2:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    metal_bottom_cabinet-bbjuqu-0-base_link material tree contains K_Table_Button_2:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    range_hood-xnokqb-0 material tree contains K_Panel:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    grill-ncdzkj-0 material tree contains K_Hood_Light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    dishwasher-xvchpe-0-top-base_link-P-upper material tree contains K_EQ_Button:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-cauldron-lbcxwi-0-lid-base_link-P-upper material tree contains K_SK_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    range_hood-xnokqb-0 material tree contains K_Oven_Flame:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    grill-ncdzkj-0 material tree contains K_Panel:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    dishwasher-xvchpe-0-top-base_link-P-upper material tree contains K_EQ_Button_2:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-cauldron-lbcxwi-0-lid-base_link-P-upper material tree contains K_SK_Lamp2:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    stove-xspxzg-0-base_link material tree contains K_Hood_Light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    grill-ncdzkj-0 material tree contains K_Oven_Flame:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    dishwasher-xvchpe-0-base_link material tree contains K_EQ_Button:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    stove-xspxzg-0-door2-base_link-R-upper material tree contains K_Hood_Light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    stove-xspxzg-0-base_link material tree contains K_Panel:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-commercial_kitchen_table-bwlfnu-0 material tree contains K_Table_Button:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    dishwasher-xvchpe-0-base_link material tree contains K_EQ_Button_2:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    stove-xspxzg-0-door2-base_link-R-upper material tree contains K_Panel:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    stove-xspxzg-0-base_link material tree contains K_Oven_Flame:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-commercial_kitchen_table-bwlfnu-0 material tree contains K_Table_Button_2:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    dishwasher-xvchpe-0-top-base_link-P-lower material tree contains K_EQ_Button:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    stove-xspxzg-0-door2-base_link-R-upper material tree contains K_Oven_Flame:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-shelf-dvnwfa-0 material tree contains K_Hood_Light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-commercial_kitchen_table-bwlfnu-1 material tree contains K_Table_Button:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    dishwasher-xvchpe-0-top-base_link-P-lower material tree contains K_EQ_Button_2:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-electric_cauldron-qmdgct-0-lid-base_link-P-lower material tree contains K_Hood_Light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-shelf-dvnwfa-0 material tree contains K_Panel:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-commercial_kitchen_table-bwlfnu-1 material tree contains K_Table_Button_2:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-hwozjd-0 material tree contains K_Yellow_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-electric_cauldron-qmdgct-0-lid-base_link-P-lower material tree contains K_Panel:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.


## scenes/commercial_kitchen_pans
  ### (.*) material tree contains .* with disallowed type .*. It should be one of .*.: 1
    ceilings-edfvqt-0 material tree contains Material #718:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.


## scenes/grocery_store_asian
  ### (.*) material tree contains .* with disallowed type .*. It should be one of .*.: 341
    walls-hukrvr-0 material tree contains Mat3d66-1545959-28-586:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-hukrvr-0 material tree contains Mat3d66-1545959-262-4821:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-hukrvr-0 material tree contains Mat3d66-1545959-264-7856:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-hukrvr-0 material tree contains Mat3d66-1545959-265-4753:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-mwnkac-0 material tree contains Mat3d66-1545959-129-9691:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    downlight-qnleip-0 material tree contains Mat3d66-1545959-124-8784:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-qnleip-1 material tree contains Mat3d66-1545959-124-8784:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-qnleip-2 material tree contains Mat3d66-1545959-124-8784:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-qnleip-3 material tree contains Mat3d66-1545959-124-8784:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-qnleip-4 material tree contains Mat3d66-1545959-124-8784:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-qnleip-5 material tree contains Mat3d66-1545959-124-8784:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-qnleip-6 material tree contains Mat3d66-1545959-124-8784:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-qnleip-7 material tree contains Mat3d66-1545959-124-8784:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-qnleip-8 material tree contains Mat3d66-1545959-124-8784:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-avpijm-0 material tree contains Mat3d66-1545959-257-9741:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-xfigfe-0 material tree contains Mat3d66-1545959-332-1670:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-xfigfe-0 material tree contains 15 - Default:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    track_light-gypkjo-0 material tree contains bulb:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-carton_of_eggs-tacdgl-0 material tree contains Mat3d66-1545959-315-4151:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-grocery_shelf-xmjzos-0 material tree contains Mat3d66-1545959-314-5230:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    wall_mounted_tv-aelksc-0 material tree contains Mat3d66-1545959-255-5292:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-0 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-durian-mttfyh-0 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-half_durian-qqnxxi-0-Tsubpart material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-half_durian-menivf-0-Tsubpart material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    cash_register-ekvdjq-0 material tree contains cgaxis_models_32_23_02:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    checkout_counter-odcaop-0 material tree contains Mat3d66-1545959-266-9766:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-wvctnb-0 material tree contains AI33_008_25:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-wvctnb-0 material tree contains AI33_008_30:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-wvctnb-1 material tree contains AI33_008_25:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-wvctnb-1 material tree contains AI33_008_30:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-wvctnb-2 material tree contains AI33_008_25:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-wvctnb-2 material tree contains AI33_008_30:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-wvctnb-3 material tree contains AI33_008_25:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-wvctnb-3 material tree contains AI33_008_30:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-wvctnb-4 material tree contains AI33_008_25:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-wvctnb-4 material tree contains AI33_008_30:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-wvctnb-7 material tree contains AI33_008_25:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-wvctnb-7 material tree contains AI33_008_30:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-wvctnb-8 material tree contains AI33_008_25:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-wvctnb-8 material tree contains AI33_008_30:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-wvctnb-9 material tree contains AI33_008_25:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-vihddz-0 material tree contains Mat3d66-1545959-28-586:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-wvctnb-9 material tree contains AI33_008_30:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-znllxg-0 material tree contains Mat3d66-1545959-265-4753:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-wvctnb-10 material tree contains AI33_008_25:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-znllxg-0 material tree contains Mat3d66-1545959-264-7856:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-wvctnb-10 material tree contains AI33_008_30:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-znllxg-0 material tree contains Mat3d66-1545959-262-4821:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-wvctnb-11 material tree contains AI33_008_25:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-znllxg-0 material tree contains Mat3d66-1545959-28-586:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-wvctnb-11 material tree contains AI33_008_30:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-hwdoys-0 material tree contains Mat3d66-1545959-265-4753:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-wvctnb-12 material tree contains AI33_008_25:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-hwdoys-0 material tree contains Mat3d66-1545959-264-7856:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-wvctnb-12 material tree contains AI33_008_30:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-hwdoys-0 material tree contains Mat3d66-1545959-262-4821:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-wvctnb-13 material tree contains AI33_008_25:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-hwdoys-0 material tree contains Mat3d66-1545959-28-586:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-wvctnb-13 material tree contains AI33_008_30:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-pmumoz-0 material tree contains Mat3d66-1545959-265-4753:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-wvctnb-14 material tree contains AI33_008_25:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-pmumoz-0 material tree contains Mat3d66-1545959-264-7856:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    cash_register-ekvdjq-0-drawer-base_link-P-upper material tree contains cgaxis_models_32_23_02:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-wvctnb-14 material tree contains AI33_008_30:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-pmumoz-0 material tree contains Mat3d66-1545959-262-4821:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    cash_register-ekvdjq-0-drawer-base_link-P-lower material tree contains cgaxis_models_32_23_02:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-wvctnb-15 material tree contains AI33_008_25:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-pmumoz-0 material tree contains Mat3d66-1545959-28-586:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-ehfawi-0 material tree contains Mat3d66-1545959-265-4753:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-wvctnb-15 material tree contains AI33_008_30:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-huoqsn-0 material tree contains Mat3d66-1545959-265-4753:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-ehfawi-0 material tree contains Mat3d66-1545959-264-7856:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-wvctnb-16 material tree contains AI33_008_25:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-huoqsn-0 material tree contains Mat3d66-1545959-264-7856:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-ehfawi-0 material tree contains Mat3d66-1545959-262-4821:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-wvctnb-16 material tree contains AI33_008_30:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-huoqsn-0 material tree contains Mat3d66-1545959-262-4821:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-ehfawi-0 material tree contains Mat3d66-1545959-28-586:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-wvctnb-17 material tree contains AI33_008_25:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-huoqsn-0 material tree contains Mat3d66-1545959-28-586:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-ytyvgd-0 material tree contains Mat3d66-1545959-265-4753:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-wvctnb-17 material tree contains AI33_008_30:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-nnxecd-0 material tree contains Mat3d66-1545959-265-4753:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-ytyvgd-0 material tree contains Mat3d66-1545959-264-7856:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-wvctnb-18 material tree contains AI33_008_25:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-nnxecd-0 material tree contains Mat3d66-1545959-264-7856:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-ytyvgd-0 material tree contains Mat3d66-1545959-262-4821:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-wvctnb-18 material tree contains AI33_008_30:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-nnxecd-0 material tree contains Mat3d66-1545959-262-4821:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-ytyvgd-0 material tree contains Mat3d66-1545959-28-586:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-wvctnb-19 material tree contains AI33_008_25:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-nnxecd-0 material tree contains Mat3d66-1545959-28-586:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-bjybja-0 material tree contains Mat3d66-1545959-265-4753:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-wvctnb-19 material tree contains AI33_008_30:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-oqsqzj-0 material tree contains Mat3d66-1545959-265-4753:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-bjybja-0 material tree contains Mat3d66-1545959-264-7856:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-wvctnb-20 material tree contains AI33_008_25:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-oqsqzj-0 material tree contains Mat3d66-1545959-264-7856:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-bjybja-0 material tree contains Mat3d66-1545959-262-4821:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-wvctnb-20 material tree contains AI33_008_30:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-oqsqzj-0 material tree contains Mat3d66-1545959-262-4821:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-bjybja-0 material tree contains Mat3d66-1545959-28-586:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-wvctnb-21 material tree contains AI33_008_25:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-oqsqzj-0 material tree contains Mat3d66-1545959-28-586:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-ncwzvh-0 material tree contains Mat3d66-1545959-265-4753:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-wvctnb-21 material tree contains AI33_008_30:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-bgmpqm-0 material tree contains Mat3d66-1545959-265-4753:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-ncwzvh-0 material tree contains Mat3d66-1545959-264-7856:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-wvctnb-22 material tree contains AI33_008_25:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-bgmpqm-0 material tree contains Mat3d66-1545959-264-7856:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-ncwzvh-0 material tree contains Mat3d66-1545959-262-4821:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-wvctnb-22 material tree contains AI33_008_30:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-bgmpqm-0 material tree contains Mat3d66-1545959-262-4821:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-ncwzvh-0 material tree contains Mat3d66-1545959-28-586:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-wvctnb-23 material tree contains AI33_008_25:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-bgmpqm-0 material tree contains Mat3d66-1545959-28-586:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-vihddz-0 material tree contains Mat3d66-1545959-265-4753:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-wvctnb-23 material tree contains AI33_008_30:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-yjyfqb-0 material tree contains Mat3d66-1545959-265-4753:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-vihddz-0 material tree contains Mat3d66-1545959-264-7856:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-wvctnb-24 material tree contains AI33_008_25:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-yjyfqb-0 material tree contains Mat3d66-1545959-264-7856:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-vihddz-0 material tree contains Mat3d66-1545959-262-4821:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-wvctnb-24 material tree contains AI33_008_30:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-80 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-yjyfqb-0 material tree contains Mat3d66-1545959-262-4821:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-wvctnb-25 material tree contains AI33_008_25:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-79 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-yjyfqb-0 material tree contains Mat3d66-1545959-28-586:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-wvctnb-25 material tree contains AI33_008_30:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-78 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-vwfllw-0 material tree contains Mat3d66-1545959-265-4753:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-wvctnb-26 material tree contains AI33_008_25:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-77 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-vwfllw-0 material tree contains Mat3d66-1545959-264-7856:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-wvctnb-26 material tree contains AI33_008_30:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-76 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-vwfllw-0 material tree contains Mat3d66-1545959-262-4821:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-wvctnb-27 material tree contains AI33_008_25:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-75 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-vwfllw-0 material tree contains Mat3d66-1545959-28-586:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-wvctnb-27 material tree contains AI33_008_30:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-74 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-qgzvcz-0 material tree contains Mat3d66-1545959-265-4753:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-wvctnb-28 material tree contains AI33_008_25:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-73 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-qgzvcz-0 material tree contains Mat3d66-1545959-264-7856:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-wvctnb-28 material tree contains AI33_008_30:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-72 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-qgzvcz-0 material tree contains Mat3d66-1545959-262-4821:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-wvctnb-29 material tree contains AI33_008_25:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-71 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-qgzvcz-0 material tree contains Mat3d66-1545959-28-586:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-wvctnb-29 material tree contains AI33_008_30:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-70 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-carton_of_eggs-tacdgl-5 material tree contains Mat3d66-1545959-315-4151:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-wvctnb-30 material tree contains AI33_008_25:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-69 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-carton_of_eggs-tacdgl-4 material tree contains Mat3d66-1545959-315-4151:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-wvctnb-30 material tree contains AI33_008_30:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-68 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-carton_of_eggs-tacdgl-3 material tree contains Mat3d66-1545959-315-4151:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-wvctnb-31 material tree contains AI33_008_25:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-67 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-carton_of_eggs-tacdgl-2 material tree contains Mat3d66-1545959-315-4151:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-wvctnb-31 material tree contains AI33_008_30:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-66 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-carton_of_eggs-tacdgl-1 material tree contains Mat3d66-1545959-315-4151:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-wvctnb-32 material tree contains AI33_008_25:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-65 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-xfigfe-4 material tree contains 15 - Default:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-wvctnb-32 material tree contains AI33_008_30:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-64 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-xfigfe-4 material tree contains Mat3d66-1545959-332-1670:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-wvctnb-33 material tree contains AI33_008_25:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-63 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-xfigfe-3 material tree contains 15 - Default:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-wvctnb-33 material tree contains AI33_008_30:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-62 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-xfigfe-3 material tree contains Mat3d66-1545959-332-1670:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-wvctnb-34 material tree contains AI33_008_25:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-61 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-xfigfe-2 material tree contains 15 - Default:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-wvctnb-34 material tree contains AI33_008_30:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-60 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-xfigfe-2 material tree contains Mat3d66-1545959-332-1670:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-wvctnb-35 material tree contains AI33_008_25:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-59 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-xfigfe-1 material tree contains 15 - Default:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-wvctnb-35 material tree contains AI33_008_30:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-58 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-xfigfe-1 material tree contains Mat3d66-1545959-332-1670:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-wvctnb-5 material tree contains AI33_008_25:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-57 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-150 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-wvctnb-5 material tree contains AI33_008_30:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-56 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-149 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-wvctnb-6 material tree contains AI33_008_25:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-55 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-148 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-wvctnb-6 material tree contains AI33_008_30:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-54 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-147 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-durian-mttfyh-1 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-53 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-146 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-durian-mttfyh-2 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-52 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-145 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-durian-mttfyh-3 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-51 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-144 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-durian-mttfyh-4 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-50 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-143 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-durian-mttfyh-5 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-142 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-durian-mttfyh-6 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-141 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-durian-mttfyh-7 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-140 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-durian-mttfyh-8 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-139 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-durian-mttfyh-9 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-138 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-durian-mttfyh-10 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-137 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-durian-mttfyh-11 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-136 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-durian-mttfyh-12 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-135 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-durian-mttfyh-13 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-134 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-durian-mttfyh-14 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-133 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-durian-mttfyh-15 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-132 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-durian-mttfyh-16 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-131 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-durian-mttfyh-17 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-130 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-1 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-129 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-2 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-128 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-3 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-127 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-4 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-126 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-5 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-125 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-6 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-124 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-7 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-123 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-8 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-122 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-9 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-121 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-10 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-120 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-11 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-119 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-12 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-118 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-13 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-117 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-14 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-116 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-15 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-115 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-16 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-114 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-17 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-113 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-18 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-112 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-19 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-111 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-20 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-110 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-21 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-109 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-22 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-108 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-23 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-107 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-24 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-106 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-25 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-105 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-26 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-104 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-27 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-103 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-28 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-102 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-29 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-101 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-30 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-100 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-31 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-99 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-32 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-98 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-33 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-97 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-34 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-96 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-35 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-95 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-36 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-94 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-37 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-93 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-38 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-92 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-39 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-91 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-40 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-90 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-41 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-89 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-42 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-88 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-43 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-87 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-44 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-86 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-45 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-85 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-46 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-84 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-47 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-83 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-48 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-82 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-49 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-rizrsp-81 material tree contains Material #158:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.


## scenes/grocery_store_cafe
  ### (.*) material tree contains .* with disallowed type .*. It should be one of .*.: 25
    C-coffee_maker-fwlabx-0 material tree contains vray_Mat3d66-1508405-119-979:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-cash_register-ekvdjq-0-drawer-base_link-P-lower material tree contains cgaxis_models_32_23_02:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-monitor-okfatx-0 material tree contains Mat3d66-1508405-180-1168:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-cash_register-ekvdjq-0-base_link material tree contains cgaxis_models_32_23_02:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    wall_mounted_tv-nmquap-0 material tree contains Mat3d66-1508405-197-2584:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    wall_mounted_tv-nmquap-1 material tree contains Mat3d66-1508405-197-2584:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    wall_mounted_tv-nmquap-0 material tree contains re224_3:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    wall_mounted_tv-nmquap-1 material tree contains re224_3:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    wall_mounted_tv-nmquap-2 material tree contains Mat3d66-1508405-197-2584:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-ffoeix-0 material tree contains Mat3d66-1508405-191-6116:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    wall_mounted_tv-nmquap-2 material tree contains re224_3:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-yqhwtb-0 material tree contains Mat3d66-1508405-191-6116:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    structural_element-kebons-0 material tree contains Mat3d66-1508405-169-5186:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-espresso_machine-qhrtyw-1 material tree contains re194:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    pillar-xnxycq-0 material tree contains Mat3d66-1508405-204-1703:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    structural_element-kebons-0 material tree contains Material #25:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    ceilings-gexutv-0 material tree contains Mat3d66-1508405-204-1703:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    pillar-xnxycq-1 material tree contains Mat3d66-1508405-204-1703:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    ceilings-gexutv-0 material tree contains Mat3d66-1508405-44-5514:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-espresso_machine-qhrtyw-0 material tree contains re194:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-cap-kaolpg-0-Tconnectedpart material tree contains vray_Mat3d66-1508405-120-794:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-cap-kaolpg-0-Tconnectedpart material tree contains vray_Mat3d66-1508405-123-187:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-cap-kaolpg-0-Tconnectedpart material tree contains vray_Mat3d66-1508405-119-979:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-coffee_maker-fwlabx-0 material tree contains vray_Mat3d66-1508405-120-794:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-coffee_maker-fwlabx-0 material tree contains vray_Mat3d66-1508405-123-187:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.


## scenes/grocery_store_convenience
  ### (.*) is in the deletion queue. Delete the object.: 7
    C-box_of_salt-azqqwx-1 is in the deletion queue. Delete the object.
    C-box_of_salt-azqqwx-3 is in the deletion queue. Delete the object.
    C-box_of_salt-azqqwx-2 is in the deletion queue. Delete the object.
    C-box_of_salt-ntmrkt-2 is in the deletion queue. Delete the object.
    C-box_of_salt-ntmrkt-0 is in the deletion queue. Delete the object.
    C-box_of_salt-ntmrkt-1 is in the deletion queue. Delete the object.
    C-box_of_salt-azqqwx-0 is in the deletion queue. Delete the object.


## scenes/grocery_store_half_stocked
  ### (.*) material tree contains .* with disallowed type .*. It should be one of .*.: 912
    C-bottle_of_wine-dawhco-5 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-qtefba-0-Tglass material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-2-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-5 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-qtefba-1 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-2-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-9 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-qtefba-1 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-2-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-9 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-qtefba-2 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-2-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-10 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-qtefba-2 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-1-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-10 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-qtefba-3 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-1-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-11 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-qtefba-3 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-1-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-11 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-15 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-1-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-12 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-15 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-17-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-12 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-13 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-17-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-13 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-13 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-17-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-13 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-11 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-17-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-14 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-11 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-16-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-14 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-9 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-16-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-15 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-9 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-16-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-15 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-7 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-16-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-16 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-7 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-15-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-16 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-16 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-15-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-17 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-16 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-15-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-17 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-24 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-15-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-18 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-24 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-14-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-18 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-23 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-14-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-13 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-23 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-14-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-13 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-22 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-14-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-12 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-22 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-13-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-12 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-21 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-13-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-11 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-21 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-13-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-11 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-20 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-13-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-10 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-20 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-12-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-10 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-19 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-12-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-9 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-19 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-12-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-9 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-13 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-4-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-8 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-11 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-3-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-8 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-11 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-3-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-7 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-12 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-2-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-7 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-12 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-2-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-6 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-jivcff-7 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-1-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-6 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-jivcff-7 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-1-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-5 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-jivcff-6 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_champagne-ahpwlg-4-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-5 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-jivcff-6 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_champagne-ahpwlg-4-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-4 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-jivcff-5 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_champagne-ahpwlg-3-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-4 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-jivcff-5 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_champagne-ahpwlg-3-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-gaqzfq-5 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-jivcff-4 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_champagne-ahpwlg-2-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-gaqzfq-5 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-jivcff-4 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_champagne-ahpwlg-2-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-gaqzfq-4 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-jivcff-3 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_champagne-ahpwlg-1-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-gaqzfq-4 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-jivcff-3 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_champagne-ahpwlg-1-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-gaqzfq-3 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-jivcff-2 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-0-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-gaqzfq-3 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-jivcff-2 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-4-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-gaqzfq-2 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-12 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-3-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-gaqzfq-2 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-12 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-3-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-19 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-11 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-2-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-19 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-11 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-2-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-18 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-10 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-1-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-18 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-10 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-1-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-1 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-9 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_champagne-ahpwlg-4-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-1 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-9 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_champagne-ahpwlg-4-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-17 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-8 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_champagne-ahpwlg-3-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-17 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-8 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_champagne-ahpwlg-3-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-4 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-7 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_champagne-ahpwlg-2-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-4 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-7 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_champagne-ahpwlg-2-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-3 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-6 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_champagne-ahpwlg-1-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-3 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-6 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_champagne-ahpwlg-1-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-2 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-2 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-12-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-gaqzfq-0-Tglass material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-2 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-11-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-gaqzfq-0-Tglass material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-1 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-11-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-bmrewz-23 material tree contains Material #1111226171:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-1 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-10-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-bmrewz-23 material tree contains vray_light006:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    C-bottle_of_wine-jivcff-0-Tglass material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-10-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-bmrewz-22 material tree contains Material #1111226171:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-jivcff-0-Tglass material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-9-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-bmrewz-22 material tree contains vray_light006:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    C-bottle_of_wine-jivcff-1 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-9-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-bmrewz-21 material tree contains Material #1111226171:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-jivcff-1 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-8-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-bmrewz-21 material tree contains vray_light006:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    C-bottle_of_wine-dawhco-4 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-7-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-bmrewz-20 material tree contains Material #1111226171:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-4 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-7-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-bmrewz-20 material tree contains vray_light006:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    C-bottle_of_wine-dawhco-3 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-6-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-bmrewz-19 material tree contains Material #1111226171:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-3 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-6-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-bmrewz-19 material tree contains vray_light006:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    C-bottle_of_wine-dawhco-2 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-5-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-bmrewz-18 material tree contains Material #1111226171:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-2 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-5-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-bmrewz-18 material tree contains vray_light006:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    C-bottle_of_wine-dawhco-1 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-4-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-bmrewz-17 material tree contains Material #1111226171:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-1 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-8-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-bmrewz-17 material tree contains vray_light006:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    C-bottle_of_wine-rfsuxc-3 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-11-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-bmrewz-16 material tree contains Material #1111226171:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-3 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-11-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-bmrewz-16 material tree contains vray_light006:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    C-bottle_of_wine-rfsuxc-2 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-10-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-bmrewz-15 material tree contains Material #1111226171:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-2 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-10-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-bmrewz-15 material tree contains vray_light006:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    C-bottle_of_wine-rfsuxc-1 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-9-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-bmrewz-14 material tree contains Material #1111226171:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-1 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-9-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-bmrewz-14 material tree contains vray_light006:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    C-bottle_of_wine-rfsuxc-0-Tglass material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-8-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-bmrewz-13 material tree contains Material #1111226171:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-0-Tglass material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-8-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-bmrewz-13 material tree contains vray_light006:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    C-bottle_of_wine-kqokhv-3 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-7-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-bmrewz-12 material tree contains Material #1111226171:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-3 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-7-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-bmrewz-12 material tree contains vray_light006:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    C-bottle_of_wine-kqokhv-0-Tglass material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-6-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-bmrewz-11 material tree contains Material #1111226171:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-0-Tglass material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-6-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-bmrewz-11 material tree contains vray_light006:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    C-bottle_of_wine-kqokhv-1 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-5-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-bmrewz-10 material tree contains Material #1111226171:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-1 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-5-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-bmrewz-10 material tree contains vray_light006:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    C-bottle_of_wine-kqokhv-2 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-4-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-19 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-bmrewz-9 material tree contains Material #1111226171:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-18 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_champagne-ahpwlg-1 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-bmrewz-9 material tree contains vray_light006:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    C-bottle_of_wine-dbccri-17 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_champagne-ahpwlg-1 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-bmrewz-8 material tree contains Material #1111226171:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-16 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-qtefba-4 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-bmrewz-8 material tree contains vray_light006:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    C-bottle_of_wine-dbccri-15 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-qtefba-4 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-bmrewz-7 material tree contains Material #1111226171:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-jivcff-11 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_rum-rduqdf-0 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-bmrewz-7 material tree contains vray_light006:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    C-bottle_of_wine-jivcff-10 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_rum-rduqdf-0 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-bmrewz-6 material tree contains Material #1111226171:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-jivcff-9 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_champagne-pxaayf-0-Tglass material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-bmrewz-6 material tree contains vray_light006:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    C-bottle_of_wine-dawhco-17 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_champagne-pxaayf-0-Tglass material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-bmrewz-5 material tree contains Material #1111226171:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-16 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-bmrewz-5 material tree contains vray_light006:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    C-bottle_of_wine-dawhco-15 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-bmrewz-4 material tree contains Material #1111226171:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-14 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-bmrewz-4 material tree contains vray_light006:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    C-bottle_of_wine-dawhco-13 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-bmrewz-3 material tree contains Material #1111226171:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-4 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-bmrewz-3 material tree contains vray_light006:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    C-bottle_of_wine-rfsuxc-4 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-bmrewz-2 material tree contains Material #1111226171:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-5 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-bmrewz-2 material tree contains vray_light006:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    C-bottle_of_wine-rfsuxc-5 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-bmrewz-1 material tree contains Material #1111226171:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-18 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-bmrewz-1 material tree contains vray_light006:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    C-bottle_of_wine-dbccri-17 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-bmrewz-0 material tree contains Material #1111226171:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-16 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-bmrewz-0 material tree contains vray_light006:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    C-bottle_of_wine-dbccri-15 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-0-Tglass material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-jivcff-11 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-0-Tglass material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-jivcff-10 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-19 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-jivcff-9 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-19 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-17 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-0-Tglass material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-16 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-0-Tglass material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-15 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-jivcff-8 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-14 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-jivcff-8 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-13 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-cash_register-ekvdjq-0-drawer-base_link-P-lower material tree contains cgaxis_models_32_23_02:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-gaqzfq-1 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-6 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-cash_register-ekvdjq-0-base_link material tree contains cgaxis_models_32_23_02:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-gaqzfq-1 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-6 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-4-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-19 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-7 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-4-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-0-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_champagne-pxaayf-3-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-2-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-12-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-4-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-0-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_champagne-pxaayf-2-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-2-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-12-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-4-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-0-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_champagne-pxaayf-2-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-2-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-12-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-5-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-0-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_champagne-pxaayf-2-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-2-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-13-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-5-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-0-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_champagne-pxaayf-2-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-1-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-13-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-5-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-0-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_champagne-pxaayf-1-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-1-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-13-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-5-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-0-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_champagne-pxaayf-1-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-1-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-13-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-6-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-jivcff-0-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_champagne-pxaayf-1-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-1-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-14-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-6-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-jivcff-0-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_champagne-pxaayf-1-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-19-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-14-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-6-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-0-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-15-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-19-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-14-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-6-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-0-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-15-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-19-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-14-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-7-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-qtefba-0-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-15-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-19-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-15-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-7-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-qtefba-0-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-15-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-qtefba-1-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-15-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-7-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_champagne-ahpwlg-0-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-13-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-qtefba-1-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-15-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-7-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_champagne-ahpwlg-0-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-13-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-qtefba-1-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-15-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-8-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_champagne-pxaayf-0-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-13-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-qtefba-1-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-16-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-8-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_champagne-pxaayf-0-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-13-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-qtefba-2-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-16-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-8-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-0-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-11-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-qtefba-2-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-16-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-8-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-0-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-11-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-qtefba-2-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-16-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-9-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-qtefba-0-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-11-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-qtefba-2-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-17-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-9-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-qtefba-0-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-11-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-qtefba-3-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-17-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-9-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-0-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-9-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-qtefba-3-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-17-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-9-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-0-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-9-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-qtefba-3-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-17-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-10-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-jivcff-0-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-9-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-qtefba-3-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-18-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-10-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-jivcff-0-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-9-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-qtefba-4-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-18-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-10-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-0-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-7-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-qtefba-4-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-18-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-10-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-0-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-7-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-qtefba-4-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-18-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-11-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-0-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-7-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-qtefba-4-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-3-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-11-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dawhco-0-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-7-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_champagne-pxaayf-3-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-3-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-11-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_champagne-pxaayf-0-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-16-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_champagne-pxaayf-3-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-3-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-11-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_champagne-pxaayf-0-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-16-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_champagne-pxaayf-3-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-3-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-12-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-7 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-9-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-16-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-3-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-16-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-8 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-8-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-16-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-3-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-16-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-rfsuxc-8 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-8-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-16-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-3-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-14-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-18 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-8-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-16-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-2-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-14-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-18 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-8-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-15-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-2-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-14-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-17 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-7-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-15-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-2-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-14-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-17 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-7-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-15-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-2-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-12-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-16 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-7-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-15-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-19-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-12-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-16 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-7-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-14-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-19-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-12-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-15 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-6-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-14-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-19-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-12-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-15 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-6-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-14-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-19-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-10-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-14 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-6-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-14-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-18-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-10-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-14 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-6-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-13-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-18-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-10-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-gaqzfq-12 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-5-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-13-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-18-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-10-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-gaqzfq-12 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-5-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-13-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-18-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-8-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-gaqzfq-11 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-5-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-13-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-1-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-8-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-gaqzfq-11 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-5-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-12-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-1-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-8-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-gaqzfq-10 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-4-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-12-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-1-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-8-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-gaqzfq-10 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-4-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-12-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-1-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-6-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-gaqzfq-9 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-4-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-12-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-17-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-6-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-gaqzfq-9 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-4-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-11-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-17-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-6-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-gaqzfq-8 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-3-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-11-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-17-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-6-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-gaqzfq-8 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-3-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-11-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-17-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-5-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-gaqzfq-7 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-3-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-11-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-18-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-5-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-gaqzfq-7 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-3-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-10-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-18-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-5-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-gaqzfq-6 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-1-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-10-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-18-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-5-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-gaqzfq-6 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-1-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-10-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-18-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-4-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-14 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-1-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-10-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-17-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-4-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-14 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-1-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-9-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-17-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-4-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-12 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-2-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-9-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-17-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-4-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-12 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-2-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-9-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-17-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-3-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_champagne-ahpwlg-0-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-gaqzfq-9-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-jivcff-4-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-2-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_champagne-ahpwlg-0-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-gaqzfq-9-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-jivcff-4-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-2-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-0-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bottle_of_wine-gaqzfq-9-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-jivcff-4-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-19-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-0-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bottle_of_wine-gaqzfq-9-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-jivcff-3-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-19-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bottle_of_wine-gaqzfq-0-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-gaqzfq-8-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-jivcff-3-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-19-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bottle_of_wine-gaqzfq-0-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-gaqzfq-8-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-jivcff-3-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-kqokhv-19-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-gaqzfq-0-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bottle_of_wine-gaqzfq-8-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-jivcff-3-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-jivcff-11-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-gaqzfq-0-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bottle_of_wine-gaqzfq-8-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-jivcff-2-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-jivcff-11-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_champagne-pxaayf-3 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-gaqzfq-7-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-jivcff-2-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-jivcff-11-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_champagne-pxaayf-3 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-gaqzfq-7-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-jivcff-2-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-jivcff-11-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_champagne-pxaayf-2 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bottle_of_wine-gaqzfq-7-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-jivcff-2-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-jivcff-10-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_champagne-pxaayf-2 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bottle_of_wine-gaqzfq-7-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-jivcff-1-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-jivcff-10-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_champagne-pxaayf-1 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-gaqzfq-6-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-jivcff-1-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-jivcff-10-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_champagne-pxaayf-1 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-gaqzfq-6-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-jivcff-1-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-jivcff-10-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_rum-rduqdf-4 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bottle_of_wine-gaqzfq-6-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-jivcff-1-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-jivcff-9-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_rum-rduqdf-4 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bottle_of_wine-gaqzfq-6-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-jivcff-8-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-jivcff-9-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_rum-rduqdf-3 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-gaqzfq-5-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-jivcff-8-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-jivcff-9-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_rum-rduqdf-3 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-gaqzfq-5-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-jivcff-8-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-jivcff-9-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_rum-rduqdf-2 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bottle_of_wine-gaqzfq-5-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-jivcff-8-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-jivcff-7-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_rum-rduqdf-2 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bottle_of_wine-gaqzfq-5-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-gaqzfq-12-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-jivcff-7-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_rum-rduqdf-1 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-gaqzfq-4-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-gaqzfq-12-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-jivcff-7-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_rum-rduqdf-1 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-gaqzfq-4-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bottle_of_wine-gaqzfq-12-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-jivcff-7-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_champagne-ahpwlg-4 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bottle_of_wine-gaqzfq-4-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bottle_of_wine-gaqzfq-12-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-jivcff-6-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_champagne-ahpwlg-4 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bottle_of_wine-gaqzfq-4-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-gaqzfq-11-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-jivcff-6-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_champagne-ahpwlg-3 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-gaqzfq-3-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-gaqzfq-11-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-jivcff-6-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_champagne-ahpwlg-3 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-gaqzfq-3-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bottle_of_wine-gaqzfq-11-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-jivcff-6-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_champagne-ahpwlg-2 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bottle_of_wine-gaqzfq-3-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bottle_of_wine-gaqzfq-11-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-jivcff-5-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_champagne-ahpwlg-2 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bottle_of_wine-gaqzfq-3-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-gaqzfq-10-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-jivcff-5-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_champagne-ahpwlg-0-Tglass material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-gaqzfq-2-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-gaqzfq-10-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-jivcff-5-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_champagne-ahpwlg-0-Tglass material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-gaqzfq-2-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bottle_of_wine-gaqzfq-10-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-jivcff-5-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-qtefba-0-Tglass material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bottle_of_wine-gaqzfq-2-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bottle_of_wine-gaqzfq-10-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-jivcff-4-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-10 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-9-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-18-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bottle_of_wine-gaqzfq-2-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-10 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-9-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-18-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-gaqzfq-1-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-8 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-9-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-17-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-gaqzfq-1-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-8 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-14-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-17-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bottle_of_wine-gaqzfq-1-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-6 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-14-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-17-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bottle_of_wine-gaqzfq-1-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-6 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-14-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-17-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-24-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-5 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-14-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-16-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-24-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-5 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-10-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-16-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-24-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-4 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-10-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-16-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-24-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-4 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-10-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-16-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-23-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-3 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-10-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-15-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-23-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-3 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-13-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-15-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-23-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-2 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-13-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-15-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-23-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-2 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-13-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-15-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-22-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-0-Tglass material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-13-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-7-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-22-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-ntzera-0-Tglass material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-11-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-7-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-22-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-7 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-11-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-7-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-22-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-7 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-11-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-7-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-21-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-5 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-11-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-5-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-21-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-5 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-12-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-5-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-21-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-6 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-12-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-5-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-21-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-6 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-12-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-5-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-20-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-8 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-12-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-6-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-20-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-8 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-4-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-6-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-20-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-9 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-4-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-6-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-20-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-9 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-4-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-6-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-19-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-14 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-4-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-8-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-19-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-14 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-3-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-8-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-19-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-10 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-3-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-8-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-19-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-10 material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-3-cap-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-8-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-18-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-13 material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-3-cap-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-9-label-base_link-F-lower material tree contains vray_Material #35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_wine-dbccri-18-label-base_link-F-lower material tree contains Material #36:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.


## scenes/hall_arch_wood
  ### (.*) material tree contains .* with disallowed type .*. It should be one of .*.: 8
    walls-odlitx-0 material tree contains Archinteriors_024_017_07:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    ceilings-zkoute-0-Tglass material tree contains Archinteriors_024_017_07:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    ceilings-utgliy-0-Tglass material tree contains Archinteriors_024_017_07:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-vtplrx-0 material tree contains Archinteriors_024_017_07:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    ceilings-xaodor-0-Tglass material tree contains Archinteriors_024_017_07:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    ceilings-oktrwd-0 material tree contains Archinteriors_024_017_07:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-uaszec-0 material tree contains Archinteriors_024_017_07:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-wynpvm-0 material tree contains Archinteriors_024_017_07:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.


## scenes/hall_conference_large
  ### (.*) material tree contains .* with disallowed type .*. It should be one of .*.: 391
    downlight-yrnfdt-297 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-328 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-359 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-390 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-296 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-327 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-358 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-389 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-295 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-326 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-357 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-388 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-294 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-325 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-356 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-387 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-293 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-324 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-355 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-386 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-292 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-323 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-354 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-385 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-291 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-322 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-353 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-384 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-290 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-321 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-352 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-383 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-289 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-320 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-351 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-382 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-288 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-319 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-350 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-381 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-287 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-318 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-349 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-380 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-286 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-317 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-348 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-379 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-285 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-316 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-347 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-378 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-284 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-315 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-346 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-377 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-283 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-314 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-345 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-376 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-282 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-313 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-344 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-375 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-281 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-312 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-343 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-374 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-280 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-311 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-342 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-373 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-279 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-310 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-341 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-372 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-278 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-309 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-340 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-371 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-277 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-308 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-339 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-370 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-276 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-307 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-338 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-369 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-275 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-306 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-337 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-368 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-274 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-305 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-336 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-367 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-273 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-304 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-335 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-366 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-272 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-303 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-334 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-365 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-271 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-302 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-333 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-364 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-270 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-301 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-332 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-363 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-269 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-300 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-331 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-362 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-268 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-299 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-330 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-361 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-267 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-298 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-329 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-360 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-204 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-235 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-266 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-203 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-234 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-265 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-202 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-233 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-264 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-201 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-232 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-263 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-200 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-231 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-262 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-199 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-230 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-261 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-198 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-229 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-260 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-197 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-228 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-259 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-196 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-227 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-258 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-195 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-226 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-254 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-194 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-225 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-250 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-193 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-224 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-244 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-192 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-223 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-257 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-191 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-222 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-253 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-190 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-221 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-249 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-189 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-220 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-248 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-188 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-219 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-256 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-187 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-218 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-252 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-186 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-217 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-247 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-185 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-216 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-246 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-184 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-215 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-255 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-183 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-214 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-251 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-182 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-213 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-245 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-181 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-212 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-243 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-180 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-211 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-242 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-179 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-210 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-241 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-178 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-209 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-240 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-177 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-208 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-239 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-176 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-207 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-238 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-175 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-206 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-237 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-174 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-205 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-236 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-142 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-173 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-141 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-172 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-140 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-171 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-139 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-170 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-138 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-169 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-137 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-168 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-136 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-167 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-135 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-166 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-134 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-165 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-133 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-164 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-132 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-163 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-131 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-162 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-130 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-161 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-129 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-160 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-128 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-159 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-127 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-158 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-126 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-157 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-125 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-156 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-124 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-155 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-123 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-154 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-122 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-153 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-121 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-152 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-120 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-151 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-119 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-150 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-118 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-149 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-117 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-148 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-116 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-147 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-115 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-146 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-114 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-145 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-113 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-144 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-112 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-143 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-18 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-49 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-80 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-111 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-17 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-48 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-79 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-110 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-16 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-47 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-78 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-109 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-15 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-46 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-77 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-108 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-14 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-45 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-76 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-107 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-13 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-44 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-75 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-106 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-12 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-43 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-74 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-105 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-11 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-42 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-73 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-104 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-10 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-41 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-72 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-103 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-9 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-40 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-71 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-102 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-8 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-39 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-70 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-101 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-7 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-38 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-69 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-100 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-6 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-37 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-68 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-99 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-5 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-36 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-67 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-98 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-4 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-35 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-66 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-97 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-34 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-65 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-3 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-96 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-2 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-33 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-64 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-95 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-1 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-32 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-63 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-94 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-0 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-31 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-62 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-93 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-30 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-61 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-92 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-29 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-60 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-91 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-28 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-59 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-90 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-27 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-58 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-89 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-26 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-57 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-88 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-25 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-56 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-87 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-24 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-55 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-86 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-23 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-54 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-85 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-22 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-53 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-84 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-21 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-52 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-83 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-20 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-51 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-82 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-19 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-50 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-yrnfdt-81 material tree contains AI53_002_VRayLight_Lamp:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.


## scenes/hall_glass_ceiling
  ### (.*) material tree contains .* with disallowed type .*. It should be one of .*.: 68
    pillar-keesuv-10 material tree contains renderwhite_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    spotlight-wtrjoa-9 material tree contains steellight_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    walls-nyrdyc-0 material tree contains wooddark_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    ceilings-nyewww-0-Tglass material tree contains steellight_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-cggyho-4 material tree contains steellight_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    spotlight-wtrjoa-7 material tree contains steellight_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    spotlight-wtrjoa-20 material tree contains steellight_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    walls-nyrdyc-0 material tree contains renderwhite_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-cggyho-3 material tree contains steellight_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    spotlight-wtrjoa-6 material tree contains steellight_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    spotlight-wtrjoa-17 material tree contains steellight_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    walls-lsfzjg-0 material tree contains wooddark_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-cggyho-2 material tree contains steellight_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    spotlight-wtrjoa-5 material tree contains steellight_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    spotlight-wtrjoa-12 material tree contains steellight_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    walls-lsfzjg-0 material tree contains renderwhite_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-cggyho-1 material tree contains steellight_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    spotlight-wtrjoa-8 material tree contains steellight_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    spotlight-wtrjoa-23 material tree contains steellight_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    walls-kiagzh-0 material tree contains wooddark_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    pillar-keesuv-9 material tree contains renderwhite_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    spotlight-wtrjoa-3 material tree contains steellight_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    spotlight-wtrjoa-19 material tree contains steellight_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    walls-kiagzh-0 material tree contains renderwhite_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    pillar-keesuv-8 material tree contains renderwhite_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    spotlight-wtrjoa-1 material tree contains steellight_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    spotlight-wtrjoa-16 material tree contains steellight_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    spotlight-wtrjoa-33 material tree contains steellight_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    fixed_window-cggyho-0 material tree contains steellight_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    spotlight-wtrjoa-0 material tree contains steellight_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    spotlight-wtrjoa-15 material tree contains steellight_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    spotlight-wtrjoa-32 material tree contains steellight_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    pillar-keesuv-7 material tree contains renderwhite_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    spotlight-wtrjoa-4 material tree contains steellight_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    spotlight-wtrjoa-22 material tree contains steellight_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    spotlight-wtrjoa-31 material tree contains steellight_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    pillar-keesuv-6 material tree contains renderwhite_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    spotlight-wtrjoa-2 material tree contains steellight_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    spotlight-wtrjoa-18 material tree contains steellight_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    spotlight-wtrjoa-30 material tree contains steellight_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    ceilings-nyewww-0-Tglass material tree contains renderwhite_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    pillar-keesuv-5 material tree contains renderwhite_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    pillar-keesuv-11 material tree contains renderwhite_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    spotlight-wtrjoa-14 material tree contains steellight_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    spotlight-wtrjoa-28 material tree contains steellight_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    spotlight-wtrjoa-29 material tree contains steellight_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    pillar-keesuv-4 material tree contains renderwhite_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    pillar-keesuv-3 material tree contains renderwhite_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    spotlight-wtrjoa-13 material tree contains steellight_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    spotlight-wtrjoa-27 material tree contains steellight_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    pillar-keesuv-1 material tree contains renderwhite_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    pillar-keesuv-2 material tree contains renderwhite_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    spotlight-wtrjoa-21 material tree contains steellight_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    spotlight-wtrjoa-26 material tree contains steellight_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    fixed_window-cggyho-4-glass-base_link-F-lower-Tglass material tree contains steellight_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    pillar-keesuv-0 material tree contains renderwhite_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    walls-dpponz-0 material tree contains wooddark_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    spotlight-wtrjoa-11 material tree contains steellight_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    spotlight-wtrjoa-25 material tree contains steellight_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    fixed_window-cggyho-2-glass-base_link-F-lower-Tglass material tree contains steellight_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-cggyho-3-glass-base_link-F-lower-Tglass material tree contains steellight_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-dpponz-0 material tree contains renderwhite_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    ceilings-fagehw-0 material tree contains renderwhite_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    spotlight-wtrjoa-10 material tree contains steellight_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    spotlight-wtrjoa-24 material tree contains steellight_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    fixed_window-cggyho-0-glass-base_link-F-lower-Tglass material tree contains steellight_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-cggyho-1-glass-base_link-F-lower-Tglass material tree contains steellight_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    ceilings-fagehw-0 material tree contains steellight_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.


## scenes/hall_train_station
  ### (.*) material tree contains .* with disallowed type .*. It should be one of .*.: 245
    fixed_window-cmuvzz-9 material tree contains rendercolour_rs_08:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-cmuvzz-40 material tree contains rendercolour_rs_08:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-zzjuyg-0 material tree contains rendercolour_rs_08:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-bhpjmj-0-Tglass material tree contains rendercolour_rs_08:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-npahfr-0-Tglass material tree contains 10 - Default:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-cmuvzz-7 material tree contains rendercolour_rs_08:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-cmuvzz-38 material tree contains rendercolour_rs_08:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-zzjuyg-0 material tree contains rendercolour_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-bhpjmj-0-Tglass material tree contains rendercolour_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-npahfr-0-Tglass material tree contains rendercolour_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-cmuvzz-5 material tree contains rendercolour_rs_08:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-cmuvzz-36 material tree contains rendercolour_rs_08:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-zzjuyg-0 material tree contains 10 - Default:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-bhpjmj-0-Tglass material tree contains 10 - Default:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-npahfr-0-Tglass material tree contains concrete_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-cmuvzz-3 material tree contains rendercolour_rs_08:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-cmuvzz-34 material tree contains rendercolour_rs_08:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-zzjuyg-0 material tree contains rendercolour_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-bhpjmj-0-Tglass material tree contains rendercolour_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-cmuvzz-1 material tree contains rendercolour_rs_08:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-cmuvzz-32 material tree contains rendercolour_rs_08:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-zzjuyg-0 material tree contains concrete_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-mhzijm-0-Tglass material tree contains vray_03 - Default:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-vsifsu-0 material tree contains rendercolour_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-cmuvzz-30 material tree contains rendercolour_rs_08:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-wzhlgy-0 material tree contains rendercolour_rs_08:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-mhzijm-0-Tglass material tree contains rendercolour_rs_08:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-vsifsu-0 material tree contains rendercolour_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-cmuvzz-28 material tree contains rendercolour_rs_08:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-wzhlgy-0 material tree contains rendercolour_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-mhzijm-0-Tglass material tree contains rendercolour_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-cmuvzz-0 material tree contains rendercolour_rs_08:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-cmuvzz-26 material tree contains rendercolour_rs_08:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-wzhlgy-0 material tree contains vray_03 - Default:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-mhzijm-0-Tglass material tree contains 10 - Default:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-hkbszu-0 material tree contains rendercolour_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-cmuvzz-24 material tree contains rendercolour_rs_08:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-wzhlgy-0 material tree contains rendercolour_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-mhzijm-0-Tglass material tree contains rendercolour_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-hkbszu-0 material tree contains rendercolour_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-cmuvzz-22 material tree contains rendercolour_rs_08:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-wzhlgy-0 material tree contains concrete_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-ckmffa-0-Tglass material tree contains vray_03 - Default:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    ceilings-emjfnt-0 material tree contains sail_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-cmuvzz-20 material tree contains rendercolour_rs_08:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-accxms-0-Tglass material tree contains vray_03 - Default:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-ckmffa-0-Tglass material tree contains rendercolour_rs_08:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    ceilings-emjfnt-0 material tree contains 08 - Default:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-cmuvzz-18 material tree contains rendercolour_rs_08:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-accxms-0-Tglass material tree contains rendercolour_rs_08:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-ckmffa-0-Tglass material tree contains rendercolour_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-rfqpbd-0 material tree contains concrete_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    fixed_window-cmuvzz-15 material tree contains rendercolour_rs_08:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-accxms-0-Tglass material tree contains rendercolour_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-ckmffa-0-Tglass material tree contains 10 - Default:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-cmuvzz-13 material tree contains rendercolour_rs_08:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-accxms-0-Tglass material tree contains 10 - Default:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-ckmffa-0-Tglass material tree contains rendercolour_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-cmuvzz-12 material tree contains rendercolour_rs_08:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-accxms-0-Tglass material tree contains rendercolour_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-zyvndj-0-Tglass material tree contains vray_03 - Default:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-cmuvzz-17 material tree contains rendercolour_rs_08:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-xrozyp-0-Tglass material tree contains vray_03 - Default:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-zyvndj-0-Tglass material tree contains rendercolour_rs_08:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-cmuvzz-8 material tree contains rendercolour_rs_08:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-cmuvzz-39 material tree contains rendercolour_rs_08:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-xrozyp-0-Tglass material tree contains rendercolour_rs_08:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-zyvndj-0-Tglass material tree contains rendercolour_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-cmuvzz-6 material tree contains rendercolour_rs_08:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-cmuvzz-37 material tree contains rendercolour_rs_08:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-xrozyp-0-Tglass material tree contains rendercolour_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-zyvndj-0-Tglass material tree contains 10 - Default:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-cmuvzz-4 material tree contains rendercolour_rs_08:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-cmuvzz-35 material tree contains rendercolour_rs_08:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-xrozyp-0-Tglass material tree contains 10 - Default:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-zyvndj-0-Tglass material tree contains rendercolour_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-cmuvzz-2 material tree contains rendercolour_rs_08:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-cmuvzz-33 material tree contains rendercolour_rs_08:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-xrozyp-0-Tglass material tree contains rendercolour_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-saldxe-0-Tglass material tree contains vray_03 - Default:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-vqbgzc-0 material tree contains rendercolour_rs_08:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-cmuvzz-31 material tree contains rendercolour_rs_08:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-foyiqv-0-Tglass material tree contains vray_03 - Default:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-saldxe-0-Tglass material tree contains rendercolour_rs_08:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-vsifsu-0 material tree contains vray_03 - Default:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-cmuvzz-29 material tree contains rendercolour_rs_08:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-foyiqv-0-Tglass material tree contains rendercolour_rs_08:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-saldxe-0-Tglass material tree contains rendercolour_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-asrqdf-0 material tree contains rendercolour_rs_08:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-cmuvzz-27 material tree contains rendercolour_rs_08:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-foyiqv-0-Tglass material tree contains rendercolour_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-saldxe-0-Tglass material tree contains 10 - Default:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-hkbszu-0 material tree contains concrete_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-cmuvzz-25 material tree contains rendercolour_rs_08:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-foyiqv-0-Tglass material tree contains 10 - Default:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-saldxe-0-Tglass material tree contains rendercolour_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-hkbszu-0 material tree contains vray_03 - Default:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-cmuvzz-23 material tree contains rendercolour_rs_08:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-foyiqv-0-Tglass material tree contains rendercolour_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-ycspus-0-Tglass material tree contains vray_03 - Default:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-hkbszu-0 material tree contains rendercolour_rs_08:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-cmuvzz-21 material tree contains rendercolour_rs_08:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-kknxpz-0-Tglass material tree contains vray_03 - Default:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-ycspus-0-Tglass material tree contains rendercolour_rs_08:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    ceilings-emjfnt-0 material tree contains 08 - Default:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-cmuvzz-19 material tree contains rendercolour_rs_08:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-kknxpz-0-Tglass material tree contains rendercolour_rs_08:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-ycspus-0-Tglass material tree contains rendercolour_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    ceilings-emjfnt-0 material tree contains steellight_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-cmuvzz-16 material tree contains rendercolour_rs_08:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-kknxpz-0-Tglass material tree contains rendercolour_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-ycspus-0-Tglass material tree contains 10 - Default:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-cmuvzz-14 material tree contains rendercolour_rs_08:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-kknxpz-0-Tglass material tree contains 10 - Default:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-ycspus-0-Tglass material tree contains rendercolour_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-cmuvzz-11 material tree contains rendercolour_rs_08:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-kknxpz-0-Tglass material tree contains rendercolour_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-cdjsgw-0-Tglass material tree contains vray_03 - Default:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-cmuvzz-10 material tree contains rendercolour_rs_08:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-bhpjmj-0-Tglass material tree contains vray_03 - Default:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-cdjsgw-0-Tglass material tree contains rendercolour_rs_08:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-vsifsu-3 material tree contains rendercolour_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-vsifsu-14 material tree contains rendercolour_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-zzjuyg-0 material tree contains vray_03 - Default:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-cdjsgw-0-Tglass material tree contains rendercolour_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-vsifsu-3 material tree contains rendercolour_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-vsifsu-13 material tree contains rendercolour_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-njrmbo-0 material tree contains rendercolour_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-cdjsgw-0-Tglass material tree contains 10 - Default:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-vsifsu-2 material tree contains vray_03 - Default:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-vsifsu-13 material tree contains vray_03 - Default:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-njrmbo-0 material tree contains rendercolour_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-cdjsgw-0-Tglass material tree contains rendercolour_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-vsifsu-1 material tree contains rendercolour_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-vsifsu-13 material tree contains rendercolour_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-njrmbo-0 material tree contains vray_03 - Default:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-ieyjnv-0-Tglass material tree contains vray_03 - Default:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-vsifsu-1 material tree contains rendercolour_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-vsifsu-12 material tree contains rendercolour_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-vsifsu-21 material tree contains rendercolour_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-ieyjnv-0-Tglass material tree contains rendercolour_rs_08:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-asrqdf-20 material tree contains rendercolour_rs_08:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-vsifsu-12 material tree contains vray_03 - Default:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-vsifsu-21 material tree contains rendercolour_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-ieyjnv-0-Tglass material tree contains rendercolour_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-asrqdf-18 material tree contains rendercolour_rs_08:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-vsifsu-12 material tree contains rendercolour_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-vsifsu-20 material tree contains vray_03 - Default:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-ieyjnv-0-Tglass material tree contains 10 - Default:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-asrqdf-16 material tree contains rendercolour_rs_08:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-vsifsu-11 material tree contains rendercolour_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-vsifsu-19 material tree contains rendercolour_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-ieyjnv-0-Tglass material tree contains rendercolour_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-asrqdf-14 material tree contains rendercolour_rs_08:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-vsifsu-11 material tree contains vray_03 - Default:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-vsifsu-19 material tree contains rendercolour_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-ywdwya-0-Tglass material tree contains vray_03 - Default:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-asrqdf-12 material tree contains rendercolour_rs_08:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-vsifsu-10 material tree contains rendercolour_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-vsifsu-18 material tree contains vray_03 - Default:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-ywdwya-0-Tglass material tree contains rendercolour_rs_08:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-asrqdf-10 material tree contains rendercolour_rs_08:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-vsifsu-10 material tree contains vray_03 - Default:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-vsifsu-17 material tree contains rendercolour_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-ywdwya-0-Tglass material tree contains rendercolour_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-asrqdf-8 material tree contains rendercolour_rs_08:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-vsifsu-10 material tree contains rendercolour_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-vsifsu-17 material tree contains rendercolour_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-ywdwya-0-Tglass material tree contains 10 - Default:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-asrqdf-6 material tree contains rendercolour_rs_08:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-vsifsu-9 material tree contains rendercolour_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-vsifsu-16 material tree contains vray_03 - Default:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-ywdwya-0-Tglass material tree contains rendercolour_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-asrqdf-4 material tree contains rendercolour_rs_08:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-vsifsu-9 material tree contains vray_03 - Default:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-vsifsu-15 material tree contains rendercolour_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-vfpjnh-0-Tglass material tree contains vray_03 - Default:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-asrqdf-2 material tree contains rendercolour_rs_08:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-vsifsu-9 material tree contains rendercolour_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-vsifsu-15 material tree contains rendercolour_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-vfpjnh-0-Tglass material tree contains rendercolour_rs_08:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-cmuvzz-41 material tree contains rendercolour_rs_08:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-vsifsu-11 material tree contains rendercolour_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-vsifsu-14 material tree contains vray_03 - Default:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-vfpjnh-0-Tglass material tree contains rendercolour_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-vsifsu-3 material tree contains vray_03 - Default:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-vsifsu-8 material tree contains rendercolour_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-njrmbo-0 material tree contains concrete_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-vfpjnh-0-Tglass material tree contains 10 - Default:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-vsifsu-2 material tree contains rendercolour_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-vsifsu-8 material tree contains vray_03 - Default:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-njrmbo-0 material tree contains 10 - Default:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-vfpjnh-0-Tglass material tree contains rendercolour_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-vsifsu-2 material tree contains rendercolour_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-vsifsu-8 material tree contains rendercolour_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-njrmbo-0 material tree contains rendercolour_rs_08:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    ceilings-srtoah-0-Tglass material tree contains steellight_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-vsifsu-1 material tree contains vray_03 - Default:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-vsifsu-7 material tree contains rendercolour_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-vqbgzc-1 material tree contains rendercolour_rs_08:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    ceilings-srtoah-0-Tglass material tree contains 08 - Default:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-asrqdf-21 material tree contains rendercolour_rs_08:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-vsifsu-7 material tree contains vray_03 - Default:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-vsifsu-21 material tree contains vray_03 - Default:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    ceilings-srtoah-0-Tglass material tree contains 08 - Default:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-asrqdf-19 material tree contains rendercolour_rs_08:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-vsifsu-7 material tree contains rendercolour_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-vsifsu-20 material tree contains rendercolour_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    ceilings-srtoah-0-Tglass material tree contains sail_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-asrqdf-17 material tree contains rendercolour_rs_08:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-vsifsu-6 material tree contains rendercolour_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-vsifsu-20 material tree contains rendercolour_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-whbgzw-0-Tglass material tree contains vray_03 - Default:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-asrqdf-15 material tree contains rendercolour_rs_08:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-vsifsu-6 material tree contains vray_03 - Default:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-vsifsu-19 material tree contains vray_03 - Default:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-whbgzw-0-Tglass material tree contains rendercolour_rs_08:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-asrqdf-13 material tree contains rendercolour_rs_08:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-vsifsu-6 material tree contains rendercolour_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-vsifsu-18 material tree contains rendercolour_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-whbgzw-0-Tglass material tree contains rendercolour_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-asrqdf-11 material tree contains rendercolour_rs_08:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-vsifsu-5 material tree contains rendercolour_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-vsifsu-18 material tree contains rendercolour_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-whbgzw-0-Tglass material tree contains 10 - Default:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-asrqdf-9 material tree contains rendercolour_rs_08:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-vsifsu-5 material tree contains vray_03 - Default:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-vsifsu-17 material tree contains vray_03 - Default:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-whbgzw-0-Tglass material tree contains rendercolour_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-asrqdf-7 material tree contains rendercolour_rs_08:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-vsifsu-5 material tree contains rendercolour_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-vsifsu-16 material tree contains rendercolour_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-whbgzw-0-Tglass material tree contains concrete_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-asrqdf-5 material tree contains rendercolour_rs_08:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-vsifsu-4 material tree contains rendercolour_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-vsifsu-16 material tree contains rendercolour_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-npahfr-0-Tglass material tree contains vray_03 - Default:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-asrqdf-3 material tree contains rendercolour_rs_08:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-vsifsu-4 material tree contains vray_03 - Default:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-vsifsu-15 material tree contains vray_03 - Default:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-npahfr-0-Tglass material tree contains rendercolour_rs_08:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-asrqdf-1 material tree contains rendercolour_rs_08:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-vsifsu-4 material tree contains rendercolour_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-vsifsu-14 material tree contains rendercolour_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-npahfr-0-Tglass material tree contains rendercolour_rs_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.


## scenes/hotel_gym_spa
  ### (.*) material tree contains .* with disallowed type .*. It should be one of .*.: 739
    downlight-vtamah-12 material tree contains spa_room_147:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-mxpwbb-0 material tree contains spa_room_147:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-kqskfv-0 material tree contains spa_room_147:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-vtamah-13 material tree contains Standard_16fsd:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-cqizpg-0 material tree contains Standard_16fsd:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-znkqca-0 material tree contains Standard_16fsd:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-vtamah-13 material tree contains Standard_16fsd:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-cqizpg-0 material tree contains Standard_16fsd:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-znkqca-0 material tree contains Standard_16fsd:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    hot_tub-nuzkuf-0 material tree contains spa_room_007:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-vtamah-13 material tree contains spa_room_147:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-cqizpg-0 material tree contains spa_room_009:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-znkqca-0 material tree contains spa_room_009:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-adwcsx-0 material tree contains spa_room_072:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-vtamah-14 material tree contains Standard_16fsd:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-cqizpg-0 material tree contains spa_room_178:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-znkqca-0 material tree contains spa_room_178:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-adwcsx-2 material tree contains spa_room_072:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-vtamah-14 material tree contains Standard_16fsd:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-cqizpg-0 material tree contains spa_room_147:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-znkqca-0 material tree contains spa_room_147:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-swimming_pool-kohria-0 material tree contains spa_room_096:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-vtamah-14 material tree contains spa_room_147:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-lnikgo-0 material tree contains Standard_16fsd:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-xidufr-0 material tree contains Standard_16fsd:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-swimming_pool-kohria-0 material tree contains spa_room_095:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-vtamah-15 material tree contains Standard_16fsd:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-lnikgo-0 material tree contains Standard_16fsd:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-xidufr-0 material tree contains Standard_16fsd:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-qdlnea-0 material tree contains spa_room_009:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-vtamah-15 material tree contains Standard_16fsd:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-lnikgo-0 material tree contains spa_room_009:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-xidufr-0 material tree contains spa_room_009:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-qdlnea-0 material tree contains gym_space_023:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-vtamah-15 material tree contains spa_room_147:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-lnikgo-0 material tree contains spa_room_178:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-xidufr-0 material tree contains spa_room_178:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-qdlnea-0 material tree contains gym_space_026:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-vtamah-16 material tree contains Standard_16fsd:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-lnikgo-0 material tree contains spa_room_147:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-xidufr-0 material tree contains spa_room_147:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    swimming_pool-qjhauf-0 material tree contains spa_room_158:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-vtamah-16 material tree contains Standard_16fsd:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-gknbia-0 material tree contains Standard_16fsd:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-yltaoz-0 material tree contains Standard_16fsd:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    swimming_pool-qjhauf-0 material tree contains spa_room_157:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-vtamah-16 material tree contains spa_room_147:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-gknbia-0 material tree contains Standard_16fsd:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-yltaoz-0 material tree contains Standard_16fsd:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    hot_tub-wbnkfk-0 material tree contains spa_room_007:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-vtamah-17 material tree contains Standard_16fsd:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-gknbia-0 material tree contains spa_room_009:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-yltaoz-0 material tree contains spa_room_009:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-kpobtc-2 material tree contains spa_room_138:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-vtamah-17 material tree contains Standard_16fsd:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-gknbia-0 material tree contains spa_room_178:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-yltaoz-0 material tree contains spa_room_178:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-kpobtc-2 material tree contains spa_room_137:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-vtamah-17 material tree contains spa_room_147:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-gknbia-0 material tree contains spa_room_147:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-yltaoz-0 material tree contains spa_room_147:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-vase-euqzpy-0 material tree contains stuff_026:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    downlight-vtamah-18 material tree contains Standard_16fsd:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-tftqjv-0 material tree contains Standard_16fsd:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-bqjvho-0 material tree contains Standard_16fsd:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-49 material tree contains spa_room_158:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-vtamah-18 material tree contains Standard_16fsd:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-tftqjv-0 material tree contains Standard_16fsd:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-bqjvho-0 material tree contains Standard_16fsd:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-49 material tree contains spa_room_157:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-vtamah-18 material tree contains spa_room_147:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-tftqjv-0 material tree contains spa_room_009:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-bqjvho-0 material tree contains spa_room_009:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-48 material tree contains spa_room_158:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-kpobtc-0 material tree contains spa_room_138:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-tftqjv-0 material tree contains spa_room_178:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-bqjvho-0 material tree contains spa_room_178:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-48 material tree contains spa_room_157:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-kpobtc-0 material tree contains spa_room_137:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-tftqjv-0 material tree contains spa_room_147:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-bqjvho-0 material tree contains spa_room_147:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pillar_candle-iotqru-7 material tree contains spa_room_141:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-kpobtc-1 material tree contains spa_room_138:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-fzfeof-0 material tree contains Standard_16fsd:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-flehto-0 material tree contains Standard_16fsd:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pillar_candle-iotqru-7 material tree contains spa_room_140:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-kpobtc-1 material tree contains spa_room_137:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-fzfeof-0 material tree contains Standard_16fsd:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-flehto-0 material tree contains Standard_16fsd:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pillar_candle-iotqru-7 material tree contains spa_room_162:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-65 material tree contains spa_room_158:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-fzfeof-0 material tree contains spa_room_009:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-flehto-0 material tree contains spa_room_009:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pillar_candle-iotqru-5 material tree contains spa_room_141:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-65 material tree contains spa_room_157:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-fzfeof-0 material tree contains spa_room_178:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-flehto-0 material tree contains spa_room_178:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pillar_candle-iotqru-5 material tree contains spa_room_140:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-66 material tree contains spa_room_158:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-fzfeof-0 material tree contains spa_room_147:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-flehto-0 material tree contains spa_room_147:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pillar_candle-iotqru-5 material tree contains spa_room_162:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-66 material tree contains spa_room_157:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-scegoa-0 material tree contains Standard_16fsd:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-uqkubr-0 material tree contains Standard_16fsd:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pillar_candle-iotqru-6 material tree contains spa_room_141:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-67 material tree contains spa_room_158:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-scegoa-0 material tree contains Standard_16fsd:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-uqkubr-0 material tree contains Standard_16fsd:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pillar_candle-iotqru-6 material tree contains spa_room_140:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-67 material tree contains spa_room_157:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-scegoa-0 material tree contains spa_room_009:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-uqkubr-0 material tree contains spa_room_009:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pillar_candle-iotqru-6 material tree contains spa_room_162:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-68 material tree contains spa_room_158:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-scegoa-0 material tree contains spa_room_178:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-uqkubr-0 material tree contains spa_room_178:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-47 material tree contains spa_room_158:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-68 material tree contains spa_room_157:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-scegoa-0 material tree contains spa_room_147:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-uqkubr-0 material tree contains spa_room_147:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-47 material tree contains spa_room_157:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-69 material tree contains spa_room_158:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-zrojoi-0 material tree contains Standard_16fsd:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-46 material tree contains spa_room_158:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-69 material tree contains spa_room_157:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-zrojoi-0 material tree contains Standard_16fsd:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-46 material tree contains spa_room_157:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-70 material tree contains spa_room_158:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-zrojoi-0 material tree contains spa_room_009:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-45 material tree contains spa_room_158:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-70 material tree contains spa_room_157:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-zrojoi-0 material tree contains spa_room_178:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-45 material tree contains spa_room_157:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-71 material tree contains spa_room_158:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-zrojoi-0 material tree contains spa_room_147:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-44 material tree contains spa_room_158:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-71 material tree contains spa_room_157:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-kjruwg-0 material tree contains Standard_16fsd:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-44 material tree contains spa_room_157:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-72 material tree contains spa_room_158:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-kjruwg-0 material tree contains Standard_16fsd:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-43 material tree contains spa_room_158:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-72 material tree contains spa_room_157:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-kjruwg-0 material tree contains spa_room_009:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-43 material tree contains spa_room_157:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-73 material tree contains spa_room_158:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-kjruwg-0 material tree contains spa_room_178:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-42 material tree contains spa_room_158:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-73 material tree contains spa_room_157:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-kjruwg-0 material tree contains spa_room_147:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-42 material tree contains spa_room_157:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-74 material tree contains spa_room_158:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-irnnhp-0 material tree contains Standard_16fsd:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-41 material tree contains spa_room_158:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-74 material tree contains spa_room_157:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-irnnhp-0 material tree contains Standard_16fsd:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-41 material tree contains spa_room_157:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-75 material tree contains spa_room_158:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-irnnhp-0 material tree contains spa_room_009:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-40 material tree contains spa_room_158:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-75 material tree contains spa_room_157:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-irnnhp-0 material tree contains spa_room_178:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-40 material tree contains spa_room_157:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-76 material tree contains spa_room_158:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-irnnhp-0 material tree contains spa_room_147:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-39 material tree contains spa_room_158:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-76 material tree contains spa_room_157:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-mlwwng-0 material tree contains Standard_16fsd:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-39 material tree contains spa_room_157:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-77 material tree contains spa_room_158:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-mlwwng-0 material tree contains Standard_16fsd:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-38 material tree contains spa_room_158:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-77 material tree contains spa_room_157:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-mlwwng-0 material tree contains spa_room_009:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-38 material tree contains spa_room_157:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-78 material tree contains spa_room_158:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-mlwwng-0 material tree contains spa_room_178:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-37 material tree contains spa_room_158:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-78 material tree contains spa_room_157:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-mlwwng-0 material tree contains spa_room_147:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-37 material tree contains spa_room_157:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-79 material tree contains spa_room_158:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-dtdcvu-0 material tree contains Standard_16fsd:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-36 material tree contains spa_room_158:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-79 material tree contains spa_room_157:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-dtdcvu-0 material tree contains Standard_16fsd:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-36 material tree contains spa_room_157:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-80 material tree contains spa_room_158:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-dtdcvu-0 material tree contains spa_room_009:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-35 material tree contains spa_room_158:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-80 material tree contains spa_room_157:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-dtdcvu-0 material tree contains spa_room_178:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-35 material tree contains spa_room_157:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-81 material tree contains spa_room_158:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-dtdcvu-0 material tree contains spa_room_147:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-34 material tree contains spa_room_158:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-81 material tree contains spa_room_157:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-aljtes-0 material tree contains Standard_16fsd:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-34 material tree contains spa_room_157:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-82 material tree contains spa_room_158:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-aljtes-0 material tree contains Standard_16fsd:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-33 material tree contains spa_room_158:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-82 material tree contains spa_room_157:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-aljtes-0 material tree contains spa_room_009:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-33 material tree contains spa_room_157:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-83 material tree contains spa_room_158:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-aljtes-0 material tree contains spa_room_178:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-32 material tree contains spa_room_158:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-83 material tree contains spa_room_157:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-aljtes-0 material tree contains spa_room_147:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-32 material tree contains spa_room_157:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-84 material tree contains spa_room_158:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-gnfmsd-0 material tree contains spa_room_009:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-31 material tree contains spa_room_158:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-84 material tree contains spa_room_157:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-rktrmr-0 material tree contains Standard_16fsd:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-gnfmsd-0 material tree contains gym_space_023:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-31 material tree contains spa_room_157:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-85 material tree contains spa_room_158:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-rktrmr-0 material tree contains Standard_16fsd:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-gnfmsd-0 material tree contains gym_space_026:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-30 material tree contains spa_room_158:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-85 material tree contains spa_room_157:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-rktrmr-0 material tree contains spa_room_009:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-zpadmw-0 material tree contains spa_room_009:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-30 material tree contains spa_room_157:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-86 material tree contains spa_room_158:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-rktrmr-0 material tree contains spa_room_178:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-zpadmw-0 material tree contains gym_space_023:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-29 material tree contains spa_room_158:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-86 material tree contains spa_room_157:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-rktrmr-0 material tree contains spa_room_147:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-zpadmw-0 material tree contains gym_space_026:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-29 material tree contains spa_room_157:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-87 material tree contains spa_room_158:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-yupwjg-0 material tree contains Standard_16fsd:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-glwcho-0 material tree contains spa_room_009:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-28 material tree contains spa_room_158:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-87 material tree contains spa_room_157:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-yupwjg-0 material tree contains Standard_16fsd:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-glwcho-0 material tree contains gym_space_023:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-28 material tree contains spa_room_157:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-88 material tree contains spa_room_158:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-yupwjg-0 material tree contains spa_room_009:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-glwcho-0 material tree contains gym_space_026:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    spotlight-xwotqa-0 material tree contains spa_room_170:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-88 material tree contains spa_room_157:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-yupwjg-0 material tree contains spa_room_178:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-oxhkpu-0 material tree contains spa_room_009:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    spotlight-xwotqa-1 material tree contains spa_room_170:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-89 material tree contains spa_room_158:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-yupwjg-0 material tree contains spa_room_147:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-oxhkpu-0 material tree contains gym_space_023:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    spotlight-xwotqa-2 material tree contains spa_room_170:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-89 material tree contains spa_room_157:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-mqmejb-0 material tree contains Standard_16fsd:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-oxhkpu-0 material tree contains gym_space_026:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pillar_candle-iotqru-4 material tree contains spa_room_141:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pillar_candle-qevocc-3 material tree contains spa_room_139:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-mqmejb-0 material tree contains Standard_16fsd:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-xawgsl-0 material tree contains spa_room_009:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pillar_candle-iotqru-4 material tree contains spa_room_140:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pillar_candle-qevocc-3 material tree contains spa_room_141:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-mqmejb-0 material tree contains spa_room_009:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-xawgsl-0 material tree contains gym_space_023:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pillar_candle-iotqru-4 material tree contains spa_room_162:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pillar_candle-qevocc-3 material tree contains spa_room_140:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-mqmejb-0 material tree contains spa_room_178:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-xawgsl-0 material tree contains gym_space_026:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-adwcsx-1 material tree contains spa_room_072:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    shower_stall-vcwkma-0-base_link material tree contains spa_room_147:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-mqmejb-0 material tree contains spa_room_147:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-buymjp-0 material tree contains spa_room_009:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-27 material tree contains spa_room_158:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    shower_stall-vcwkma-1-base_link material tree contains spa_room_147:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-ibczvg-0 material tree contains Standard_16fsd:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-buymjp-0 material tree contains gym_space_023:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-27 material tree contains spa_room_157:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-dkoolt-0 material tree contains Standard_16fsd:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-ibczvg-0 material tree contains Standard_16fsd:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-buymjp-0 material tree contains gym_space_026:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-26 material tree contains spa_room_158:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-dkoolt-0 material tree contains Standard_16fsd:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-ibczvg-0 material tree contains spa_room_009:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-ghutyr-0 material tree contains spa_room_009:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-26 material tree contains spa_room_157:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-dkoolt-0 material tree contains spa_room_009:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-ibczvg-0 material tree contains spa_room_178:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-ghutyr-0 material tree contains gym_space_023:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-25 material tree contains spa_room_158:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-dkoolt-0 material tree contains spa_room_178:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-ibczvg-0 material tree contains spa_room_147:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-ghutyr-0 material tree contains gym_space_026:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-25 material tree contains spa_room_157:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-dkoolt-0 material tree contains spa_room_147:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-avabgo-0 material tree contains Standard_16fsd:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-rphmwm-0 material tree contains spa_room_009:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-24 material tree contains spa_room_158:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-ykgvny-0 material tree contains Standard_16fsd:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-avabgo-0 material tree contains Standard_16fsd:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-rphmwm-0 material tree contains gym_space_023:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-24 material tree contains spa_room_157:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-ykgvny-0 material tree contains Standard_16fsd:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-avabgo-0 material tree contains spa_room_009:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-rphmwm-0 material tree contains gym_space_026:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-23 material tree contains spa_room_158:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-ykgvny-0 material tree contains spa_room_009:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-avabgo-0 material tree contains spa_room_178:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-90 material tree contains spa_room_158:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-23 material tree contains spa_room_157:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-ykgvny-0 material tree contains spa_room_178:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-avabgo-0 material tree contains spa_room_147:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-90 material tree contains spa_room_157:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-22 material tree contains spa_room_158:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-ykgvny-0 material tree contains spa_room_147:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-nmekjn-0 material tree contains Standard_16fsd:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-91 material tree contains spa_room_158:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-22 material tree contains spa_room_157:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-levbrm-0 material tree contains Standard_16fsd:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-nmekjn-0 material tree contains Standard_16fsd:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-91 material tree contains spa_room_157:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-21 material tree contains spa_room_158:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-levbrm-0 material tree contains Standard_16fsd:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-nmekjn-0 material tree contains spa_room_009:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-92 material tree contains spa_room_158:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-21 material tree contains spa_room_157:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-levbrm-0 material tree contains spa_room_009:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-nmekjn-0 material tree contains spa_room_178:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-92 material tree contains spa_room_157:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-20 material tree contains spa_room_158:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-levbrm-0 material tree contains spa_room_178:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-nmekjn-0 material tree contains spa_room_147:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-20 material tree contains spa_room_157:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-levbrm-0 material tree contains spa_room_147:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-cxmuxl-0 material tree contains Standard_16fsd:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-19 material tree contains spa_room_158:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pillar_candle-aqryfd-0 material tree contains spa_room_139:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-cxmuxl-0 material tree contains Standard_16fsd:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-19 material tree contains spa_room_157:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pillar_candle-aqryfd-0 material tree contains spa_room_141:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-cxmuxl-0 material tree contains spa_room_009:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-18 material tree contains spa_room_158:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pillar_candle-aqryfd-0 material tree contains spa_room_140:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-cxmuxl-0 material tree contains spa_room_178:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-18 material tree contains spa_room_157:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pillar_candle-acycbi-1 material tree contains spa_room_139:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-cxmuxl-0 material tree contains spa_room_147:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-17 material tree contains spa_room_158:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pillar_candle-acycbi-1 material tree contains spa_room_141:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-drwyhj-0 material tree contains Standard_16fsd:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-17 material tree contains spa_room_157:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pillar_candle-acycbi-1 material tree contains spa_room_140:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-drwyhj-0 material tree contains Standard_16fsd:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-16 material tree contains spa_room_158:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pillar_candle-acycbi-2 material tree contains spa_room_139:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-drwyhj-0 material tree contains spa_room_009:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-16 material tree contains spa_room_157:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pillar_candle-acycbi-2 material tree contains spa_room_141:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-drwyhj-0 material tree contains spa_room_178:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-15 material tree contains spa_room_158:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pillar_candle-acycbi-2 material tree contains spa_room_140:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-drwyhj-0 material tree contains spa_room_147:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-15 material tree contains spa_room_157:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pillar_candle-acycbi-3 material tree contains spa_room_139:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-iivarc-0 material tree contains Standard_16fsd:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-14 material tree contains spa_room_158:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pillar_candle-acycbi-3 material tree contains spa_room_141:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-iivarc-0 material tree contains Standard_16fsd:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-14 material tree contains spa_room_157:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pillar_candle-acycbi-3 material tree contains spa_room_140:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-iivarc-0 material tree contains spa_room_009:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-13 material tree contains spa_room_158:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pillar_candle-qevocc-1 material tree contains spa_room_139:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-iivarc-0 material tree contains spa_room_178:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-13 material tree contains spa_room_157:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pillar_candle-qevocc-1 material tree contains spa_room_141:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-iivarc-0 material tree contains spa_room_147:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-12 material tree contains spa_room_158:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pillar_candle-qevocc-1 material tree contains spa_room_140:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-likzkw-0 material tree contains Standard_16fsd:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-12 material tree contains spa_room_157:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pillar_candle-qevocc-2 material tree contains spa_room_139:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-likzkw-0 material tree contains Standard_16fsd:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-11 material tree contains spa_room_158:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pillar_candle-qevocc-2 material tree contains spa_room_141:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-likzkw-0 material tree contains spa_room_009:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-11 material tree contains spa_room_157:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pillar_candle-qevocc-2 material tree contains spa_room_140:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-likzkw-0 material tree contains spa_room_178:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-10 material tree contains spa_room_158:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pillar_candle-qevocc-4 material tree contains spa_room_139:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-likzkw-0 material tree contains spa_room_147:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-10 material tree contains spa_room_157:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pillar_candle-qevocc-4 material tree contains spa_room_141:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-kkibut-0 material tree contains Standard_16fsd:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-56 material tree contains spa_room_158:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pillar_candle-qevocc-4 material tree contains spa_room_140:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-kkibut-0 material tree contains Standard_16fsd:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-56 material tree contains spa_room_157:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pillar_candle-qevocc-5 material tree contains spa_room_139:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-kkibut-0 material tree contains spa_room_009:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-63 material tree contains spa_room_158:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pillar_candle-qevocc-5 material tree contains spa_room_141:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-kkibut-0 material tree contains spa_room_178:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-63 material tree contains spa_room_157:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pillar_candle-qevocc-5 material tree contains spa_room_140:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-kkibut-0 material tree contains spa_room_147:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-62 material tree contains spa_room_158:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pillar_candle-qevocc-6 material tree contains spa_room_139:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-yeozsn-0 material tree contains Standard_16fsd:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-62 material tree contains spa_room_157:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pillar_candle-qevocc-6 material tree contains spa_room_141:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-yeozsn-0 material tree contains Standard_16fsd:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-64 material tree contains spa_room_158:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pillar_candle-qevocc-6 material tree contains spa_room_140:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-yeozsn-0 material tree contains spa_room_009:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-64 material tree contains spa_room_157:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-vtamah-1 material tree contains Standard_16fsd:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-yeozsn-0 material tree contains spa_room_178:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-57 material tree contains spa_room_158:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-vtamah-1 material tree contains Standard_16fsd:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-yeozsn-0 material tree contains spa_room_147:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-57 material tree contains spa_room_157:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-vtamah-1 material tree contains spa_room_147:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-pitgiq-0 material tree contains Standard_16fsd:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-58 material tree contains spa_room_158:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-vtamah-2 material tree contains Standard_16fsd:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-pitgiq-0 material tree contains Standard_16fsd:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-58 material tree contains spa_room_157:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-vmmqus-0 material tree contains Standard_16fsd:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-pitgiq-0 material tree contains spa_room_009:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-59 material tree contains spa_room_158:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-vmmqus-0 material tree contains Standard_16fsd:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-pitgiq-0 material tree contains spa_room_178:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-59 material tree contains spa_room_157:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-vmmqus-0 material tree contains spa_room_009:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-pitgiq-0 material tree contains spa_room_147:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-60 material tree contains spa_room_158:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-vmmqus-0 material tree contains spa_room_178:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-ibhpug-0 material tree contains Standard_16fsd:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-60 material tree contains spa_room_157:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-vmmqus-0 material tree contains spa_room_147:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-ibhpug-0 material tree contains Standard_16fsd:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-61 material tree contains spa_room_158:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-rnwjlm-0 material tree contains Standard_16fsd:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-ibhpug-0 material tree contains spa_room_009:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-61 material tree contains spa_room_157:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-rnwjlm-0 material tree contains Standard_16fsd:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-ibhpug-0 material tree contains spa_room_178:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pillar_candle-iotqru-3 material tree contains spa_room_141:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-rnwjlm-0 material tree contains spa_room_009:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-ibhpug-0 material tree contains spa_room_147:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pillar_candle-iotqru-3 material tree contains spa_room_140:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-rnwjlm-0 material tree contains spa_room_178:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-vbdlka-0 material tree contains Standard_16fsd:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pillar_candle-iotqru-3 material tree contains spa_room_162:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-rnwjlm-0 material tree contains spa_room_147:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-vbdlka-0 material tree contains Standard_16fsd:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pillar_candle-iotqru-2 material tree contains spa_room_141:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-kbfyvb-0 material tree contains Standard_16fsd:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-vbdlka-0 material tree contains spa_room_009:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pillar_candle-iotqru-2 material tree contains spa_room_140:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-kbfyvb-0 material tree contains Standard_16fsd:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-vbdlka-0 material tree contains spa_room_178:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pillar_candle-iotqru-2 material tree contains spa_room_162:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-kbfyvb-0 material tree contains spa_room_009:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-vbdlka-0 material tree contains spa_room_147:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pillar_candle-iotqru-1 material tree contains spa_room_141:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-kbfyvb-0 material tree contains spa_room_178:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-afsofl-0 material tree contains Standard_16fsd:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pillar_candle-iotqru-1 material tree contains spa_room_140:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-kbfyvb-0 material tree contains spa_room_147:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-afsofl-0 material tree contains Standard_16fsd:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pillar_candle-iotqru-1 material tree contains spa_room_162:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-wkknqq-0 material tree contains Standard_16fsd:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-afsofl-0 material tree contains spa_room_009:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pillar_candle-iotqru-0 material tree contains spa_room_141:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-wkknqq-0 material tree contains Standard_16fsd:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-afsofl-0 material tree contains spa_room_178:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pillar_candle-iotqru-0 material tree contains spa_room_140:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-wkknqq-0 material tree contains spa_room_009:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-afsofl-0 material tree contains spa_room_147:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pillar_candle-iotqru-0 material tree contains spa_room_162:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-wkknqq-0 material tree contains spa_room_178:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-zqspqa-0 material tree contains Standard_16fsd:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    spotlight-xwotqa-3 material tree contains spa_room_170:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-wkknqq-0 material tree contains spa_room_147:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-zqspqa-0 material tree contains Standard_16fsd:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    spotlight-xwotqa-4 material tree contains spa_room_170:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-fhydgh-0 material tree contains Standard_16fsd:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-zqspqa-0 material tree contains spa_room_009:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    spotlight-xwotqa-5 material tree contains spa_room_170:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-fhydgh-0 material tree contains Standard_16fsd:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-zqspqa-0 material tree contains spa_room_178:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-ignqog-0 material tree contains Standard_16fsd:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-fhydgh-0 material tree contains spa_room_009:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-zqspqa-0 material tree contains spa_room_147:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-ignqog-0 material tree contains Standard_16fsd:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-fhydgh-0 material tree contains spa_room_178:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-cfcpxc-0 material tree contains Standard_16fsd:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-ignqog-0 material tree contains spa_room_009:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-fhydgh-0 material tree contains spa_room_147:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-cfcpxc-0 material tree contains Standard_16fsd:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-ignqog-0 material tree contains spa_room_178:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-rrsfyc-0 material tree contains Standard_16fsd:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-cfcpxc-0 material tree contains spa_room_009:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-ignqog-0 material tree contains spa_room_147:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-rrsfyc-0 material tree contains Standard_16fsd:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-cfcpxc-0 material tree contains spa_room_178:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    spotlight-gbvzvy-0 material tree contains spa_room_170:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-rrsfyc-0 material tree contains spa_room_009:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-cfcpxc-0 material tree contains spa_room_147:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    spotlight-gbvzvy-1 material tree contains spa_room_170:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-rrsfyc-0 material tree contains spa_room_178:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-pshkih-0 material tree contains Standard_16fsd:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    spotlight-gbvzvy-2 material tree contains spa_room_170:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-rrsfyc-0 material tree contains spa_room_147:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-pshkih-0 material tree contains Standard_16fsd:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    spotlight-gbvzvy-3 material tree contains spa_room_170:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-amllyu-0 material tree contains Standard_16fsd:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-pshkih-0 material tree contains spa_room_009:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    spotlight-gbvzvy-4 material tree contains spa_room_170:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-vzkdpu-0 material tree contains spa_room_009:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-pshkih-0 material tree contains spa_room_178:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    spotlight-gbvzvy-5 material tree contains spa_room_170:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-vzkdpu-0 material tree contains spa_room_178:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-pshkih-0 material tree contains spa_room_147:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    spotlight-gbvzvy-6 material tree contains spa_room_170:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-vzkdpu-0 material tree contains spa_room_147:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-zxbzcp-0 material tree contains Standard_16fsd:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    spotlight-gbvzvy-7 material tree contains spa_room_170:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-jcsihj-0 material tree contains Standard_16fsd:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-zxbzcp-0 material tree contains Standard_16fsd:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    spotlight-gbvzvy-8 material tree contains spa_room_170:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-jcsihj-0 material tree contains Standard_16fsd:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-zxbzcp-0 material tree contains spa_room_009:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    spotlight-gbvzvy-9 material tree contains spa_room_170:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-jcsihj-0 material tree contains spa_room_009:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-zxbzcp-0 material tree contains spa_room_178:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    spotlight-gbvzvy-10 material tree contains spa_room_170:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-jcsihj-0 material tree contains spa_room_178:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-zxbzcp-0 material tree contains spa_room_147:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    spotlight-gbvzvy-11 material tree contains spa_room_170:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-jcsihj-0 material tree contains spa_room_147:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-affyzw-0 material tree contains Standard_16fsd:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    spotlight-gbvzvy-12 material tree contains spa_room_170:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-ldwqlk-0 material tree contains Standard_16fsd:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-affyzw-0 material tree contains Standard_16fsd:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    spotlight-gbvzvy-13 material tree contains spa_room_170:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-ldwqlk-0 material tree contains Standard_16fsd:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-affyzw-0 material tree contains spa_room_009:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    spotlight-gbvzvy-14 material tree contains spa_room_170:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-ldwqlk-0 material tree contains spa_room_009:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-affyzw-0 material tree contains spa_room_178:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    spotlight-gbvzvy-15 material tree contains spa_room_170:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-ldwqlk-0 material tree contains spa_room_178:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-affyzw-0 material tree contains spa_room_147:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    spotlight-gbvzvy-16 material tree contains spa_room_170:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-ldwqlk-0 material tree contains spa_room_147:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-zxxxro-0 material tree contains Standard_16fsd:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    spotlight-gbvzvy-17 material tree contains spa_room_170:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-jfyqek-0 material tree contains Standard_16fsd:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-zxxxro-0 material tree contains Standard_16fsd:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    shower_stall-vcwkma-2-base_link material tree contains spa_room_147:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-jfyqek-0 material tree contains Standard_16fsd:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-zxxxro-0 material tree contains spa_room_009:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-vtamah-0 material tree contains Standard_16fsd:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-jfyqek-0 material tree contains spa_room_009:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-zxxxro-0 material tree contains spa_room_178:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-vtamah-0 material tree contains Standard_16fsd:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-jfyqek-0 material tree contains spa_room_178:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-zxxxro-0 material tree contains spa_room_147:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-vtamah-0 material tree contains spa_room_147:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-jfyqek-0 material tree contains spa_room_147:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-ooodeb-0 material tree contains Standard_16fsd:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-9 material tree contains spa_room_158:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-xvqqob-0 material tree contains Standard_16fsd:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-ooodeb-0 material tree contains Standard_16fsd:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-9 material tree contains spa_room_157:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-xvqqob-0 material tree contains Standard_16fsd:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-ooodeb-0 material tree contains spa_room_009:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-8 material tree contains spa_room_158:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-xvqqob-0 material tree contains spa_room_009:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-ooodeb-0 material tree contains spa_room_178:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-8 material tree contains spa_room_157:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-xvqqob-0 material tree contains spa_room_178:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-ooodeb-0 material tree contains spa_room_147:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-6 material tree contains spa_room_158:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-xvqqob-0 material tree contains spa_room_147:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-bxewmp-0 material tree contains Standard_16fsd:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-6 material tree contains spa_room_157:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-upcnna-0 material tree contains Standard_16fsd:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-bxewmp-0 material tree contains Standard_16fsd:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-7 material tree contains spa_room_158:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-upcnna-0 material tree contains Standard_16fsd:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-bxewmp-0 material tree contains spa_room_009:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-7 material tree contains spa_room_157:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-upcnna-0 material tree contains spa_room_009:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-bxewmp-0 material tree contains spa_room_178:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-5 material tree contains spa_room_158:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-upcnna-0 material tree contains spa_room_178:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-bxewmp-0 material tree contains spa_room_147:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-5 material tree contains spa_room_157:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-upcnna-0 material tree contains spa_room_147:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-kqskfv-0 material tree contains Standard_16fsd:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-4 material tree contains spa_room_158:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-lhyxmc-0 material tree contains Standard_16fsd:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-kqskfv-0 material tree contains Standard_16fsd:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-4 material tree contains spa_room_157:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-lhyxmc-0 material tree contains Standard_16fsd:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-kqskfv-0 material tree contains spa_room_009:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-3 material tree contains spa_room_158:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-lhyxmc-0 material tree contains spa_room_009:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-kqskfv-0 material tree contains spa_room_178:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-3 material tree contains spa_room_157:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-amllyu-0 material tree contains Standard_16fsd:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-2 material tree contains spa_room_158:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-amllyu-0 material tree contains spa_room_009:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-2 material tree contains spa_room_157:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-amllyu-0 material tree contains spa_room_178:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-1 material tree contains spa_room_158:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-amllyu-0 material tree contains spa_room_147:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-1 material tree contains spa_room_157:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-hdicxf-0 material tree contains Standard_16fsd:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-0 material tree contains spa_room_158:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-hdicxf-0 material tree contains Standard_16fsd:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-0 material tree contains spa_room_157:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-hdicxf-0 material tree contains spa_room_009:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-55 material tree contains spa_room_158:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-hdicxf-0 material tree contains spa_room_178:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-55 material tree contains spa_room_157:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-hdicxf-0 material tree contains spa_room_147:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-54 material tree contains spa_room_158:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-dswbqz-0 material tree contains Standard_16fsd:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-54 material tree contains spa_room_157:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-dswbqz-0 material tree contains Standard_16fsd:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-53 material tree contains spa_room_158:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-dswbqz-0 material tree contains spa_room_009:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-53 material tree contains spa_room_157:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-dswbqz-0 material tree contains spa_room_178:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-52 material tree contains spa_room_158:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-dswbqz-0 material tree contains spa_room_147:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-52 material tree contains spa_room_157:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-utfrrn-0 material tree contains Standard_16fsd:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-51 material tree contains spa_room_158:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-utfrrn-0 material tree contains Standard_16fsd:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-51 material tree contains spa_room_157:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-utfrrn-0 material tree contains spa_room_009:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-50 material tree contains spa_room_158:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-utfrrn-0 material tree contains spa_room_178:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hbqwek-50 material tree contains spa_room_157:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-utfrrn-0 material tree contains spa_room_147:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pillar_candle-acycbi-0 material tree contains spa_room_139:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-lbvctz-0 material tree contains Standard_16fsd:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pillar_candle-acycbi-0 material tree contains spa_room_141:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-lbvctz-0 material tree contains Standard_16fsd:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pillar_candle-acycbi-0 material tree contains spa_room_140:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-lbvctz-0 material tree contains spa_room_009:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pillar_candle-aqryfd-2 material tree contains spa_room_139:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-lbvctz-0 material tree contains spa_room_178:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pillar_candle-aqryfd-2 material tree contains spa_room_141:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-lbvctz-0 material tree contains spa_room_147:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pillar_candle-aqryfd-2 material tree contains spa_room_140:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-eltakf-0 material tree contains Standard_16fsd:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pillar_candle-qevocc-0 material tree contains spa_room_139:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-eltakf-0 material tree contains Standard_16fsd:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pillar_candle-qevocc-0 material tree contains spa_room_141:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-eltakf-0 material tree contains spa_room_009:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pillar_candle-qevocc-0 material tree contains spa_room_140:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-eltakf-0 material tree contains spa_room_178:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pillar_candle-aqryfd-1 material tree contains spa_room_139:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-eltakf-0 material tree contains spa_room_147:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pillar_candle-aqryfd-1 material tree contains spa_room_141:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-vzkdpu-0 material tree contains Standard_16fsd:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pillar_candle-aqryfd-1 material tree contains spa_room_140:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-vzkdpu-0 material tree contains Standard_16fsd:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-vtamah-2 material tree contains Standard_16fsd:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-lhyxmc-0 material tree contains spa_room_178:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-vtamah-2 material tree contains spa_room_147:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-lhyxmc-0 material tree contains spa_room_147:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-vtamah-3 material tree contains Standard_16fsd:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-inysln-0 material tree contains Standard_16fsd:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-vtamah-3 material tree contains Standard_16fsd:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-inysln-0 material tree contains Standard_16fsd:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-vtamah-3 material tree contains spa_room_147:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-inysln-0 material tree contains spa_room_009:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-vtamah-4 material tree contains Standard_16fsd:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-inysln-0 material tree contains spa_room_178:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-vtamah-4 material tree contains Standard_16fsd:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-inysln-0 material tree contains spa_room_147:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-vtamah-4 material tree contains spa_room_147:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-uejdci-0 material tree contains Standard_16fsd:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-vtamah-5 material tree contains Standard_16fsd:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-uejdci-0 material tree contains Standard_16fsd:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-vtamah-5 material tree contains Standard_16fsd:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-uejdci-0 material tree contains spa_room_009:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-vtamah-5 material tree contains spa_room_147:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-uejdci-0 material tree contains spa_room_178:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-vtamah-6 material tree contains Standard_16fsd:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-uejdci-0 material tree contains spa_room_147:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-vtamah-6 material tree contains Standard_16fsd:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-jngdxy-0 material tree contains Standard_16fsd:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-vtamah-6 material tree contains spa_room_147:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-jngdxy-0 material tree contains Standard_16fsd:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-vtamah-7 material tree contains Standard_16fsd:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-jngdxy-0 material tree contains spa_room_009:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-vtamah-7 material tree contains Standard_16fsd:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-jngdxy-0 material tree contains spa_room_178:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-vtamah-7 material tree contains spa_room_147:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-jngdxy-0 material tree contains spa_room_147:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-vtamah-8 material tree contains Standard_16fsd:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-jxhdba-0 material tree contains Standard_16fsd:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-vtamah-8 material tree contains Standard_16fsd:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-jxhdba-0 material tree contains Standard_16fsd:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-vtamah-8 material tree contains spa_room_147:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-jxhdba-0 material tree contains spa_room_009:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-vtamah-9 material tree contains Standard_16fsd:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-jxhdba-0 material tree contains spa_room_178:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-vtamah-9 material tree contains Standard_16fsd:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-jxhdba-0 material tree contains spa_room_147:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-vtamah-9 material tree contains spa_room_147:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-zwbaru-0 material tree contains Standard_16fsd:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-vtamah-10 material tree contains Standard_16fsd:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-zwbaru-0 material tree contains Standard_16fsd:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-vtamah-10 material tree contains Standard_16fsd:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-zwbaru-0 material tree contains spa_room_009:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-vtamah-10 material tree contains spa_room_147:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-zwbaru-0 material tree contains spa_room_178:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-vtamah-11 material tree contains Standard_16fsd:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-zwbaru-0 material tree contains spa_room_147:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-vtamah-11 material tree contains Standard_16fsd:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-mxpwbb-0 material tree contains Standard_16fsd:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-vtamah-11 material tree contains spa_room_147:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-mxpwbb-0 material tree contains Standard_16fsd:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-vtamah-12 material tree contains Standard_16fsd:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-mxpwbb-0 material tree contains spa_room_009:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-vtamah-12 material tree contains Standard_16fsd:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-mxpwbb-0 material tree contains spa_room_178:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.


## scenes/hotel_suite_large
  ### (.*) material tree contains .* with disallowed type .*. It should be one of .*.: 79
    downlight-miioxs-0 material tree contains halogen_004:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-miioxs-0 material tree contains halogen_003:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-miioxs-1 material tree contains halogen_004:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-miioxs-1 material tree contains halogen_003:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-miioxs-2 material tree contains halogen_004:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-miioxs-2 material tree contains halogen_003:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-miioxs-16 material tree contains halogen_004:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-miioxs-16 material tree contains halogen_003:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-miioxs-17 material tree contains halogen_004:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-miioxs-17 material tree contains halogen_003:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-miioxs-18 material tree contains halogen_004:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-miioxs-18 material tree contains halogen_003:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-miioxs-19 material tree contains halogen_004:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-miioxs-19 material tree contains halogen_003:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-miioxs-4 material tree contains halogen_004:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-miioxs-4 material tree contains halogen_003:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-miioxs-5 material tree contains halogen_004:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-miioxs-5 material tree contains halogen_003:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-miioxs-6 material tree contains halogen_004:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-miioxs-6 material tree contains halogen_003:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-miioxs-7 material tree contains halogen_004:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-miioxs-7 material tree contains halogen_003:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-miioxs-20 material tree contains halogen_004:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-miioxs-20 material tree contains halogen_003:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-miioxs-23 material tree contains halogen_004:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-miioxs-23 material tree contains halogen_003:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-miioxs-21 material tree contains halogen_004:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-miioxs-21 material tree contains halogen_003:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-miioxs-22 material tree contains halogen_004:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-table_lamp-uzrovn-1 material tree contains element_004:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-miioxs-22 material tree contains halogen_003:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-table_lamp-uzrovn-0 material tree contains element_004:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-miioxs-8 material tree contains halogen_004:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-blanket-uksirc-1 material tree contains furniture_028:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-miioxs-8 material tree contains halogen_003:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pillow-eghwza-1 material tree contains furniture_032:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    downlight-miioxs-9 material tree contains halogen_004:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-blanket-uksirc-0 material tree contains furniture_028:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-miioxs-9 material tree contains halogen_003:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pillow-eghwza-0 material tree contains furniture_032:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    downlight-miioxs-10 material tree contains halogen_004:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pillow-ahvodn-0 material tree contains furniture_020:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    downlight-miioxs-10 material tree contains halogen_003:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-laptop-sngiih-0 material tree contains decor_stuff_015:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-miioxs-11 material tree contains halogen_004:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-acksaa-0 material tree contains element_004:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-miioxs-11 material tree contains halogen_003:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-statue-cfrsik-0 material tree contains decor_stuff_051:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-miioxs-12 material tree contains halogen_004:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-ottoman-hnmndv-1 material tree contains microsGroup2112604805:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-miioxs-12 material tree contains halogen_003:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-ottoman-hnmndv-0 material tree contains microsGroup2112604805:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-laptop-sngiih-0-screen-base_link-R-lower material tree contains decor_stuff_015:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pillow-qtgqhd-0 material tree contains furniture_028:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-laptop-sngiih-0-screen-base_link-R-upper material tree contains decor_stuff_015:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bar_soap-lyigsj-0 material tree contains decor_stuff_090:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-mjfjhn-1 material tree contains interior_058:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-mjfjhn-0 material tree contains interior_058:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-mjfjhn-1 material tree contains interior_057:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-mjfjhn-0 material tree contains interior_057:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-mjfjhn-1 material tree contains interior_054:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-mjfjhn-0 material tree contains interior_054:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-plate-ihrjrb-0 material tree contains decor_stuff_090:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    door-ydokma-0-leaf-base_link-R-upper material tree contains interior_030:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    door-ydokma-0-leaf-base_link-R-lower-Tglass material tree contains interior_030:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    towel_rack-ibrplo-0 material tree contains interior_030:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    door-gjtspo-0-leaf-base_link-R-upper material tree contains interior_030:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    door-gjtspo-0-leaf-base_link-R-lower-Tglass material tree contains interior_030:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pillow-ahvodn-1 material tree contains furniture_020:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    C-pillow-ahvodn-2 material tree contains furniture_020:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    C-pillow-ahvodn-3 material tree contains furniture_020:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    downlight-miioxs-3 material tree contains halogen_004:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-miioxs-3 material tree contains halogen_003:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-miioxs-13 material tree contains halogen_004:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-miioxs-13 material tree contains halogen_003:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-miioxs-14 material tree contains halogen_004:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-miioxs-14 material tree contains halogen_003:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-miioxs-15 material tree contains halogen_004:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-miioxs-15 material tree contains halogen_003:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.


## scenes/hotel_suite_small
  ### (.*) material tree contains .* with disallowed type .*. It should be one of .*.: 20
    ceilings-axkzor-0 material tree contains VRayMtl_20:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-hevnoq-0 material tree contains VRayMtl_20:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    ceilings-axkzor-0 material tree contains VRayMtl_20:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    ceilings-ocoolw-0 material tree contains VRayMtl_20:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-dfjhxs-0 material tree contains VRayMtl_20:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    ceilings-ocoolw-0 material tree contains VRayMtl_20:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-dfjhxs-0 material tree contains VRayMtl_20:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-lllojr-0 material tree contains Material #4dsads59585:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    makeup_mirror-vtccbu-0 material tree contains VRayMtl_20:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    makeup_mirror-vtccbu-0 material tree contains VRayMtl_20:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-dfjhxs-1 material tree contains VRayMtl_20:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-dfjhxs-1 material tree contains VRayMtl_20:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-dfjhxs-2 material tree contains VRayMtl_20:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-dfjhxs-2 material tree contains VRayMtl_20:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-dfjhxs-3 material tree contains VRayMtl_20:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-dfjhxs-3 material tree contains VRayMtl_20:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-ovzcgo-0 material tree contains VRayMtl_20:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-ovzcgo-0 material tree contains VRayMtl_20:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-hevnoq-0 material tree contains VRayMtl_20:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-iylkzm-0 material tree contains Material #4dsads59585:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.


## scenes/house_double_floor_lower
  ### (.*) material tree contains .* with disallowed type .*. It should be one of .*.: 298
    downlight-pawfmw-3 material tree contains AE31_mat_003_Standard_10:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-table_knife-nvwmkx-11 material tree contains Arch145_009_010:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-washer-ziomqg-0-door-base_link-R-upper material tree contains AE31_19 - light_ON:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-mgvxov-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-pawfmw-2 material tree contains AE31_mat_003_Standard_10:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-table_knife-nvwmkx-12 material tree contains Arch145_009_010:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-washer-ziomqg-0-door-base_link-R-upper material tree contains vray_ArchModels_68_043_014:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-sfobco-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-pawfmw-0 material tree contains AE31_mat_003_Standard_10:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-table_knife-nvwmkx-13 material tree contains Arch145_009_010:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-washer-ziomqg-0-door-base_link-R-upper material tree contains vray_ArchModels_68_043_009:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-ueqyqb-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-pawfmw-4 material tree contains AE31_mat_003_Standard_10:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-table_knife-nvwmkx-14 material tree contains Arch145_009_010:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-washer-ziomqg-0-door-base_link-R-upper material tree contains vray_ArchModels_68_043_011:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-pqzkrj-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bottom_cabinet-rhdbzv-0-left_drawer-base_link-P-lower material tree contains AM_144_048_022:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-pawfmw-5 material tree contains AE31_mat_003_Standard_10:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-table_knife-nvwmkx-15 material tree contains Arch145_009_010:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-washer-ziomqg-0-door-base_link-R-upper material tree contains vray_ArchModels_68_043_010:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-mazmaa-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bottom_cabinet-rhdbzv-0-left_drawer-base_link-P-lower material tree contains AM_144_048_023:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-pawfmw-6 material tree contains AE31_mat_003_Standard_10:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-table_knife-nvwmkx-16 material tree contains Arch145_009_010:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-washer-ziomqg-0-door-base_link-R-upper material tree contains AE31_mat #117:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-gholmm-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bottom_cabinet-rhdbzv-0-left_drawer-base_link-P-lower material tree contains AM_144_048_024:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-pawfmw-7 material tree contains AE31_mat_003_Standard_10:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-table_knife-nvwmkx-17 material tree contains Arch145_009_010:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-washer-ziomqg-0-door-base_link-R-upper material tree contains vray_ArchModels_68_043_015:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-trfdmp-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bottom_cabinet-rhdbzv-0-right_drawer-base_link-P-lower material tree contains AM_144_048_022:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-pawfmw-8 material tree contains AE31_mat_003_Standard_10:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    track_light-ltnnrc-1 material tree contains AE31_10 - Defaultghgh:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    garden_light-qyfrwj-0 material tree contains AM107_015_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-yaieuq-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bottom_cabinet-rhdbzv-0-right_drawer-base_link-P-lower material tree contains AM_144_048_023:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-pawfmw-1 material tree contains AE31_mat_003_Standard_10:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    track_light-ltnnrc-0 material tree contains AE31_10 - Defaultghgh:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    garden_light-qyfrwj-0 material tree contains AM107_015_metal_brushed_linear:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-fpfyos-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bottom_cabinet-rhdbzv-0-right_drawer-base_link-P-lower material tree contains AM_144_048_024:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    top_cabinet-tynnnw-0 material tree contains Arch137_040_mat_047:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-pawfmw-9 material tree contains AE31_mat_003_Standard_10:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    garden_light-qyfrwj-2 material tree contains AM107_015_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-eculrb-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bottom_cabinet-rhdbzv-0-base_link material tree contains AM_144_048_022:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    top_cabinet-tynnnw-0 material tree contains Arch137_040_mat_051:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-pawfmw-10 material tree contains AE31_mat_003_Standard_10:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    garden_light-qyfrwj-2 material tree contains AM107_015_metal_brushed_linear:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-kczhvj-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-C-knife_block-mjywpg-0 material tree contains Arch145_013_004:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    top_cabinet-tynnnw-0 material tree contains Arch137_040_mat_050:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-pawfmw-11 material tree contains AE31_mat_003_Standard_10:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    garden_light-qyfrwj-1 material tree contains AM107_015_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-jchtfh-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-C-knife_block-mjywpg-0 material tree contains Arch145_013_000:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    range_hood-xvdfpp-0 material tree contains Arch137_040_mat_047:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-pawfmw-12 material tree contains AE31_mat_003_Standard_10:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    garden_light-qyfrwj-1 material tree contains AM107_015_metal_brushed_linear:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-wnhkmr-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-chopping_board-bzauwp-0 material tree contains Arch145_034_015:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    range_hood-xvdfpp-0 material tree contains Arch137_040_mat_051:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-pawfmw-13 material tree contains AE31_mat_003_Standard_10:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    garden_light-qyfrwj-4 material tree contains AM107_015_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-croxvf-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-kiuayi-0 material tree contains AE31_glass_bulb:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    range_hood-xvdfpp-0 material tree contains Arch137_040_mat_050:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-pawfmw-14 material tree contains AE31_mat_003_Standard_10:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    garden_light-qyfrwj-4 material tree contains AM107_015_metal_brushed_linear:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-sdxjmq-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-bar-byvbuc-0 material tree contains Arch137_040_mat_047:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-kiuayi-0 material tree contains AE31_rubber:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-eqbjaz-0 material tree contains AE31_mat_003_metal_V5:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    garden_light-qyfrwj-3 material tree contains AM107_015_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-pghswh-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-kiuayi-0 material tree contains AE31_metal_chrome_dirty:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-bar-byvbuc-0 material tree contains Arch137_040_mat_051:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-eqbjaz-0 material tree contains AE31_mat_003_chrome_glossy:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    garden_light-qyfrwj-3 material tree contains AM107_015_metal_brushed_linear:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-dqarfl-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-kiuayi-0 material tree contains AE31_metal_bright_anizo:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-bar-byvbuc-0 material tree contains Arch137_040_mat_050:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-eqbjaz-0 material tree contains AE31_mat_003_lightbar:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-ghetev-0 material tree contains AE31_mat_003_granite:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-ajeqmf-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-flower-qokboq-0 material tree contains AM108_067_03:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-C-pencil_holder-qwqsgi-0 material tree contains Arch145_009_007:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    room_light-eqbjaz-0 material tree contains AE31_metal_white_matte:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    lawn-ogzsvf-0 material tree contains AE31_mat_003_granite:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-gfyaka-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-flower-qokboq-0 material tree contains AM108_067_01:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-C-pencil_holder-qwqsgi-1 material tree contains Arch145_009_007:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    room_light-eqbjaz-0 material tree contains AE31_mat_003_metal_black_matte:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    garden_light-qyfrwj-5 material tree contains AM107_015_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-qeczib-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-pxrmnt-0 material tree contains AE31_10 - Defaultghgh:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-C-pencil_holder-qwqsgi-2 material tree contains Arch145_009_007:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    room_light-eqbjaz-0 material tree contains AE31_metal_V6:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    garden_light-qyfrwj-5 material tree contains AM107_015_metal_brushed_linear:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-lagpjt-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-pxrmnt-1 material tree contains AE31_10 - Defaultghgh:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-C-carving_knife-oxvnsa-0 material tree contains Arch145_013_004:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-pawfmw-15 material tree contains AE31_mat_003_Standard_10:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    garden_light-qyfrwj-6 material tree contains AM107_015_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-nobmrq-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-pxrmnt-2 material tree contains AE31_10 - Defaultghgh:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-C-carving_knife-oxvnsa-0 material tree contains Arch145_013_000:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-pawfmw-16 material tree contains AE31_mat_003_Standard_10:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    garden_light-qyfrwj-6 material tree contains AM107_015_metal_brushed_linear:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-eooxoa-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-pxrmnt-3 material tree contains AE31_10 - Defaultghgh:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-C-carving_knife-xsjhji-0 material tree contains Arch145_013_004:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-pawfmw-17 material tree contains AE31_mat_003_Standard_10:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    garden_light-qyfrwj-7 material tree contains AM107_015_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-ixwdkv-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-pxrmnt-4 material tree contains AE31_10 - Defaultghgh:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-C-carving_knife-xsjhji-0 material tree contains Arch145_013_000:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bottom_cabinet-rhdbzv-0-left_drawer-base_link-P-upper material tree contains AM_144_048_022:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    garden_light-qyfrwj-7 material tree contains AM107_015_metal_brushed_linear:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-gkezra-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-pxrmnt-5 material tree contains AE31_10 - Defaultghgh:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-C-carving_knife-icxnkd-0 material tree contains Arch145_013_004:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bottom_cabinet-rhdbzv-0-left_drawer-base_link-P-upper material tree contains AM_144_048_023:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    garden_light-qyfrwj-8 material tree contains AM107_015_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-nmnycd-0 material tree contains AE31_mat_003_floor_Marble:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    wall_mounted_light-mbhxbg-0 material tree contains AE31_Standard_2:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-C-carving_knife-icxnkd-0 material tree contains Arch145_013_000:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bottom_cabinet-rhdbzv-0-left_drawer-base_link-P-upper material tree contains AM_144_048_024:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    garden_light-qyfrwj-8 material tree contains AM107_015_metal_brushed_linear:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-nmnycd-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-soap_dispenser-ycvrcg-0 material tree contains vray_1:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-teaspoon-wzsvzv-0 material tree contains Arch145_034_003:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bottom_cabinet-rhdbzv-0-right_drawer-base_link-P-upper material tree contains AM_144_048_022:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-csxnnw-0 material tree contains AE31_mat_003_floor_Marble:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    top_cabinet-tynnnw-0-leaf0-base_link-R-lower material tree contains Arch137_040_mat_047:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    flower-ldkhmx-0 material tree contains AE31_mat #830:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-C-mug-dhnxww-0 material tree contains Arch145_034_003:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bottom_cabinet-rhdbzv-0-right_drawer-base_link-P-upper material tree contains AM_144_048_023:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-csxnnw-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    top_cabinet-tynnnw-0-leaf0-base_link-R-lower material tree contains Arch137_040_mat_051:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    flower-ldkhmx-0 material tree contains AM124_097_v1_02:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-C-mug-dhnxww-1 material tree contains Arch145_034_003:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bottom_cabinet-rhdbzv-0-right_drawer-base_link-P-upper material tree contains AM_144_048_024:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-ewbqsp-0 material tree contains AE31_mat_003_floor_Marble:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    top_cabinet-tynnnw-0-leaf0-base_link-R-lower material tree contains Arch137_040_mat_050:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    flower-ldkhmx-0 material tree contains AM155_010__012_v3_02:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-C-mug-dhnxww-2 material tree contains Arch145_034_003:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-ottoman-wxodip-0 material tree contains AM159_002_005:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-ewbqsp-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    top_cabinet-tynnnw-0-leaf1-base_link-R-lower material tree contains Arch137_040_mat_047:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    flower-ldkhmx-0 material tree contains AM155_010_012_v3_01:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-C-mug-dhnxww-3 material tree contains Arch145_034_003:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-shelf-vanltx-0 material tree contains AM149_set12__aluminium:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-adimyi-0 material tree contains AE31_mat_003_floor_Marble:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    top_cabinet-tynnnw-0-leaf1-base_link-R-lower material tree contains Arch137_040_mat_051:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-hair_dryer-qacdxx-0 material tree contains vray_AE31_plastic25:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-teaspoon-wzsvzv-1 material tree contains Arch145_034_003:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-pcbfvo-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-adimyi-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    top_cabinet-tynnnw-0-leaf1-base_link-R-lower material tree contains Arch137_040_mat_050:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-washer-ziomqg-0-base_link material tree contains AE31_mat #101:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-tablefork-osfjvs-0 material tree contains Arch145_009_010:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    ceilings-pgwmer-0 material tree contains AI34_005_77sssasa:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-uddigl-0 material tree contains AE31_mat_003_floor_Marble:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    top_cabinet-tynnnw-0-leaf2-base_link-R-lower material tree contains Arch137_040_mat_047:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-washer-ziomqg-0-base_link material tree contains vray_ArchModels_68_043_007:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-tablefork-osfjvs-1 material tree contains Arch145_009_010:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    ceilings-pgwmer-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-uddigl-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    top_cabinet-tynnnw-0-leaf2-base_link-R-lower material tree contains Arch137_040_mat_051:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-washer-ziomqg-0-base_link material tree contains AE31_19 - light_OFF:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-tablefork-osfjvs-2 material tree contains Arch145_009_010:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    ceilings-fzisrr-0 material tree contains AE31_mat_003_lights:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-rhwsaa-0 material tree contains AE31_mat_003_floor_Marble:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    top_cabinet-tynnnw-0-leaf2-base_link-R-lower material tree contains Arch137_040_mat_050:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-washer-ziomqg-0-base_link material tree contains vray_ArchModels_68_043_008:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-tablefork-osfjvs-3 material tree contains Arch145_009_010:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    ceilings-fzisrr-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-rhwsaa-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    top_cabinet-tynnnw-0-leaf3-base_link-R-lower material tree contains Arch137_040_mat_047:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-washer-ziomqg-0-base_link material tree contains AE31_19 - light_ON:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-tablefork-osfjvs-4 material tree contains Arch145_009_010:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    floors-xqgwla-0 material tree contains AE31_mat_003_floor_Marble:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-yhvncx-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    top_cabinet-tynnnw-0-leaf3-base_link-R-lower material tree contains Arch137_040_mat_051:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-washer-ziomqg-0-base_link material tree contains vray_ArchModels_68_043_014:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-tablefork-osfjvs-5 material tree contains Arch145_009_010:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    floors-xqgwla-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-jhyczx-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    top_cabinet-tynnnw-0-leaf3-base_link-R-lower material tree contains Arch137_040_mat_050:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-washer-ziomqg-0-base_link material tree contains vray_ArchModels_68_043_009:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-tablefork-osfjvs-6 material tree contains Arch145_009_010:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    floors-qwzkxn-0 material tree contains AE31_mat_003_granite:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-mojnsh-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    top_cabinet-tynnnw-0-leaf0-base_link-R-upper material tree contains Arch137_040_mat_047:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-washer-ziomqg-0-base_link material tree contains vray_ArchModels_68_043_011:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-tablefork-osfjvs-7 material tree contains Arch145_009_010:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    F-burner-mdanhg-0 material tree contains Arch137_040_mat_047:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-phzkls-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    top_cabinet-tynnnw-0-leaf0-base_link-R-upper material tree contains Arch137_040_mat_051:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-washer-ziomqg-0-base_link material tree contains vray_ArchModels_68_043_010:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-tablefork-osfjvs-8 material tree contains Arch145_009_010:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    F-burner-mdanhg-0 material tree contains Arch137_040_mat_051:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-kfadgw-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    top_cabinet-tynnnw-0-leaf0-base_link-R-upper material tree contains Arch137_040_mat_050:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-washer-ziomqg-0-base_link material tree contains AE31_mat #117:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-tablefork-osfjvs-9 material tree contains Arch145_009_010:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    F-burner-mdanhg-0 material tree contains Arch137_040_mat_050:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-ouhzch-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    top_cabinet-tynnnw-0-leaf1-base_link-R-upper material tree contains Arch137_040_mat_047:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-washer-ziomqg-0-base_link material tree contains vray_ArchModels_68_043_015:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-tablefork-osfjvs-10 material tree contains Arch145_009_010:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    furniture_sink-egwapq-0 material tree contains Arch137_040_mat_047:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-knntqk-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    top_cabinet-tynnnw-0-leaf1-base_link-R-upper material tree contains Arch137_040_mat_051:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-sofa-frlxgz-0 material tree contains AM159_002_005:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-tablefork-osfjvs-11 material tree contains Arch145_009_010:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    furniture_sink-egwapq-0 material tree contains Arch137_040_mat_051:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-cvnmjb-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    top_cabinet-tynnnw-0-leaf1-base_link-R-upper material tree contains Arch137_040_mat_050:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bowl-xplzbo-0 material tree contains AM149_set24_plate:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    C-tablefork-osfjvs-12 material tree contains Arch145_009_010:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    furniture_sink-egwapq-0 material tree contains Arch137_040_mat_050:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-apakqv-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    top_cabinet-tynnnw-0-leaf2-base_link-R-upper material tree contains Arch137_040_mat_047:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-C-hingeless_jar-ifgcmr-0-Tglass material tree contains AM149_set24_chrome:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-tablefork-osfjvs-13 material tree contains Arch145_009_010:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    driveway-qbxmgb-0 material tree contains AE31_mat_003_granite:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-uqliim-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    top_cabinet-tynnnw-0-leaf2-base_link-R-upper material tree contains Arch137_040_mat_051:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-C-lid-fvebnu-0-Tconnectedpart material tree contains AM149_set24_chrome:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-tablefork-osfjvs-14 material tree contains Arch145_009_010:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-washer-ziomqg-0-door-base_link-R-lower material tree contains AE31_mat #101:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-tbivvy-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    top_cabinet-tynnnw-0-leaf2-base_link-R-upper material tree contains Arch137_040_mat_050:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pot_plant-nwuuca-0 material tree contains AM145_set24_leaf:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-tablefork-osfjvs-15 material tree contains Arch145_009_010:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-washer-ziomqg-0-door-base_link-R-lower material tree contains vray_ArchModels_68_043_007:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-judhst-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    top_cabinet-tynnnw-0-leaf3-base_link-R-upper material tree contains Arch137_040_mat_047:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    flower-tfvsai-0 material tree contains AE31_mat_006_379:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-tablefork-osfjvs-16 material tree contains Arch145_009_010:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-washer-ziomqg-0-door-base_link-R-lower material tree contains AE31_19 - light_OFF:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-kxxjkr-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    top_cabinet-tynnnw-0-leaf3-base_link-R-upper material tree contains Arch137_040_mat_051:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    flower-tfvsai-0 material tree contains AE31_mat_006_358:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-tablefork-osfjvs-17 material tree contains Arch145_009_010:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-washer-ziomqg-0-door-base_link-R-lower material tree contains vray_ArchModels_68_043_008:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-gatabi-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    top_cabinet-tynnnw-0-leaf3-base_link-R-upper material tree contains Arch137_040_mat_050:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    flower-tfvsai-0 material tree contains AE31_mat_006_377:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-table_knife-nvwmkx-0 material tree contains Arch145_009_010:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-washer-ziomqg-0-door-base_link-R-lower material tree contains AE31_19 - light_ON:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-fpvunc-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-blender-dhfqid-0-base_link material tree contains Arch145_034_003:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    flower-tfvsai-0 material tree contains AE31_mat_006_378:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-table_knife-nvwmkx-1 material tree contains Arch145_009_010:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-washer-ziomqg-0-door-base_link-R-lower material tree contains vray_ArchModels_68_043_014:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-fapfsm-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-blender-dhfqid-0-lid-base_link-R-lower material tree contains Arch145_034_003:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    flower-tfvsai-0 material tree contains AE31_mat_006_375:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-table_knife-nvwmkx-2 material tree contains Arch145_009_010:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-washer-ziomqg-0-door-base_link-R-lower material tree contains vray_ArchModels_68_043_009:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-nqmair-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-blender-dhfqid-0-glass-base_link-F-lower material tree contains Arch145_034_003:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-gqwker-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-table_knife-nvwmkx-3 material tree contains Arch145_009_010:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-washer-ziomqg-0-door-base_link-R-lower material tree contains vray_ArchModels_68_043_011:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-hzstma-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-iuzbqu-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-table_knife-nvwmkx-4 material tree contains Arch145_009_010:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-washer-ziomqg-0-door-base_link-R-lower material tree contains vray_ArchModels_68_043_010:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-vupsfl-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-C-vase-dfjcsi-0 material tree contains AM108_067_03:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-table_knife-nvwmkx-5 material tree contains Arch145_009_010:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-washer-ziomqg-0-door-base_link-R-lower material tree contains AE31_mat #117:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-ksqvpy-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-C-vase-dfjcsi-0 material tree contains AM108_067_01:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-table_knife-nvwmkx-6 material tree contains Arch145_009_010:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-washer-ziomqg-0-door-base_link-R-lower material tree contains vray_ArchModels_68_043_015:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-btxexl-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-vase-vmbzmm-0-Tglass material tree contains AE31_mat #830:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-table_knife-nvwmkx-7 material tree contains Arch145_009_010:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-washer-ziomqg-0-door-base_link-R-upper material tree contains AE31_mat #101:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-hnxiju-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-vase-vmbzmm-0-Tglass material tree contains AM124_097_v1_02:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-table_knife-nvwmkx-8 material tree contains Arch145_009_010:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-washer-ziomqg-0-door-base_link-R-upper material tree contains vray_ArchModels_68_043_007:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-ntwvgk-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-vase-vmbzmm-0-Tglass material tree contains AM155_010__012_v3_02:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-table_knife-nvwmkx-9 material tree contains Arch145_009_010:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-washer-ziomqg-0-door-base_link-R-upper material tree contains AE31_19 - light_OFF:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-lovfhw-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-vase-vmbzmm-0-Tglass material tree contains AM155_010_012_v3_01:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-table_knife-nvwmkx-10 material tree contains Arch145_009_010:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-washer-ziomqg-0-door-base_link-R-upper material tree contains vray_ArchModels_68_043_008:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-hcbqvq-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.


## scenes/house_double_floor_upper
  ### (.*) material tree contains .* with disallowed type .*. It should be one of .*.: 112
    C-hardback-vywxev-0 material tree contains AM149_set22_black_plastic_dials:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-nnxsjd-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-mug-kewbyf-0 material tree contains AM149_set22_cup_2:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    walls-jamapi-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    folderal-ydrlth-0 material tree contains AM149_set22_metal_noise_dark:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    walls-tcrhve-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-hardback-mkxbhk-0 material tree contains AM149_set22_black_plastic_dials:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-avbnnx-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-C-plate-nikfgd-0 material tree contains AM149_set22_gold_noise_black2:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    walls-acugsx-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-digital_camera-tymxet-0 material tree contains AM149_set22_black_plastic_dials:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-tghwgm-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-sound_system-wllxyl-0 material tree contains AM149_set22_dial:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-vqgvuw-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-taboret-cgwxtq-0 material tree contains AM149_set22_metal_noise_dark:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-ffybck-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-statue-bpqnof-0 material tree contains AM149_set22_metal_noise_dark:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-xsfcrp-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-table_lamp-htmtsr-0 material tree contains AM149_set22_glass_1:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-rqcytb-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_water-qaceen-0 material tree contains AM149_set22_metal_noise_dark:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-rstyxz-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-cap-amryfj-0-Tconnectedpart material tree contains AM149_set22_metal_noise_dark:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-vtwnqs-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-table_lamp-vbjpad-0 material tree contains ArchModels_123_031_018:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-autzye-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-table_lamp-vbjpad-0 material tree contains ArchModels_123_031_021:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-ndfsrm-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-flower-xxvkkw-0 material tree contains AM155_026_flovers_leaf:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-potacy-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-flower-xxvkkw-0 material tree contains AM155_026_flovers_001:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-yijzob-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pot_plant-kyrcpw-0 material tree contains AE31_mat_002_AM141_007_001:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-orjpqs-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-penqpk-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-ogrxqz-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-curtain-zmodhv-0 material tree contains AI34_005_77sssasa:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    walls-edumiy-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-table_lamp-wbcgev-0 material tree contains ArchModels_123_014_013:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-pohzfc-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-pqbutt-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-pffhuv-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-beaker-uzgibd-0-Tglass material tree contains AE31_mat_002_AM134_08_chrome:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-onrszv-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-tray-iaaiyi-0 material tree contains AE31_mat_002_AM134_08_chrome:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-cegpja-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-beaker-uzgibd-1 material tree contains AE31_mat_002_AM134_08_chrome:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-xayqcu-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-beaker-uzgibd-2 material tree contains AE31_mat_002_AM134_08_chrome:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-hltkbe-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-beaker-uzgibd-3 material tree contains AE31_mat_002_AM134_08_chrome:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-zpsuqe-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-plate-uumkbl-0 material tree contains AE31_mat_002_porcelain_white:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-qwwinf-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-plate-uumkbl-0 material tree contains AE31_mat_002_black_shiny:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-klcxyf-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-plate-uumkbl-1 material tree contains AE31_mat_002_porcelain_white:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-yorsom-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-plate-uumkbl-1 material tree contains AE31_mat_002_black_shiny:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-kuhhbl-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-teacup-kccqwj-1 material tree contains AE31_mat_002_porcelain_white:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    walls-iobzfe-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-armchair-axcwcm-0 material tree contains AM149_set01_aluminium:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-dohvvk-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-table_lamp-lshxmu-0 material tree contains AM149_set01_glass_bottle_lamp:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-keugzu-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-table_lamp-lshxmu-0 material tree contains AM149_set01_globe_lamp_chrome:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-ynhwyn-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-beer_bottle-nigfha-0 material tree contains AM155_026_flovers_leaf:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-ynporc-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-beer_bottle-nigfha-0 material tree contains AM155_026_flovers_001:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-nvykqp-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-floor_lamp-mqfdjk-0 material tree contains vray_Archmodels72_INSIDEOUT_Table_silver:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-etjpqr-0-glass-base_link-F-lower-Tglass material tree contains AI34_005_77sssasa:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-floor_lamp-mqfdjk-0 material tree contains vray_Archmodels72_INSIDEOUT_Table_white:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-etjpqr-1-glass-base_link-F-lower-Tglass material tree contains AI34_005_77sssasa:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-oxglnt-3 material tree contains AE31_mat_003_Standard_16:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-etjpqr-2-glass-base_link-F-lower-Tglass material tree contains AI34_005_77sssasa:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-oxglnt-2 material tree contains AE31_mat_003_Standard_16:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-etjpqr-3-glass-base_link-F-lower-Tglass material tree contains AI34_005_77sssasa:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-oxglnt-1 material tree contains AE31_mat_003_Standard_16:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-etjpqr-4-glass-base_link-F-lower-Tglass material tree contains AI34_005_77sssasa:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-oxglnt-0 material tree contains AE31_mat_003_Standard_16:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-etjpqr-5-glass-base_link-F-lower-Tglass material tree contains AI34_005_77sssasa:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-etjpqr-0 material tree contains AI34_005_77sssasa:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    picture-rypaxo-0 material tree contains AM149_set13_metal_frame:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    picture-hyjqxp-0 material tree contains AM149_set13_metal_frame:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    picture-yrkfct-0 material tree contains AM149_set13_metal_frame:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    picture-nzctlq-0 material tree contains AM149_set13_metal_frame:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-picture-krwzus-0 material tree contains AM149_set13_metal_frame:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    picture-vezfau-0 material tree contains AM149_set13_metal_frame:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    picture-ttwhjj-0 material tree contains AM149_set13_metal_frame:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    picture-aqahkx-0 material tree contains AM149_set13_metal_frame:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    shelf-sapyzl-0 material tree contains AM149_set13_metal_frame:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    shelf-sapyzl-1 material tree contains AM149_set13_metal_frame:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-ispfoo-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-teacup-kccqwj-0 material tree contains AE31_mat_002_porcelain_white:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    ceilings-pcpvvu-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    chandelier-fveyao-0 material tree contains vray_Archmodels72_INSIDEOUT_silver:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-gfgnbf-0 material tree contains AE31_mat_003_MarbleNoisy2:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    chandelier-fveyao-0 material tree contains vray_Archmodels72_INSIDEOUT_white:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-gfgnbf-0 material tree contains AE31_mat_003_Concerte_building:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-hardback-vywxev-1 material tree contains AM149_set22_black_plastic_dials:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-etjpqr-1 material tree contains AI34_005_77sssasa:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-hardback-vywxev-2 material tree contains AM149_set22_black_plastic_dials:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-etjpqr-2 material tree contains AI34_005_77sssasa:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-curtain-didlbp-0 material tree contains AI34_005_77sssasa:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    fixed_window-etjpqr-3 material tree contains AI34_005_77sssasa:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-mug-yiamah-0 material tree contains ArchModels_123_001_007:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-etjpqr-4 material tree contains AI34_005_77sssasa:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-table_lamp-erfbgw-0 material tree contains ArchModels_123_013_018:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-etjpqr-5 material tree contains AI34_005_77sssasa:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.


## scenes/house_single_floor
  ### (.*) material tree contains .* with disallowed type .*. It should be one of .*.: 416
    electric_switch-gashan-19 material tree contains AE27_153:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-bottom_cabinet-rhdbzv-1-left_drawer-base_link-P-lower material tree contains AM_144_048_024:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    electric_switch-gashan-20 material tree contains AE27_153:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-bottom_cabinet-rhdbzv-1-right_drawer-base_link-P-lower material tree contains AM_144_048_023:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    electric_switch-gashan-21 material tree contains AE27_153:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-bottom_cabinet-rhdbzv-1-right_drawer-base_link-P-lower material tree contains AM_144_048_022:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    electric_switch-gashan-22 material tree contains AE27_153:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-bottom_cabinet-rhdbzv-1-right_drawer-base_link-P-lower material tree contains AM_144_048_024:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-half_apple-jjaskz-0 material tree contains AM130_028_003:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-toy_box-aviadj-1 material tree contains AE27_64:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-half_apple-jjaskz-0 material tree contains AE27_644:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-toy_box-aviadj-1 material tree contains AE27_65:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-half_apple-jjaskz-0 material tree contains AE27_643:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-toy_box-aviadj-2 material tree contains AE27_64:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-half_apple-jjaskz-0 material tree contains AM130_028_004:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-toy_box-aviadj-2 material tree contains AE27_65:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    wall_mounted_light-urhftt-0 material tree contains AE27_1222:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-toy_box-aviadj-3 material tree contains AE27_64:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-wooden_spoon-ybytvw-0 material tree contains AE27_669:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-toy_box-aviadj-3 material tree contains AE27_65:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-wooden_spoon-ybytvw-0 material tree contains AE27_652:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-toy_box-aviadj-4 material tree contains AE27_64:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-wooden_spoon-ybytvw-0 material tree contains AE27_668:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-toy_box-aviadj-4 material tree contains AE27_65:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-wooden_spoon-rfrzjn-0 material tree contains AE27_669:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-toy_box-aviadj-5 material tree contains AE27_64:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-wooden_spoon-rfrzjn-0 material tree contains AE27_652:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-toy_box-aviadj-5 material tree contains AE27_65:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-wooden_spoon-rfrzjn-0 material tree contains AE27_668:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-toy_box-aviadj-6 material tree contains AE27_64:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-spatula-yppdij-0 material tree contains AE27_669:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-toy_box-aviadj-6 material tree contains AE27_65:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-spatula-yppdij-0 material tree contains AE27_652:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-toy_box-aviadj-7 material tree contains AE27_64:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-spatula-yppdij-0 material tree contains AE27_668:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-toy_box-aviadj-7 material tree contains AE27_65:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-loudspeaker-xpoajn-1 material tree contains AE27_1138:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-toy_box-aviadj-8 material tree contains AE27_64:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-notebook-qehiwi-0 material tree contains AE27_1239:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-toy_box-aviadj-8 material tree contains AE27_65:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-paperback_book-tmiacj-0 material tree contains AE27_1239:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-toy_box-aviadj-9 material tree contains AE27_64:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-notebook-djtnsj-0 material tree contains AE27_1239:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-toy_box-aviadj-9 material tree contains AE27_65:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-paperback_book-nbuuwf-0 material tree contains AE27_1239:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-toy_box-aviadj-10 material tree contains AE27_64:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-paperback_book-nbuuwf-1 material tree contains AE27_1239:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-toy_box-aviadj-10 material tree contains AE27_65:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-paperback_book-hccdxo-0 material tree contains AE27_1239:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-paperback_book-ewjgzf-0 material tree contains AE27_1239:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-paperback_book-ufgtdl-0 material tree contains AE27_1239:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-paperback_book-ufgtdl-1 material tree contains AE27_1239:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-paperback_book-umtflg-0 material tree contains AE27_1239:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-notebook-qehiwi-1 material tree contains AE27_1239:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-toy_box-aviadj-0 material tree contains AE27_64:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-toy_box-aviadj-0 material tree contains AE27_65:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-lid-mlkmlk-0-Tconnectedpart material tree contains AE27_64:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-lid-mlkmlk-0-Tconnectedpart material tree contains AE27_65:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-salt_shaker-fztfsi-1 material tree contains AE27_653:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-salt_shaker-fztfsi-1 material tree contains AE27_652:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-wooden_spoon-tczanw-1 material tree contains AE27_652:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    washer-ynwamu-0-door-base_link-R-lower material tree contains vray_AE27_1597:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    washer-ynwamu-0-door-base_link-R-lower material tree contains vray_AE27_1587:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    washer-ynwamu-0-door-base_link-R-lower material tree contains vray_AE27_1601:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    washer-ynwamu-0-door-base_link-R-lower material tree contains vray_AE27_1595:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    washer-ynwamu-0-door-base_link-R-lower material tree contains vray_AE27_1594:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    washer-ynwamu-0-door-base_link-R-lower material tree contains Material #101:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    washer-ynwamu-0-door-base_link-R-lower material tree contains vray_AE27_1598:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    washer-ynwamu-0-door-base_link-R-lower material tree contains AE27_1599:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    washer-ynwamu-0-door-base_link-R-lower material tree contains AE27_1600:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    washer-ynwamu-0-door-base_link-R-lower material tree contains Material #117:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    washer-ynwamu-0-door-base_link-R-lower material tree contains vray_AE27_1602:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    washer-ynwamu-0-door-base_link-R-upper material tree contains vray_AE27_1597:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    washer-ynwamu-0-door-base_link-R-upper material tree contains vray_AE27_1587:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    washer-ynwamu-0-door-base_link-R-upper material tree contains vray_AE27_1601:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    washer-ynwamu-0-door-base_link-R-upper material tree contains vray_AE27_1595:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    washer-ynwamu-0-door-base_link-R-upper material tree contains vray_AE27_1594:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    washer-ynwamu-0-door-base_link-R-upper material tree contains Material #101:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    washer-ynwamu-0-door-base_link-R-upper material tree contains vray_AE27_1598:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    washer-ynwamu-0-door-base_link-R-upper material tree contains AE27_1599:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    washer-ynwamu-0-door-base_link-R-upper material tree contains AE27_1600:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    washer-ynwamu-0-door-base_link-R-upper material tree contains Material #117:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    washer-ynwamu-0-door-base_link-R-upper material tree contains vray_AE27_1602:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-wardrobe-hibwic-0 material tree contains AE27_1222:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-podyre-0 material tree contains AE31_metal_white_matte:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-podyre-0 material tree contains AE31_mat_003_metal_V5:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-podyre-0 material tree contains AE31_mat_003_metal_black_matte:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-podyre-0 material tree contains AE31_metal_V6:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-podyre-0 material tree contains AE31_mat_003_chrome_glossy:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-podyre-0 material tree contains AE31_mat_003_lightbar:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-podyre-1 material tree contains AE31_metal_white_matte:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-podyre-1 material tree contains AE31_mat_003_metal_V5:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-podyre-1 material tree contains AE31_mat_003_metal_black_matte:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-podyre-1 material tree contains AE31_metal_V6:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-podyre-1 material tree contains AE31_mat_003_chrome_glossy:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-podyre-1 material tree contains AE31_mat_003_lightbar:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    countertop-gjeoer-0 material tree contains AE27_554:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    drop_in_sink-awvzkn-0 material tree contains AE27_562:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    burner-mjvqii-0 material tree contains AE27_554:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-taboret-ivtoxu-0 material tree contains AE27_516:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-taboret-ivtoxu-0 material tree contains AE27_515:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    gas_fireplace-gwqitr-0 material tree contains AE27_613:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-zutnsf-0 material tree contains AM130_028_003:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-zutnsf-0 material tree contains AE27_644:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-zutnsf-0 material tree contains AE27_643:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-zutnsf-0 material tree contains AM130_028_004:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bowl-tyczoo-0 material tree contains AE27_652:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-hammock-aiftuk-0 material tree contains AE27_1746:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-taboret-ivtoxu-1 material tree contains AE27_516:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-taboret-ivtoxu-1 material tree contains AE27_515:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-taboret-ivtoxu-2 material tree contains AE27_516:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-taboret-ivtoxu-2 material tree contains AE27_515:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-taboret-ivtoxu-3 material tree contains AE27_516:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-taboret-ivtoxu-3 material tree contains AE27_515:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-taboret-ivtoxu-4 material tree contains AE27_516:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-taboret-ivtoxu-4 material tree contains AE27_515:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-spatula-kqpoxk-0 material tree contains AE27_562:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    C-tongs-yljsbm-0 material tree contains AE27_562:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    C-soupspoon-qqvgml-0 material tree contains AE27_562:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    C-tablespoon-oacqcl-0 material tree contains AE27_677:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-tablespoon-oacqcl-0 material tree contains AE27_562:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bowl-npuuir-0 material tree contains AE27_677:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bowl-npuuir-0 material tree contains AE27_562:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-hingeless_jar-vxqpnm-0-Tglass material tree contains AE27_677:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-cap-zggjif-0-Tconnectedpart material tree contains AE27_677:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bowl-hitnkv-0 material tree contains AE27_676:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bowl-hitnkv-0 material tree contains AE27_677:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-vase-rfegnv-0 material tree contains AE27_676:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-floor_lamp-disyzd-0 material tree contains AE27_92:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-vase-rfegnv-0 material tree contains AE27_677:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    wall_mounted_light-yylrlu-0 material tree contains AE27_226:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-jar_of_sugar-pnbbfb-0 material tree contains AE27_653:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-hat-ljyldk-0 material tree contains AE27_290:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-jar_of_sugar-pnbbfb-0 material tree contains AE27_652:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-dddvjo-0 material tree contains AE27_367:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-salt_shaker-zkpgkl-0 material tree contains AE27_653:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-dddvjo-0 material tree contains AE27_366:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-salt_shaker-zkpgkl-0 material tree contains AE27_652:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-dddvjo-0 material tree contains AE27_369:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-box_of_cane_sugar-pozpqi-0 material tree contains AE27_653:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-iron-vlvslm-0 material tree contains vray_AE27_468:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-box_of_cane_sugar-pozpqi-0 material tree contains AE27_652:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-table_lamp-jbjckk-0 material tree contains AE27_526:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-tray-tkgsho-0 material tree contains AE27_653:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bar-udatjt-0 material tree contains AE27_554:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-tray-tkgsho-0 material tree contains AE27_652:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_olive_oil-ajswsh-0 material tree contains AE27_653:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-pot_plant-bcirge-0 material tree contains AE27_539:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_olive_oil-ajswsh-0 material tree contains AE27_652:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-pot_plant-bcirge-0 material tree contains AE27_545:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-carving_knife-usqmjc-0 material tree contains AE27_669:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-pot_plant-bcirge-0 material tree contains AE27_540:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-carving_knife-usqmjc-0 material tree contains AE27_652:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-pot_plant-bcirge-0 material tree contains AE27_538:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-carving_knife-usqmjc-0 material tree contains AE27_668:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-pot_plant-bcirge-0 material tree contains AE27_541:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pestle-dzhzot-0 material tree contains AE27_652:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-pot_plant-nwfojy-0 material tree contains AE27_539:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pillow-phtzvr-0 material tree contains AE27_690:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-pot_plant-nwfojy-0 material tree contains AE27_545:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bust-tnnbxo-0 material tree contains AE27_867:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-pot_plant-nwfojy-0 material tree contains AE27_540:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-statue-lcgvzq-0 material tree contains AE27_868:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-pot_plant-nwfojy-0 material tree contains AE27_538:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-picture-pblkaf-0 material tree contains AE27_993:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-pot_plant-nwfojy-0 material tree contains AE27_541:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-ottoman-wkmbra-0 material tree contains AE27_333:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    C-vase-dwspgo-0 material tree contains AE27_539:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-hat-rfwrrv-0 material tree contains AE27_1000:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    C-vase-dwspgo-0 material tree contains AE27_545:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-eames_chair-hndxiw-0 material tree contains AE27_1131:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-vase-dwspgo-0 material tree contains AE27_540:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-statue-krsfzi-0 material tree contains AE27_1041:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    C-vase-dwspgo-0 material tree contains AE27_538:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-vase-bbduix-0 material tree contains AE27_1042:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    C-vase-dwspgo-0 material tree contains AE27_541:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-bookcase-jaysra-0 material tree contains AE27_1051:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pillow-ajxhun-0 material tree contains AE27_691:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    F-desk-bhkhxo-0 material tree contains AE27_1056:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pillow-tvmuxs-0 material tree contains AE27_691:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    C-loudspeaker-xpoajn-0 material tree contains AE27_1138:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    shelf-pfusrd-0 material tree contains AE27_554:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    chandelier-knunaz-0 material tree contains AE27_1144:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    shelf-pfusrd-1 material tree contains AE27_554:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-table_lamp-btdcou-0 material tree contains AE27_1144:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    top_cabinet-lkxmne-0-base_link material tree contains AE27_554:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-table_lamp-btdcou-0 material tree contains AE27_1056:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bottom_cabinet-fancyy-0-base_link material tree contains AE27_554:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    electric_switch-gashan-1 material tree contains AE27_153:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    C-banana-wmglhc-0 material tree contains AM130_028_003:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-picture_frame-cvyfaw-0 material tree contains AE27_1320:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-banana-wmglhc-0 material tree contains AE27_644:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_water-lojipo-0 material tree contains AE27_1324:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-banana-wmglhc-0 material tree contains AE27_643:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_water-lojipo-0 material tree contains AE27_1320:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-banana-wmglhc-0 material tree contains AM130_028_004:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-cap-tkwpyr-0-Tconnectedpart material tree contains AE27_1324:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-ymhxqk-0 material tree contains AM130_028_003:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-cap-tkwpyr-0-Tconnectedpart material tree contains AE27_1320:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-ymhxqk-0 material tree contains AE27_644:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-digital_camera-yptdmt-0 material tree contains AE27_1359:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-ymhxqk-0 material tree contains AE27_643:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-hingeless_jar-ociqav-0 material tree contains AE27_1370:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-apple-ymhxqk-0 material tree contains AM130_028_004:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-cap-urmqxk-0-Tconnectedpart material tree contains AE27_1370:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-gravy_boat-krgqwl-0 material tree contains AE27_652:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    C-laptop-izydvb-0 material tree contains AE27_1379:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-beaker-sstojv-0 material tree contains AE27_669:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    washer-ynwamu-0-base_link material tree contains vray_AE27_1597:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-beaker-sstojv-0 material tree contains AE27_652:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    washer-ynwamu-0-base_link material tree contains vray_AE27_1587:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-beaker-sstojv-0 material tree contains AE27_668:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    washer-ynwamu-0-base_link material tree contains vray_AE27_1601:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-wooden_spoon-qnrthd-0 material tree contains AE27_669:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    washer-ynwamu-0-base_link material tree contains vray_AE27_1595:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-wooden_spoon-qnrthd-0 material tree contains AE27_652:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    washer-ynwamu-0-base_link material tree contains vray_AE27_1594:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-wooden_spoon-qnrthd-0 material tree contains AE27_668:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    washer-ynwamu-0-base_link material tree contains Material #101:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-spatula-ckpgkc-0 material tree contains AE27_669:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    washer-ynwamu-0-base_link material tree contains vray_AE27_1598:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-spatula-ckpgkc-0 material tree contains AE27_652:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    washer-ynwamu-0-base_link material tree contains AE27_1599:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-spatula-ckpgkc-0 material tree contains AE27_668:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    washer-ynwamu-0-base_link material tree contains AE27_1600:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-water_glass-bbpraa-0-Tglass material tree contains AE27_677:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    washer-ynwamu-0-base_link material tree contains Material #117:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-water_glass-bbpraa-0-Tglass material tree contains AE27_562:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    washer-ynwamu-0-base_link material tree contains vray_AE27_1602:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    boulder-vxbkjq-0 material tree contains AE27_1682:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-pot_plant-rkqvba-0 material tree contains AE27_1611:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    baseboard-dsajeh-0 material tree contains AE27_554:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    wall_mounted_light-cstgzt-3 material tree contains AE27_1615:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_olive_oil-jocrsz-0 material tree contains AE27_653:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    wall_mounted_light-cstgzt-2 material tree contains AE27_1615:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_olive_oil-jocrsz-0 material tree contains AE27_652:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    wall_mounted_light-cstgzt-1 material tree contains AE27_1615:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-tablespoon-oacqcl-1 material tree contains AE27_677:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    wall_mounted_light-cstgzt-0 material tree contains AE27_1615:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-tablespoon-oacqcl-1 material tree contains AE27_562:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-pot_plant-lligdl-0 material tree contains AE27_1611:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-tablespoon-oacqcl-2 material tree contains AE27_677:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-beeswax_candle-aiuhyv-0 material tree contains AE27_1757:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-tablespoon-oacqcl-2 material tree contains AE27_562:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-beeswax_candle-aiuhyv-0 material tree contains AM139_039_020:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    electric_switch-gashan-5 material tree contains AE27_153:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    C-beeswax_candle-aiuhyv-1 material tree contains AE27_1757:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    electric_switch-gashan-6 material tree contains AE27_153:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    C-beeswax_candle-aiuhyv-1 material tree contains AM139_039_020:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-wardrobe-pyuwgc-0 material tree contains AE27_1222:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-beeswax_candle-aiuhyv-2 material tree contains AE27_1757:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-wardrobe-pyuwgc-0 material tree contains Material #7860:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-beeswax_candle-aiuhyv-2 material tree contains AM139_039_020:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    countertop-rkgjer-0 material tree contains AE27_554:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-beeswax_candle-aiuhyv-3 material tree contains AE27_1757:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    top_cabinet-lkxmne-0-door1-base_link-R-lower material tree contains AE27_554:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-beeswax_candle-aiuhyv-3 material tree contains AM139_039_020:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    top_cabinet-lkxmne-0-door2-base_link-R-lower material tree contains AE27_554:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-beeswax_candle-aiuhyv-4 material tree contains AE27_1757:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    top_cabinet-lkxmne-0-door3-base_link-R-lower material tree contains AE27_554:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-beeswax_candle-aiuhyv-4 material tree contains AM139_039_020:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    top_cabinet-lkxmne-0-door3-base_link-R-upper material tree contains AE27_554:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-beeswax_candle-aiuhyv-5 material tree contains AE27_1757:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    top_cabinet-lkxmne-0-door1-base_link-R-upper material tree contains AE27_554:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-beeswax_candle-aiuhyv-5 material tree contains AM139_039_020:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    top_cabinet-lkxmne-0-door2-base_link-R-upper material tree contains AE27_554:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-beeswax_candle-aiuhyv-6 material tree contains AE27_1757:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-wjlqlb-0 material tree contains AE27_1222:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-beeswax_candle-aiuhyv-6 material tree contains AM139_039_020:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-wjlqlb-0 material tree contains Material #7860:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-beeswax_candle-aiuhyv-7 material tree contains AE27_1757:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    countertop-fjkase-0 material tree contains AE27_1222:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-beeswax_candle-aiuhyv-7 material tree contains AM139_039_020:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    countertop-fjkase-0 material tree contains Material #7860:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-beeswax_candle-aiuhyv-8 material tree contains AE27_1757:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    shelf-zlaser-0 material tree contains AE27_1222:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-beeswax_candle-aiuhyv-8 material tree contains AM139_039_020:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    shelf-zlaser-0 material tree contains Material #7860:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-beeswax_candle-aiuhyv-9 material tree contains AE27_1757:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    shelf-amyper-0 material tree contains AE27_1222:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-beeswax_candle-aiuhyv-9 material tree contains AM139_039_020:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    shelf-amyper-0 material tree contains Material #7860:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-beeswax_candle-aiuhyv-10 material tree contains AE27_1757:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-wjlqlb-1 material tree contains AE27_1222:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-beeswax_candle-aiuhyv-10 material tree contains AM139_039_020:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-wjlqlb-1 material tree contains Material #7860:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-beeswax_candle-aiuhyv-11 material tree contains AE27_1757:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-wjlqlb-2 material tree contains AE27_1222:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-beeswax_candle-aiuhyv-11 material tree contains AM139_039_020:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-wjlqlb-2 material tree contains Material #7860:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-wooden_spoon-tczanw-0 material tree contains AE27_652:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-wjlqlb-3 material tree contains AE27_1222:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-carafe-mdtkkv-0-Tglass material tree contains AE27_1646:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-wjlqlb-3 material tree contains Material #7860:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-water_glass-ggpnlr-0-Tglass material tree contains AE27_1646:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-wjlqlb-4 material tree contains AE27_1222:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-water_glass-ggpnlr-1-Tglass material tree contains AE27_1646:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-wjlqlb-4 material tree contains Material #7860:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-carafe-mdtkkv-1-Tglass material tree contains AE27_1646:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-wjlqlb-5 material tree contains AE27_1222:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-whisk-xglhcs-0 material tree contains AE27_562:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    walls-wjlqlb-5 material tree contains Material #7860:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-spatula-vnymjm-0 material tree contains AE27_562:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    walls-wjlqlb-6 material tree contains AE27_1222:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-hamper-stibna-0 material tree contains AE27_677:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-wjlqlb-6 material tree contains Material #7860:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-hamper-stibna-0 material tree contains AE27_562:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-beeswax_candle-aiuhyv-12 material tree contains AE27_1757:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-lid-kgjkzr-0-Tconnectedpart material tree contains AE27_677:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-beeswax_candle-aiuhyv-12 material tree contains AM139_039_020:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-lid-kgjkzr-0-Tconnectedpart material tree contains AE27_562:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-beeswax_candle-aiuhyv-13 material tree contains AE27_1757:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-hamper-otswdv-0 material tree contains AE27_677:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-beeswax_candle-aiuhyv-13 material tree contains AM139_039_020:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-hamper-otswdv-0 material tree contains AE27_562:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-beeswax_candle-aiuhyv-14 material tree contains AE27_1757:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-lid-lkaasa-0-Tconnectedpart material tree contains AE27_677:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-beeswax_candle-aiuhyv-14 material tree contains AM139_039_020:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-lid-lkaasa-0-Tconnectedpart material tree contains AE27_562:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-beeswax_candle-aiuhyv-15 material tree contains AE27_1757:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-hingeless_jar-pjaljg-0-Tglass material tree contains AE27_677:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-beeswax_candle-aiuhyv-15 material tree contains AM139_039_020:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-cap-dduopd-0-Tconnectedpart material tree contains AE27_677:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-carafe-mdtkkv-2-Tglass material tree contains AE27_1646:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-hingeless_jar-jdwvyt-0-Tglass material tree contains AE27_677:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-water_glass-ggpnlr-2-Tglass material tree contains AE27_1646:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-cap-oyofsv-0-Tconnectedpart material tree contains AE27_677:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-water_glass-ggpnlr-3-Tglass material tree contains AE27_1646:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    wall_mounted_light-yylrlu-1 material tree contains AE27_226:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-clothesline_pole-smrqrw-0 material tree contains Material #8:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    wall_mounted_light-yylrlu-2 material tree contains AE27_226:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-clothesline_pole-smrqrw-1 material tree contains Material #8:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    wall_mounted_light-yylrlu-3 material tree contains AE27_226:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-bottom_cabinet_no_top-gjrero-0-link_0-base_link-R-lower material tree contains AE27_554:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    wall_mounted_light-yylrlu-4 material tree contains AE27_226:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-bottom_cabinet_no_top-gjrero-0-link_1-base_link-R-lower material tree contains AE27_554:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    mirror-ffoudz-0 material tree contains AE27_341:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-bottom_cabinet_no_top-gjrero-1-link_0-base_link-R-lower material tree contains AE27_554:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-pot_plant-lligdl-1 material tree contains AE27_1611:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-bottom_cabinet_no_top-gjrero-1-link_1-base_link-R-lower material tree contains AE27_554:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-pot_plant-lligdl-2 material tree contains AE27_1611:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-bottom_cabinet_no_top-gjrero-2-link_0-base_link-R-lower material tree contains AE27_554:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    C-mug-kitxam-0 material tree contains AE27_1324:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-bottom_cabinet_no_top-gjrero-2-link_1-base_link-R-lower material tree contains AE27_554:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    C-mug-kitxam-0 material tree contains AE27_1320:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-bottom_cabinet_no_top-gjrero-3-link_0-base_link-R-lower material tree contains AE27_554:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    C-shears-glbxff-0 material tree contains AE27_1370:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-bottom_cabinet_no_top-gjrero-3-link_1-base_link-R-lower material tree contains AE27_554:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    C-paintbrush-tqbfsl-0 material tree contains AE27_1370:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-bottom_cabinet_no_top-gjrero-4-link_0-base_link-R-lower material tree contains AE27_554:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    C-paintbrush-tnchbi-0 material tree contains AE27_1370:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-bottom_cabinet_no_top-gjrero-4-link_1-base_link-R-lower material tree contains AE27_554:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    C-paintbrush-qkuaff-0 material tree contains AE27_1370:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-bottom_cabinet_no_top-gjrero-5-link_0-base_link-R-lower material tree contains AE27_554:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    C-paintbrush-hkwatn-0 material tree contains AE27_1370:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-bottom_cabinet_no_top-gjrero-5-link_1-base_link-R-lower material tree contains AE27_554:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    C-beaker-qwoqqr-0 material tree contains AE27_1370:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-bottom_cabinet_no_top-gjrero-6-link_0-base_link-R-lower material tree contains AE27_554:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    room_light-dddvjo-1 material tree contains AE27_367:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-bottom_cabinet_no_top-gjrero-6-link_1-base_link-R-lower material tree contains AE27_554:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    room_light-dddvjo-1 material tree contains AE27_366:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-bottom_cabinet_no_top-gjrero-7-link_0-base_link-R-lower material tree contains AE27_554:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    room_light-dddvjo-1 material tree contains AE27_369:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-bottom_cabinet_no_top-gjrero-7-link_1-base_link-R-lower material tree contains AE27_554:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    C-salt_shaker-fztfsi-0 material tree contains AE27_653:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-bottom_cabinet_no_top-gjrero-8-link_0-base_link-R-lower material tree contains AE27_554:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    C-salt_shaker-fztfsi-0 material tree contains AE27_652:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-bottom_cabinet_no_top-gjrero-8-link_1-base_link-R-lower material tree contains AE27_554:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-pot_plant-yfqgfd-0 material tree contains AE27_539:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-bottom_cabinet_no_top-gjrero-9-link_0-base_link-R-lower material tree contains AE27_554:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-pot_plant-yfqgfd-0 material tree contains AE27_545:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-bottom_cabinet_no_top-gjrero-9-link_1-base_link-R-lower material tree contains AE27_554:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-pot_plant-yfqgfd-0 material tree contains AE27_540:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-bottom_cabinet_no_top-gjrero-10-link_0-base_link-R-lower material tree contains AE27_554:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-pot_plant-yfqgfd-0 material tree contains AE27_538:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-bottom_cabinet_no_top-gjrero-10-link_1-base_link-R-lower material tree contains AE27_554:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-pot_plant-yfqgfd-0 material tree contains AE27_541:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-vase-rfegnv-1 material tree contains AE27_676:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pillow-inhmhc-0 material tree contains AE27_690:VRayFastSSS2 with disallowed type VRayFastSSS2. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    C-vase-rfegnv-1 material tree contains AE27_677:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-straight_chair-rljebp-0 material tree contains AE27_92:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-laptop-izydvb-0-screen-base_link-R-lower material tree contains AE27_1379:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-straight_chair-rljebp-1 material tree contains AE27_92:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-laptop-izydvb-0-screen-base_link-R-upper material tree contains AE27_1379:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    electric_switch-gashan-2 material tree contains AE27_153:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    bottom_cabinet-fancyy-0-door1-base_link-R-lower material tree contains AE27_554:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    electric_switch-gashan-0 material tree contains AE27_153:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    bottom_cabinet-fancyy-0-door2-base_link-R-lower material tree contains AE27_554:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    electric_switch-gashan-3 material tree contains AE27_153:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    bottom_cabinet-fancyy-0-door3-base_link-R-lower material tree contains AE27_554:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    electric_switch-gashan-4 material tree contains AE27_153:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    bottom_cabinet-fancyy-0-door1-base_link-R-upper material tree contains AE27_554:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    electric_switch-gashan-7 material tree contains AE27_153:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    bottom_cabinet-fancyy-0-door2-base_link-R-upper material tree contains AE27_554:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    electric_switch-gashan-8 material tree contains AE27_153:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    bottom_cabinet-fancyy-0-door3-base_link-R-upper material tree contains AE27_554:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    electric_switch-gashan-9 material tree contains AE27_153:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-bottom_cabinet-rhdbzv-0-base_link material tree contains AM_144_048_022:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    electric_switch-gashan-10 material tree contains AE27_153:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-bottom_cabinet-rhdbzv-0-left_drawer-base_link-P-lower material tree contains AM_144_048_023:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    electric_switch-gashan-11 material tree contains AE27_153:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-bottom_cabinet-rhdbzv-0-left_drawer-base_link-P-lower material tree contains AM_144_048_022:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    electric_switch-gashan-12 material tree contains AE27_153:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-bottom_cabinet-rhdbzv-0-left_drawer-base_link-P-lower material tree contains AM_144_048_024:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    electric_switch-gashan-13 material tree contains AE27_153:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-bottom_cabinet-rhdbzv-0-right_drawer-base_link-P-lower material tree contains AM_144_048_023:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    electric_switch-gashan-14 material tree contains AE27_153:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-bottom_cabinet-rhdbzv-0-right_drawer-base_link-P-lower material tree contains AM_144_048_022:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    electric_switch-gashan-15 material tree contains AE27_153:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-bottom_cabinet-rhdbzv-0-right_drawer-base_link-P-lower material tree contains AM_144_048_024:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    electric_switch-gashan-16 material tree contains AE27_153:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-bottom_cabinet-rhdbzv-1-base_link material tree contains AM_144_048_022:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    electric_switch-gashan-17 material tree contains AE27_153:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-bottom_cabinet-rhdbzv-1-left_drawer-base_link-P-lower material tree contains AM_144_048_023:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    electric_switch-gashan-18 material tree contains AE27_153:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-bottom_cabinet-rhdbzv-1-left_drawer-base_link-P-lower material tree contains AM_144_048_022:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.


## scenes/Ihlen_0_int
  ### (.*) material tree contains .* with disallowed type .*. It should be one of .*.: 1
    floors-qrzmkt-0 material tree contains Tiles_Simple_J01_100cm #0:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.


## scenes/Merom_0_garden
  ### (.*) material tree contains .* with disallowed type .*. It should be one of .*.: 4
    gate-jnzgyw-0-leaf-base_link-R-lower material tree contains 05 - BLACK STEEL2:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    gate-jnzgyw-0-leaf-base_link-R-upper material tree contains 05 - BLACK STEEL2:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    gate-bzotkq-0-leaf-base_link-R-upper material tree contains 05 - BLACK STEEL2:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    gate-bzotkq-0-leaf-base_link-R-lower material tree contains 05 - BLACK STEEL2:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.


## scenes/office_bike
  ### (.*) material tree contains .* with disallowed type .*. It should be one of .*.: 140
    C-hamper-mkxkwm-21 material tree contains AI36_006_124:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-table_lamp-qsajsm-13 material tree contains AI36_006_108:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-hamper-mkxkwm-18 material tree contains AI36_006_124:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-table_lamp-qsajsm-13 material tree contains AI36_006_107:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-hamper-mkxkwm-19 material tree contains AI36_006_124:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-table_lamp-qsajsm-5 material tree contains AI36_006_108:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-hamper-mkxkwm-20 material tree contains AI36_006_124:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-table_lamp-qsajsm-5 material tree contains AI36_006_107:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-hamper-mkxkwm-9 material tree contains AI36_006_124:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-table_lamp-qsajsm-7 material tree contains AI36_006_108:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-hamper-mkxkwm-11 material tree contains AI36_006_124:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-table_lamp-qsajsm-7 material tree contains AI36_006_107:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-hamper-mkxkwm-1 material tree contains AI36_006_124:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-table_lamp-qsajsm-3 material tree contains AI36_006_108:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-hamper-mkxkwm-3 material tree contains AI36_006_124:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-table_lamp-qsajsm-3 material tree contains AI36_006_107:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-hamper-mkxkwm-5 material tree contains AI36_006_124:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-table_lamp-qsajsm-0 material tree contains AI36_006_108:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-hamper-mkxkwm-7 material tree contains AI36_006_124:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-table_lamp-qsajsm-0 material tree contains AI36_006_107:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-hamper-mkxkwm-16 material tree contains AI36_006_124:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-table_lamp-qsajsm-2 material tree contains AI36_006_108:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-hamper-mkxkwm-13 material tree contains AI36_006_124:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-table_lamp-qsajsm-2 material tree contains AI36_006_107:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-hamper-mkxkwm-17 material tree contains AI36_006_124:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-table_lamp-qsajsm-4 material tree contains AI36_006_108:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-hamper-mkxkwm-14 material tree contains AI36_006_124:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-table_lamp-qsajsm-4 material tree contains AI36_006_107:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-taboret-zxoexh-1 material tree contains AI36_006_151:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    C-table_lamp-qsajsm-6 material tree contains AI36_006_108:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-taboret-zxoexh-0 material tree contains AI36_006_151:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    C-table_lamp-qsajsm-6 material tree contains AI36_006_107:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-desk-nafjjg-0 material tree contains AI36_006_121:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-table_lamp-qsajsm-8 material tree contains AI36_006_108:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-desk-nafjjg-2 material tree contains AI36_006_121:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-table_lamp-qsajsm-8 material tree contains AI36_006_107:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-desk-nafjjg-4 material tree contains AI36_006_121:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-table_lamp-qsajsm-10 material tree contains AI36_006_108:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-desk-nafjjg-6 material tree contains AI36_006_121:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-table_lamp-qsajsm-10 material tree contains AI36_006_107:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-desk-nafjjg-8 material tree contains AI36_006_121:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-table_lamp-qsajsm-12 material tree contains AI36_006_108:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-desk-nafjjg-10 material tree contains AI36_006_121:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-table_lamp-qsajsm-12 material tree contains AI36_006_107:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-desk-nafjjg-12 material tree contains AI36_006_121:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-table_lamp-qsajsm-15 material tree contains AI36_006_108:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-desk-nafjjg-14 material tree contains AI36_006_121:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-table_lamp-qsajsm-15 material tree contains AI36_006_107:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-desk-nafjjg-16 material tree contains AI36_006_121:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-table_lamp-qsajsm-18 material tree contains AI36_006_108:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-desk-nafjjg-18 material tree contains AI36_006_121:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-table_lamp-qsajsm-18 material tree contains AI36_006_107:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-desk-nafjjg-21 material tree contains AI36_006_121:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-table_lamp-qsajsm-19 material tree contains AI36_006_108:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-desk-nafjjg-27 material tree contains AI36_006_121:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-table_lamp-qsajsm-19 material tree contains AI36_006_107:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-desk-nafjjg-24 material tree contains AI36_006_121:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-table_lamp-qsajsm-21 material tree contains AI36_006_108:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-architectural_plan-upkcfq-0 material tree contains AI36_006_62:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-desk-nafjjg-28 material tree contains AI36_006_121:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-table_lamp-qsajsm-21 material tree contains AI36_006_107:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    ceilings-eiskrn-0 material tree contains AI36_006_05:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-desk-nafjjg-29 material tree contains AI36_006_121:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-table_lamp-qsajsm-20 material tree contains AI36_006_108:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-architectural_plan-upkcfq-1 material tree contains AI36_006_62:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-desk-nafjjg-30 material tree contains AI36_006_121:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-table_lamp-qsajsm-20 material tree contains AI36_006_107:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-monitor-acltvl-21 material tree contains AI36_006_93:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-desk-nafjjg-31 material tree contains AI36_006_121:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-table_lamp-qsajsm-11 material tree contains AI36_006_108:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-monitor-acltvl-20 material tree contains AI36_006_93:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-desk-nafjjg-15 material tree contains AI36_006_121:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-table_lamp-qsajsm-11 material tree contains AI36_006_107:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-monitor-acltvl-19 material tree contains AI36_006_93:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-desk-nafjjg-13 material tree contains AI36_006_121:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-table_lamp-qsajsm-9 material tree contains AI36_006_108:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-monitor-acltvl-18 material tree contains AI36_006_93:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-desk-nafjjg-32 material tree contains AI36_006_121:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-table_lamp-qsajsm-9 material tree contains AI36_006_107:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-monitor-acltvl-17 material tree contains AI36_006_93:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-desk-nafjjg-17 material tree contains AI36_006_121:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-table_lamp-qsajsm-1 material tree contains AI36_006_108:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-monitor-acltvl-16 material tree contains AI36_006_93:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-desk-nafjjg-3 material tree contains AI36_006_121:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-table_lamp-qsajsm-1 material tree contains AI36_006_107:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-monitor-acltvl-15 material tree contains AI36_006_93:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-desk-nafjjg-1 material tree contains AI36_006_121:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pot_plant-jaofyw-0 material tree contains AI36_006_181:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-monitor-acltvl-14 material tree contains AI36_006_93:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-desk-nafjjg-5 material tree contains AI36_006_121:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pot_plant-ztnfnw-0 material tree contains AI36_006_177:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-monitor-acltvl-13 material tree contains AI36_006_93:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-desk-nafjjg-9 material tree contains AI36_006_121:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pot_plant-ztnfnw-0 material tree contains AI36_006_175:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-monitor-acltvl-12 material tree contains AI36_006_93:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-desk-nafjjg-7 material tree contains AI36_006_121:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pot_plant-ztnfnw-0 material tree contains AI36_006_172:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-monitor-acltvl-11 material tree contains AI36_006_93:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-desk-nafjjg-11 material tree contains AI36_006_121:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pot_plant-ztnfnw-0 material tree contains AI36_006_173:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-monitor-acltvl-10 material tree contains AI36_006_93:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-desk-nafjjg-26 material tree contains AI36_006_121:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pot_plant-ztnfnw-0 material tree contains AI36_006_169:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-monitor-acltvl-9 material tree contains AI36_006_93:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-desk-nafjjg-20 material tree contains AI36_006_121:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    ceilings-fvacxv-0 material tree contains AI36_006_05:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-monitor-acltvl-8 material tree contains AI36_006_93:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-desk-nafjjg-23 material tree contains AI36_006_121:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-monitor-acltvl-7 material tree contains AI36_006_93:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-desk-nafjjg-25 material tree contains AI36_006_121:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-monitor-acltvl-6 material tree contains AI36_006_93:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-desk-nafjjg-19 material tree contains AI36_006_121:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-monitor-acltvl-5 material tree contains AI36_006_93:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-desk-nafjjg-22 material tree contains AI36_006_121:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-monitor-acltvl-4 material tree contains AI36_006_93:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    vending_machine-wogoig-1 material tree contains AI36_006_50:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-monitor-acltvl-3 material tree contains AI36_006_93:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    vending_machine-wogoig-1 material tree contains AI36_006_53:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-monitor-acltvl-2 material tree contains AI36_006_93:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    vending_machine-wogoig-1 material tree contains AI36_006_54:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-monitor-acltvl-1 material tree contains AI36_006_93:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    vending_machine-wogoig-0 material tree contains AI36_006_50:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-monitor-acltvl-0 material tree contains AI36_006_93:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    vending_machine-wogoig-0 material tree contains AI36_006_53:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-hamper-mkxkwm-0 material tree contains AI36_006_124:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    vending_machine-wogoig-0 material tree contains AI36_006_54:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-hamper-mkxkwm-2 material tree contains AI36_006_124:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-pedestal_table-gkkxaj-0 material tree contains AI36_006_162:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-hamper-mkxkwm-4 material tree contains AI36_006_124:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-table_lamp-qsajsm-17 material tree contains AI36_006_108:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-hamper-mkxkwm-6 material tree contains AI36_006_124:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-table_lamp-qsajsm-17 material tree contains AI36_006_107:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-hamper-mkxkwm-8 material tree contains AI36_006_124:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-table_lamp-qsajsm-14 material tree contains AI36_006_108:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-hamper-mkxkwm-10 material tree contains AI36_006_124:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-table_lamp-qsajsm-14 material tree contains AI36_006_107:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-hamper-mkxkwm-12 material tree contains AI36_006_124:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-table_lamp-qsajsm-16 material tree contains AI36_006_108:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-hamper-mkxkwm-15 material tree contains AI36_006_124:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-table_lamp-qsajsm-16 material tree contains AI36_006_107:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.


## scenes/office_cubicles_left
  ### (.*) material tree contains .* with disallowed type .*. It should be one of .*.: 222
    C-pot_plant-iblutb-4 material tree contains 64_mat(2002192):VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-downlight-hpkvem-28 material tree contains 311_mat(2002192):VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-odlphz-19 material tree contains 331_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-downlight-hpkvem-38 material tree contains 320_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    rectangular_light-jnnhzm-4 material tree contains 324_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pot_plant-dbjcic-0 material tree contains 314_mat(2002192):VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-odlphz-27 material tree contains 331_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-downlight-hpkvem-38 material tree contains 311_mat(2002192):VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    rectangular_light-jnnhzm-5 material tree contains 324_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pot_plant-dbjcic-0 material tree contains 315_mat(2002192):VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-odlphz-2 material tree contains 331_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-downlight-hpkvem-39 material tree contains 320_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    rectangular_light-jnnhzm-2 material tree contains 324_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pot_plant-dbjcic-0 material tree contains 316_mat(2002192):VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-odlphz-14 material tree contains 331_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-downlight-hpkvem-39 material tree contains 311_mat(2002192):VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    rectangular_light-jnnhzm-1 material tree contains 324_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pot_plant-dbjcic-6 material tree contains 314_mat(2002192):VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-odlphz-22 material tree contains 331_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-downlight-hpkvem-40 material tree contains 320_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    spotlight-vojnfz-0 material tree contains 328_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pot_plant-dbjcic-6 material tree contains 315_mat(2002192):VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-odlphz-1 material tree contains 331_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-downlight-hpkvem-40 material tree contains 311_mat(2002192):VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    spotlight-vojnfz-1 material tree contains 328_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pot_plant-dbjcic-6 material tree contains 316_mat(2002192):VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-odlphz-10 material tree contains 331_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-downlight-hpkvem-41 material tree contains 320_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    spotlight-vojnfz-2 material tree contains 328_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-table_lamp-gycqmt-0 material tree contains 323_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-odlphz-18 material tree contains 331_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-downlight-hpkvem-41 material tree contains 311_mat(2002192):VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-xmgdee-0 material tree contains 62_mat(2002192):VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pot_plant-dbjcic-3 material tree contains 314_mat(2002192):VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-odlphz-26 material tree contains 331_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-downlight-hpkvem-42 material tree contains 320_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-downlight-hpkvem-13 material tree contains 320_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pot_plant-dbjcic-3 material tree contains 315_mat(2002192):VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-odlphz-9 material tree contains 331_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-downlight-hpkvem-42 material tree contains 311_mat(2002192):VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-downlight-hpkvem-13 material tree contains 311_mat(2002192):VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pot_plant-dbjcic-3 material tree contains 316_mat(2002192):VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-odlphz-0 material tree contains 331_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-downlight-hpkvem-43 material tree contains 320_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-downlight-hpkvem-27 material tree contains 320_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pot_plant-dbjcic-5 material tree contains 314_mat(2002192):VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    rectangular_light-nsjice-0 material tree contains 324_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-downlight-hpkvem-43 material tree contains 311_mat(2002192):VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-downlight-hpkvem-27 material tree contains 311_mat(2002192):VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pot_plant-dbjcic-5 material tree contains 315_mat(2002192):VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    rectangular_light-nsjice-1 material tree contains 324_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-downlight-hpkvem-44 material tree contains 320_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-downlight-hpkvem-1 material tree contains 320_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pot_plant-dbjcic-5 material tree contains 316_mat(2002192):VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    rectangular_light-nsjice-4 material tree contains 324_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-downlight-hpkvem-44 material tree contains 311_mat(2002192):VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-downlight-hpkvem-1 material tree contains 311_mat(2002192):VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-table_lamp-gycqmt-3 material tree contains 323_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    rectangular_light-nsjice-2 material tree contains 324_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-ojzcvk-0 material tree contains 62_mat(2002192):VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-downlight-hpkvem-2 material tree contains 320_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pot_plant-dbjcic-8 material tree contains 314_mat(2002192):VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    rectangular_light-nsjice-5 material tree contains 324_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-glpnus-0 material tree contains 62_mat(2002192):VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-downlight-hpkvem-2 material tree contains 311_mat(2002192):VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pot_plant-dbjcic-8 material tree contains 315_mat(2002192):VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    rectangular_light-nsjice-3 material tree contains 324_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-aruzuz-0 material tree contains 62_mat(2002192):VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-downlight-hpkvem-21 material tree contains 320_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pot_plant-dbjcic-8 material tree contains 316_mat(2002192):VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    rectangular_light-jnnhzm-3 material tree contains 324_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-zegxds-0 material tree contains 62_mat(2002192):VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-downlight-hpkvem-21 material tree contains 311_mat(2002192):VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pot_plant-dbjcic-9 material tree contains 314_mat(2002192):VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-downlight-hpkvem-25 material tree contains 320_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-besanj-0 material tree contains 62_mat(2002192):VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-downlight-hpkvem-32 material tree contains 320_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pot_plant-dbjcic-9 material tree contains 315_mat(2002192):VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-downlight-hpkvem-25 material tree contains 311_mat(2002192):VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-downlight-hpkvem-32 material tree contains 311_mat(2002192):VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pot_plant-dbjcic-9 material tree contains 316_mat(2002192):VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-downlight-hpkvem-19 material tree contains 320_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-downlight-hpkvem-11 material tree contains 320_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-table_lamp-gycqmt-2 material tree contains 323_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-downlight-hpkvem-19 material tree contains 311_mat(2002192):VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-downlight-hpkvem-11 material tree contains 311_mat(2002192):VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pot_plant-dbjcic-10 material tree contains 314_mat(2002192):VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-downlight-hpkvem-23 material tree contains 320_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-downlight-hpkvem-12 material tree contains 320_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pot_plant-dbjcic-10 material tree contains 315_mat(2002192):VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-downlight-hpkvem-23 material tree contains 311_mat(2002192):VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-downlight-hpkvem-12 material tree contains 311_mat(2002192):VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pot_plant-dbjcic-10 material tree contains 316_mat(2002192):VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-downlight-hpkvem-9 material tree contains 320_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-downlight-hpkvem-26 material tree contains 320_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pot_plant-dbjcic-11 material tree contains 314_mat(2002192):VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-downlight-hpkvem-9 material tree contains 311_mat(2002192):VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-downlight-hpkvem-26 material tree contains 311_mat(2002192):VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pot_plant-dbjcic-11 material tree contains 315_mat(2002192):VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-downlight-hpkvem-29 material tree contains 320_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-downlight-hpkvem-3 material tree contains 320_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pot_plant-dbjcic-11 material tree contains 316_mat(2002192):VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-downlight-hpkvem-29 material tree contains 311_mat(2002192):VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-downlight-hpkvem-3 material tree contains 311_mat(2002192):VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-table_lamp-gycqmt-5 material tree contains 323_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-downlight-hpkvem-17 material tree contains 320_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-downlight-hpkvem-0 material tree contains 320_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pot_plant-dbjcic-1 material tree contains 314_mat(2002192):VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-downlight-hpkvem-17 material tree contains 311_mat(2002192):VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-downlight-hpkvem-0 material tree contains 311_mat(2002192):VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pot_plant-dbjcic-1 material tree contains 315_mat(2002192):VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-downlight-hpkvem-33 material tree contains 320_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-downlight-hpkvem-20 material tree contains 320_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pot_plant-dbjcic-1 material tree contains 316_mat(2002192):VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-downlight-hpkvem-33 material tree contains 311_mat(2002192):VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-downlight-hpkvem-20 material tree contains 311_mat(2002192):VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pot_plant-dbjcic-2 material tree contains 314_mat(2002192):VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-downlight-hpkvem-5 material tree contains 320_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-downlight-hpkvem-31 material tree contains 320_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pot_plant-dbjcic-2 material tree contains 315_mat(2002192):VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-downlight-hpkvem-5 material tree contains 311_mat(2002192):VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-downlight-hpkvem-31 material tree contains 311_mat(2002192):VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pot_plant-dbjcic-2 material tree contains 316_mat(2002192):VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-znxlvq-0 material tree contains 62_mat(2002192):VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-downlight-hpkvem-7 material tree contains 320_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-table_lamp-gycqmt-4 material tree contains 323_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-oxwqrw-0 material tree contains 62_mat(2002192):VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-downlight-hpkvem-7 material tree contains 311_mat(2002192):VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pot_plant-dbjcic-4 material tree contains 314_mat(2002192):VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-ydhbul-0 material tree contains 62_mat(2002192):VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-downlight-hpkvem-10 material tree contains 320_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pot_plant-dbjcic-4 material tree contains 315_mat(2002192):VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-tsqhlh-0 material tree contains 62_mat(2002192):VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-downlight-hpkvem-10 material tree contains 311_mat(2002192):VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pot_plant-dbjcic-4 material tree contains 316_mat(2002192):VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-qtumre-0 material tree contains 62_mat(2002192):VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-downlight-hpkvem-24 material tree contains 320_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pot_plant-dbjcic-7 material tree contains 314_mat(2002192):VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-kraeyl-0 material tree contains 62_mat(2002192):VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-downlight-hpkvem-24 material tree contains 311_mat(2002192):VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pot_plant-dbjcic-7 material tree contains 315_mat(2002192):VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-wmzbql-0 material tree contains 62_mat(2002192):VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-downlight-hpkvem-8 material tree contains 320_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pot_plant-dbjcic-7 material tree contains 316_mat(2002192):VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-muywul-0 material tree contains 62_mat(2002192):VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-downlight-hpkvem-8 material tree contains 311_mat(2002192):VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-table_lamp-gycqmt-1 material tree contains 323_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-xubdst-0 material tree contains 62_mat(2002192):VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-downlight-hpkvem-16 material tree contains 320_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-odlphz-8 material tree contains 331_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    floors-nhcbxy-0 material tree contains 62_mat(2002192):VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-downlight-hpkvem-16 material tree contains 311_mat(2002192):VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-odlphz-17 material tree contains 331_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    floors-hgmnot-0 material tree contains 62_mat(2002192):VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    ceilings-gpnify-0 material tree contains 326_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-downlight-hpkvem-18 material tree contains 320_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-odlphz-25 material tree contains 331_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    ceilings-ipmtjd-0 material tree contains 326_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-downlight-hpkvem-18 material tree contains 311_mat(2002192):VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-legal_document-uchhwn-0 material tree contains 317_mat(2002192):VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-odlphz-7 material tree contains 331_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    ceilings-pmniqn-0 material tree contains 326_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-legal_document-uchhwn-0 material tree contains 313_mat(2002192):VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-downlight-hpkvem-30 material tree contains 320_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-odlphz-13 material tree contains 331_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    ceilings-pqkmzt-0 material tree contains 326_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-printer-kqmlho-0 material tree contains 310_mat(2002192):VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-downlight-hpkvem-30 material tree contains 311_mat(2002192):VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-odlphz-21 material tree contains 331_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    ceilings-emgmmn-0 material tree contains 326_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-printer-kqmlho-0 material tree contains 309_mat(2002192):VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-downlight-hpkvem-4 material tree contains 320_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-odlphz-29 material tree contains 331_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    ceilings-mkpnye-0 material tree contains 326_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-printer-kqmlho-0 material tree contains 308_mat(2002192):VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-downlight-hpkvem-4 material tree contains 311_mat(2002192):VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-odlphz-6 material tree contains 331_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    ceilings-byckax-0 material tree contains 326_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    rectangular_light-jnnhzm-0 material tree contains 324_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-downlight-hpkvem-6 material tree contains 320_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-odlphz-16 material tree contains 331_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    ceilings-qsmgcw-0 material tree contains 326_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-swivel_chair-uauzsu-0 material tree contains 63_mat(2002192):VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-downlight-hpkvem-6 material tree contains 311_mat(2002192):VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-odlphz-24 material tree contains 331_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    ceilings-heevjg-0 material tree contains 326_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-swivel_chair-uauzsu-3 material tree contains 63_mat(2002192):VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-downlight-hpkvem-22 material tree contains 320_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-odlphz-5 material tree contains 331_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    ceilings-ztkfuh-0 material tree contains 326_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-swivel_chair-uauzsu-4 material tree contains 63_mat(2002192):VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-downlight-hpkvem-22 material tree contains 311_mat(2002192):VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-odlphz-12 material tree contains 331_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    ceilings-cxkvpr-0 material tree contains 326_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-swivel_chair-uauzsu-5 material tree contains 63_mat(2002192):VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-downlight-hpkvem-34 material tree contains 320_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-odlphz-20 material tree contains 331_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    ceilings-lrvszp-0 material tree contains 326_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-swivel_chair-uauzsu-1 material tree contains 63_mat(2002192):VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-downlight-hpkvem-34 material tree contains 311_mat(2002192):VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-odlphz-28 material tree contains 331_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-downlight-hpkvem-35 material tree contains 320_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-swivel_chair-uauzsu-2 material tree contains 63_mat(2002192):VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-downlight-hpkvem-14 material tree contains 320_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-odlphz-4 material tree contains 331_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-downlight-hpkvem-35 material tree contains 311_mat(2002192):VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pot_plant-iblutb-0 material tree contains 64_mat(2002192):VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-downlight-hpkvem-14 material tree contains 311_mat(2002192):VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-odlphz-15 material tree contains 331_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-downlight-hpkvem-36 material tree contains 320_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pot_plant-iblutb-1 material tree contains 64_mat(2002192):VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-downlight-hpkvem-15 material tree contains 320_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-odlphz-23 material tree contains 331_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-downlight-hpkvem-36 material tree contains 311_mat(2002192):VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pot_plant-iblutb-2 material tree contains 64_mat(2002192):VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-downlight-hpkvem-15 material tree contains 311_mat(2002192):VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-odlphz-3 material tree contains 331_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-downlight-hpkvem-37 material tree contains 320_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pot_plant-iblutb-3 material tree contains 64_mat(2002192):VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-downlight-hpkvem-28 material tree contains 320_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-odlphz-11 material tree contains 331_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-downlight-hpkvem-37 material tree contains 311_mat(2002192):VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.


## scenes/office_cubicles_right
  ### (.*) material tree contains .* with disallowed type .*. It should be one of .*.: 122
    downlight-hpkvem-40 material tree contains 311_mat(2002192):VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-odlphz-13 material tree contains 331_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    downlight-hpkvem-40 material tree contains 320_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-odlphz-1 material tree contains 331_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    downlight-hpkvem-2 material tree contains 311_mat(2002192):VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-odlphz-2 material tree contains 331_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    downlight-hpkvem-2 material tree contains 320_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-odlphz-15 material tree contains 331_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    downlight-hpkvem-7 material tree contains 311_mat(2002192):VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-odlphz-8 material tree contains 331_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    downlight-hpkvem-7 material tree contains 320_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hpkvem-27 material tree contains 311_mat(2002192):VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hpkvem-13 material tree contains 311_mat(2002192):VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hpkvem-27 material tree contains 320_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hpkvem-13 material tree contains 320_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hpkvem-32 material tree contains 311_mat(2002192):VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hpkvem-16 material tree contains 311_mat(2002192):VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hpkvem-32 material tree contains 320_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hpkvem-16 material tree contains 320_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hpkvem-1 material tree contains 311_mat(2002192):VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hpkvem-20 material tree contains 311_mat(2002192):VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hpkvem-1 material tree contains 320_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hpkvem-20 material tree contains 320_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hpkvem-37 material tree contains 311_mat(2002192):VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hpkvem-24 material tree contains 311_mat(2002192):VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hpkvem-37 material tree contains 320_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hpkvem-24 material tree contains 320_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hpkvem-6 material tree contains 311_mat(2002192):VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hpkvem-26 material tree contains 311_mat(2002192):VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hpkvem-6 material tree contains 320_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hpkvem-26 material tree contains 320_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hpkvem-9 material tree contains 311_mat(2002192):VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hpkvem-29 material tree contains 311_mat(2002192):VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hpkvem-9 material tree contains 320_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hpkvem-29 material tree contains 320_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hpkvem-18 material tree contains 311_mat(2002192):VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hpkvem-30 material tree contains 311_mat(2002192):VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hpkvem-18 material tree contains 320_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    ceilings-lezgrj-0 material tree contains 326_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hpkvem-30 material tree contains 320_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hpkvem-22 material tree contains 311_mat(2002192):VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-odlphz-3 material tree contains 331_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    downlight-hpkvem-33 material tree contains 311_mat(2002192):VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hpkvem-22 material tree contains 320_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-naavhr-0 material tree contains 62_mat(2002192):VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hpkvem-33 material tree contains 320_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hpkvem-31 material tree contains 311_mat(2002192):VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-printer-kqzshp-0 material tree contains 312_mat(2002192):VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hpkvem-34 material tree contains 311_mat(2002192):VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hpkvem-31 material tree contains 320_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hpkvem-0 material tree contains 311_mat(2002192):VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hpkvem-34 material tree contains 320_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hpkvem-35 material tree contains 311_mat(2002192):VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hpkvem-0 material tree contains 320_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hpkvem-36 material tree contains 311_mat(2002192):VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hpkvem-35 material tree contains 320_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hpkvem-21 material tree contains 311_mat(2002192):VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hpkvem-36 material tree contains 320_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hpkvem-39 material tree contains 311_mat(2002192):VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hpkvem-21 material tree contains 320_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hpkvem-38 material tree contains 311_mat(2002192):VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hpkvem-39 material tree contains 320_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hpkvem-25 material tree contains 311_mat(2002192):VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hpkvem-38 material tree contains 320_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hpkvem-4 material tree contains 311_mat(2002192):VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hpkvem-25 material tree contains 320_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hpkvem-10 material tree contains 311_mat(2002192):VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hpkvem-4 material tree contains 320_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hpkvem-28 material tree contains 311_mat(2002192):VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hpkvem-10 material tree contains 320_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-vxanqe-0 material tree contains 62_mat(2002192):VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hpkvem-28 material tree contains 320_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hpkvem-12 material tree contains 311_mat(2002192):VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-lmgvcb-0 material tree contains 62_mat(2002192):VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hpkvem-12 material tree contains 320_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-schwxn-0 material tree contains 62_mat(2002192):VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hpkvem-15 material tree contains 311_mat(2002192):VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-furgou-0 material tree contains 62_mat(2002192):VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hpkvem-15 material tree contains 320_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-curkiw-0 material tree contains 62_mat(2002192):VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hpkvem-19 material tree contains 311_mat(2002192):VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-zfaowb-0 material tree contains 62_mat(2002192):VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hpkvem-19 material tree contains 320_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-pchrtt-0 material tree contains 62_mat(2002192):VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hpkvem-23 material tree contains 311_mat(2002192):VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-qrrbfo-0 material tree contains 62_mat(2002192):VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hpkvem-23 material tree contains 320_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-vrggjj-0 material tree contains 62_mat(2002192):VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hpkvem-3 material tree contains 311_mat(2002192):VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    ceilings-vsoozg-0 material tree contains 326_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hpkvem-3 material tree contains 320_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    ceilings-pqcxos-0 material tree contains 326_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hpkvem-5 material tree contains 311_mat(2002192):VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    ceilings-igbhdx-0 material tree contains 326_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hpkvem-5 material tree contains 320_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    ceilings-vonsdl-0 material tree contains 326_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hpkvem-8 material tree contains 311_mat(2002192):VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    ceilings-unwrsb-0 material tree contains 326_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hpkvem-8 material tree contains 320_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    ceilings-stmwcr-0 material tree contains 326_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hpkvem-11 material tree contains 311_mat(2002192):VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    ceilings-wlglon-0 material tree contains 326_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hpkvem-11 material tree contains 320_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    ceilings-bkqhio-0 material tree contains 326_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hpkvem-14 material tree contains 311_mat(2002192):VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    ceilings-vhsxcg-0 material tree contains 326_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hpkvem-14 material tree contains 320_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    ceilings-xvlngn-0 material tree contains 326_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hpkvem-17 material tree contains 311_mat(2002192):VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-hpkvem-17 material tree contains 320_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-odlphz-6 material tree contains 331_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-square_light-odlphz-0 material tree contains 331_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-square_light-odlphz-10 material tree contains 331_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-square_light-odlphz-14 material tree contains 331_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-square_light-odlphz-17 material tree contains 331_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-square_light-odlphz-5 material tree contains 331_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-square_light-odlphz-12 material tree contains 331_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-square_light-odlphz-4 material tree contains 331_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-square_light-odlphz-11 material tree contains 331_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-square_light-odlphz-16 material tree contains 331_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-square_light-odlphz-7 material tree contains 331_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-square_light-odlphz-9 material tree contains 331_mat(2002192):VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.


## scenes/office_large
  ### (.*) material tree contains .* with disallowed type .*. It should be one of .*.: 15
    C-laptop-rocupn-0 material tree contains vray_keyboard:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    rectangular_light-lmicjz-0 material tree contains 20 - Default:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-laptop-rocupn-0-screen-base_link-R-upper material tree contains vray_holes:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-fbuipj-0 material tree contains Bricks_Tiles_D01_50cm #0:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    C-floor_lamp-lnacuc-0 material tree contains Material #827713059:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-laptop-rocupn-0 material tree contains screen:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-laptop-rocupn-0-screen-base_link-R-lower material tree contains vray_holes:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-laptop-rocupn-0-screen-base_link-R-upper material tree contains vray_keyboard:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-floor_lamp-lnacuc-1 material tree contains Material #827713059:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    rectangular_light-lmicjz-1 material tree contains 20 - Default:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    rectangular_light-lmicjz-2 material tree contains 20 - Default:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-laptop-rocupn-0-screen-base_link-R-lower material tree contains vray_keyboard:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-laptop-rocupn-0-screen-base_link-R-lower material tree contains screen:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-laptop-rocupn-0-screen-base_link-R-upper material tree contains screen:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-laptop-rocupn-0 material tree contains vray_holes:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.


## scenes/office_vendor_machine
  ### (.*) material tree contains .* with disallowed type .*. It should be one of .*.: 88
    C-monitor-fpkdou-2 material tree contains AI33_008_282:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-bookcase-zdsmhe-1 material tree contains AI33_008_35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    C-monitor-fpkdou-3 material tree contains AI33_008_282:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-bookcase-zdsmhe-2 material tree contains AI33_008_35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    C-monitor-fpkdou-4 material tree contains AI33_008_282:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-dthvxn-0 material tree contains AI33_008_30:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-monitor-fpkdou-5 material tree contains AI33_008_282:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-dthvxn-0 material tree contains AI33_008_25:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-monitor-fpkdou-6 material tree contains AI33_008_282:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-dthvxn-1 material tree contains AI33_008_30:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-monitor-fpkdou-7 material tree contains AI33_008_282:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-dthvxn-1 material tree contains AI33_008_25:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-motorcycle-aocuum-0 material tree contains vray_AI33_008_62:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-dthvxn-2 material tree contains AI33_008_30:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-motorcycle-aocuum-0 material tree contains vray_AI33_008_60:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-dthvxn-2 material tree contains AI33_008_25:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-motorcycle-aocuum-0 material tree contains vray_AI33_008_61:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-dthvxn-3 material tree contains AI33_008_30:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-motorcycle-aocuum-0 material tree contains vray_AI33_008_52:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-dthvxn-3 material tree contains AI33_008_25:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-conference_table-hdomxc-0 material tree contains AI33_008_35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-dthvxn-4 material tree contains AI33_008_30:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-conference_table-hdomxc-0 material tree contains AI33_008_172:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-dthvxn-4 material tree contains AI33_008_25:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-conference_table-hdomxc-1 material tree contains AI33_008_35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-dthvxn-5 material tree contains AI33_008_30:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-conference_table-hdomxc-1 material tree contains AI33_008_172:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-dthvxn-5 material tree contains AI33_008_25:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-bookcase-xsfeyi-0 material tree contains AI33_008_35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    L-bookcase-zdsmhe-0 material tree contains AI33_008_35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    room_light-dthvxn-6 material tree contains AI33_008_30:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-dthvxn-6 material tree contains AI33_008_25:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-bookcase-xsfeyi-1 material tree contains AI33_008_35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    room_light-dthvxn-7 material tree contains AI33_008_30:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-bookcase-xsfeyi-2 material tree contains AI33_008_35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    room_light-dthvxn-7 material tree contains AI33_008_25:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-bookcase-xsfeyi-3 material tree contains AI33_008_35:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    room_light-dthvxn-8 material tree contains AI33_008_30:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    ceilings-qtktaq-0 material tree contains AI33_008_23:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-dthvxn-8 material tree contains AI33_008_25:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    ceilings-qtktaq-0 material tree contains AI33_008_09:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-dthvxn-9 material tree contains AI33_008_30:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    ceilings-yzbfge-0 material tree contains AI33_008_23:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-dthvxn-9 material tree contains AI33_008_25:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    ceilings-yzbfge-0 material tree contains AI33_008_09:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-dthvxn-10 material tree contains AI33_008_30:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    ceilings-lvnwja-0 material tree contains AI33_008_23:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-dthvxn-10 material tree contains AI33_008_25:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    ceilings-lvnwja-0 material tree contains AI33_008_09:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-dthvxn-11 material tree contains AI33_008_30:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-dthvxn-27 material tree contains AI33_008_30:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-dthvxn-11 material tree contains AI33_008_25:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-dthvxn-27 material tree contains AI33_008_25:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-dthvxn-12 material tree contains AI33_008_30:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-dthvxn-12 material tree contains AI33_008_25:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-dthvxn-13 material tree contains AI33_008_30:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-dthvxn-13 material tree contains AI33_008_25:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-dthvxn-14 material tree contains AI33_008_30:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-dthvxn-14 material tree contains AI33_008_25:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-dthvxn-15 material tree contains AI33_008_30:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-dthvxn-15 material tree contains AI33_008_25:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-dthvxn-16 material tree contains AI33_008_30:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-dthvxn-16 material tree contains AI33_008_25:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-dthvxn-17 material tree contains AI33_008_30:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-dthvxn-17 material tree contains AI33_008_25:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-dthvxn-18 material tree contains AI33_008_30:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-dthvxn-18 material tree contains AI33_008_25:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-dthvxn-19 material tree contains AI33_008_30:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-dthvxn-19 material tree contains AI33_008_25:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-dthvxn-20 material tree contains AI33_008_30:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-dthvxn-20 material tree contains AI33_008_25:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-dthvxn-21 material tree contains AI33_008_30:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-dthvxn-21 material tree contains AI33_008_25:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-dthvxn-22 material tree contains AI33_008_30:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-dthvxn-22 material tree contains AI33_008_25:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-dthvxn-23 material tree contains AI33_008_30:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-dthvxn-23 material tree contains AI33_008_25:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-dthvxn-24 material tree contains AI33_008_30:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-dthvxn-24 material tree contains AI33_008_25:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-dthvxn-25 material tree contains AI33_008_30:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-dthvxn-25 material tree contains AI33_008_25:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-dthvxn-26 material tree contains AI33_008_30:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-dthvxn-26 material tree contains AI33_008_25:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    vending_machine-ocnbil-0 material tree contains AI33_008_94:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    vending_machine-ocnbil-0 material tree contains AI33_008_98:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    vending_machine-ocnbil-0 material tree contains AI33_008_97:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-monitor-fpkdou-0 material tree contains AI33_008_282:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-monitor-fpkdou-1 material tree contains AI33_008_282:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.


## scenes/Pomaria_0_garden
  ### (.*) material tree contains .* with disallowed type .*. It should be one of .*.: 42
    L-flower-mylblj-0 material tree contains AM248_020_flower_04:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bush-waafqw-0 material tree contains iBushes-Leaf:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bush-zwhhlc-0 material tree contains iBushes-Leaf:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-flower-mylblj-0 material tree contains AM248_020_flower_03:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bush-waafqw-0 material tree contains iBushes-Leaf:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bush-zwhhlc-0 material tree contains iBushes-Leaf:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-flower-mylblj-0 material tree contains AM248_020_flower_02:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bush-waafqw-0 material tree contains iBushes-Leaf:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bush-zwhhlc-0 material tree contains iBushes-Leaf:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-flower-mylblj-0 material tree contains AM248_020_flower_12:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bush-jivsft-0 material tree contains iBushes-Leaf:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bush-jonbyb-0 material tree contains iBushes-Leaf:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-flower-mylblj-0 material tree contains AM248_020_flower_10:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bush-jivsft-0 material tree contains iBushes-Leaf:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bush-jonbyb-0 material tree contains iBushes-Leaf:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bush-ujrbvp-0 material tree contains iBushes-Petal:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bush-jivsft-0 material tree contains iBushes-Leaf:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bush-jonbyb-0 material tree contains iBushes-Leaf:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bush-ujrbvp-0 material tree contains iBushes-Leaf:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bush-nhorlu-0 material tree contains iBushes-Leaf:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    greenery-ccisua-0 material tree contains AM248_020_creeper_2s:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bush-ujrbvp-0 material tree contains iBushes-Leaf:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bush-nhorlu-0 material tree contains iBushes-Leaf:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bush-ujrbvp-0 material tree contains iBushes-Leaf:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bush-nhorlu-0 material tree contains iBushes-Leaf:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bush-jylruu-0 material tree contains iBushes-Petal:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bush-rkxgdi-0 material tree contains iBushes-Leaf:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bush-jylruu-0 material tree contains iBushes-Leaf:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bush-rkxgdi-0 material tree contains iBushes-Leaf:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bush-jylruu-0 material tree contains iBushes-Leaf:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-flower-mylblj-0 material tree contains AM248_020_flower_06:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bush-rkxgdi-0 material tree contains iBushes-Leaf:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-flower-mylblj-0 material tree contains AM248_020_flower_17:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bush-jylruu-0 material tree contains iBushes-Leaf:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bush-waafqw-1 material tree contains iBushes-Leaf:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-flower-mylblj-0 material tree contains AM248_020_flower_05:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bush-llcsvc-0 material tree contains iBushes-Leaf:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bush-waafqw-1 material tree contains iBushes-Leaf:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-flower-mylblj-0 material tree contains AM248_020_flower_16:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bush-llcsvc-0 material tree contains iBushes-Leaf:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bush-waafqw-1 material tree contains iBushes-Leaf:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bush-llcsvc-0 material tree contains iBushes-Leaf:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.


## scenes/public_restroom_blue
  ### (.*) material tree contains .* with disallowed type .*. It should be one of .*.: 1
    floors-krgyyj-0 material tree contains Tiles_Simple_J01_100cm #0:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.


## scenes/public_restroom_futuristic
  ### (.*) material tree contains .* with disallowed type .*. It should be one of .*.: 10
    downlight-xhoqgk-1 material tree contains fsdfdsf:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-xhoqgk-5 material tree contains fsdfdsf:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-xhoqgk-9 material tree contains fsdfdsf:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-xhoqgk-8 material tree contains fsdfdsf:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-xhoqgk-2 material tree contains fsdfdsf:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-xhoqgk-6 material tree contains fsdfdsf:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-xhoqgk-0 material tree contains fsdfdsf:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-xhoqgk-3 material tree contains fsdfdsf:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-xhoqgk-7 material tree contains fsdfdsf:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-xhoqgk-4 material tree contains fsdfdsf:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.


## scenes/public_restroom_marble
  ### (.*) material tree contains .* with disallowed type .*. It should be one of .*.: 10
    downlight-jexffy-2 material tree contains svet:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-jexffy-1 material tree contains svet:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-jexffy-4 material tree contains svet:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-jexffy-3 material tree contains svet:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-jexffy-7 material tree contains svet:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-jexffy-6 material tree contains svet:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-jexffy-5 material tree contains svet:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-jexffy-9 material tree contains svet:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-jexffy-0 material tree contains svet:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-jexffy-8 material tree contains svet:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.


## scenes/public_restroom_white
  ### (.*) material tree contains .* with disallowed type .*. It should be one of .*.: 6
    square_light-tgtowc-2 material tree contains Lights_Neon:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-tgtowc-5 material tree contains Lights_Neon:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-tgtowc-1 material tree contains Lights_Neon:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-tgtowc-4 material tree contains Lights_Neon:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-tgtowc-3 material tree contains Lights_Neon:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-tgtowc-0 material tree contains Lights_Neon:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.


## scenes/restaurant_asian
  ### (.*) material tree contains .* with disallowed type .*. It should be one of .*.: 196
    chandelier-qghfol-9 material tree contains Mat3d66-1514387-79-6171:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-can_of_soda-ixrfxv-10 material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-lkxreb-0 material tree contains Mat3d66-1329970-28-3759:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    chandelier-qghfol-10 material tree contains Mat3d66-1514387-79-6171:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-can_of_soda-ixrfxv-11 material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-lkxreb-0 material tree contains Mat3d66-1514387-33-9800:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    chandelier-qghfol-11 material tree contains Mat3d66-1514387-79-6171:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-can_of_soda-ixrfxv-12 material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-etksmo-0 material tree contains Mat3d66-1329970-28-3759:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    chandelier-zgwevt-1 material tree contains Mat3d66-1514387-112-8171:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-can_of_soda-ixrfxv-13 material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-etksmo-0 material tree contains Mat3d66-1514387-33-9800:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    chandelier-zgwevt-2 material tree contains Mat3d66-1514387-112-8171:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-can_of_soda-ixrfxv-14 material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-rikals-0 material tree contains Mat3d66-1329970-28-3759:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    chandelier-zgwevt-3 material tree contains Mat3d66-1514387-112-8171:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-can_of_soda-ixrfxv-15 material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-rikals-0 material tree contains Mat3d66-1514387-33-9800:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    chandelier-zgwevt-4 material tree contains Mat3d66-1514387-112-8171:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-can_of_soda-ixrfxv-16 material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-foactj-0 material tree contains Mat3d66-1329970-28-3759:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    chandelier-zgwevt-5 material tree contains Mat3d66-1514387-112-8171:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-can_of_soda-ixrfxv-17 material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-foactj-0 material tree contains Mat3d66-1514387-33-9800:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    paper_lantern-hcieco-1 material tree contains Mat3d66-1514387-48-8577:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-can_of_soda-ixrfxv-18 material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-oglkuk-0 material tree contains Mat3d66-1329970-28-3759:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    paper_lantern-hcieco-2 material tree contains Mat3d66-1514387-48-8577:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-can_of_soda-ixrfxv-19 material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-oglkuk-0 material tree contains Mat3d66-1514387-33-9800:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    paper_lantern-hcieco-3 material tree contains Mat3d66-1514387-48-8577:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_pop-zsisxf-0 material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-jyvcbs-0 material tree contains Mat3d66-1329970-28-3759:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    chandelier-xtwiwi-5 material tree contains Mat3d66-1514387-112-8171:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_pop-zsisxf-1 material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-jyvcbs-0 material tree contains Mat3d66-1514387-33-9800:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    paper_lantern-cvzqeb-1 material tree contains Mat3d66-1514387-48-8577:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_pop-zsisxf-2 material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fridge-seyhuo-0-glass-base_link-F-lower-Tglass material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    paper_lantern-cvzqeb-2 material tree contains Mat3d66-1514387-48-8577:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_pop-zsisxf-3 material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fridge-seyhuo-0-glass_door-door-F-lower-Tglass material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    paper_lantern-cvzqeb-3 material tree contains Mat3d66-1514387-48-8577:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_pop-zsisxf-4 material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_pop-tfvmik-0-glass-base_link-F-lower-Tglass material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    paper_lantern-cvzqeb-4 material tree contains Mat3d66-1514387-48-8577:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_pop-zsisxf-5 material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_pop-tfvmik-1-glass-base_link-F-lower-Tglass material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    paper_lantern-cvzqeb-5 material tree contains Mat3d66-1514387-48-8577:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_pop-zsisxf-6 material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_pop-tfvmik-2-glass-base_link-F-lower-Tglass material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    paper_lantern-cvzqeb-6 material tree contains Mat3d66-1514387-48-8577:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_pop-zsisxf-7 material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_pop-tfvmik-3-glass-base_link-F-lower-Tglass material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    door-izyvmu-0-base_link material tree contains Mat3d66-1329970-28-3759:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_pop-zsisxf-8 material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_pop-tfvmik-4-glass-base_link-F-lower-Tglass material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    door-izyvmu-0-base_link material tree contains Mat3d66-1514387-33-9800:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_pop-zsisxf-9 material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_pop-tfvmik-5-glass-base_link-F-lower-Tglass material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fridge-seyhuo-0-door-base_link-P-lower material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_pop-zsisxf-10 material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_pop-tfvmik-6-glass-base_link-F-lower-Tglass material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fridge-seyhuo-0-door-base_link-P-upper material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_pop-zsisxf-11 material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_pop-tfvmik-7-glass-base_link-F-lower-Tglass material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_pop-tfvmik-0 material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-bwgquk-0 material tree contains Mat3d66-1329970-28-3759:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_pop-tfvmik-8-glass-base_link-F-lower-Tglass material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_pop-tfvmik-1 material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-bwgquk-0 material tree contains Mat3d66-1514387-33-9800:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_pop-tfvmik-9-glass-base_link-F-lower-Tglass material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_pop-tfvmik-2 material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-kbfwsp-0 material tree contains Mat3d66-1329970-28-3759:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_pop-tfvmik-10-glass-base_link-F-lower-Tglass material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_pop-tfvmik-3 material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-kbfwsp-0 material tree contains Mat3d66-1514387-33-9800:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_pop-tfvmik-11-glass-base_link-F-lower-Tglass material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_pop-tfvmik-4 material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-moxkxl-0 material tree contains Mat3d66-1329970-28-3759:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_pop-tfvmik-12-glass-base_link-F-lower-Tglass material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_pop-tfvmik-5 material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-moxkxl-0 material tree contains Mat3d66-1514387-33-9800:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_pop-tfvmik-13-glass-base_link-F-lower-Tglass material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_pop-tfvmik-6 material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-lnyvlo-0 material tree contains Mat3d66-1329970-28-3759:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_pop-tfvmik-14-glass-base_link-F-lower-Tglass material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_pop-tfvmik-7 material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-lnyvlo-0 material tree contains Mat3d66-1514387-33-9800:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_pop-tfvmik-15-glass-base_link-F-lower-Tglass material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_pop-tfvmik-8 material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-bfdewd-0 material tree contains Mat3d66-1329970-28-3759:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_pop-tfvmik-16-glass-base_link-F-lower-Tglass material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_pop-tfvmik-9 material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-bfdewd-0 material tree contains Mat3d66-1514387-33-9800:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_pop-tfvmik-17-glass-base_link-F-lower-Tglass material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_pop-tfvmik-10 material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-fpwgxq-0 material tree contains Mat3d66-1329970-28-3759:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_pop-tfvmik-18-glass-base_link-F-lower-Tglass material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_pop-tfvmik-11 material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-fpwgxq-0 material tree contains Mat3d66-1514387-33-9800:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_pop-tfvmik-19-glass-base_link-F-lower-Tglass material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-nyzjes-0 material tree contains Mat3d66-1329970-28-3759:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_pop-tfvmik-12 material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-fctnym-0 material tree contains Mat3d66-1329970-28-3759:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_pop-tfvmik-20-glass-base_link-F-lower-Tglass material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-nyzjes-0 material tree contains Mat3d66-1514387-33-9800:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_pop-tfvmik-13 material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-fctnym-0 material tree contains Mat3d66-1514387-33-9800:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_pop-tfvmik-21-glass-base_link-F-lower-Tglass material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    ceilings-axvuwm-0 material tree contains Mat3d66-1514387-14-6327:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_pop-tfvmik-14 material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-blfeln-0 material tree contains Mat3d66-1329970-28-3759:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_pop-tfvmik-22-glass-base_link-F-lower-Tglass material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-armchair-tcscts-5 material tree contains Mat3d66-1514387-30-2170:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_pop-tfvmik-15 material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-blfeln-0 material tree contains Mat3d66-1514387-33-9800:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_pop-tfvmik-23-glass-base_link-F-lower-Tglass material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-armchair-tcscts-4 material tree contains Mat3d66-1514387-30-2170:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_pop-tfvmik-16 material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-veumfu-0 material tree contains Mat3d66-1329970-28-3759:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_pop-tfvmik-24-glass-base_link-F-lower-Tglass material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-armchair-tcscts-3 material tree contains Mat3d66-1514387-30-2170:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_pop-tfvmik-17 material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-veumfu-0 material tree contains Mat3d66-1514387-33-9800:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_pop-tfvmik-25-glass-base_link-F-lower-Tglass material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-armchair-tcscts-2 material tree contains Mat3d66-1514387-30-2170:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_pop-tfvmik-18 material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-xplvcn-0 material tree contains Mat3d66-1329970-28-3759:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_pop-tfvmik-26-glass-base_link-F-lower-Tglass material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-armchair-tcscts-1 material tree contains Mat3d66-1514387-30-2170:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_pop-tfvmik-19 material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-xplvcn-0 material tree contains Mat3d66-1514387-33-9800:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_pop-tfvmik-27-glass-base_link-F-lower-Tglass material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-armchair-tcscts-0 material tree contains Mat3d66-1514387-30-2170:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_pop-tfvmik-20 material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-uuhmcc-0 material tree contains Mat3d66-1329970-28-3759:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_pop-tfvmik-28-glass-base_link-F-lower-Tglass material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    chandelier-qghfol-0 material tree contains Mat3d66-1514387-79-6171:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_pop-tfvmik-21 material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-uuhmcc-0 material tree contains Mat3d66-1514387-33-9800:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_pop-tfvmik-29-glass-base_link-F-lower-Tglass material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    chandelier-zgwevt-0 material tree contains Mat3d66-1514387-112-8171:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_pop-tfvmik-22 material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-adgmxy-0 material tree contains Mat3d66-1329970-28-3759:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fridge-seyhuo-0 material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_pop-tfvmik-23 material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-adgmxy-0 material tree contains Mat3d66-1514387-33-9800:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_pop-tfvmik-24 material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    chandelier-xtwiwi-4 material tree contains Mat3d66-1514387-112-8171:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-xtajqv-0 material tree contains Mat3d66-1329970-28-3759:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    chandelier-xtwiwi-0 material tree contains Mat3d66-1514387-112-8171:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_pop-tfvmik-25 material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-xtajqv-0 material tree contains Mat3d66-1514387-33-9800:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    chandelier-xtwiwi-1 material tree contains Mat3d66-1514387-112-8171:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_pop-tfvmik-26 material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-tttkqd-0 material tree contains Mat3d66-1329970-28-3759:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    chandelier-xtwiwi-2 material tree contains Mat3d66-1514387-112-8171:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_pop-tfvmik-27 material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-tttkqd-0 material tree contains Mat3d66-1514387-33-9800:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    chandelier-xtwiwi-3 material tree contains Mat3d66-1514387-112-8171:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_pop-tfvmik-28 material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-dfenga-0 material tree contains Mat3d66-1329970-28-3759:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    paper_lantern-cvzqeb-0 material tree contains Mat3d66-1514387-48-8577:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-bottle_of_pop-tfvmik-29 material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-dfenga-0 material tree contains Mat3d66-1514387-33-9800:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    painting-zfyrna-0 material tree contains Mat3d66-1514387-42-9040:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-can_of_soda-ixrfxv-0 material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-zhuaij-0 material tree contains Mat3d66-1329970-28-3759:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    paper_lantern-hcieco-0 material tree contains Mat3d66-1514387-48-8577:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-can_of_soda-ixrfxv-1 material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-zhuaij-0 material tree contains Mat3d66-1514387-33-9800:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    chandelier-qghfol-1 material tree contains Mat3d66-1514387-79-6171:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-can_of_soda-ixrfxv-2 material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-efuaku-0 material tree contains Mat3d66-1329970-28-3759:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    chandelier-qghfol-2 material tree contains Mat3d66-1514387-79-6171:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-can_of_soda-ixrfxv-3 material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-efuaku-0 material tree contains Mat3d66-1514387-33-9800:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    chandelier-qghfol-3 material tree contains Mat3d66-1514387-79-6171:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-can_of_soda-ixrfxv-4 material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-sgzzkd-0 material tree contains Mat3d66-1329970-28-3759:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    chandelier-qghfol-4 material tree contains Mat3d66-1514387-79-6171:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-can_of_soda-ixrfxv-5 material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-sgzzkd-0 material tree contains Mat3d66-1514387-33-9800:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    chandelier-qghfol-5 material tree contains Mat3d66-1514387-79-6171:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-can_of_soda-ixrfxv-6 material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-oalpcj-0 material tree contains Mat3d66-1329970-28-3759:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    chandelier-qghfol-6 material tree contains Mat3d66-1514387-79-6171:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-can_of_soda-ixrfxv-7 material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-oalpcj-0 material tree contains Mat3d66-1514387-33-9800:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    chandelier-qghfol-7 material tree contains Mat3d66-1514387-79-6171:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-can_of_soda-ixrfxv-8 material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-ttljge-0 material tree contains Mat3d66-1329970-28-3759:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    chandelier-qghfol-8 material tree contains Mat3d66-1514387-79-6171:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-can_of_soda-ixrfxv-9 material tree contains Mat3d66-1514387-99-1070:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-ttljge-0 material tree contains Mat3d66-1514387-33-9800:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.


## scenes/restaurant_brunch
  ### (.*) material tree contains .* with disallowed type .*. It should be one of .*.: 67
    F-vine-hchlkz-0 material tree contains AI55_008_Dypsis_004:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-C-pot_plant-kndwfb-0 material tree contains AM201_leafs3:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-hanging_plant-xfecjy-8 material tree contains AM141_035_001:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-hanging_plant-bedomm-2 material tree contains AM141_035_001:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-C-bouquet-hrvulc-1 material tree contains Am185_12_mat_035:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-vine-xzdszm-0 material tree contains AI55_008_Dypsis_004:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-C-pot_plant-kndwfb-0 material tree contains AM201_leafs4:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-hanging_plant-xfecjy-9 material tree contains AM141_035_001:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-console_table-tzkfqa-1 material tree contains AI55_008_table_metal:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-C-bouquet-hrvulc-2 material tree contains Am185_12_mat_035:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-vine-nxvleg-0 material tree contains AI55_008_Dypsis_004:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-C-pot_plant-niavbx-0 material tree contains AM201_leafs3:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-vine-voaeps-1 material tree contains AI55_008_Dypsis_004:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-console_table-tzkfqa-2 material tree contains AI55_008_table_metal:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-vine-xwzycj-0 material tree contains AI55_008_Dypsis_004:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-C-pot_plant-niavbx-0 material tree contains AM201_leafs4:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-hanging_plant-xfecjy-7 material tree contains AM141_035_001:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-vine-evmhsg-1 material tree contains AI55_008_Dypsis_004:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-console_table-tzkfqa-3 material tree contains AI55_008_table_metal:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-vine-evmhsg-0 material tree contains AI55_008_Dypsis_004:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-hanging_plant-xfecjy-0 material tree contains AM141_035_001:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-vine-sfqnem-1 material tree contains AI55_008_Dypsis_004:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-console_table-tzkfqa-4 material tree contains AI55_008_table_metal:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-C-vase-rfigof-2 material tree contains Am185_12_mat_035:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-vine-voaeps-0 material tree contains AI55_008_Dypsis_004:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-hanging_plant-zlgfak-0 material tree contains AM141_035_001:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-vine-xwzycj-1 material tree contains AI55_008_Dypsis_004:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-console_table-tzkfqa-5 material tree contains AI55_008_table_metal:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-vine-sfqnem-0 material tree contains AI55_008_Dypsis_004:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-hanging_plant-ipimgx-0 material tree contains AM141_035_001:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-vine-xjcnky-1 material tree contains AI55_008_Dypsis_004:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-console_table-tzkfqa-6 material tree contains AI55_008_table_metal:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-vine-xjcnky-0 material tree contains AI55_008_Dypsis_004:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-hanging_plant-bedomm-0 material tree contains AM141_035_001:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-hanging_plant-zlgfak-1 material tree contains AM141_035_001:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pot_plant-spmhnj-1 material tree contains Am185_12_mat_045:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-C-bouquet-hrvulc-0 material tree contains Am185_12_mat_035:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-console_table-tzkfqa-0 material tree contains AI55_008_table_metal:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-hanging_plant-zlgfak-2 material tree contains AM141_035_001:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pot_plant-spmhnj-2 material tree contains Am185_12_mat_045:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pot_plant-spmhnj-0 material tree contains Am185_12_mat_045:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-hanging_plant-xfecjy-1 material tree contains AM141_035_001:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-hanging_plant-zlgfak-3 material tree contains AM141_035_001:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pot_plant-spmhnj-3 material tree contains Am185_12_mat_045:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-vine-fotcaz-0 material tree contains AI55_008_Dypsis_004:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-hanging_plant-xfecjy-2 material tree contains AM141_035_001:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-hanging_plant-zlgfak-4 material tree contains AM141_035_001:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-C-pot_plant-peypuq-1 material tree contains Am185_12_mat_048:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-vine-bftzho-0 material tree contains AI55_008_Dypsis_004:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-hanging_plant-xfecjy-3 material tree contains AM141_035_001:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-hanging_plant-zlgfak-5 material tree contains AM141_035_001:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-C-pot_plant-peypuq-2 material tree contains Am185_12_mat_048:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-vine-kcajqg-0 material tree contains AI55_008_Dypsis_004:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-hanging_plant-xfecjy-4 material tree contains AM141_035_001:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-hanging_plant-zlgfak-6 material tree contains AM141_035_001:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-C-pot_plant-peypuq-3 material tree contains Am185_12_mat_048:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-vine-gccsfy-0 material tree contains AI55_008_Dypsis_004:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-C-pot_plant-qlyaks-0 material tree contains Am185_12_mat_048:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-hanging_plant-xfecjy-5 material tree contains AM141_035_001:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-hanging_plant-ipimgx-1 material tree contains AM141_035_001:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-C-vase-rfigof-0 material tree contains Am185_12_mat_035:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-vine-nuhdae-0 material tree contains AI55_008_Dypsis_004:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-C-pot_plant-peypuq-0 material tree contains Am185_12_mat_048:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-hanging_plant-xfecjy-6 material tree contains AM141_035_001:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-hanging_plant-bedomm-1 material tree contains AM141_035_001:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-C-vase-rfigof-1 material tree contains Am185_12_mat_035:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-vine-hdbgla-0 material tree contains AI55_008_Dypsis_004:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.


## scenes/restaurant_cafeteria
  ### (.*) material tree contains .* with disallowed type .*. It should be one of .*.: 150
    chandelier-nflhjl-2 material tree contains AM152_015_04:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-ldapja-24 material tree contains AM152_010_brushed_alu_black:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-cgchwh-0-glass-base_link-F-lower-Tglass material tree contains Oak Matt Light 2:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    chandelier-nflhjl-2 material tree contains AM152_015_03:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-ldapja-24 material tree contains AM152_015_04:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-djqiqc-0-glass-base_link-F-lower-Tglass material tree contains Oak Matt Light 2:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    chandelier-nflhjl-1 material tree contains AM152_015_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-ldapja-25 material tree contains AM152_010_brushed_alu_black:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-guhrnv-0-glass-base_link-F-lower-Tglass material tree contains Oak Matt Light 2:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    chandelier-nflhjl-1 material tree contains AM152_015_04:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-ldapja-25 material tree contains AM152_015_04:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-oszzno-0-glass-base_link-F-lower-Tglass material tree contains Oak Matt Light 2:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    chandelier-nflhjl-1 material tree contains AM152_015_03:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-ldapja-26 material tree contains AM152_010_brushed_alu_black:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-whujjc-0-glass-base_link-F-lower-Tglass material tree contains Oak Matt Light 2:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    chandelier-nflhjl-4 material tree contains AM152_015_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-ldapja-26 material tree contains AM152_015_04:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-ldapja-30 material tree contains AM152_010_brushed_alu_black:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    chandelier-nflhjl-4 material tree contains AM152_015_04:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-ldapja-27 material tree contains AM152_010_brushed_alu_black:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-ldapja-30 material tree contains AM152_015_04:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    chandelier-nflhjl-4 material tree contains AM152_015_03:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-ldapja-27 material tree contains AM152_015_04:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-ldapja-31 material tree contains AM152_010_brushed_alu_black:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    ceiling_air_conditioner-hapein-0 material tree contains Ceiling AC:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-ldapja-28 material tree contains AM152_010_brushed_alu_black:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-ldapja-31 material tree contains AM152_015_04:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-room_divider-vzwwqo-0 material tree contains Oak Matt Light 2:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-ldapja-28 material tree contains AM152_015_04:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-ldapja-32 material tree contains AM152_010_brushed_alu_black:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    loudspeaker-eylavt-0 material tree contains vray_Snow Fabric:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-ldapja-29 material tree contains AM152_010_brushed_alu_black:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-ldapja-32 material tree contains AM152_015_04:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    wall_mounted_tv-eqnopt-0 material tree contains CARPAINT::Non-Metallic (Solid)::Supergloss Jet Black:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-ldapja-29 material tree contains AM152_015_04:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-ldapja-33 material tree contains AM152_010_brushed_alu_black:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    handset-ilmhxf-0 material tree contains 06 - Default:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    door-esgehq-0-base_link material tree contains Oak Matt Light 2:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-ldapja-33 material tree contains AM152_015_04:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    handset-ilmhxf-0 material tree contains Material #15:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    door-esgehq-0-leaf0-base_link-R-lower material tree contains Oak Matt Light 2:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-ldapja-0 material tree contains AM152_010_brushed_alu_black:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    door-esgehq-0-leaf1-base_link-R-lower material tree contains Oak Matt Light 2:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-ldapja-0 material tree contains AM152_015_04:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-commercial_kitchen_table-wodpss-0 material tree contains Logo SMEG:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-ldapja-1 material tree contains AM152_010_brushed_alu_black:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    door-esgehq-0-leaf0-base_link-R-upper material tree contains Oak Matt Light 2:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-ldapja-1 material tree contains AM152_015_04:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    door-esgehq-0-leaf1-base_link-R-upper material tree contains Oak Matt Light 2:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-ldapja-2 material tree contains AM152_010_brushed_alu_black:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    stove-urgtyy-0 material tree contains switch 5:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-ldapja-2 material tree contains AM152_015_04:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-oszzno-0 material tree contains Oak Matt Light 2:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-ldapja-3 material tree contains AM152_010_brushed_alu_black:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-djqiqc-0 material tree contains Oak Matt Light 2:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-ldapja-3 material tree contains AM152_015_04:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-guhrnv-0 material tree contains Oak Matt Light 2:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-ldapja-4 material tree contains AM152_010_brushed_alu_black:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-cgchwh-0 material tree contains Oak Matt Light 2:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-ldapja-4 material tree contains AM152_015_04:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-ayvcgc-0 material tree contains Oak Matt Light 2:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-ldapja-5 material tree contains AM152_010_brushed_alu_black:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-cciwhk-0 material tree contains Oak Matt Light 2:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-ldapja-5 material tree contains AM152_015_04:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-whujjc-0 material tree contains Oak Matt Light 2:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-ldapja-6 material tree contains AM152_010_brushed_alu_black:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    door-axhbtu-0-leaf-base_link-R-lower material tree contains Oak Matt Light 2:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-ldapja-6 material tree contains AM152_015_04:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    loudspeaker-eylavt-1 material tree contains vray_Snow Fabric:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-ldapja-7 material tree contains AM152_010_brushed_alu_black:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    door-hzfirv-0 material tree contains Worn Pocedural 7:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-ldapja-7 material tree contains AM152_015_04:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    door-hzfirv-0-leaf-base_link-R-lower material tree contains Worn Pocedural 7:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-ldapja-8 material tree contains AM152_010_brushed_alu_black:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    door-hzfirv-0-leaf-base_link-R-upper material tree contains Worn Pocedural 7:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-ldapja-8 material tree contains AM152_015_04:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    door-hzfirv-1 material tree contains Worn Pocedural 7:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-ldapja-9 material tree contains AM152_010_brushed_alu_black:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    door-hzfirv-1-leaf-base_link-R-lower material tree contains Worn Pocedural 7:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-ldapja-9 material tree contains AM152_015_04:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    door-hzfirv-2 material tree contains Worn Pocedural 7:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-ldapja-10 material tree contains AM152_010_brushed_alu_black:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    door-hzfirv-2-leaf-base_link-R-lower material tree contains Worn Pocedural 7:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-ldapja-10 material tree contains AM152_015_04:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    door-axhbtu-0 material tree contains Oak Matt Light 2:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-ldapja-11 material tree contains AM152_010_brushed_alu_black:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    door-axhbtu-0-leaf-base_link-R-upper material tree contains Oak Matt Light 2:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-ldapja-11 material tree contains AM152_015_04:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    pillar-oukzkz-1 material tree contains Oak Matt Light 2:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    downlight-ldapja-12 material tree contains AM152_010_brushed_alu_black:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-ijouus-0 material tree contains Oak Matt Light 2:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-ldapja-12 material tree contains AM152_015_04:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-abeith-0 material tree contains Oak Matt Light 2:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-ldapja-13 material tree contains AM152_010_brushed_alu_black:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-qroopu-0 material tree contains Oak Matt Light 2:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-ldapja-13 material tree contains AM152_015_04:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-ruczgi-0 material tree contains Oak Matt Light 2:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-ldapja-14 material tree contains AM152_010_brushed_alu_black:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-inhngq-0 material tree contains Oak Matt Light 2:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-ldapja-14 material tree contains AM152_015_04:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-avdwms-0 material tree contains wood_66:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-ldapja-15 material tree contains AM152_010_brushed_alu_black:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    ceilings-jdvtxw-0 material tree contains Oak Matt Light 2:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-ldapja-15 material tree contains AM152_015_04:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    ceilings-ngbkuj-0 material tree contains Oak Matt Light 2:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-ldapja-16 material tree contains AM152_010_brushed_alu_black:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    ceilings-jwqbkz-0 material tree contains Oak Matt Light 2:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-ldapja-16 material tree contains AM152_015_04:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    door-axhbtu-0-glass-leaf-F-lower-Tglass material tree contains Oak Matt Light 2:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-ldapja-17 material tree contains AM152_010_brushed_alu_black:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    door-esgehq-0-glass0-leaf0-F-lower-Tglass material tree contains Oak Matt Light 2:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-nrkqre-0 material tree contains wood_66:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-ldapja-17 material tree contains AM152_015_04:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    door-esgehq-0-glass1-leaf1-F-lower-Tglass material tree contains Oak Matt Light 2:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    pillar-oukzkz-0 material tree contains Oak Matt Light 2:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    downlight-ldapja-18 material tree contains AM152_010_brushed_alu_black:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-ayvcgc-0-glass0-base_link-F-lower-Tglass material tree contains Oak Matt Light 2:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    ceilings-jrpolo-0 material tree contains Oak Matt Light 2:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-ldapja-18 material tree contains AM152_015_04:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-ayvcgc-0-glass1-base_link-F-lower-Tglass material tree contains Oak Matt Light 2:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    chandelier-nflhjl-5 material tree contains AM152_015_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-ldapja-19 material tree contains AM152_010_brushed_alu_black:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-ayvcgc-0-glass2-base_link-F-lower-Tglass material tree contains Oak Matt Light 2:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    chandelier-nflhjl-5 material tree contains AM152_015_04:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-ldapja-19 material tree contains AM152_015_04:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-ayvcgc-0-glass3-base_link-F-lower-Tglass material tree contains Oak Matt Light 2:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    chandelier-nflhjl-5 material tree contains AM152_015_03:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-ldapja-20 material tree contains AM152_010_brushed_alu_black:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-cciwhk-0-glass0-base_link-F-lower-Tglass material tree contains Oak Matt Light 2:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    chandelier-nflhjl-0 material tree contains AM152_015_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-ldapja-20 material tree contains AM152_015_04:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-cciwhk-0-glass1-base_link-F-lower-Tglass material tree contains Oak Matt Light 2:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    chandelier-nflhjl-0 material tree contains AM152_015_04:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-ldapja-21 material tree contains AM152_010_brushed_alu_black:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-cciwhk-0-glass2-base_link-F-lower-Tglass material tree contains Oak Matt Light 2:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    chandelier-nflhjl-0 material tree contains AM152_015_03:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-ldapja-21 material tree contains AM152_015_04:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-cciwhk-0-glass3-base_link-F-lower-Tglass material tree contains Oak Matt Light 2:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    chandelier-nflhjl-3 material tree contains AM152_015_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-ldapja-22 material tree contains AM152_010_brushed_alu_black:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-cciwhk-0-glass4-base_link-F-lower-Tglass material tree contains Oak Matt Light 2:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    chandelier-nflhjl-3 material tree contains AM152_015_04:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-ldapja-22 material tree contains AM152_015_04:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-cciwhk-0-glass5-base_link-F-lower-Tglass material tree contains Oak Matt Light 2:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    chandelier-nflhjl-3 material tree contains AM152_015_03:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-ldapja-23 material tree contains AM152_010_brushed_alu_black:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-cciwhk-0-glass6-base_link-F-lower-Tglass material tree contains Oak Matt Light 2:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    chandelier-nflhjl-2 material tree contains AM152_015_01:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-ldapja-23 material tree contains AM152_015_04:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    fixed_window-cciwhk-0-glass7-base_link-F-lower-Tglass material tree contains Oak Matt Light 2:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.


## scenes/restaurant_diner
  ### (.*) material tree contains .* with disallowed type .*. It should be one of .*.: 39
    L-popcorn_machine-ignowl-0 material tree contains D_Ppm_Red_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-ijvlre-5 material tree contains D_Lamp_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-popcorn_machine-ignowl-0-glass_right-base_link-F-lower-Tglass material tree contains D_Ppm_Yellow_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-popcorn_machine-ignowl-0 material tree contains D_Ppm_Yellow_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-ijvlre-6 material tree contains D_Round_Lamps:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-popcorn_machine-ignowl-0-glass_back-base_link-F-lower-Tglass material tree contains D_Ppm_Red_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-cucdgm-0 material tree contains D_Round_Lamps:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-ijvlre-6 material tree contains D_Lamp_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-popcorn_machine-ignowl-0-glass_back-base_link-F-lower-Tglass material tree contains D_Ppm_Yellow_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-cucdgm-0 material tree contains D_Lamp_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-ijvlre-7 material tree contains D_Round_Lamps:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-popcorn_machine-ignowl-0-glass_left-base_link-F-lower-Tglass material tree contains D_Ppm_Red_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-ijvlre-0 material tree contains D_Round_Lamps:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-ijvlre-7 material tree contains D_Lamp_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-popcorn_machine-ignowl-0-glass_left-base_link-F-lower-Tglass material tree contains D_Ppm_Yellow_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-ijvlre-0 material tree contains D_Lamp_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-cucdgm-1 material tree contains D_Round_Lamps:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-ijvlre-1 material tree contains D_Round_Lamps:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-cucdgm-1 material tree contains D_Lamp_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-ijvlre-1 material tree contains D_Lamp_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-cucdgm-2 material tree contains D_Round_Lamps:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-ijvlre-2 material tree contains D_Round_Lamps:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-cucdgm-2 material tree contains D_Lamp_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-ijvlre-2 material tree contains D_Lamp_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-cucdgm-3 material tree contains D_Round_Lamps:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-ijvlre-3 material tree contains D_Round_Lamps:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-cucdgm-3 material tree contains D_Lamp_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-popcorn_machine-ignowl-0-glass_right-base_link-F-lower-Tglass material tree contains D_Ppm_Red_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-ijvlre-3 material tree contains D_Lamp_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-cucdgm-4 material tree contains D_Round_Lamps:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-ijvlre-4 material tree contains D_Round_Lamps:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-cucdgm-4 material tree contains D_Lamp_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    wall_mounted_tv-hviblz-0 material tree contains D_TV_Screen:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-ijvlre-4 material tree contains D_Lamp_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-popcorn_machine-ignowl-0-glass_front-base_link-F-lower-Tglass material tree contains D_Ppm_Red_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    wall_mounted_tv-hviblz-1 material tree contains D_TV_Screen:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-ijvlre-5 material tree contains D_Round_Lamps:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-popcorn_machine-ignowl-0-glass_front-base_link-F-lower-Tglass material tree contains D_Ppm_Yellow_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    jukebox-nijgrl-0 material tree contains D_JP_Light_panel:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.


## scenes/restaurant_hotel
  ### (.*) material tree contains .* with disallowed type .*. It should be one of .*.: 175
    room_light-bgvpxl-3 material tree contains Lamp:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-room_divider-ispwfg-0 material tree contains Blind:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-jrrugm-24 material tree contains element_object0019:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-bgvpxl-4 material tree contains Lamp:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    window_blind-rdacud-1 material tree contains Blind:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-jrrugm-24 material tree contains element_object0018:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-bgvpxl-4 material tree contains Lamp:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    carpet-gvwhfn-0 material tree contains ceramic:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-jrrugm-25 material tree contains element_object0019:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-bgvpxl-5 material tree contains Lamp:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    carpet-gvwhfn-0 material tree contains ceramic2:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-jrrugm-25 material tree contains element_object0018:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-bgvpxl-5 material tree contains Lamp:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    carpet-gvwhfn-0 material tree contains Floor_tiling_2:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    downlight-jrrugm-26 material tree contains element_object0019:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-bgvpxl-6 material tree contains Lamp:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    carpet-gvwhfn-1 material tree contains ceramic:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-jrrugm-26 material tree contains element_object0018:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-bgvpxl-6 material tree contains Lamp:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    carpet-gvwhfn-1 material tree contains ceramic2:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-jrrugm-27 material tree contains element_object0019:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-bgvpxl-7 material tree contains Lamp:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    carpet-gvwhfn-1 material tree contains Floor_tiling_2:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    downlight-jrrugm-27 material tree contains element_object0018:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-bgvpxl-7 material tree contains Lamp:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    carpet-iqnove-1 material tree contains ceramic:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-jrrugm-28 material tree contains element_object0019:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    picture-verboh-0 material tree contains Standard_4das:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    carpet-iqnove-1 material tree contains ceramic2:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-jrrugm-28 material tree contains element_object0018:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    picture-verboh-1 material tree contains Standard_4das:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    carpet-iqnove-1 material tree contains Floor_tiling_2:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    downlight-jrrugm-29 material tree contains element_object0019:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-uptmal-0 material tree contains Lamp:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-room_divider-oufvle-0 material tree contains Blind:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-jrrugm-29 material tree contains element_object0018:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-uptmal-0 material tree contains Lamp:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    window_blind-rfdlor-0 material tree contains Blind:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    walls-bxvhde-0 material tree contains VRayMtl_15:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-ylheyu-0 material tree contains element_object0133:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    window_blind-rfdlor-1 material tree contains Blind:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    walls-bxvhde-0 material tree contains Floor_tiling:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-ylheyu-0 material tree contains element_object0132:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    window_blind-rfdlor-2 material tree contains Blind:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    walls-bxvhde-0 material tree contains Kafil pol:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-ylheyu-1 material tree contains element_object0133:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    window_blind-rfdlor-3 material tree contains Blind:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    walls-vcravt-0 material tree contains VRayMtl_15:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-ylheyu-1 material tree contains element_object0132:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-jrrugm-1 material tree contains element_object0019:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-vcravt-0 material tree contains Floor_tiling:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-ylheyu-3 material tree contains element_object0133:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-jrrugm-1 material tree contains element_object0018:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-vcravt-0 material tree contains Kafil pol:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-ylheyu-3 material tree contains element_object0132:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-jrrugm-2 material tree contains element_object0019:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-bigboy-0 material tree contains VRayMtl_15:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-ylheyu-2 material tree contains element_object0133:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-jrrugm-2 material tree contains element_object0018:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-bigboy-0 material tree contains Floor_tiling:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-ylheyu-2 material tree contains element_object0132:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-jrrugm-3 material tree contains element_object0019:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-bigboy-0 material tree contains Kafil pol:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    painting-xxmpoi-1 material tree contains element_object0019:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-jrrugm-3 material tree contains element_object0018:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-qdoclq-0 material tree contains VRayMtl_15:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    painting-xxmpoi-1 material tree contains element_object0018:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-jrrugm-4 material tree contains element_object0019:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-qdoclq-0 material tree contains Floor_tiling:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    carpet-brrqkq-0 material tree contains ceramic:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-jrrugm-4 material tree contains element_object0018:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-qdoclq-0 material tree contains Kafil pol:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    carpet-brrqkq-0 material tree contains ceramic2:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-jrrugm-5 material tree contains element_object0019:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-xsosmv-0 material tree contains VRayMtl_15:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    carpet-brrqkq-0 material tree contains Floor_tiling_2:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    downlight-jrrugm-5 material tree contains element_object0018:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-xsosmv-0 material tree contains Floor_tiling:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    carpet-brrqkq-1 material tree contains ceramic:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-jrrugm-6 material tree contains element_object0019:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-xsosmv-0 material tree contains Kafil pol:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    carpet-brrqkq-1 material tree contains ceramic2:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-jrrugm-6 material tree contains element_object0018:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-qovwxg-0 material tree contains VRayMtl_15:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    carpet-brrqkq-1 material tree contains Floor_tiling_2:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    downlight-jrrugm-7 material tree contains element_object0019:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-qovwxg-0 material tree contains Floor_tiling:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    carpet-iqnove-0 material tree contains ceramic:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-jrrugm-7 material tree contains element_object0018:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-qovwxg-0 material tree contains Kafil pol:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    carpet-iqnove-0 material tree contains ceramic2:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-jrrugm-8 material tree contains element_object0019:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-wifija-0 material tree contains VRayMtl_15:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    carpet-iqnove-0 material tree contains Floor_tiling_2:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    downlight-jrrugm-8 material tree contains element_object0018:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-wifija-0 material tree contains Floor_tiling:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-jrrugm-0 material tree contains element_object0019:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-jrrugm-9 material tree contains element_object0019:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-wifija-0 material tree contains Kafil pol:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-jrrugm-0 material tree contains element_object0018:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-jrrugm-9 material tree contains element_object0018:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-goiiwo-0 material tree contains VRayMtl_15:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    painting-xxmpoi-0 material tree contains element_object0019:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-jrrugm-10 material tree contains element_object0019:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-goiiwo-0 material tree contains Floor_tiling:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    painting-xxmpoi-0 material tree contains element_object0018:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-jrrugm-10 material tree contains element_object0018:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-goiiwo-0 material tree contains Kafil pol:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-xyvxhs-0 material tree contains Lamp:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-jrrugm-11 material tree contains element_object0019:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-wtwwvl-0 material tree contains Floor_tiling:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-xyvxhs-0 material tree contains Lamp:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-jrrugm-11 material tree contains element_object0018:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-wtwwvl-0 material tree contains Blind:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    window_blind-ewdlor-3 material tree contains Blind:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    downlight-jrrugm-12 material tree contains element_object0019:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-hwglfw-0 material tree contains VRayMtl_15:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    window_blind-ewdlor-2 material tree contains Blind:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    downlight-jrrugm-12 material tree contains element_object0018:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-hwglfw-0 material tree contains Floor_tiling:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    window_blind-ewdlor-1 material tree contains Blind:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    downlight-jrrugm-13 material tree contains element_object0019:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-hwglfw-0 material tree contains Kafil pol:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    window_blind-ewdlor-0 material tree contains Blind:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    downlight-jrrugm-13 material tree contains element_object0018:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-ylheyu-5 material tree contains element_object0133:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-jrrugm-14 material tree contains element_object0019:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-ylheyu-5 material tree contains element_object0132:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-jrrugm-14 material tree contains element_object0018:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-ylheyu-4 material tree contains element_object0133:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-jrrugm-15 material tree contains element_object0019:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-ylheyu-4 material tree contains element_object0132:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-jrrugm-15 material tree contains element_object0018:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    window_blind-rdacud-0 material tree contains Blind:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-jrrugm-16 material tree contains element_object0019:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    shelf-ubsonj-0 material tree contains Blind:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-jrrugm-16 material tree contains element_object0018:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    picture-sjmxgo-0 material tree contains Blind:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-jrrugm-17 material tree contains element_object0019:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-reed_diffuser-ngcvaw-0 material tree contains Blind:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-jrrugm-17 material tree contains element_object0018:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    picture-vzomxk-0 material tree contains Blind:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-jrrugm-18 material tree contains element_object0019:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    picture-sjmxgo-1 material tree contains Blind:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-jrrugm-18 material tree contains element_object0018:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-hubdfx-0 material tree contains VRayMtl_15:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    picture-vzomxk-1 material tree contains Blind:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-jrrugm-19 material tree contains element_object0019:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-hubdfx-0 material tree contains Floor_tiling:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-eivonw-0 material tree contains Floor_tiling:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-jrrugm-19 material tree contains element_object0018:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-hubdfx-0 material tree contains Kafil pol:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-eivonw-0 material tree contains Blind:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-jrrugm-20 material tree contains element_object0019:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-bgvpxl-0 material tree contains Lamp:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-xyvxhs-1 material tree contains Lamp:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-jrrugm-20 material tree contains element_object0018:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-bgvpxl-0 material tree contains Lamp:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-xyvxhs-1 material tree contains Lamp:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-jrrugm-21 material tree contains element_object0019:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-bgvpxl-1 material tree contains Lamp:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    shelf-ubsonj-1 material tree contains Blind:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-jrrugm-21 material tree contains element_object0018:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-bgvpxl-1 material tree contains Lamp:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    shelf-ubsonj-2 material tree contains Blind:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-jrrugm-22 material tree contains element_object0019:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-bgvpxl-2 material tree contains Lamp:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    shelf-ubsonj-3 material tree contains Blind:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-jrrugm-22 material tree contains element_object0018:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-bgvpxl-2 material tree contains Lamp:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-reed_diffuser-ngcvaw-1 material tree contains Blind:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-jrrugm-23 material tree contains element_object0019:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    room_light-bgvpxl-3 material tree contains Lamp:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    door-scfdua-0-base_link material tree contains Floor_tiling:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    downlight-jrrugm-23 material tree contains element_object0018:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.


## scenes/restaurant_urban
  ### (.*) material tree contains .* with disallowed type .*. It should be one of .*.: 4
    C-pot_plant-jtkcip-1 material tree contains AI55_001_258:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pot_plant-jtkcip-0 material tree contains AI55_001_258:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pot_plant-jtkcip-2 material tree contains AI55_001_258:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pot_plant-jtkcip-3 material tree contains AI55_001_258:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.


## scenes/Rs_garden
  ### (.*) material tree contains .* with disallowed type .*. It should be one of .*.: 27
    bush-lcktfs-0 material tree contains iBushes-Petal:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bush-vxulvs-0 material tree contains iBushes-Petal:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bush-lcktfs-0 material tree contains iBushes-Petal:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bush-vxulvs-0 material tree contains iBushes-Petal:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bush-lcktfs-0 material tree contains iBushes-Petal:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bush-vxulvs-0 material tree contains iBushes-Petal:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bush-lcktfs-0 material tree contains iBushes-Leaf:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bush-vxulvs-0 material tree contains iBushes-Leaf:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bush-lcktfs-0 material tree contains iBushes-Berry:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bush-vxulvs-0 material tree contains iBushes-Berry:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bush-bqdskl-0 material tree contains iBushes-Petal:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bush-dsjegk-0 material tree contains iBushes-Petal:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bush-bqdskl-0 material tree contains iBushes-Petal:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bush-dsjegk-0 material tree contains iBushes-Petal:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bush-bqdskl-0 material tree contains iBushes-Petal:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bush-dsjegk-0 material tree contains iBushes-Petal:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bush-bqdskl-0 material tree contains iBushes-Leaf:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bush-dsjegk-0 material tree contains iBushes-Leaf:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bush-bqdskl-0 material tree contains iBushes-Berry:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bush-dsjegk-0 material tree contains iBushes-Berry:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bush-albnbz-0 material tree contains iBushes-Petal:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bush-albnbz-0 material tree contains iBushes-Petal:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bush-albnbz-0 material tree contains iBushes-Petal:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bush-albnbz-0 material tree contains iBushes-Leaf:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-pot_plant-cecdwu-0 material tree contains AM248_014_flower1:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    bush-albnbz-0 material tree contains iBushes-Berry:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-pot_plant-cecdwu-0 material tree contains AM248_014_flower3:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.


## scenes/school_biology
  ### (.*) material tree contains .* with disallowed type .*. It should be one of .*.: 508
    B-locker-gemgfz-6-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-49-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-22-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-54-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-16-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-54-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-25-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-54-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-30-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-59-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-26-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-59-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-18-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-59-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-1-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-3-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-34-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-3-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-37-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-3-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-40-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-8-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-43-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-8-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-11-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-8-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-7-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-13-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-0-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-13-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-17-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-13-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-12-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-18-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-23-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-18-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-29-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-18-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-24-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-23-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-14-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-23-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-32-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-23-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-33-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-29-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-36-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-29-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-39-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-29-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-42-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-35-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-8-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-35-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-3-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-35-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    C-pot_plant-qyarus-0 material tree contains terra_green_plant001_sub:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-41-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    C-pot_plant-cvthyv-0 material tree contains leaf3_Vray2SidedMtl:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-41-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    C-pot_plant-cvthyv-0 material tree contains leaf1_Vray2SidedMtl:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-41-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    C-pot_plant-cvthyv-0 material tree contains leaf2_Vray2SidedMtl:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-47-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-square_light-qebtel-43 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-47-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-square_light-qebtel-49 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-47-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-square_light-qebtel-55 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-0-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-square_light-qebtel-60 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-0-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-square_light-qebtel-1 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-0-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-square_light-qebtel-10 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-dsuxez-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-17 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-mndzqa-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-6 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-gbbxxg-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-13 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-dyajzl-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-19 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-asahjx-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-23 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-xbjiwl-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-28 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-yqaokb-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-33 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-oaivry-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-38 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-tzxsfc-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-44 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-lvpqgb-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-50 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-ihirsz-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-56 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-wpkkqn-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-61 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-fhxrzm-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-47 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-fjhbza-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-8 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-ohlvsg-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-15 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-rdbpkn-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-7 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-hjezae-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-14 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-jlndff-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-20 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-uzbwif-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-24 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-pirsax-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-29 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-ebfjro-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-34 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-hdbypk-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-39 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-jhnhwj-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-45 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-vxxvmu-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-51 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-bfhmxg-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-57 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-gvpsby-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-62 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-zdipdo-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-41 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-zwkfzz-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-5 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-galkwn-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-0 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-rjqxhp-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-9 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-bsblwz-0 material tree contains Firstaid_floor_title_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-16 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-bsblwz-0 material tree contains Gym_floor_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-21 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-rzbksf-0 material tree contains Firstaid_floor_title_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-25 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-rzbksf-0 material tree contains Gym_floor_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-30 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-ijwnhc-0 material tree contains Firstaid_floor_title_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-35 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-ijwnhc-0 material tree contains Gym_floor_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-40 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-terrarium-rxwfcx-0-glass_front-base_link-F-lower-Tglass material tree contains leaf3_Vray2SidedMtl:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-46 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-terrarium-rxwfcx-0-glass_front-base_link-F-lower-Tglass material tree contains leaf2_Vray2SidedMtl:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-52 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-terrarium-rxwfcx-0-glass_front-base_link-F-lower-Tglass material tree contains leaf1_Vray2SidedMtl:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-58 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-terrarium-rxwfcx-0-glass_left-base_link-F-lower-Tglass material tree contains leaf3_Vray2SidedMtl:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-63 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-terrarium-rxwfcx-0-glass_left-base_link-F-lower-Tglass material tree contains leaf2_Vray2SidedMtl:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-37 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-terrarium-rxwfcx-0-glass_left-base_link-F-lower-Tglass material tree contains leaf1_Vray2SidedMtl:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-4 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-terrarium-rxwfcx-0-glass_right-base_link-F-lower-Tglass material tree contains leaf3_Vray2SidedMtl:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-2 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-terrarium-rxwfcx-0-glass_right-base_link-F-lower-Tglass material tree contains leaf2_Vray2SidedMtl:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-11 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-terrarium-rxwfcx-0-glass_right-base_link-F-lower-Tglass material tree contains leaf1_Vray2SidedMtl:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-18 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-door-ewqpfv-0-left_glass-left_door-F-lower material tree contains leaf3_Vray2SidedMtl:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-22 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-door-ewqpfv-0-left_glass-left_door-F-lower material tree contains leaf2_Vray2SidedMtl:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-26 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-door-ewqpfv-0-left_glass-left_door-F-lower material tree contains leaf1_Vray2SidedMtl:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-31 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-door-ewqpfv-0-right_glass-right_door-F-lower material tree contains leaf3_Vray2SidedMtl:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-36 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-door-ewqpfv-0-right_glass-right_door-F-lower material tree contains leaf2_Vray2SidedMtl:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-42 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-door-ewqpfv-0-right_glass-right_door-F-lower material tree contains leaf1_Vray2SidedMtl:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-48 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-door-nwlbit-0-glass-leaf-F-lower material tree contains Firstaid_floor_title_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-54 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-door-nwlbit-0-glass-leaf-F-lower material tree contains Gym_floor_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-59 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-door-nwlbit-1-glass-leaf-F-lower material tree contains Firstaid_floor_title_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-53 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-door-nwlbit-1-glass-leaf-F-lower material tree contains Gym_floor_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-32 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-door-nwlbit-2-glass-leaf-F-lower material tree contains Firstaid_floor_title_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-27 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-door-nwlbit-2-glass-leaf-F-lower material tree contains Gym_floor_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-3 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-door-nwlbit-3-glass-leaf-F-lower material tree contains Firstaid_floor_title_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-12 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-door-nwlbit-3-glass-leaf-F-lower material tree contains Gym_floor_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-terrarium-rxwfcx-0 material tree contains leaf3_Vray2SidedMtl:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-door-nwlbit-4-glass-leaf-F-lower material tree contains Firstaid_floor_title_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-terrarium-rxwfcx-0 material tree contains leaf2_Vray2SidedMtl:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-door-nwlbit-4-glass-leaf-F-lower material tree contains Gym_floor_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-terrarium-rxwfcx-0 material tree contains leaf1_Vray2SidedMtl:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-door-xxipyh-0-reddoor2-base_link-R-lower material tree contains leaf3_Vray2SidedMtl:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-26-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-door-xxipyh-0-reddoor2-base_link-R-lower material tree contains leaf1_Vray2SidedMtl:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-26-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-door-xxipyh-0-reddoor2-base_link-R-lower material tree contains leaf2_Vray2SidedMtl:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-26-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-door-xxipyh-0-reddoor1-base_link-R-lower material tree contains terra_green_plant002_sub:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-32-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-door-xxipyh-0-glass2-reddoor2-F-lower material tree contains Firstaid_floor_title_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-32-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-door-xxipyh-0-glass2-reddoor2-F-lower material tree contains Gym_floor_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-32-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-door-xxipyh-1-reddoor2-base_link-R-lower material tree contains leaf3_Vray2SidedMtl:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-38-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-door-xxipyh-1-reddoor2-base_link-R-lower material tree contains leaf1_Vray2SidedMtl:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-38-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-door-xxipyh-1-reddoor2-base_link-R-lower material tree contains leaf2_Vray2SidedMtl:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-38-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-door-xxipyh-1-reddoor1-base_link-R-lower material tree contains terra_green_plant002_sub:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-44-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-door-xxipyh-1-glass2-reddoor2-F-lower material tree contains Firstaid_floor_title_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-44-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-door-xxipyh-1-glass2-reddoor2-F-lower material tree contains Gym_floor_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-44-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-door-xxipyh-2-reddoor2-base_link-R-lower material tree contains leaf3_Vray2SidedMtl:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-50-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-door-xxipyh-2-reddoor2-base_link-R-lower material tree contains leaf1_Vray2SidedMtl:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-50-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-door-xxipyh-2-reddoor2-base_link-R-lower material tree contains leaf2_Vray2SidedMtl:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-50-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-door-xxipyh-2-reddoor1-base_link-R-lower material tree contains terra_green_plant002_sub:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-55-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-door-xxipyh-2-glass2-reddoor2-F-lower material tree contains Firstaid_floor_title_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-55-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-door-xxipyh-2-glass2-reddoor2-F-lower material tree contains Gym_floor_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-55-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-openable_window-rgmujm-0-base_link material tree contains leaf3_Vray2SidedMtl:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-60-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-openable_window-rgmujm-0-base_link material tree contains leaf1_Vray2SidedMtl:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-60-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-openable_window-rgmujm-0-base_link material tree contains leaf2_Vray2SidedMtl:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-60-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-openable_window-rgmujm-1-base_link material tree contains leaf3_Vray2SidedMtl:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-5-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-openable_window-rgmujm-1-base_link material tree contains leaf1_Vray2SidedMtl:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-5-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-openable_window-rgmujm-1-base_link material tree contains leaf2_Vray2SidedMtl:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-5-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-openable_window-rgmujm-2-base_link material tree contains leaf3_Vray2SidedMtl:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-9-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-openable_window-rgmujm-2-base_link material tree contains leaf1_Vray2SidedMtl:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-9-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-openable_window-rgmujm-2-base_link material tree contains leaf2_Vray2SidedMtl:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-9-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-openable_window-rgmujm-3-base_link material tree contains leaf3_Vray2SidedMtl:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-14-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-openable_window-rgmujm-3-base_link material tree contains leaf1_Vray2SidedMtl:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-14-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-openable_window-rgmujm-3-base_link material tree contains leaf2_Vray2SidedMtl:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-14-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-openable_window-rgmujm-4-base_link material tree contains leaf3_Vray2SidedMtl:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-19-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-openable_window-rgmujm-4-base_link material tree contains leaf1_Vray2SidedMtl:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-19-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-openable_window-rgmujm-4-base_link material tree contains leaf2_Vray2SidedMtl:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-19-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-openable_window-rgmujm-5-base_link material tree contains leaf3_Vray2SidedMtl:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-24-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-openable_window-rgmujm-5-base_link material tree contains leaf1_Vray2SidedMtl:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-24-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-openable_window-rgmujm-5-base_link material tree contains leaf2_Vray2SidedMtl:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pot_plant-yrakhf-0 material tree contains Biolab_pot_flower:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-24-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-openable_window-rgmujm-6-base_link material tree contains leaf3_Vray2SidedMtl:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    wall_mounted_light-pqomwb-0 material tree contains Firstaid_lamp_light_white:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-30-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-openable_window-rgmujm-6-base_link material tree contains leaf1_Vray2SidedMtl:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pot_plant-zkbbdm-0 material tree contains fig_tree:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-30-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-openable_window-rgmujm-6-base_link material tree contains leaf2_Vray2SidedMtl:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pot_plant-arwvvc-0 material tree contains terra_green_plant002_sub:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-30-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-openable_window-rgmujm-7-base_link material tree contains leaf3_Vray2SidedMtl:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pot_plant-qftnsu-0 material tree contains leaf3_Vray2SidedMtl:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-36-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-openable_window-rgmujm-7-base_link material tree contains leaf1_Vray2SidedMtl:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pot_plant-qftnsu-0 material tree contains leaf1_Vray2SidedMtl:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-36-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-openable_window-rgmujm-7-base_link material tree contains leaf2_Vray2SidedMtl:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    C-pot_plant-qftnsu-0 material tree contains leaf2_Vray2SidedMtl:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-36-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-openable_window-rgmujm-8-base_link material tree contains leaf3_Vray2SidedMtl:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-3-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-42-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-openable_window-rgmujm-8-base_link material tree contains leaf1_Vray2SidedMtl:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-8-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-42-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-openable_window-rgmujm-8-base_link material tree contains leaf2_Vray2SidedMtl:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-42-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-42-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    walls-soekre-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-39-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-48-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    walls-zssera-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-36-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-48-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    walls-xxveqj-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-33-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-48-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-32-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-53-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-14-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-53-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-24-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-53-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-29-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-58-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-23-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-58-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-12-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-58-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-17-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-2-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-0-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-2-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-7-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-2-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-11-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-7-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-43-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-7-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-40-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-7-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-37-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-12-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-34-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-12-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-1-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-12-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-18-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-17-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-26-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-17-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-30-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-17-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-25-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-22-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-16-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-22-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-22-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-22-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-6-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-28-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-10-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-28-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-15-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-28-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-44-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-34-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-41-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-34-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-38-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-34-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-35-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-40-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-5-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-40-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-21-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-40-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-28-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-46-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-31-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-46-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-27-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-46-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-20-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-52-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-4-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-52-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-9-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-52-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-13-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-57-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-19-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-57-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-2-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-57-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-2-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-4-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-19-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-4-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-13-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-4-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-9-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-6-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-4-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-6-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-20-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-6-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-27-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-11-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-31-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-11-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-28-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-11-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-21-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-16-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-5-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-16-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-35-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-16-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-38-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-21-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-41-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-21-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-44-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-21-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-15-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-27-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-10-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-27-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-6-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-27-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-22-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-33-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-16-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-33-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-25-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-33-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-30-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-39-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-26-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-39-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-18-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-39-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-1-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-45-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-34-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-45-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-37-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-45-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-40-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-51-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-43-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-51-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-11-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-51-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-7-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-56-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-0-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-56-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-17-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-56-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-12-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-61-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-23-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-61-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-29-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-61-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-24-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-1-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-14-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-1-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-32-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-1-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-33-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-10-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-36-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-10-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-39-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-10-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-42-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-15-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-8-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-15-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-3-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-15-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-2-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-20-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-19-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-20-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-13-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-20-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-9-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-25-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-4-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-25-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-20-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-25-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-27-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-31-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-31-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-31-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-28-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-31-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-21-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-37-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-5-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-37-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-35-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-37-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-38-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-43-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-41-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-43-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-44-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-43-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-15-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-49-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-10-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-49-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.


## scenes/school_chemistry
  ### (.*) material tree contains .* with disallowed type .*. It should be one of .*.: 468
    B-locker-gemgfz-42-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-37-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-41-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-37-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-9-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-42-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-22-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-42-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-2-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-42-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-17-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-47-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-3-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-47-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-11-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-47-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-24-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-53-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-4-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-53-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-19-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-53-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-30-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-59-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-33-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-59-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-36-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-59-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-40-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-3-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-39-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-3-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-5-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-3-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-18-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-10-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-29-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-10-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-15-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-10-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-1-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-15-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-7-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-15-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-21-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-15-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-0-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-20-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-16-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-20-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-28-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-20-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-13-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-25-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-26-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-25-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-32-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-25-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-35-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-30-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-38-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-30-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-44-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-30-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-43-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-35-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-12-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-35-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-25-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-35-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-6-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-40-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-20-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-40-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-8-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-40-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-14-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-45-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-27-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-45-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-10-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-45-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-23-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-51-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-31-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-51-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-34-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-51-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-37-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-57-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-42-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-57-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-41-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-57-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-9-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-6-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-22-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-6-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-2-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-6-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-17-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-8-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-3-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-8-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-11-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-8-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-24-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-0-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-4-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-0-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-19-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-0-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-30-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    walls-juqfoo-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-33-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    walls-ikwaza-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-36-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    walls-rlujfv-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-40-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    walls-kmywfp-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-39-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    walls-ajtqcv-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-5-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    walls-kntomh-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-18-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    walls-hguvbe-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-29-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    walls-ylaasu-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-15-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    walls-wjmwgi-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-1-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    walls-bhvyap-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-7-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    walls-kjzouu-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-21-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    walls-mwpnqm-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-0-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    walls-upyvbg-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-16-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    walls-lgvfbm-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-22 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-oipebp-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-28 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-ahzqhk-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-32 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-ofwbho-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-38 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-nlhetj-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-43 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-xcbayw-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-42 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-xllowg-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-7 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-ktjnvh-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-37 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-unxitj-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-16 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-utnmys-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-17 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-ezmpyn-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-23 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-kmxvob-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-19 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-abqeqz-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-25 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-bzkvvl-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-29 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-oubyzg-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-34 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-zejbxt-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-39 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-uitoic-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-44 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-pcmcub-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-18 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-cjrrdo-0 material tree contains Gym_floor_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-6 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-cjrrdo-0 material tree contains Firstaid_floor_title_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-33 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-lttvnn-0 material tree contains Gym_floor_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-15 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-lttvnn-0 material tree contains Firstaid_floor_title_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-1 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-ngibqw-0 material tree contains Gym_floor_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-0 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-ngibqw-0 material tree contains Firstaid_floor_title_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-9 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-L-lab_exhaust_hood-sdlkyk-0-glass-base_link-F-lower-Tglass material tree contains Chamlab_vent_cab_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-14 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-C-reagent_bottle-tnjpsf-0-glass-base_link-F-lower-Tglass material tree contains Chamlab_bottle_400ml_paper2s:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-21 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-C-reagent_bottle-trtrsl-0-glass-base_link-F-lower-Tglass material tree contains Chamlab_bottle_400ml_paper2s:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-27 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-C-reagent_bottle-ukayce-0-glass-base_link-F-lower-Tglass material tree contains Chamlab_bottle_400ml_paper2s:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-31 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-C-reagent_bottle-xstykf-0-glass-base_link-F-lower-Tglass material tree contains Chamlab_bottle_400ml_paper2s:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-35 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-door-ewqpfv-0-right_glass-right_door-F-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-square_light-qebtel-40 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-C-reagent_bottle-tnjpsf-1-glass-base_link-F-lower-Tglass material tree contains Chamlab_bottle_400ml_paper2s:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-45 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-C-reagent_bottle-tnjpsf-2-glass-base_link-F-lower-Tglass material tree contains Chamlab_bottle_400ml_paper2s:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-10 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-C-reagent_bottle-tnjpsf-3-glass-base_link-F-lower-Tglass material tree contains Chamlab_bottle_400ml_paper2s:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-5 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-C-reagent_bottle-trtrsl-1-glass-base_link-F-lower-Tglass material tree contains Chamlab_bottle_400ml_paper2s:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-24 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-C-reagent_bottle-trtrsl-4-glass-base_link-F-lower-Tglass material tree contains Chamlab_bottle_400ml_paper2s:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-12 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-C-reagent_bottle-trtrsl-2-glass-base_link-F-lower-Tglass material tree contains Chamlab_bottle_400ml_paper2s:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-3 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-C-reagent_bottle-trtrsl-3-glass-base_link-F-lower-Tglass material tree contains Chamlab_bottle_400ml_paper2s:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-4 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-C-reagent_bottle-trtrsl-5-glass-base_link-F-lower-Tglass material tree contains Chamlab_bottle_400ml_paper2s:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-8 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-C-reagent_bottle-ukayce-4-glass-base_link-F-lower-Tglass material tree contains Chamlab_bottle_400ml_paper2s:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-13 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-C-reagent_bottle-ukayce-1-glass-base_link-F-lower-Tglass material tree contains Chamlab_bottle_400ml_paper2s:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-20 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-C-reagent_bottle-ukayce-2-glass-base_link-F-lower-Tglass material tree contains Chamlab_bottle_400ml_paper2s:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-26 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-C-reagent_bottle-ukayce-3-glass-base_link-F-lower-Tglass material tree contains Chamlab_bottle_400ml_paper2s:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-30 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-C-reagent_bottle-xstykf-5-glass-base_link-F-lower-Tglass material tree contains Chamlab_bottle_400ml_paper2s:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-36 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-C-reagent_bottle-xstykf-1-glass-base_link-F-lower-Tglass material tree contains Chamlab_bottle_400ml_paper2s:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-41 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-C-reagent_bottle-xstykf-2-glass-base_link-F-lower-Tglass material tree contains Chamlab_bottle_400ml_paper2s:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-46 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-C-reagent_bottle-xstykf-4-glass-base_link-F-lower-Tglass material tree contains Chamlab_bottle_400ml_paper2s:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-11 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    F-C-reagent_bottle-xstykf-3-glass-base_link-F-lower-Tglass material tree contains Chamlab_bottle_400ml_paper2s:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-47 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-yhsjer-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-2 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-oytuni-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-48-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-48-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-48-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-54-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-54-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-54-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-60-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-60-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-60-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-5-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-5-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-5-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-11-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-11-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-11-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-16-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-16-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-16-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-21-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-21-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-21-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-26-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-26-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-26-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-31-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-31-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-31-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-36-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    F-L-lab_exhaust_hood-sdlkyk-0 material tree contains Chamlab_vent_cab_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-36-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    F-C-reagent_bottle-xstykf-5 material tree contains Chamlab_bottle_400ml_paper2s:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-36-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    F-C-reagent_bottle-ukayce-4 material tree contains Chamlab_bottle_400ml_paper2s:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-41-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    F-C-reagent_bottle-trtrsl-1 material tree contains Chamlab_bottle_400ml_paper2s:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-41-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    F-C-reagent_bottle-tnjpsf-1 material tree contains Chamlab_bottle_400ml_paper2s:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-41-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    F-C-reagent_bottle-trtrsl-4 material tree contains Chamlab_bottle_400ml_paper2s:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-46-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    F-C-reagent_bottle-xstykf-1 material tree contains Chamlab_bottle_400ml_paper2s:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-46-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    F-C-reagent_bottle-ukayce-1 material tree contains Chamlab_bottle_400ml_paper2s:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-46-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    F-C-reagent_bottle-trtrsl-2 material tree contains Chamlab_bottle_400ml_paper2s:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-52-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    F-C-reagent_bottle-tnjpsf-2 material tree contains Chamlab_bottle_400ml_paper2s:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-52-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    F-C-reagent_bottle-xstykf-2 material tree contains Chamlab_bottle_400ml_paper2s:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-52-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    F-C-reagent_bottle-ukayce-2 material tree contains Chamlab_bottle_400ml_paper2s:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-58-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    F-C-reagent_bottle-trtrsl-3 material tree contains Chamlab_bottle_400ml_paper2s:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-58-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    F-C-reagent_bottle-tnjpsf-3 material tree contains Chamlab_bottle_400ml_paper2s:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-58-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    F-C-reagent_bottle-xstykf-4 material tree contains Chamlab_bottle_400ml_paper2s:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-2-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    F-C-reagent_bottle-trtrsl-5 material tree contains Chamlab_bottle_400ml_paper2s:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-2-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    F-C-reagent_bottle-ukayce-3 material tree contains Chamlab_bottle_400ml_paper2s:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-2-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    F-C-reagent_bottle-xstykf-3 material tree contains Chamlab_bottle_400ml_paper2s:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-9-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    F-C-reagent_bottle-tnjpsf-0 material tree contains Chamlab_bottle_400ml_paper2s:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-9-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    F-C-cap-nkjxbc-2-Tconnectedpart material tree contains Chamlab_bottle_400ml_paper2s:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-9-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    F-C-reagent_bottle-xstykf-0 material tree contains Chamlab_bottle_400ml_paper2s:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-14-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    F-C-cap-nkjxbc-3-Tconnectedpart material tree contains Chamlab_bottle_400ml_paper2s:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-14-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    F-C-reagent_bottle-trtrsl-0 material tree contains Chamlab_bottle_400ml_paper2s:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-14-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    F-C-cap-nkjxbc-1-Tconnectedpart material tree contains Chamlab_bottle_400ml_paper2s:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-19-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    F-C-reagent_bottle-ukayce-0 material tree contains Chamlab_bottle_400ml_paper2s:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-19-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    F-C-cap-nkjxbc-0-Tconnectedpart-Tglass material tree contains Chamlab_bottle_400ml_paper2s:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-19-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-square_light-hylkmj-7 material tree contains light_on:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-24-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-square_light-hylkmj-2 material tree contains light_on:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-24-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-square_light-hylkmj-0 material tree contains light_on:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-24-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-square_light-hylkmj-5 material tree contains light_on:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-29-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-square_light-hylkmj-1 material tree contains light_on:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-29-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-square_light-hylkmj-8 material tree contains light_on:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-29-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-square_light-hylkmj-4 material tree contains light_on:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-34-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-square_light-hylkmj-9 material tree contains light_on:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-34-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-square_light-hylkmj-3 material tree contains light_on:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-34-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-square_light-hylkmj-10 material tree contains light_on:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-39-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-square_light-hylkmj-6 material tree contains light_on:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-39-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-16-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-39-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-0-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-44-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-21-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-44-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-7-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-44-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-1-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-50-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-15-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-50-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-29-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-50-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-18-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-56-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-5-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-56-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-39-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-56-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-40-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-4-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-36-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-4-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-33-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-4-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-30-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-7-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-19-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-7-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-4-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-7-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-24-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-13-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-11-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-13-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-3-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-13-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-17-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-18-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-2-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-18-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-22-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-18-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-9-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-23-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-41-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-23-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-42-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-23-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-37-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-28-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-34-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-28-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-31-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-28-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-23-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-33-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-10-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-33-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-27-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-33-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-14-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-38-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-8-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-38-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-20-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-38-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-6-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-43-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-25-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-43-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-12-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-43-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-43-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-49-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-44-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-49-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-38-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-49-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-35-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-55-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-32-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-55-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-26-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-55-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-13-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-61-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-28-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-61-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-28-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-61-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-13-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-1-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-26-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-1-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-32-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-1-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-35-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-12-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-38-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-12-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-44-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-12-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-43-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-17-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-12-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-17-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-25-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-17-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-6-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-22-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-20-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-22-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-8-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-22-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-14-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-27-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-27-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-27-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-10-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-27-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-23-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-32-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-31-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-32-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-34-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-32-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-37-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-37-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.


## scenes/school_computer_lab_and_infirmary
  ### (.*) material tree contains .* with disallowed type .*. It should be one of .*.: 486
    B-locker-gemgfz-36-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-26-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-39-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-26-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-square_light-qebtel-70 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-7-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-square_light-qebtel-82 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-23-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-square_light-qebtel-78 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-14-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-square_light-qebtel-67 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-13-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-square_light-qebtel-57 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-2-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-square_light-qebtel-49 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-1-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-square_light-qebtel-44 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-frqhms-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-63 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-xdyqvv-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-74 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-phzmoh-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-86 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-zcablu-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-84 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-zccppf-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-72 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-xqohyh-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-61 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-xiepks-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-51 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-anjroq-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-5 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-qoatai-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-19 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-nwuzeb-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-30 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-wvwoad-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-28 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-wcbbpb-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-16 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-pshgqt-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-3 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-iwdnuy-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-55 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-bafezw-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-65 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-hspvgd-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-77 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-mvcnbj-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-89 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-xxehnj-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-88 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-hpsypc-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-76 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-hxvyjx-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-64 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-xaypxr-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-56 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-vfafud-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-9 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-anksht-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-22 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-yphspl-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-34 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-mkhdgc-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-32 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-pschoa-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-21 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-ebhgft-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-10 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-bogber-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-58 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-ldphwm-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-68 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-yqwxuv-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-80 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-uarvek-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-92 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-ttbneh-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-91 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-qcleor-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-81 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-xibegv-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-69 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-bgsmqx-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-59 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-zelkfr-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-11 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-thfrem-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-24 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    walls-nasnsa-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-35 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-qloiuu-0 material tree contains Gym_floor_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-40 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-qloiuu-0 material tree contains Firstaid_floor_title_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-45 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-vedzwu-0 material tree contains Gym_floor_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-50 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-vedzwu-0 material tree contains Firstaid_floor_title_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-62 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-zelrbj-0 material tree contains Gym_floor_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-73 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-zelrbj-0 material tree contains Firstaid_floor_title_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-85 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-bijmgk-0 material tree contains Gym_floor_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-94 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-bijmgk-0 material tree contains Firstaid_floor_title_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-1 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-zysogn-0 material tree contains Gym_floor_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-14 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-zysogn-0 material tree contains Firstaid_floor_title_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-26 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-door-nwlbit-0-base_link material tree contains Firstaid_screen_sub:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-2 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-door-nwlbit-0-base_link material tree contains Firstaid_screen_2sd:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-17 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-door-nwlbit-0-glass-leaf-F-lower material tree contains Gym_floor_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-29 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-door-nwlbit-0-glass-leaf-F-lower material tree contains Firstaid_floor_title_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-38 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-door-nwlbit-1-base_link material tree contains Firstaid_screen_sub:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-42 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-door-nwlbit-1-base_link material tree contains Firstaid_screen_2sd:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-47 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-door-nwlbit-1-glass-leaf-F-lower material tree contains Gym_floor_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-53 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-door-nwlbit-1-glass-leaf-F-lower material tree contains Firstaid_floor_title_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-79 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-door-nwlbit-2-base_link material tree contains Firstaid_screen_sub:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-90 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-door-nwlbit-2-base_link material tree contains Firstaid_screen_2sd:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-96 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-door-nwlbit-2-glass-leaf-F-lower material tree contains Gym_floor_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-99 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-door-nwlbit-2-glass-leaf-F-lower material tree contains Firstaid_floor_title_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-100 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-door-nwlbit-5-base_link material tree contains Firstaid_screen_sub:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-13 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-door-nwlbit-5-base_link material tree contains Firstaid_screen_2sd:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-8 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-door-nwlbit-5-glass-leaf-F-lower material tree contains Gym_floor_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-23 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-door-nwlbit-5-glass-leaf-F-lower material tree contains Firstaid_floor_title_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-31 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-door-nwlbit-6-base_link material tree contains Firstaid_screen_sub:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-39 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-door-nwlbit-6-base_link material tree contains Firstaid_screen_2sd:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-43 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-door-nwlbit-6-glass-leaf-F-lower material tree contains Gym_floor_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-48 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-door-nwlbit-6-glass-leaf-F-lower material tree contains Firstaid_floor_title_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-54 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-door-xxipyh-0-glass2-reddoor2-F-lower material tree contains Gym_floor_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-66 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-door-xxipyh-0-glass2-reddoor2-F-lower material tree contains Firstaid_floor_title_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-75 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-door-xxipyh-0-glass1-reddoor1-F-lower material tree contains Gym_floor_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-87 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-door-xxipyh-0-glass1-reddoor1-F-lower material tree contains Firstaid_floor_title_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-95 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-door-xxipyh-0-glass-base_link-F-lower material tree contains Gym_floor_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-98 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-door-xxipyh-0-glass-base_link-F-lower material tree contains Firstaid_floor_title_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-4 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-door-xxipyh-1-glass2-reddoor2-F-lower material tree contains Gym_floor_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-18 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-door-xxipyh-1-glass2-reddoor2-F-lower material tree contains Firstaid_floor_title_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-6 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-door-xxipyh-1-glass1-reddoor1-F-lower material tree contains Gym_floor_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-15 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-door-xxipyh-1-glass1-reddoor1-F-lower material tree contains Firstaid_floor_title_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-27 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-door-xxipyh-1-glass-base_link-F-lower material tree contains Gym_floor_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-37 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-door-xxipyh-1-glass-base_link-F-lower material tree contains Firstaid_floor_title_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-41 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-door-xxipyh-2-glass2-reddoor2-F-lower material tree contains Gym_floor_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-46 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-door-xxipyh-2-glass2-reddoor2-F-lower material tree contains Firstaid_floor_title_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-52 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-door-xxipyh-2-glass1-reddoor1-F-lower material tree contains Gym_floor_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-60 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-door-xxipyh-2-glass1-reddoor1-F-lower material tree contains Firstaid_floor_title_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-71 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-door-xxipyh-2-glass-base_link-F-lower material tree contains Gym_floor_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-83 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-door-xxipyh-2-glass-base_link-F-lower material tree contains Firstaid_floor_title_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-93 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-door-xxipyh-3-glass2-reddoor2-F-lower material tree contains Gym_floor_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-97 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-door-xxipyh-3-glass2-reddoor2-F-lower material tree contains Firstaid_floor_title_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-7 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-door-xxipyh-3-glass1-reddoor1-F-lower material tree contains Gym_floor_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-20 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-door-xxipyh-3-glass1-reddoor1-F-lower material tree contains Firstaid_floor_title_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-33 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-door-xxipyh-3-glass-base_link-F-lower material tree contains Gym_floor_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-12 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-door-xxipyh-3-glass-base_link-F-lower material tree contains Firstaid_floor_title_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-25 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-door-nwlbit-4-base_link material tree contains Firstaid_screen_sub:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-36 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-door-nwlbit-4-base_link material tree contains Firstaid_screen_2sd:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-0 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-door-nwlbit-4-glass-leaf-F-lower material tree contains Gym_floor_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-27-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-door-nwlbit-4-glass-leaf-F-lower material tree contains Firstaid_floor_title_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-27-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-door-nwlbit-3-base_link material tree contains Firstaid_screen_sub:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-27-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-door-nwlbit-3-base_link material tree contains Firstaid_screen_2sd:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-44-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-door-nwlbit-3-glass-leaf-F-lower material tree contains Gym_floor_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-44-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-door-nwlbit-3-glass-leaf-F-lower material tree contains Firstaid_floor_title_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-44-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-0-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-0-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-0-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-30-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-30-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-30-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-47-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-47-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-47-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    wall_mounted_light-kzpogm-1 material tree contains Firstaid_lamp_light_white:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-15-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    bed_screen-ofpyax-0-fabric-base_link-P-lower material tree contains Firstaid_screen_sub:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-15-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    bed_screen-ofpyax-0-fabric-base_link-P-lower material tree contains Firstaid_screen_2sd:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-15-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    wall_mounted_light-kzpogm-2 material tree contains Firstaid_lamp_light_white:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-33-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    wall_mounted_light-kzpogm-0 material tree contains Firstaid_lamp_light_white:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-33-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    bed_screen-ofpyax-0-fabric-base_link-P-upper material tree contains Firstaid_screen_sub:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-33-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    bed_screen-ofpyax-0-fabric-base_link-P-upper material tree contains Firstaid_screen_2sd:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-11-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    bed_screen-klvmfw-0-fabric-base_link-P-lower material tree contains Firstaid_screen_sub:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-11-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    bed_screen-klvmfw-0-fabric-base_link-P-lower material tree contains Firstaid_screen_2sd:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-11-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    bed_screen-klvmfw-0-fabric-base_link-P-upper material tree contains Firstaid_screen_sub:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-18-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    bed_screen-klvmfw-0-fabric-base_link-P-upper material tree contains Firstaid_screen_2sd:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-18-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    C-pot_plant-jrvtso-0 material tree contains Firstaid_ficus_sub:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-18-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-39-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-36-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-36-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-36-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-33-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-36-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-5-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-9-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-19-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-9-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-25-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-9-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-30-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-21-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-24-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-21-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-18-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-21-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-6-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-39-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-11-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-39-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-16-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-39-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-0-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-7-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-4-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-7-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-40-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-7-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-37-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-24-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-34-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-24-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-8-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-24-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-20-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-42-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-26-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-42-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-31-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-42-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-27-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-4-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-21-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-4-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-10-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-4-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-15-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-28-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-17-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-28-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-3-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-28-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-9-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-45-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-41-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-45-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-38-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-45-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-35-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-3-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-12-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-3-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-22-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-3-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-28-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-31-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-32-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-31-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-29-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-31-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-13-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-48-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-7-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-48-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-1-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-48-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-2-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-16-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-14-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-16-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-23-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-16-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-29-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-34-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-32-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-34-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-28-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-34-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-22-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-14-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-12-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-14-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-35-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-14-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-38-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-19-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-41-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-19-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-9-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-19-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-3-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-37-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-17-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-37-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-15-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-37-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-10-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-12-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-21-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-12-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-27-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-12-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-31-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-22-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-26-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-22-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-20-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-22-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-8-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-40-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-34-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-40-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-37-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-40-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-40-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-10-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-4-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-10-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-0-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-10-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-16-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-25-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-11-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-25-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-6-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-25-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-18-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-43-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-24-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-43-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-30-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-43-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-25-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-8-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-19-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-8-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-5-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-8-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-33-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-29-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-36-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-29-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-39-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-29-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-13-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-46-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-7-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-46-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-1-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-46-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-2-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-5-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-14-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-5-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-23-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-5-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-29-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-32-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-32-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-32-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-28-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-32-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-22-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-2-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-12-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-2-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-35-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-2-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-38-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-17-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-41-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-17-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-9-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-17-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-3-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-35-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-17-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-35-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-15-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-35-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-10-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-6-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-21-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-6-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-27-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-6-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-31-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-20-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-26-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-20-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-20-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-20-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-8-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-38-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-34-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-38-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-37-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-38-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-40-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-1-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-4-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-1-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-0-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-1-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-16-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-23-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-11-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-23-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-6-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-23-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-18-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-41-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-24-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-41-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-30-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-41-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-25-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-13-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-19-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-13-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-5-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-13-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-33-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-26-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.


## scenes/school_geography
  ### (.*) material tree contains .* with disallowed type .*. It should be one of .*.: 432
    B-locker-gemgfz-12-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-17-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-15-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-17-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-16-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-20-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-17-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-20-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-18-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-20-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-19-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-23-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-20-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-23-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-21-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-23-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-22-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    walls-enzltq-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-23-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    walls-spesoz-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-24-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    walls-pknttk-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-26-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    walls-breuts-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-28-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    walls-ozufch-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-30-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    walls-vjvwsq-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-32-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    walls-rneejf-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-35-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    walls-mxdvsa-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-38-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    walls-dpphuu-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-41-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    walls-wecpnn-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-44-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    walls-kgyjuy-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-47-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    walls-yqgagw-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-50-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    walls-fcwcwf-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-53-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    walls-gxvcis-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-56-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    walls-detmse-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-59-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    walls-eefxlg-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-62-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    walls-xkhdgd-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-66-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    walls-bubmvu-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-70-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    walls-ixkwwc-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-68-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    walls-jsgjgn-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-64-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    walls-lnvsue-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-61-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    walls-vuwkbz-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-58-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    walls-ngxkqy-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-55-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    walls-mgubmm-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-52-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    walls-zzxpxu-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-49-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    walls-mqvlev-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-46-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    floors-hxdqav-0 material tree contains Firstaid_floor_title_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-43-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    floors-hxdqav-0 material tree contains Gym_floor_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-40-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    floors-fdcjvm-0 material tree contains Firstaid_floor_title_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-37-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    floors-fdcjvm-0 material tree contains Gym_floor_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-34-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    floors-kwjfig-0 material tree contains Firstaid_floor_title_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-31-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    floors-kwjfig-0 material tree contains Gym_floor_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-65-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    floors-bqaini-0 material tree contains Firstaid_floor_title_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-69-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    floors-bqaini-0 material tree contains Gym_floor_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-72-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-L-display_case-khxpgc-0-base_link material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-71-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-L-display_case-khxpgc-1-base_link material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-67-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-openable_window-rgmujm-0-base_link material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-63-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-openable_window-rgmujm-0-moving-base_link-P-lower material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-60-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-openable_window-rgmujm-1-base_link material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-57-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-openable_window-rgmujm-1-moving-base_link-P-lower material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-54-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-openable_window-rgmujm-2-base_link material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-51-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-openable_window-rgmujm-2-moving-base_link-P-lower material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-48-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-openable_window-rgmujm-3-base_link material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-45-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-openable_window-rgmujm-3-moving-base_link-P-lower material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-42-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-openable_window-rgmujm-4-base_link material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-39-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-openable_window-rgmujm-4-moving-base_link-P-lower material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-36-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-openable_window-rgmujm-5-base_link material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-33-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-openable_window-rgmujm-5-moving-base_link-P-lower material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-29-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-openable_window-rgmujm-6-base_link material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-27-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-openable_window-rgmujm-6-moving-base_link-P-lower material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-25-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-openable_window-rgmujm-7-base_link material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-3-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-openable_window-rgmujm-7-moving-base_link-P-lower material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-7-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-openable_window-rgmujm-8-base_link material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-11-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-openable_window-rgmujm-8-moving-base_link-P-lower material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-14-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-openable_window-rgmujm-9-base_link material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-5-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-openable_window-rgmujm-9-moving-base_link-P-lower material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-8-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-openable_window-rgmujm-10-base_link material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-13-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-openable_window-rgmujm-10-moving-base_link-P-lower material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-2-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-door-xxipyh-3-base_link material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-6-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-door-xxipyh-2-base_link material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-10-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-door-xxipyh-1-base_link material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-9-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-door-xxipyh-0-base_link material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-4-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-door-xxipyh-3-glass-base_link-F-lower-Tglass material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-1-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-door-xxipyh-2-glass-base_link-F-lower-Tglass material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-12-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-door-xxipyh-1-glass-base_link-F-lower-Tglass material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-15-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-door-xxipyh-0-glass-base_link-F-lower-Tglass material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-16-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    walls-vqxato-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-17-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-18-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-19-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-20-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-21-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-22-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-23-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-24-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-26-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-28-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-30-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-32-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-35-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-38-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-41-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-44-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-47-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-50-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-53-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-56-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-59-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-62-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-66-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-70-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-68-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-64-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-61-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-58-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-55-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-52-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-49-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-46-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-43-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    C-pot_plant-irtlsj-0 material tree contains Geo_class_pot_flower:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-40-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-square_light-hylkmj-0 material tree contains light_on:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-37-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-31-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-34-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-34-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-31-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-37-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-square_light-qebtel-21 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-40-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-square_light-qebtel-37 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-43-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-square_light-qebtel-46 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-46-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-square_light-qebtel-44 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-49-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-square_light-qebtel-36 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-52-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-square_light-qebtel-29 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-55-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-square_light-qebtel-22 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-58-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-square_light-qebtel-14 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-61-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-square_light-qebtel-7 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-64-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-square_light-qebtel-0 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-68-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-square_light-qebtel-2 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-70-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-square_light-qebtel-9 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-16 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-24 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-31 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-39 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-47 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-45 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-38 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-30 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-23 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-15 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-8 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-1 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-3 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-10 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-18 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-26 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-33 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-41 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-49 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-48 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-40 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-32 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-25 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-17 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-11 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-4 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-5 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-12 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-19 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-27 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-square_light-qebtel-35 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-66-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-square_light-qebtel-43 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-62-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-square_light-qebtel-51 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-59-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-square_light-qebtel-50 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-56-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-square_light-qebtel-42 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-53-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-square_light-qebtel-34 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-50-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-square_light-qebtel-28 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-47-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-square_light-qebtel-20 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-44-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-square_light-qebtel-13 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-41-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-square_light-qebtel-6 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-38-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-square_light-hylkmj-3 material tree contains light_on:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-35-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-square_light-hylkmj-4 material tree contains light_on:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-32-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-square_light-hylkmj-5 material tree contains light_on:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-30-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-square_light-hylkmj-7 material tree contains light_on:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-28-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-square_light-hylkmj-1 material tree contains light_on:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-26-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-square_light-hylkmj-2 material tree contains light_on:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-24-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-square_light-hylkmj-6 material tree contains light_on:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-23-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-square_light-hylkmj-8 material tree contains light_on:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-22-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-10-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-21-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-10-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-20-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-10-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-19-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-13-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-18-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-13-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-17-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-13-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-16-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-16-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-15-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-16-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-12-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-16-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-1-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-19-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-4-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-19-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-9-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-19-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-10-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-22-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-6-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-22-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-2-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-22-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-13-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-25-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-8-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-25-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-5-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-25-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-14-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-27-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-11-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-27-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-7-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-27-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-3-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-0-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-25-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-0-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-27-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-0-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-29-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-3-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-33-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-3-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-36-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-3-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-39-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-5-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-42-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-5-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-45-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-5-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-48-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-7-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-51-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-7-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-54-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-7-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-57-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-9-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-60-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-9-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-63-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-9-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-67-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-12-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-71-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-12-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-72-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-12-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-69-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-15-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-65-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-15-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-0-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-15-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-0-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-18-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-0-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-18-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-65-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-18-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-69-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-21-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-72-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-21-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-71-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-21-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-67-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-24-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-63-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-24-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-60-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-24-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-57-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-26-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-54-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-26-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-51-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-26-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-48-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-1-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-45-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-1-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-42-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-1-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-39-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-2-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-36-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-2-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-33-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-2-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-29-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-4-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-27-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-4-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-25-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-4-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-3-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-6-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-7-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-6-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-11-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-6-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-14-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-8-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-5-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-8-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-8-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-8-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-13-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-11-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-2-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-11-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-6-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-11-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-10-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-14-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-9-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-14-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-4-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-14-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-1-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-17-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.


## scenes/school_gym
  ### (.*) material tree contains .* with disallowed type .*. It should be one of .*.: 444
    B-locker-gemgfz-9-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    walls-nbobru-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-14-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    walls-dgwjzr-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-18-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    walls-avkurm-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-22-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    walls-ehzlaj-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-26-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    walls-zoaezm-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-29-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    floors-gzjodz-0 material tree contains Gym_floor_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-32-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    floors-gzjodz-0 material tree contains Firstaid_floor_title_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-35-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    floors-lzecin-0 material tree contains Gym_floor_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-38-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    floors-lzecin-0 material tree contains Firstaid_floor_title_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-40-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    floors-gvzlnj-0 material tree contains Gym_floor_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-42-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    floors-gvzlnj-0 material tree contains Firstaid_floor_title_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-44-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-square_light-qebtel-11 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-47-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-square_light-qebtel-10 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-50-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-square_light-qebtel-9 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-0-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-square_light-qebtel-8 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-5-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-square_light-qebtel-7 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-10-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-square_light-qebtel-6 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-15-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-square_light-qebtel-5 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-19-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-square_light-qebtel-4 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-23-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-square_light-qebtel-3 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-27-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-square_light-qebtel-2 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-30-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-square_light-qebtel-1 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-33-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-square_light-qebtel-0 material tree contains Lockerroom_light:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-36-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-39-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-41-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-43-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-45-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-48-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-51-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-3-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-8-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-13-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-31-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-34-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-37-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-1-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-6-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-11-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-16-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-20-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-24-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-28-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-25-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-21-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-17-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-12-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-7-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-2-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-46-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-49-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-52-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-4-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-9-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-14-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-18-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-22-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-26-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-29-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-32-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-35-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-38-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-40-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-42-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-44-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-47-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-50-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-0-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-5-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-10-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-15-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-19-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-23-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-27-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-30-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    walls-hcljaj-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-33-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-36 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-36-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-52 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-39-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-66 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-41-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-80 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-43-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-94 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-45-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-7 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-48-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-24 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-51-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-39 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-3-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-54 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-8-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-68 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-13-topdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-82 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-sjywtq-0 material tree contains Gym_floor_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-gqjmyu-96 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    floors-sjywtq-0 material tree contains Firstaid_floor_title_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    square_light-gqjmyu-10 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-2-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-26 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-2-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-41 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-2-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-55 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-7-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-69 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-7-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-83 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-7-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-97 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-12-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-12 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-12-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-27 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-12-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-42 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-17-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-56 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-17-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-70 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-17-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-84 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-22-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-98 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-22-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-14 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-22-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-28 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-27-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-43 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-27-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-57 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-27-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-71 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-31-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-85 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-31-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-2 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-31-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-15 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-35-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-29 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-35-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-44 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-35-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-58 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-39-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-72 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-39-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-86 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-39-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-4 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-44-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-16 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-44-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-30 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-44-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-45 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-48-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-59 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-48-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-73 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-48-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-87 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-0-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-6 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-0-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-17 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-0-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-31 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-5-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-46 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-5-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-60 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-5-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-74 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-10-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-88 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-10-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-9 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-10-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-18 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-15-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-32 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-15-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-47 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-15-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-61 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-20-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-75 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-20-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-89 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-20-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-11 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-25-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-19 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-25-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-33 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-25-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-48 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-30-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-62 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-30-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-76 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-30-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-90 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-34-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-13 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-34-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-20 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-34-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-34 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-38-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-49 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-38-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-63 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-38-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-77 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-42-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-91 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-42-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-1 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-42-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-21 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-47-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-35 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-47-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-50 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-47-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-64 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-51-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-78 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-51-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-92 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-51-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-3 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-3-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-22 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-3-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-37 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-3-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-51 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-9-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-65 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-9-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-79 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-9-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-93 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-14-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-5 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-14-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-23 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-14-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-38 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-19-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-53 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-19-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-67 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-19-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-81 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-24-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-95 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-24-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-8 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-24-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-25 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-29-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-40 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-29-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    square_light-gqjmyu-0 material tree contains Gymlight:VRayLightMtl with disallowed type VRayLightMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-yhthgr-29-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-13-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-33-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-8-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-33-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-3-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-33-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-51-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-37-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-48-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-37-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-45-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-37-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-43-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-41-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-41-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-41-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-39-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-41-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-36-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-46-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-33-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-46-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-30-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-46-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-27-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-50-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-23-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-50-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-19-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-50-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-15-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-4-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-10-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-4-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-5-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-4-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-0-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-8-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-50-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-8-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-47-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-8-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-44-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-13-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-42-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-13-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-40-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-13-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-38-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-18-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-35-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-18-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-32-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-18-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-29-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-23-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-26-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-23-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-22-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-23-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-18-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-28-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-14-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-28-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-9-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-28-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-4-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-32-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-52-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-32-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-49-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-32-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-46-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-36-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-2-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-36-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-7-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-36-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-12-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-40-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-17-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-40-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-21-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-40-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-25-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-45-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-28-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-45-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-24-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-45-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-20-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-49-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-16-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-49-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-11-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-49-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-6-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-1-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-1-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-1-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-37-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-1-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-34-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-6-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-31-base_link material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-6-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-31-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-6-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-34-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-11-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-37-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-11-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-1-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-11-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-6-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-16-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-11-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-16-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-16-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-16-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-20-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-21-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-24-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-21-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-28-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-21-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-25-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-26-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-21-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-26-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-17-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-26-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-12-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-43-doorbottom-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-7-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-43-base_link material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-2-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-yhthgr-43-doortop-base_link-R-lower material tree contains Lockerroom_LockerBlue_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    B-locker-gemgfz-46-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    walls-mwseii-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-49-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    walls-hjkqrr-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-52-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    walls-gfiikc-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    B-locker-gemgfz-4-bottomdoor-base_link-R-lower material tree contains Lockerroom_LockerRed_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>, <material<Shell_Material>>}.
    walls-nhingy-0 material tree contains Comp_class_wall_override:VRayOverrideMtl with disallowed type VRayOverrideMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.


## scenes/Wainscott_0_garden
  ### (.*) material tree contains .* with disallowed type .*. It should be one of .*.: 16
    background-miopfw-0 material tree contains AE19_010_105:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-pot_plant-koloae-0 material tree contains iFlowers-Leaf:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-pot_plant-eixiaq-0 material tree contains leaf:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-pot_plant-eixiaq-1 material tree contains leaf:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    background-miopfw-0 material tree contains AE19_010_102:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-pot_plant-koloae-0 material tree contains lilac_flower_pink:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-pot_plant-eixiaq-0 material tree contains lilac_flower_pink:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    background-miopfw-0 material tree contains AE19_010_93:VRayBlendMtl with disallowed type VRayBlendMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-pot_plant-eixiaq-1 material tree contains lilac_flower_pink:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-pot_plant-koloae-1 material tree contains lilac_flower_pink:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-pot_plant-eixiaq-2 material tree contains lilac_flower_pink:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    background-miopfw-0 material tree contains AE19_010_104:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-pot_plant-dxypbs-0 material tree contains Mat3d66-1715589-259-2036:VRayMtlWrapper with disallowed type VRayMtlWrapper. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-pot_plant-koloae-1 material tree contains iFlowers-Leaf:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    L-pot_plant-eixiaq-2 material tree contains leaf:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.
    background-miopfw-0 material tree contains AE19_010_103:VRay2SidedMtl with disallowed type VRay2SidedMtl. It should be one of {<material<Multimaterial>>, <material<VRayMtl>>}.


In [59]:
from collections import defaultdict
import re

warning_re = re.compile(r"(.*) has too many vertices: (\d+) > 20000")
warn_objs = defaultdict(dict)
for name, x in jsons.items():
    for w in x["warnings"]:
        m = warning_re.fullmatch(w)
        if m:
            warn_objs[name][m.group(1)] = int(m.group(2))
            
print(warn_objs)

defaultdict(<class 'dict'>, {'objects/batch-00': {'L-tag-ohegnp-0': 21794, 'chicken_wire-czoiqw-0': 52305}, 'objects/batch-01': {'L-shoe_rack-spdrzb-0': 59456}, 'objects/batch-02': {'L-radio-vgioak-0': 50123}, 'objects/batch-05': {'L-punching_bag-svkdji-0': 20027}, 'objects/batch-06': {'zipper-qwsvid-0': 27426}, 'objects/legacy_batch-01': {'swivel_chair-fqbnuk-0-base_link': 36405, 'swivel_chair-lizuow-0-base_link': 25743, 'swivel_chair-lxupys-0-base_link': 35465, 'swivel_chair-mqxrim-0-base_link': 21060, 'swivel_chair-opkoam-0-base_link': 38343}, 'objects/legacy_batch-02': {'table_lamp-pmekkr-0-arm-body-R-lower': 23529, 'table_lamp-pmekkr-0-head-neck-R-lower': 44331, 'table_lamp-pmekkr-0-arm-body-R-upper': 23529, 'table_lamp-pmekkr-0-head-neck-R-upper': 44331, 'table_lamp-pxdhzm-0-head-arm-R-lower': 33474, 'table_lamp-pxdhzm-0-head-arm-R-upper': 33474}, 'objects/legacy_batch-03': {'desk-egzdgz-0-base_link': 20943, 'dishwasher-znfvmj-0-link_0-base_link-P-lower': 31812, 'dishwasher-znfvm

In [46]:
# Worst offenders
sorted((cnt, obj) for warn_target in warn_objs.values() for obj, cnt in warn_target.items())

[(20082, 'barbecue_sauce_bottle-gfxrnj-0'),
 (20082, 'catsup_bottle-qfvqfm-0'),
 (20082, 'chocolate_sauce_bottle-yegrkf-0'),
 (20082, 'glaze_bottle-zdxagk-0'),
 (20082, 'hot_sauce_bottle-qvpthd-0'),
 (20082, 'mustard_bottle-lgxfyv-0'),
 (20082, 'salsa_bottle-kydilb-0'),
 (20082, 'soy_sauce_bottle-saujjl-0'),
 (20082, 'squeeze_bottle-uzmhdn-0'),
 (20082, 'sugar_syrup_bottle-kdlbbq-0'),
 (20082, 'vanilla_bottle-drevku-0'),
 (20082, 'white_sauce_bottle-gtwngf-0'),
 (20082, 'wine_sauce_bottle-vqtevv-0'),
 (20279, 'pot_plant-kxmvco-0-base_link'),
 (20943, 'desk-egzdgz-0-base_link'),
 (21060, 'swivel_chair-mqxrim-0-base_link'),
 (21794, 'L-tag-ohegnp-0'),
 (22097, 'L-garden_chair-deuron-0'),
 (22097, 'L-garden_chair-deuron-1'),
 (22560, 'room_light-fxtqtn-0'),
 (22560, 'room_light-fxtqtn-1'),
 (22740, 'fridge-jtqazu-0-base_link'),
 (22956, 'L-pedestal_table-ifmhpn-0'),
 (22956, 'L-pedestal_table-ifmhpn-1'),
 (22998, 'C-stockpot-gxiqbw-0'),
 (22998, 'C-stockpot-gxiqbw-1'),
 (23465, 'floor_lam

In [47]:
# Check which of the "articulated object has diff scale" errors are on bad objects
providers = json.loads(pathlib.Path(r"D:\ig_pipeline\artifacts\pipeline\object_inventory.json").read_text())["providers"]
id_providers = {k.split("-")[-1]: v for k, v in providers.items()}
categories = dict([tuple(reversed(k.split("-"))) for k in providers.keys()])
articulation_tmpl = "Articulated object (.*) instances have different scales for base links. This may have broken things during the match links script."
for target, objs in error_matches_by_file[articulation_tmpl].items():
    if not objs:
        continue
    bad_objs = set()
    for match in objs:
        obj = match.group(1)
        if id_providers[obj] != target:
            name = f"{categories[obj]}-{obj}"
            bad_objs.add(name)

    if not bad_objs:
        continue
    
    print(f"{target}: {len(bad_objs)} {bad_objs}")

scenes/Beechwood_0_garden: 6 {'top_cabinet-dmwxyl', 'bottom_cabinet_no_top-qohxjq', 'bottom_cabinet_no_top-spojpj', 'bottom_cabinet-immwzb', 'window-ulnafj', 'door-lvgliq'}
scenes/Beechwood_0_int: 7 {'top_cabinet-dmwxyl', 'bottom_cabinet_no_top-qohxjq', 'bottom_cabinet_no_top-spojpj', 'bottom_cabinet-immwzb', 'window-ulnafj', 'door-ohagsq', 'door-lvgliq'}
scenes/Beechwood_1_int: 9 {'top_cabinet-dmwxyl', 'bottom_cabinet_no_top-pluwfl', 'window-ithrgo', 'bottom_cabinet-jhymlr', 'toilet-kfmkbm', 'bottom_cabinet-kubcdk', 'window-ulnafj', 'door-ohagsq', 'door-lvgliq'}
scenes/Benevolence_0_int: 1 {'door-lvgliq'}
scenes/Benevolence_1_int: 6 {'bottom_cabinet_no_top-ufhpbn', 'top_cabinet-dmwxyl', 'top_cabinet-eobsmt', 'top_cabinet-lsyzkh', 'window-ulnafj', 'cedar_chest-fwstpx'}
scenes/Benevolence_2_int: 2 {'window-ithrgo', 'door-lvgliq'}
scenes/hotel_gym_spa: 2 {'door-ceddpg', 'door-nfrbch'}
scenes/house_single_floor: 1 {'bottom_cabinet_no_top-gjrero'}
scenes/Ihlen_0_int: 3 {'door-ktydvs', 'win

In [48]:
# Get a list of all the objects that show up in the match links relevant errors
mismatched_link_offset_tmpl = "(.*) link .* has different .* in instance .* compared to instance .*. .* difference: .*."
objects_for_match_links = error_matches[mismatched_link_offset_tmpl] | error_matches[articulation_tmpl]
print(", ".join(f'"{x.group(1)}"' for x in sorted(objects_for_match_links, key=lambda x: x.group(1))))

"bamfsz", "bamfsz", "bamfsz", "bamfsz", "bmsclc", "ceddpg", "ceddpg", "ceddpg", "dladgw", "dladgw", "dladgw", "dmwxyl", "dmwxyl", "dmwxyl", "dmwxyl", "dmwxyl", "dmwxyl", "dmwxyl", "dnyzym", "dqnbsj", "dqnbsj", "ehwmol", "ehwmol", "emeeke", "emeeke", "eobsmt", "fexqbj", "fexqbj", "fexqbj", "fqhdne", "fqhdne", "fqhdne", "fwstpx", "fwstpx", "gjrero", "gpkbiw", "gpkbiw", "ijnwlp", "ijnwlp", "ijnwlp", "ijnwlp", "ijnwlp", "ijnwlp", "ijnwlp", "ijnwlp", "ijnwlp", "ijnwlp", "ijnwlp", "immwzb", "immwzb", "immwzb", "inmymj", "inmymj", "ithrgo", "ithrgo", "ithrgo", "ithrgo", "ithrgo", "ithrgo", "ithrgo", "ithrgo", "jhymlr", "jhymlr", "jhymlr", "jrhgeu", "kfmkbm", "kfmkbm", "kfmkbm", "ktydvs", "ktydvs", "ktydvs", "kubcdk", "kwbnhy", "kwbnhy", "kwbnhy", "kwbnhy", "kwbnhy", "lcrefl", "lsyzkh", "lsyzkh", "lsyzkh", "lsyzkh", "lsyzkh", "lsyzkh", "lvgliq", "lvgliq", "lvgliq", "lvgliq", "lvgliq", "lvgliq", "lvgliq", "lvgliq", "lvgliq", "lvgliq", "lvgliq", "lvgliq", "lvgliq", "lvgliq", "lvgliq", "lvgliq", 

In [49]:
import sys
sys.path.append(r"D:\ig_pipeline")
from b1k_pipeline.utils import parse_name

# Get a link of all the objects that are bad cloth
cloth_tmpl = "Cloth object (.*) should consist of exactly 1 element. Currently it has .* elements."
objects_for_cloth = [parse_name(x.group(1)).group('model_id') for x in error_matches[cloth_tmpl]]
print(", ".join(f'"{x}"' for x in sorted(objects_for_cloth)))

"agftpm", "aiftuk", "ajcjzr", "bgvwes", "bpjqwj", "bpjqwj", "bpjqwj", "bpjqwj", "brebdk", "crbvcg", "cuoayr", "dnypbe", "efmfdf", "ekcjci", "eknmzl", "faedft", "feohwy", "fglfga", "ficvss", "fmapwe", "fzldgi", "getwzm", "girtqm", "gtghon", "gxmcpy", "htcikj", "imdhfp", "itrkhr", "ivmsdt", "jfsovh", "jkcugp", "jmujjo", "kclcrj", "kiiium", "klhkgd", "laksie", "ldpbcu", "lqweda", "mksdlu", "mpxtgf", "mxxyva", "njidcf", "nmvvil", "nogevo", "nowqqh", "nzbhjq", "ohvomi", "ohvomi", "ojgwec", "omsbux", "padfds", "pbytey", "pchzob", "prhems", "pvzxyp", "qewdqa", "qofxar", "rcgdde", "remcyk", "rhgweu", "rygqoc", "sehjcp", "shbakk", "shbakk", "stcrad", "sullco", "szdonp", "szdonp", "szdonp", "szdonp", "tfnwti", "thmepr", "tnirgd", "trwest", "ttgike", "uaidmc", "uksirc", "uksirc", "uwzqxm", "uxgjdv", "vfknnl", "vpafgj", "vqbvph", "wbhliu", "wczokr", "xeskbh", "xzuywj", "ydgivr", "ykiiqr", "ylycdz"
